In [7]:
def create_data_model(problem):
    if problem == "CMT1":
        return data_CMT1()
    elif problem == "CMT2":
        return data_CMT2()
    elif problem == "CMT3":
        return data_CMT3()
    elif problem == "CMT11":
        return data_CMT11()

In [8]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import time


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = ''
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0}'.format(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0}'.format(manager.IndexToNode(index))
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model("CMT3")

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(60)

    # Solve the problem.
    starttime = time.time()
    solution = routing.SolveWithParameters(search_parameters)
    duration = time.time() - starttime
    
    print("Solver status: ", routing.status())

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    print("duration", duration)

if __name__ == '__main__':
    main()

Solver status:  1
Objective: 821
 0 18 60 84 17 45 46 47 36 49 64 63 11 19 48 52 0
 0 1 70 30 51 9 35 71 65 66 20 32 90 10 62 0
 0 6 92 98 37 100 91 44 38 86 16 61 85 96 0
 0 58 2 57 42 14 43 15 41 22 75 74 72 73 21 40 53 0
 0 28 76 77 3 68 80 24 29 79 78 34 81 33 50 0
 0 27 69 31 88 7 82 8 83 5 93 59 99 89 0
 0 94 95 97 87 13 0
 0 12 54 55 25 39 67 23 56 4 26 0
Total distance of all routes: 821m
Total load of all routes: 1458
duration 60.0100302696228


In [1]:
def data_CMT1():
    data = {}
    data["distance_matrix"] = [[0.0,
          13.892443989449804,
          21.02379604162864,
          32.55764119219941,
          17.204650534085253,
          14.142135623730951,
          11.40175425099138,
          26.419689627245813,
          22.02271554554524,
          23.08679276123039,
          28.319604517012593,
          12.041594578792296,
          8.06225774829855,
          29.154759474226502,
          18.110770276274835,
          24.73863375370596,
          22.02271554554524,
          17.26267650163207,
          14.7648230602334,
          31.906112267087632,
          32.449961479175904,
          32.0624390837628,
          20.808652046684813,
          22.02271554554524,
          25.059928172283335,
          23.08679276123039,
          28.160255680657446,
          8.0,
          29.966648127543394,
          29.120439557122072,
          30.870698080866262,
          29.832867780352597,
          10.0,
          34.0,
          31.78049716414141,
          39.408120990476064,
          43.93176527297759,
          18.110770276274835,
          15.811388300841896,
          38.28837943815329,
          42.20189569201838,
          30.479501308256342,
          31.32091952673165,
          34.655446902326915,
          25.0,
          31.32091952673165,
          2.23606797749979,
          9.433981132056603,
          15.811388300841896,
          21.633307652783937,
          26.1725046566048],
         [13.892443989449804,
          0.0,
          12.36931687685298,
          19.209372712298546,
          31.064449134018133,
          22.20360331117452,
          16.76305461424021,
          22.825424421026653,
          11.661903789690601,
          24.20743687382041,
          34.0147027033899,
          12.083045973594572,
          20.8806130178211,
          41.86884283091664,
          26.92582403567252,
          36.013886210738214,
          18.601075237738275,
          30.675723300355934,
          27.586228448267445,
          45.79301256742124,
          20.8806130178211,
          26.92582403567252,
          7.0710678118654755,
          21.587033144922902,
          29.0,
          33.1058907144937,
          18.867962264113206,
          8.06225774829855,
          16.15549442140351,
          21.37755832643195,
          32.64965543462902,
          17.0,
          6.082762530298219,
          42.95346318982906,
          30.610455730027933,
          27.313000567495326,
          31.064449134018133,
          30.4138126514911,
          18.788294228055936,
          43.04648650006177,
          56.0357029044876,
          44.204072210600685,
          44.94441010848846,
          34.17601498127012,
          37.656340767525464,
          42.04759208325728,
          13.92838827718412,
          23.323807579381203,
          12.36931687685298,
          26.40075756488817,
          24.20743687382041],
         [21.02379604162864,
          12.36931687685298,
          0.0,
          15.297058540778355,
          37.013511046643494,
          21.02379604162864,
          28.071337695236398,
          34.92849839314596,
          22.20360331117452,
          16.278820596099706,
          28.071337695236398,
          10.63014581273465,
          24.758836806279895,
          50.11985634456667,
          37.656340767525464,
          35.4682957019364,
          8.54400374531753,
          34.058772731852805,
          35.77708763999664,
          50.91168824543142,
          12.041594578792296,
          14.7648230602334,
          10.63014581273465,
          33.95585369269929,
          41.10960958218893,
          43.41658669218482,
          29.068883707497267,
          19.026297590440446,
          18.973665961010276,
          9.055385138137417,
          23.769728648009426,
          23.323807579381203,
          11.40175425099138,
          39.11521443121589,
          20.0,
          19.1049731745428,
          24.413111231467404,
          31.906112267087632,
          14.560219778561036,
          35.4400902933387,
          61.5223536610881,
          50.44799302251776,
          48.010415536631214,
          46.486557196677836,
          38.948684188300895,
          40.26164427839479,
          19.72308292331602,
          29.410882339705484,
          24.73863375370596,
          21.02379604162864,
          13.892443989449804],
         [32.55764119219941,
          19.209372712298546,
          15.297058540778355,
          0.0,
          49.678969393496885,
          36.05551275463989,
          35.35533905932738,
          35.014282800023196,
          21.095023109728988,
          31.0,
          43.01162633521314,
          25.079872407968907,
          38.27531841800928,
          61.07372593840988,
          45.65084884205331,
          50.59644256269407,
          23.0,
          48.02082881417188,
          46.75467891024384,
          64.20280367709809,
          7.810249675906654,
          24.166091947189145,
          12.206555615733702,
          36.6742416417845,
          45.60701700396552,
          51.97114584074513,
          25.317977802344327,
          27.202941017470888,
          9.486832980505138,
          17.08800749063506,
          37.48332962798263,
          15.811388300841896,
          22.80350850198276,
          54.3323108288245,
          32.28002478313795,
          10.04987562112089,
          12.083045973594572,
          46.51881339845203,
          29.832867780352597,
          49.49747468305833,
          74.65252842335616,
          63.0317380372777,
          62.26556030423239,
          47.0,
          53.71219600798314,
          55.54277630799526,
          32.01562118716424,
          41.86884283091664,
          28.460498941515414,
          36.22154055254967,
          27.294688127912362],
         [17.204650534085253,
          31.064449134018133,
          37.013511046643494,
          49.678969393496885,
          0.0,
          20.396078054371138,
          21.02379604162864,
          37.12142238654117,
          37.64306044943742,
          32.7566787083184,
          31.400636936215164,
          26.627053911388696,
          12.529964086141668,
          15.033296378372908,
          17.88854381999832,
          18.867962264113206,
          35.34119409414458,
          7.615773105863909,
          7.615773105863909,
          14.7648230602334,
          48.91829923454004,
          44.94441010848846,
          38.01315561749642,
          31.25699921617557,
          28.635642126552707,
          17.69180601295413,
          42.579337712087536,
          24.166091947189145,
          47.01063709417264,
          43.9089968002003,
          38.01315561749642,
          46.238512086787566,
          26.90724809414742,
          30.528675044947494,
          41.593268686170845,
          55.97320787662612,
          60.81118318204309,
          12.649110640673518,
          26.570660511172846,
          40.52159917870962,
          25.0,
          13.45362404707371,
          16.0312195418814,
          40.85339643163099,
          14.866068747318506,
          24.839484696748443,
          17.69180601295413,
          7.810249675906654,
          29.427877939124322,
          28.071337695236398,
          37.64306044943742],
         [14.142135623730951,
          22.20360331117452,
          21.02379604162864,
          36.05551275463989,
          20.396078054371138,
          0.0,
          25.495097567963924,
          40.22437072223753,
          33.24154027718932,
          12.36931687685298,
          14.212670403551895,
          11.180339887498949,
          9.219544457292887,
          35.35533905932738,
          30.463092423455635,
          14.560219778561036,
          16.278820596099706,
          14.7648230602334,
          23.194827009486403,
          31.906112267087632,
          32.7566787083184,
          25.059928172283335,
          27.073972741361768,
          36.124783736376884,
          38.8329756778952,
          33.95585369269929,
          40.162171256046406,
          20.591260281974,
          37.12142238654117,
          25.45584412271571,
          18.24828759089466,
          39.11521443121589,
          16.1245154965971,
          20.8806130178211,
          21.213203435596427,
          39.66106403010388,
          45.27692569068709,
          11.313708498984761,
          7.0710678118654755,
          24.20743687382041,
          42.43819034784589,
          32.69556544854363,
          27.586228448267445,
          48.79549159502341,
          18.027756377319946,
          20.024984394500787,
          12.041594578792296,
          15.132745950421556,
          29.154759474226502,
          8.246211251235321,
          17.46424919657298],
         [11.40175425099138,
          16.76305461424021,
          28.071337695236398,
          35.35533905932738,
          21.02379604162864,
          25.495097567963924,
          0.0,
          16.492422502470642,
          18.027756377319946,
          34.0147027033899,
          39.698866482558415,
          21.840329667841555,
          18.027756377319946,
          27.202941017470888,
          10.295630140987,
          34.438350715445125,
          31.575306807693888,
          24.73863375370596,
          14.560219778561036,
          34.92849839314596,
          37.64306044943742,
          41.30375285612676,
          23.259406699226016,
          11.180339887498949,
          13.92838827718412,
          16.64331697709324,
          21.840329667841555,
          9.055385138137417,
          29.732137494637012,
          37.013511046643494,
          42.05948168962618,
          27.202941017470888,
          17.029386365926403,
          44.654227123532216,
          42.37924020083418,
          44.01136216933077,
          47.41307836451879,
          27.313000567495326,
          26.832815729997478,
          49.678969393496885,
          44.01136216933077,
          31.953090617340916,
          37.0,
          23.345235059857504,
          33.24154027718932,
          41.14608122288197,
          13.601470508735444,
          15.524174696260024,
          8.94427190999916,
          33.015148038438355,
          36.40054944640259],
         [26.419689627245813,
          22.825424421026653,
          34.92849839314596,
          35.014282800023196,
          37.12142238654117,
          40.22437072223753,
          16.492422502470642,
          0.0,
          14.035668847618199,
          46.09772228646444,
          54.037024344425184,
          33.301651610693426,
          34.0147027033899,
          39.84971769034255,
          21.587033144922902,
          50.695167422546305,
          41.340053217188775,
          41.23105625617661,
          30.0,
          50.15974481593781,
          40.311288741492746,
          49.658836071740545,
          25.709920264364882,
          6.082762530298219,
          14.212670403551895,
          26.92582403567252,
          11.180339887498949,
          19.849433241279208,
          26.30589287593181,
          43.657759905886145,
          54.56189146281496,
          20.8806130178211,
          27.018512172212592,
          60.41522986797286,
          53.25410782277739,
          45.0,
          46.389654018972806,
          43.657759905886145,
          39.59797974644666,
          63.7808748764079,
          58.249463516842795,
          46.52956049652737,
          53.150729063673246,
          12.041594578792296,
          49.72926703662542,
          57.38466694161429,
          28.30194339616981,
          32.01562118716424,
          11.313708498984761,
          46.75467891024384,
          46.87216658103186],
         [22.02271554554524,
          11.661903789690601,
          22.20360331117452,
          21.095023109728988,
          37.64306044943742,
          33.24154027718932,
          18.027756377319946,
          14.035668847618199,
          0.0,
          35.805027579936315,
          45.617978911828175,
          23.706539182259394,
          30.0,
          45.221676218380054,
          27.586228448267445,
          46.2709412050371,
          29.698484809834994,
          39.20459156782532,
          32.202484376209235,
          52.20153254455275,
          26.30589287593181,
          36.89173349139343,
          12.083045973594572,
          15.811388300841896,
          25.079872407968907,
          33.94112549695428,
          7.211102550927978,
          14.035668847618199,
          13.0,
          30.4138126514911,
          44.204072210600685,
          9.219544457292887,
          17.46424919657298,
          54.120236510939236,
          41.72529209005013,
          31.016124838541646,
          32.7566787083184,
          40.01249804748511,
          30.4138126514911,
          54.70831746635972,
          61.741396161732524,
          49.658836071740545,
          52.952809179494906,
          26.076809620810597,
          47.01063709417264,
          52.61178575186362,
          23.021728866442675,
          30.59411708155671,
          9.219544457292887,
          38.01315561749642,
          35.35533905932738],
         [23.08679276123039,
          24.20743687382041,
          16.278820596099706,
          31.0,
          32.7566787083184,
          12.36931687685298,
          34.0147027033899,
          46.09772228646444,
          35.805027579936315,
          0.0,
          12.041594578792296,
          12.806248474865697,
          21.02379604162864,
          47.67598976424087,
          41.0,
          23.345235059857504,
          8.0,
          26.92582403567252,
          35.0,
          43.829214001622255,
          25.495097567963924,
          13.45362404707371,
          26.0,
          43.266615305567875,
          47.92702786528704,
          45.27692569068709,
          43.01162633521314,
          26.627053911388696,
          35.17101079013795,
          16.15549442140351,
          8.48528137423857,
          39.0,
          19.1049731745428,
          23.769728648009426,
          9.0,
          31.622776601683793,
          37.64306044943742,
          22.825424421026653,
          7.280109889280518,
          19.313207915827967,
          54.20332093147061,
          44.94441010848846,
          38.600518131237564,
          56.302753041036986,
          28.42534080710379,
          26.419689627245813,
          20.8806130178211,
          27.018512172212592,
          34.828149534535996,
          6.4031242374328485,
          5.656854249492381],
         [28.319604517012593,
          34.0147027033899,
          28.071337695236398,
          43.01162633521314,
          31.400636936215164,
          14.212670403551895,
          39.698866482558415,
          54.037024344425184,
          45.617978911828175,
          12.041594578792296,
          0.0,
          21.93171219946131,
          22.825424421026653,
          46.17358552246078,
          44.294469180700204,
          15.811388300841896,
          20.024984394500787,
          24.08318915758459,
          36.05551275463989,
          38.8329756778952,
          37.48332962798263,
          23.706539182259394,
          37.107950630558946,
          50.20956084253277,
          53.009433122794285,
          47.16990566028302,
          52.773099207835045,
          34.20526275297414,
          46.69047011971501,
          27.892651361962706,
          9.219544457292887,
          50.0,
          28.178005607210743,
          12.083045973594572,
          15.620499351813308,
          43.41658669218482,
          49.47726750741192,
          19.026297590440446,
          15.231546211727817,
          10.0,
          48.38388161361178,
          41.19465984809196,
          31.953090617340916,
          62.96824596572466,
          21.840329667841555,
          16.278820596099706,
          26.1725046566048,
          28.231188426986208,
          42.80186911806539,
          7.615773105863909,
          16.76305461424021],
         [12.041594578792296,
          12.083045973594572,
          10.63014581273465,
          25.079872407968907,
          26.627053911388696,
          11.180339887498949,
          21.840329667841555,
          33.301651610693426,
          23.706539182259394,
          12.806248474865697,
          21.93171219946131,
          0.0,
          14.212670403551895,
          40.311288741492746,
          30.01666203960727,
          25.709920264364882,
          10.0,
          23.430749027719962,
          26.248809496813376,
          40.311288741492746,
          22.67156809750927,
          20.024984394500787,
          16.0,
          30.528675044947494,
          35.73513677041127,
          35.12833614050059,
          30.886890422961002,
          13.892443989449804,
          26.019223662515376,
          17.46424919657298,
          21.2602916254693,
          28.442925306655784,
          6.4031242374328485,
          31.25699921617557,
          20.615528128088304,
          29.732137494637012,
          35.0,
          21.470910553583888,
          6.708203932499369,
          31.064449134018133,
          50.93132631298737,
          40.0,
          37.44329045369811,
          43.56604182158393,
          28.635642126552707,
          31.144823004794873,
          10.198039027185569,
          19.235384061671343,
          22.02271554554524,
          14.317821063276353,
          14.560219778561036],
         [8.06225774829855,
          20.8806130178211,
          24.758836806279895,
          38.27531841800928,
          12.529964086141668,
          9.219544457292887,
          18.027756377319946,
          34.0147027033899,
          30.0,
          21.02379604162864,
          22.825424421026653,
          14.212670403551895,
          0.0,
          26.92582403567252,
          21.470910553583888,
          16.76305461424021,
          22.847319317591726,
          9.848857801796104,
          14.035668847618199,
          26.1725046566048,
          36.76955262170047,
          32.57299494980466,
          27.313000567495326,
          29.154759474226502,
          30.479501308256342,
          24.73863375370596,
          36.22154055254967,
          16.0312195418814,
          37.0,
          31.38470965295043,
          27.459060435491963,
          37.48332962798263,
          15.652475842498529,
          26.627053911388696,
          30.01666203960727,
          43.840620433565945,
          48.91829923454004,
          10.04987562112089,
          14.317821063276353,
          32.7566787083184,
          36.76955262170047,
          25.80697580112788,
          24.166091947189145,
          41.23105625617661,
          17.029386365926403,
          23.40939982143925,
          7.0710678118654755,
          6.0,
          23.769728648009426,
          17.46424919657298,
          25.495097567963924],
         [29.154759474226502,
          41.86884283091664,
          50.11985634456667,
          61.07372593840988,
          15.033296378372908,
          35.35533905932738,
          27.202941017470888,
          39.84971769034255,
          45.221676218380054,
          47.67598976424087,
          46.17358552246078,
          40.311288741492746,
          26.92582403567252,
          0.0,
          18.384776310850235,
          32.28002478313795,
          49.64876634922564,
          22.090722034374522,
          14.422205101855956,
          14.422205101855956,
          61.587336360651285,
          59.481089431852205,
          48.91829923454004,
          33.83784863137726,
          27.16615541441225,
          13.152946437965905,
          48.30113870293329,
          33.97057550292606,
          56.639209034025185,
          57.77542730261716,
          53.03772242470448,
          54.405882034941776,
          39.11521443121589,
          43.657759905886145,
          56.568542494923804,
          68.5054742338158,
          72.80109889280519,
          27.16615541441225,
          41.23105625617661,
          54.91812087098393,
          19.0,
          9.433981132056603,
          21.93171219946131,
          39.0,
          26.92582403567252,
          37.16180835212409,
          30.4138126514911,
          21.18962010041709,
          36.05551275463989,
          43.104524124504614,
          52.392747589718944],
         [18.110770276274835,
          26.92582403567252,
          37.656340767525464,
          45.65084884205331,
          17.88854381999832,
          30.463092423455635,
          10.295630140987,
          21.587033144922902,
          27.586228448267445,
          41.0,
          44.294469180700204,
          30.01666203960727,
          21.470910553583888,
          18.384776310850235,
          0.0,
          35.38361202590826,
          40.01249804748511,
          24.20743687382041,
          10.295630140987,
          29.017236257093817,
          47.75981574503821,
          50.0,
          33.54101966249684,
          15.524174696260024,
          10.770329614269007,
          6.4031242374328485,
          30.01666203960727,
          18.973665961010276,
          39.824615503479755,
          46.389654018972806,
          48.38388161361178,
          36.796738985948195,
          26.30589287593181,
          46.69047011971501,
          49.8196748283246,
          54.230987451824994,
          57.706152185014034,
          28.284271247461902,
          33.734255586866,
          54.20332093147061,
          36.6742416417845,
          25.079872407968907,
          33.24154027718932,
          23.08679276123039,
          32.449961479175904,
          41.86884283091664,
          20.223748416156685,
          16.401219466856727,
          18.384776310850235,
          38.62641583165593,
          44.28317965096906],
         [24.73863375370596,
          36.013886210738214,
          35.4682957019364,
          50.59644256269407,
          18.867962264113206,
          14.560219778561036,
          34.438350715445125,
          50.695167422546305,
          46.2709412050371,
          23.345235059857504,
          15.811388300841896,
          25.709920264364882,
          16.76305461424021,
          32.28002478313795,
          35.38361202590826,
          0.0,
          29.68164415931166,
          11.40175425099138,
          25.495097567963924,
          23.194827009486403,
          46.95742752749558,
          36.76955262170047,
          41.43669871020132,
          45.617978911828175,
          45.60701700396552,
          36.40054944640259,
          52.773099207835045,
          32.55764119219941,
          51.478150704935004,
          38.8329756778952,
          24.596747752497688,
          53.009433122794285,
          30.066592756745816,
          11.661903789690601,
          30.23243291566195,
          53.71219600798314,
          59.481089431852205,
          7.211102550927978,
          21.02379604162864,
          23.021728866442675,
          32.57299494980466,
          26.019223662515376,
          16.15549442140351,
          57.14017850864661,
          6.082762530298219,
          6.708203932499369,
          23.345235059857504,
          19.4164878389476,
          40.52159917870962,
          16.97056274847714,
          29.0],
         [22.02271554554524,
          18.601075237738275,
          8.54400374531753,
          23.0,
          35.34119409414458,
          16.278820596099706,
          31.575306807693888,
          41.340053217188775,
          29.698484809834994,
          8.0,
          20.024984394500787,
          10.0,
          22.847319317591726,
          49.64876634922564,
          40.01249804748511,
          29.68164415931166,
          0.0,
          30.805843601498726,
          35.90264614203248,
          48.010415536631214,
          17.72004514666935,
          10.04987562112089,
          18.867962264113206,
          39.395431207184416,
          45.35416188179427,
          45.09988913511872,
          36.796738985948195,
          23.08679276123039,
          27.51363298439521,
          9.219544457292887,
          15.231546211727817,
          31.76476034853718,
          14.866068747318506,
          31.575306807693888,
          12.041594578792296,
          24.166091947189145,
          30.083217912982647,
          27.586228448267445,
          9.219544457292887,
          26.92582403567252,
          58.60034129593445,
          48.373546489791295,
          43.840620433565945,
          52.32590180780452,
          34.058772731852805,
          33.61547262794322,
          20.09975124224178,
          28.460498941515414,
          30.4138126514911,
          13.601470508735444,
          5.656854249492381],
         [17.26267650163207,
          30.675723300355934,
          34.058772731852805,
          48.02082881417188,
          7.615773105863909,
          14.7648230602334,
          24.73863375370596,
          41.23105625617661,
          39.20459156782532,
          26.92582403567252,
          24.08318915758459,
          23.430749027719962,
          9.848857801796104,
          22.090722034374522,
          24.20743687382041,
          11.40175425099138,
          30.805843601498726,
          0.0,
          14.142135623730951,
          17.204650534085253,
          46.09772228646444,
          39.824615503479755,
          37.16180835212409,
          35.73513677041127,
          34.66987164671943,
          25.0,
          45.0,
          25.179356624028344,
          46.8187996428785,
          39.824615503479755,
          31.25699921617557,
          47.07440918375928,
          25.495097567963924,
          23.021728866442675,
          35.4400902933387,
          53.150729063673246,
          58.412327466040935,
          5.0990195135927845,
          21.633307652783937,
          32.984845004941285,
          27.80287754891569,
          18.027756377319946,
          14.317821063276353,
          46.52956049652737,
          8.54400374531753,
          17.69180601295413,
          16.76305461424021,
          9.219544457292887,
          32.0624390837628,
          21.587033144922902,
          32.202484376209235],
         [14.7648230602334,
          27.586228448267445,
          35.77708763999664,
          46.75467891024384,
          7.615773105863909,
          23.194827009486403,
          14.560219778561036,
          30.0,
          32.202484376209235,
          35.0,
          36.05551275463989,
          26.248809496813376,
          14.035668847618199,
          14.422205101855956,
          10.295630140987,
          25.495097567963924,
          35.90264614203248,
          14.142135623730951,
          0.0,
          20.396078054371138,
          47.16990566028302,
          45.89117562233506,
          34.655446902326915,
          24.020824298928627,
          21.02379604162864,
          11.180339887498949,
          36.40054944640259,
          19.849433241279208,
          42.80186911806539,
          43.657759905886145,
          41.43669871020132,
          41.182520563948,
          24.698178070456937,
          37.013511046643494,
          44.0,
          54.08326913195984,
          58.412327466040935,
          18.601075237738275,
          28.071337695236398,
          45.69463863518345,
          29.546573405388315,
          17.46424919657298,
          23.345235059857504,
          33.24154027718932,
          22.20360331117452,
          31.827660925679098,
          16.15549442140351,
          8.06225774829855,
          23.40939982143925,
          31.400636936215164,
          39.20459156782532],
         [31.906112267087632,
          45.79301256742124,
          50.91168824543142,
          64.20280367709809,
          14.7648230602334,
          31.906112267087632,
          34.92849839314596,
          50.15974481593781,
          52.20153254455275,
          43.829214001622255,
          38.8329756778952,
          40.311288741492746,
          26.1725046566048,
          14.422205101855956,
          29.017236257093817,
          23.194827009486403,
          48.010415536631214,
          17.204650534085253,
          20.396078054371138,
          0.0,
          62.93647591023825,
          56.938563381947034,
          52.69724850502159,
          44.10215414239989,
          39.319206502675,
          25.709920264364882,
          56.753854494650845,
          38.91015291668744,
          61.773780845922005,
          57.0087712549569,
          47.12748667179272,
          60.92618484691127,
          41.400483088968905,
          33.13608305156178,
          52.0,
          70.00714249274856,
          75.07329751649384,
          21.02379604162864,
          38.8329756778952,
          46.04345773288535,
          10.63014581273465,
          5.0,
          8.54400374531753,
          51.62363799656123,
          17.11724276862369,
          26.1725046566048,
          32.202484376209235,
          22.47220505424423,
          43.68065933568311,
          38.07886552931954,
          49.24428900898052],
         [32.449961479175904,
          20.8806130178211,
          12.041594578792296,
          7.810249675906654,
          48.91829923454004,
          32.7566787083184,
          37.64306044943742,
          40.311288741492746,
          26.30589287593181,
          25.495097567963924,
          37.48332962798263,
          22.67156809750927,
          36.76955262170047,
          61.587336360651285,
          47.75981574503821,
          46.95742752749558,
          17.72004514666935,
          46.09772228646444,
          47.16990566028302,
          62.93647591023825,
          0.0,
          16.76305461424021,
          15.033296378372908,
          41.012193308819754,
          49.36598018878993,
          53.85164807134504,
          31.622776601683793,
          28.792360097775937,
          16.64331697709324,
          10.04987562112089,
          31.016124838541646,
          22.825424421026653,
          22.47220505424423,
          49.24428900898052,
          25.317977802344327,
          7.0710678118654755,
          12.529964086141668,
          43.829214001622255,
          25.942243542145693,
          43.04648650006177,
          73.53910524340094,
          62.369864518050704,
          60.0,
          52.3450093132096,
          50.774009099144415,
          51.264022471905186,
          31.400636936215164,
          41.23105625617661,
          32.14031735997639,
          31.32091952673165,
          21.02379604162864],
         [32.0624390837628,
          26.92582403567252,
          14.7648230602334,
          24.166091947189145,
          44.94441010848846,
          25.059928172283335,
          41.30375285612676,
          49.658836071740545,
          36.89173349139343,
          13.45362404707371,
          23.706539182259394,
          20.024984394500787,
          32.57299494980466,
          59.481089431852205,
          50.0,
          36.76955262170047,
          10.04987562112089,
          39.824615503479755,
          45.89117562233506,
          56.938563381947034,
          16.76305461424021,
          0.0,
          25.0,
          48.38388161361178,
          54.91812087098393,
          55.14526271584895,
          43.60045871318328,
          32.55764119219941,
          31.400636936215164,
          7.211102550927978,
          15.524174696260024,
          36.796738985948195,
          24.331050121192877,
          35.77708763999664,
          9.055385138137417,
          21.0,
          27.018512172212592,
          36.05551275463989,
          18.384776310850235,
          27.16615541441225,
          67.4166151627327,
          57.697486947006624,
          52.009614495783374,
          61.09828148156051,
          41.86884283091664,
          39.408120990476064,
          30.14962686336267,
          38.3275357934736,
          39.21734310225516,
          19.79898987322333,
          7.810249675906654],
         [20.808652046684813,
          7.0710678118654755,
          10.63014581273465,
          12.206555615733702,
          38.01315561749642,
          27.073972741361768,
          23.259406699226016,
          25.709920264364882,
          12.083045973594572,
          26.0,
          37.107950630558946,
          16.0,
          27.313000567495326,
          48.91829923454004,
          33.54101966249684,
          41.43669871020132,
          18.867962264113206,
          37.16180835212409,
          34.655446902326915,
          52.69724850502159,
          15.033296378372908,
          25.0,
          0.0,
          26.0,
          34.36568055487916,
          39.824615503479755,
          18.601075237738275,
          15.0,
          10.04987562112089,
          18.35755975068582,
          34.0,
          13.0,
          11.704699910719626,
          47.16990566028302,
          30.610455730027933,
          20.8806130178211,
          24.186773244895647,
          36.40054944640259,
          22.20360331117452,
          45.31004303683677,
          63.00793600809346,
          51.22499389946279,
          51.478150704935004,
          37.656340767525464,
          43.68065933568311,
          47.095647357266465,
          20.591260281974,
          30.23243291566195,
          17.11724276862369,
          29.614185789921695,
          24.413111231467404],
         [22.02271554554524,
          21.587033144922902,
          33.95585369269929,
          36.6742416417845,
          31.25699921617557,
          36.124783736376884,
          11.180339887498949,
          6.082762530298219,
          15.811388300841896,
          43.266615305567875,
          50.20956084253277,
          30.528675044947494,
          29.154759474226502,
          33.83784863137726,
          15.524174696260024,
          45.617978911828175,
          39.395431207184416,
          35.73513677041127,
          24.020824298928627,
          44.10215414239989,
          41.012193308819754,
          48.38388161361178,
          26.0,
          0.0,
          9.433981132056603,
          21.02379604162864,
          15.556349186104045,
          16.64331697709324,
          28.792360097775937,
          42.95346318982906,
          51.61395160225576,
          24.186773244895647,
          24.596747752497688,
          55.758407437802596,
          51.0,
          46.389654018972806,
          48.507731342539614,
          38.48376280978771,
          36.40054944640259,
          60.108235708594876,
          52.172789842982326,
          40.44749683231337,
          47.265209192385896,
          13.038404810405298,
          44.27188724235731,
          52.32590180780452,
          24.08318915758459,
          26.570660511172846,
          9.219544457292887,
          43.18564576337837,
          44.721359549995796],
         [25.059928172283335,
          29.0,
          41.10960958218893,
          45.60701700396552,
          28.635642126552707,
          38.8329756778952,
          13.92838827718412,
          14.212670403551895,
          25.079872407968907,
          47.92702786528704,
          53.009433122794285,
          35.73513677041127,
          30.479501308256342,
          27.16615541441225,
          10.770329614269007,
          45.60701700396552,
          45.35416188179427,
          34.66987164671943,
          21.02379604162864,
          39.319206502675,
          49.36598018878993,
          54.91812087098393,
          34.36568055487916,
          9.433981132056603,
          0.0,
          14.035668847618199,
          24.839484696748443,
          22.360679774997898,
          38.07886552931954,
          50.15974481593781,
          55.90169943749474,
          33.61547262794322,
          30.59411708155671,
          56.639209034025185,
          56.302753041036986,
          55.10898293381942,
          57.56735185849702,
          38.41874542459709,
          40.718546143004666,
          63.00793600809346,
          46.09772228646444,
          35.05709628591621,
          43.965895873961216,
          12.36931687685298,
          43.04648650006177,
          52.20153254455275,
          27.294688127912362,
          26.248809496813376,
          17.26267650163207,
          46.647615158762406,
          50.28916384272063],
         [23.08679276123039,
          33.1058907144937,
          43.41658669218482,
          51.97114584074513,
          17.69180601295413,
          33.95585369269929,
          16.64331697709324,
          26.92582403567252,
          33.94112549695428,
          45.27692569068709,
          47.16990566028302,
          35.12833614050059,
          24.73863375370596,
          13.152946437965905,
          6.4031242374328485,
          36.40054944640259,
          45.09988913511872,
          25.0,
          11.180339887498949,
          25.709920264364882,
          53.85164807134504,
          55.14526271584895,
          39.824615503479755,
          21.02379604162864,
          14.035668847618199,
          0.0,
          36.05551275463989,
          25.079872407968907,
          46.22769732530488,
          51.97114584074513,
          52.172789842982326,
          43.139309220245984,
          32.01562118716424,
          48.010415536631214,
          54.230987451824994,
          60.41522986797286,
          64.00781202322104,
          29.68164415931166,
          38.118237105091836,
          56.859475903318,
          32.0624390837628,
          21.213203435596427,
          31.304951684997057,
          26.076809620810597,
          32.526911934581186,
          42.5205832509386,
          25.019992006393608,
          18.973665961010276,
          24.758836806279895,
          42.20189569201838,
          49.01020301937138],
         [28.160255680657446,
          18.867962264113206,
          29.068883707497267,
          25.317977802344327,
          42.579337712087536,
          40.162171256046406,
          21.840329667841555,
          11.180339887498949,
          7.211102550927978,
          43.01162633521314,
          52.773099207835045,
          30.886890422961002,
          36.22154055254967,
          48.30113870293329,
          30.01666203960727,
          52.773099207835045,
          36.796738985948195,
          45.0,
          36.40054944640259,
          56.753854494650845,
          31.622776601683793,
          43.60045871318328,
          18.601075237738275,
          15.556349186104045,
          24.839484696748443,
          36.05551275463989,
          0.0,
          20.223748416156685,
          16.0312195418814,
          36.89173349139343,
          51.40038910358559,
          10.04987562112089,
          24.596747752497688,
          61.032778078668514,
          48.79549159502341,
          35.35533905932738,
          36.013886210738214,
          46.2709412050371,
          37.589892258425,
          61.91122676865643,
          65.7875368135941,
          53.75872022286245,
          58.309518948453004,
          22.360679774997898,
          53.08483775994799,
          59.22837157984339,
          29.427877939124322,
          36.05551275463989,
          13.152946437965905,
          45.17742799230607,
          42.44997055358225],
         [8.0,
          8.06225774829855,
          19.026297590440446,
          27.202941017470888,
          24.166091947189145,
          20.591260281974,
          9.055385138137417,
          19.849433241279208,
          14.035668847618199,
          26.627053911388696,
          34.20526275297414,
          13.892443989449804,
          16.0312195418814,
          33.97057550292606,
          18.973665961010276,
          32.55764119219941,
          23.08679276123039,
          25.179356624028344,
          19.849433241279208,
          38.91015291668744,
          28.792360097775937,
          32.55764119219941,
          15.0,
          16.64331697709324,
          22.360679774997898,
          25.079872407968907,
          20.223748416156685,
          0.0,
          23.021728866442675,
          28.0,
          35.0,
          22.135943621178654,
          8.246211251235321,
          41.23105625617661,
          34.438350715445125,
          35.34119409414458,
          39.11521443121589,
          26.076809620810597,
          19.849433241279208,
          43.93176527297759,
          48.877397639399746,
          36.89173349139343,
          39.05124837953327,
          29.68164415931166,
          33.0,
          39.05124837953327,
          9.219544457292887,
          16.76305461424021,
          8.602325267042627,
          26.90724809414742,
          28.231188426986208],
         [29.966648127543394,
          16.15549442140351,
          18.973665961010276,
          9.486832980505138,
          47.01063709417264,
          37.12142238654117,
          29.732137494637012,
          26.30589287593181,
          13.0,
          35.17101079013795,
          46.69047011971501,
          26.019223662515376,
          37.0,
          56.639209034025185,
          39.824615503479755,
          51.478150704935004,
          27.51363298439521,
          46.8187996428785,
          42.80186911806539,
          61.773780845922005,
          16.64331697709324,
          31.400636936215164,
          10.04987562112089,
          28.792360097775937,
          38.07886552931954,
          46.22769732530488,
          16.0312195418814,
          23.021728866442675,
          0.0,
          24.20743687382041,
          42.720018726587654,
          6.324555320336759,
          21.587033144922902,
          57.07889277132134,
          38.47076812334269,
          19.4164878389476,
          20.09975124224178,
          46.32493928760188,
          32.0624390837628,
          54.405882034941776,
          71.8679344353238,
          59.90826320300064,
          61.09828148156051,
          38.118237105091836,
          53.600373133029585,
          57.14017850864661,
          30.083217912982647,
          39.35733730830886,
          21.633307652783937,
          39.319206502675,
          32.69556544854363],
         [29.120439557122072,
          21.37755832643195,
          9.055385138137417,
          17.08800749063506,
          43.9089968002003,
          25.45584412271571,
          37.013511046643494,
          43.657759905886145,
          30.4138126514911,
          16.15549442140351,
          27.892651361962706,
          17.46424919657298,
          31.38470965295043,
          57.77542730261716,
          46.389654018972806,
          38.8329756778952,
          9.219544457292887,
          39.824615503479755,
          43.657759905886145,
          57.0087712549569,
          10.04987562112089,
          7.211102550927978,
          18.35755975068582,
          42.95346318982906,
          50.15974481593781,
          51.97114584074513,
          36.89173349139343,
          28.0,
          24.20743687382041,
          0.0,
          21.0,
          29.698484809834994,
          20.09975124224178,
          39.84971769034255,
          15.297058540778355,
          15.524174696260024,
          21.587033144922902,
          36.76955262170047,
          18.384776310850235,
          33.015148038438355,
          67.62396025078685,
          57.14017850864661,
          53.03772242470448,
          55.362442142665635,
          43.278170016764804,
          42.485291572496,
          27.51363298439521,
          36.6742416417845,
          33.734255586866,
          22.360679774997898,
          11.180339887498949],
         [30.870698080866262,
          32.64965543462902,
          23.769728648009426,
          37.48332962798263,
          38.01315561749642,
          18.24828759089466,
          42.05948168962618,
          54.56189146281496,
          44.204072210600685,
          8.48528137423857,
          9.219544457292887,
          21.2602916254693,
          27.459060435491963,
          53.03772242470448,
          48.38388161361178,
          24.596747752497688,
          15.231546211727817,
          31.25699921617557,
          41.43669871020132,
          47.12748667179272,
          31.016124838541646,
          15.524174696260024,
          34.0,
          51.61395160225576,
          55.90169943749474,
          52.172789842982326,
          51.40038910358559,
          35.0,
          42.720018726587654,
          21.0,
          0.0,
          46.95742752749558,
          27.586228448267445,
          20.808652046684813,
          6.708203932499369,
          36.22154055254967,
          42.2965719651132,
          26.476404589747453,
          15.264337522473747,
          12.041594578792296,
          57.0087712549569,
          49.03060268852505,
          40.718546143004666,
          64.63745044476924,
          30.463092423455635,
          25.495097567963924,
          28.635642126552707,
          33.37663853655727,
          43.278170016764804,
          10.04987562112089,
          10.198039027185569],
         [29.832867780352597,
          17.0,
          23.323807579381203,
          15.811388300841896,
          46.238512086787566,
          39.11521443121589,
          27.202941017470888,
          20.8806130178211,
          9.219544457292887,
          39.0,
          50.0,
          28.442925306655784,
          37.48332962798263,
          54.405882034941776,
          36.796738985948195,
          53.009433122794285,
          31.76476034853718,
          47.07440918375928,
          41.182520563948,
          60.92618484691127,
          22.825424421026653,
          36.796738985948195,
          13.0,
          24.186773244895647,
          33.61547262794322,
          43.139309220245984,
          10.04987562112089,
          22.135943621178654,
          6.324555320336759,
          29.698484809834994,
          46.95742752749558,
          0.0,
          23.021728866442675,
          59.682493245507096,
          43.266615305567875,
          25.709920264364882,
          26.0,
          47.265209192385896,
          34.92849839314596,
          58.309518948453004,
          70.66116330771806,
          58.59180830116101,
          61.1310068623117,
          32.38826948140329,
          54.45181356024793,
          59.033888572581766,
          30.4138126514911,
          38.897300677553446,
          18.439088914585774,
          42.44997055358225,
          37.21558813185679],
         [10.0,
          6.082762530298219,
          11.40175425099138,
          22.80350850198276,
          26.90724809414742,
          16.1245154965971,
          17.029386365926403,
          27.018512172212592,
          17.46424919657298,
          19.1049731745428,
          28.178005607210743,
          6.4031242374328485,
          15.652475842498529,
          39.11521443121589,
          26.30589287593181,
          30.066592756745816,
          14.866068747318506,
          25.495097567963924,
          24.698178070456937,
          41.400483088968905,
          22.47220505424423,
          24.331050121192877,
          11.704699910719626,
          24.596747752497688,
          30.59411708155671,
          32.01562118716424,
          24.596747752497688,
          8.246211251235321,
          21.587033144922902,
          20.09975124224178,
          27.586228448267445,
          23.021728866442675,
          0.0,
          36.87817782917155,
          26.419689627245813,
          29.410882339705484,
          33.97057550292606,
          24.73863375370596,
          13.038404810405298,
          37.44329045369811,
          51.85556864985669,
          40.311288741492746,
          39.81205847478876,
          37.589892258425,
          32.01562118716424,
          36.013886210738214,
          9.219544457292887,
          19.1049731745428,
          15.811388300841896,
          20.591260281974,
          20.12461179749811],
         [34.0,
          42.95346318982906,
          39.11521443121589,
          54.3323108288245,
          30.528675044947494,
          20.8806130178211,
          44.654227123532216,
          60.41522986797286,
          54.120236510939236,
          23.769728648009426,
          12.083045973594572,
          31.25699921617557,
          26.627053911388696,
          43.657759905886145,
          46.69047011971501,
          11.661903789690601,
          31.575306807693888,
          23.021728866442675,
          37.013511046643494,
          33.13608305156178,
          49.24428900898052,
          35.77708763999664,
          47.16990566028302,
          55.758407437802596,
          56.639209034025185,
          48.010415536631214,
          61.032778078668514,
          41.23105625617661,
          57.07889277132134,
          39.84971769034255,
          20.808652046684813,
          59.682493245507096,
          36.87817782917155,
          0.0,
          27.459060435491963,
          55.362442142665635,
          61.40032573203501,
          18.439088914585774,
          25.019992006393608,
          13.92838827718412,
          41.19465984809196,
          36.6742416417845,
          25.0,
          67.80117993073571,
          16.76305461424021,
          7.0,
          32.202484376209235,
          30.4138126514911,
          49.658836071740545,
          18.110770276274835,
          28.792360097775937],
         [31.78049716414141,
          30.610455730027933,
          20.0,
          32.28002478313795,
          41.593268686170845,
          21.213203435596427,
          42.37924020083418,
          53.25410782277739,
          41.72529209005013,
          9.0,
          15.620499351813308,
          20.615528128088304,
          30.01666203960727,
          56.568542494923804,
          49.8196748283246,
          30.23243291566195,
          12.041594578792296,
          35.4400902933387,
          44.0,
          52.0,
          25.317977802344327,
          9.055385138137417,
          30.610455730027933,
          51.0,
          56.302753041036986,
          54.230987451824994,
          48.79549159502341,
          34.438350715445125,
          38.47076812334269,
          15.297058540778355,
          6.708203932499369,
          43.266615305567875,
          26.419689627245813,
          27.459060435491963,
          0.0,
          30.01666203960727,
          36.05551275463989,
          31.016124838541646,
          16.1245154965971,
          18.110770276274835,
          62.16912416947821,
          53.45091205957107,
          46.14108798023731,
          64.00781202322104,
          35.84689665786984,
          31.827660925679098,
          29.614185789921695,
          36.013886210738214,
          42.190046219457976,
          13.92838827718412,
          6.4031242374328485],
         [39.408120990476064,
          27.313000567495326,
          19.1049731745428,
          10.04987562112089,
          55.97320787662612,
          39.66106403010388,
          44.01136216933077,
          45.0,
          31.016124838541646,
          31.622776601683793,
          43.41658669218482,
          29.732137494637012,
          43.840620433565945,
          68.5054742338158,
          54.230987451824994,
          53.71219600798314,
          24.166091947189145,
          53.150729063673246,
          54.08326913195984,
          70.00714249274856,
          7.0710678118654755,
          21.0,
          20.8806130178211,
          46.389654018972806,
          55.10898293381942,
          60.41522986797286,
          35.35533905932738,
          35.34119409414458,
          19.4164878389476,
          15.524174696260024,
          36.22154055254967,
          25.709920264364882,
          29.410882339705484,
          55.362442142665635,
          30.01666203960727,
          0.0,
          6.082762530298219,
          50.80354318352215,
          32.7566787083184,
          48.093658625644196,
          80.61017305526642,
          69.42621983083913,
          67.00746227100382,
          57.0087712549569,
          57.688820407423826,
          57.77542730261716,
          38.41874542459709,
          48.27007354458868,
          37.8549864614954,
          37.69615364994153,
          26.68332812825267],
         [43.93176527297759,
          31.064449134018133,
          24.413111231467404,
          12.083045973594572,
          60.81118318204309,
          45.27692569068709,
          47.41307836451879,
          46.389654018972806,
          32.7566787083184,
          37.64306044943742,
          49.47726750741192,
          35.0,
          48.91829923454004,
          72.80109889280519,
          57.706152185014034,
          59.481089431852205,
          30.083217912982647,
          58.412327466040935,
          58.412327466040935,
          75.07329751649384,
          12.529964086141668,
          27.018512172212592,
          24.186773244895647,
          48.507731342539614,
          57.56735185849702,
          64.00781202322104,
          36.013886210738214,
          39.11521443121589,
          20.09975124224178,
          21.587033144922902,
          42.2965719651132,
          26.0,
          33.97057550292606,
          61.40032573203501,
          36.05551275463989,
          6.082762530298219,
          0.0,
          56.302753041036986,
          38.47076812334269,
          54.147945482723536,
          85.63293758829018,
          74.24957912338628,
          72.42237223399962,
          58.215118311311535,
          63.28506932918696,
          63.694583757176716,
          43.139309220245984,
          53.03772242470448,
          40.496913462633174,
          43.657759905886145,
          32.7566787083184],
         [18.110770276274835,
          30.4138126514911,
          31.906112267087632,
          46.51881339845203,
          12.649110640673518,
          11.313708498984761,
          27.313000567495326,
          43.657759905886145,
          40.01249804748511,
          22.825424421026653,
          19.026297590440446,
          21.470910553583888,
          10.04987562112089,
          27.16615541441225,
          28.284271247461902,
          7.211102550927978,
          27.586228448267445,
          5.0990195135927845,
          18.601075237738275,
          21.02379604162864,
          43.829214001622255,
          36.05551275463989,
          36.40054944640259,
          38.48376280978771,
          38.41874542459709,
          29.68164415931166,
          46.2709412050371,
          26.076809620810597,
          46.32493928760188,
          36.76955262170047,
          26.476404589747453,
          47.265209192385896,
          24.73863375370596,
          18.439088914585774,
          31.016124838541646,
          50.80354318352215,
          56.302753041036986,
          0.0,
          18.384776310850235,
          27.892651361962706,
          31.38470965295043,
          22.561028345356956,
          16.278820596099706,
          49.92995093127971,
          7.280109889280518,
          13.892443989449804,
          17.0,
          12.206555615733702,
          33.734255586866,
          17.08800749063506,
          28.30194339616981],
         [15.811388300841896,
          18.788294228055936,
          14.560219778561036,
          29.832867780352597,
          26.570660511172846,
          7.0710678118654755,
          26.832815729997478,
          39.59797974644666,
          30.4138126514911,
          7.280109889280518,
          15.231546211727817,
          6.708203932499369,
          14.317821063276353,
          41.23105625617661,
          33.734255586866,
          21.02379604162864,
          9.219544457292887,
          21.633307652783937,
          28.071337695236398,
          38.8329756778952,
          25.942243542145693,
          18.384776310850235,
          22.20360331117452,
          36.40054944640259,
          40.718546143004666,
          38.118237105091836,
          37.589892258425,
          19.849433241279208,
          32.0624390837628,
          18.384776310850235,
          15.264337522473747,
          34.92849839314596,
          13.038404810405298,
          25.019992006393608,
          16.1245154965971,
          32.7566787083184,
          38.47076812334269,
          18.384776310850235,
          0.0,
          24.413111231467404,
          49.4064773081425,
          39.35733730830886,
          34.655446902326915,
          49.4064773081425,
          25.0,
          25.709920264364882,
          13.601470508735444,
          20.223748416156685,
          28.284271247461902,
          7.615773105863909,
          11.180339887498949],
         [38.28837943815329,
          43.04648650006177,
          35.4400902933387,
          49.49747468305833,
          40.52159917870962,
          24.20743687382041,
          49.678969393496885,
          63.7808748764079,
          54.70831746635972,
          19.313207915827967,
          10.0,
          31.064449134018133,
          32.7566787083184,
          54.91812087098393,
          54.20332093147061,
          23.021728866442675,
          26.92582403567252,
          32.984845004941285,
          45.69463863518345,
          46.04345773288535,
          43.04648650006177,
          27.16615541441225,
          45.31004303683677,
          60.108235708594876,
          63.00793600809346,
          56.859475903318,
          61.91122676865643,
          43.93176527297759,
          54.405882034941776,
          33.015148038438355,
          12.041594578792296,
          58.309518948453004,
          37.44329045369811,
          13.92838827718412,
          18.110770276274835,
          48.093658625644196,
          54.147945482723536,
          27.892651361962706,
          24.413111231467404,
          0.0,
          54.74486277268398,
          49.040799340956916,
          38.3275357934736,
          72.91776189653656,
          29.0,
          20.615528128088304,
          36.124783736376884,
          38.01315561749642,
          52.49761899362675,
          17.029386365926403,
          22.20360331117452],
         [42.20189569201838,
          56.0357029044876,
          61.5223536610881,
          74.65252842335616,
          25.0,
          42.43819034784589,
          44.01136216933077,
          58.249463516842795,
          61.741396161732524,
          54.20332093147061,
          48.38388161361178,
          50.93132631298737,
          36.76955262170047,
          19.0,
          36.6742416417845,
          32.57299494980466,
          58.60034129593445,
          27.80287754891569,
          29.546573405388315,
          10.63014581273465,
          73.53910524340094,
          67.4166151627327,
          63.00793600809346,
          52.172789842982326,
          46.09772228646444,
          32.0624390837628,
          65.7875368135941,
          48.877397639399746,
          71.8679344353238,
          67.62396025078685,
          57.0087712549569,
          70.66116330771806,
          51.85556864985669,
          41.19465984809196,
          62.16912416947821,
          80.61017305526642,
          85.63293758829018,
          31.38470965295043,
          49.4064773081425,
          54.74486277268398,
          0.0,
          12.083045973594572,
          16.492422502470642,
          58.0,
          26.570660511172846,
          34.23448553724738,
          42.638011210655684,
          32.802438933713454,
          52.92447448959697,
          48.30113870293329,
          59.682493245507096],
         [30.479501308256342,
          44.204072210600685,
          50.44799302251776,
          63.0317380372777,
          13.45362404707371,
          32.69556544854363,
          31.953090617340916,
          46.52956049652737,
          49.658836071740545,
          44.94441010848846,
          41.19465984809196,
          40.0,
          25.80697580112788,
          9.433981132056603,
          25.079872407968907,
          26.019223662515376,
          48.373546489791295,
          18.027756377319946,
          17.46424919657298,
          5.0,
          62.369864518050704,
          57.697486947006624,
          51.22499389946279,
          40.44749683231337,
          35.05709628591621,
          21.213203435596427,
          53.75872022286245,
          36.89173349139343,
          59.90826320300064,
          57.14017850864661,
          49.03060268852505,
          58.59180830116101,
          40.311288741492746,
          36.6742416417845,
          53.45091205957107,
          69.42621983083913,
          74.24957912338628,
          22.561028345356956,
          39.35733730830886,
          49.040799340956916,
          12.083045973594572,
          0.0,
          13.038404810405298,
          47.265209192385896,
          20.09975124224178,
          29.832867780352597,
          31.11269837220809,
          21.213203435596427,
          40.85339643163099,
          39.56008088970496,
          50.15974481593781],
         [31.32091952673165,
          44.94441010848846,
          48.010415536631214,
          62.26556030423239,
          16.0312195418814,
          27.586228448267445,
          37.0,
          53.150729063673246,
          52.952809179494906,
          38.600518131237564,
          31.953090617340916,
          37.44329045369811,
          24.166091947189145,
          21.93171219946131,
          33.24154027718932,
          16.15549442140351,
          43.840620433565945,
          14.317821063276353,
          23.345235059857504,
          8.54400374531753,
          60.0,
          52.009614495783374,
          51.478150704935004,
          47.265209192385896,
          43.965895873961216,
          31.304951684997057,
          58.309518948453004,
          39.05124837953327,
          61.09828148156051,
          53.03772242470448,
          40.718546143004666,
          61.1310068623117,
          39.81205847478876,
          25.0,
          46.14108798023731,
          67.00746227100382,
          72.42237223399962,
          16.278820596099706,
          34.655446902326915,
          38.3275357934736,
          16.492422502470642,
          13.038404810405298,
          0.0,
          56.32051136131489,
          10.295630140987,
          18.0,
          31.016124838541646,
          22.360679774997898,
          45.17742799230607,
          32.449961479175904,
          44.204072210600685],
         [34.655446902326915,
          34.17601498127012,
          46.486557196677836,
          47.0,
          40.85339643163099,
          48.79549159502341,
          23.345235059857504,
          12.041594578792296,
          26.076809620810597,
          56.302753041036986,
          62.96824596572466,
          43.56604182158393,
          41.23105625617661,
          39.0,
          23.08679276123039,
          57.14017850864661,
          52.32590180780452,
          46.52956049652737,
          33.24154027718932,
          51.62363799656123,
          52.3450093132096,
          61.09828148156051,
          37.656340767525464,
          13.038404810405298,
          12.36931687685298,
          26.076809620810597,
          22.360679774997898,
          29.68164415931166,
          38.118237105091836,
          55.362442142665635,
          64.63745044476924,
          32.38826948140329,
          37.589892258425,
          67.80117993073571,
          64.00781202322104,
          57.0087712549569,
          58.215118311311535,
          49.92995093127971,
          49.4064773081425,
          72.91776189653656,
          58.0,
          47.265209192385896,
          56.32051136131489,
          0.0,
          55.00909015790027,
          63.812224534175265,
          36.796738985948195,
          37.73592452822641,
          21.93171219946131,
          56.08029957123981,
          57.706152185014034],
         [25.0,
          37.656340767525464,
          38.948684188300895,
          53.71219600798314,
          14.866068747318506,
          18.027756377319946,
          33.24154027718932,
          49.72926703662542,
          47.01063709417264,
          28.42534080710379,
          21.840329667841555,
          28.635642126552707,
          17.029386365926403,
          26.92582403567252,
          32.449961479175904,
          6.082762530298219,
          34.058772731852805,
          8.54400374531753,
          22.20360331117452,
          17.11724276862369,
          50.774009099144415,
          41.86884283091664,
          43.68065933568311,
          44.27188724235731,
          43.04648650006177,
          32.526911934581186,
          53.08483775994799,
          33.0,
          53.600373133029585,
          43.278170016764804,
          30.463092423455635,
          54.45181356024793,
          32.01562118716424,
          16.76305461424021,
          35.84689665786984,
          57.688820407423826,
          63.28506932918696,
          7.280109889280518,
          25.0,
          29.0,
          26.570660511172846,
          20.09975124224178,
          10.295630140987,
          55.00909015790027,
          0.0,
          10.295630140987,
          24.08318915758459,
          17.72004514666935,
          40.311288741492746,
          22.20360331117452,
          34.058772731852805],
         [31.32091952673165,
          42.04759208325728,
          40.26164427839479,
          55.54277630799526,
          24.839484696748443,
          20.024984394500787,
          41.14608122288197,
          57.38466694161429,
          52.61178575186362,
          26.419689627245813,
          16.278820596099706,
          31.144823004794873,
          23.40939982143925,
          37.16180835212409,
          41.86884283091664,
          6.708203932499369,
          33.61547262794322,
          17.69180601295413,
          31.827660925679098,
          26.1725046566048,
          51.264022471905186,
          39.408120990476064,
          47.095647357266465,
          52.32590180780452,
          52.20153254455275,
          42.5205832509386,
          59.22837157984339,
          39.05124837953327,
          57.14017850864661,
          42.485291572496,
          25.495097567963924,
          59.033888572581766,
          36.013886210738214,
          7.0,
          31.827660925679098,
          57.77542730261716,
          63.694583757176716,
          13.892443989449804,
          25.709920264364882,
          20.615528128088304,
          34.23448553724738,
          29.832867780352597,
          18.0,
          63.812224534175265,
          10.295630140987,
          0.0,
          29.832867780352597,
          26.076809620810597,
          47.12748667179272,
          20.12461179749811,
          31.906112267087632],
         [2.23606797749979,
          13.92838827718412,
          19.72308292331602,
          32.01562118716424,
          17.69180601295413,
          12.041594578792296,
          13.601470508735444,
          28.30194339616981,
          23.021728866442675,
          20.8806130178211,
          26.1725046566048,
          10.198039027185569,
          7.0710678118654755,
          30.4138126514911,
          20.223748416156685,
          23.345235059857504,
          20.09975124224178,
          16.76305461424021,
          16.15549442140351,
          32.202484376209235,
          31.400636936215164,
          30.14962686336267,
          20.591260281974,
          24.08318915758459,
          27.294688127912362,
          25.019992006393608,
          29.427877939124322,
          9.219544457292887,
          30.083217912982647,
          27.51363298439521,
          28.635642126552707,
          30.4138126514911,
          9.219544457292887,
          32.202484376209235,
          29.614185789921695,
          38.41874542459709,
          43.139309220245984,
          17.0,
          13.601470508735444,
          36.124783736376884,
          42.638011210655684,
          31.11269837220809,
          31.016124838541646,
          36.796738985948195,
          24.08318915758459,
          29.832867780352597,
          0.0,
          9.899494936611665,
          17.46424919657298,
          19.4164878389476,
          24.08318915758459],
         [9.433981132056603,
          23.323807579381203,
          29.410882339705484,
          41.86884283091664,
          7.810249675906654,
          15.132745950421556,
          15.524174696260024,
          32.01562118716424,
          30.59411708155671,
          27.018512172212592,
          28.231188426986208,
          19.235384061671343,
          6.0,
          21.18962010041709,
          16.401219466856727,
          19.4164878389476,
          28.460498941515414,
          9.219544457292887,
          8.06225774829855,
          22.47220505424423,
          41.23105625617661,
          38.3275357934736,
          30.23243291566195,
          26.570660511172846,
          26.248809496813376,
          18.973665961010276,
          36.05551275463989,
          16.76305461424021,
          39.35733730830886,
          36.6742416417845,
          33.37663853655727,
          38.897300677553446,
          19.1049731745428,
          30.4138126514911,
          36.013886210738214,
          48.27007354458868,
          53.03772242470448,
          12.206555615733702,
          20.223748416156685,
          38.01315561749642,
          32.802438933713454,
          21.213203435596427,
          22.360679774997898,
          37.73592452822641,
          17.72004514666935,
          26.076809620810597,
          9.899494936611665,
          0.0,
          23.0,
          23.345235059857504,
          31.400636936215164],
         [15.811388300841896,
          12.36931687685298,
          24.73863375370596,
          28.460498941515414,
          29.427877939124322,
          29.154759474226502,
          8.94427190999916,
          11.313708498984761,
          9.219544457292887,
          34.828149534535996,
          42.80186911806539,
          22.02271554554524,
          23.769728648009426,
          36.05551275463989,
          18.384776310850235,
          40.52159917870962,
          30.4138126514911,
          32.0624390837628,
          23.40939982143925,
          43.68065933568311,
          32.14031735997639,
          39.21734310225516,
          17.11724276862369,
          9.219544457292887,
          17.26267650163207,
          24.758836806279895,
          13.152946437965905,
          8.602325267042627,
          21.633307652783937,
          33.734255586866,
          43.278170016764804,
          18.439088914585774,
          15.811388300841896,
          49.658836071740545,
          42.190046219457976,
          37.8549864614954,
          40.496913462633174,
          33.734255586866,
          28.284271247461902,
          52.49761899362675,
          52.92447448959697,
          40.85339643163099,
          45.17742799230607,
          21.93171219946131,
          40.311288741492746,
          47.12748667179272,
          17.46424919657298,
          23.0,
          0.0,
          35.4682957019364,
          35.84689665786984],
         [21.633307652783937,
          26.40075756488817,
          21.02379604162864,
          36.22154055254967,
          28.071337695236398,
          8.246211251235321,
          33.015148038438355,
          46.75467891024384,
          38.01315561749642,
          6.4031242374328485,
          7.615773105863909,
          14.317821063276353,
          17.46424919657298,
          43.104524124504614,
          38.62641583165593,
          16.97056274847714,
          13.601470508735444,
          21.587033144922902,
          31.400636936215164,
          38.07886552931954,
          31.32091952673165,
          19.79898987322333,
          29.614185789921695,
          43.18564576337837,
          46.647615158762406,
          42.20189569201838,
          45.17742799230607,
          26.90724809414742,
          39.319206502675,
          22.360679774997898,
          10.04987562112089,
          42.44997055358225,
          20.591260281974,
          18.110770276274835,
          13.92838827718412,
          37.69615364994153,
          43.657759905886145,
          17.08800749063506,
          7.615773105863909,
          17.029386365926403,
          48.30113870293329,
          39.56008088970496,
          32.449961479175904,
          56.08029957123981,
          22.20360331117452,
          20.12461179749811,
          19.4164878389476,
          23.345235059857504,
          35.4682957019364,
          0.0,
          12.041594578792296],
         [26.1725046566048,
          24.20743687382041,
          13.892443989449804,
          27.294688127912362,
          37.64306044943742,
          17.46424919657298,
          36.40054944640259,
          46.87216658103186,
          35.35533905932738,
          5.656854249492381,
          16.76305461424021,
          14.560219778561036,
          25.495097567963924,
          52.392747589718944,
          44.28317965096906,
          29.0,
          5.656854249492381,
          32.202484376209235,
          39.20459156782532,
          49.24428900898052,
          21.02379604162864,
          7.810249675906654,
          24.413111231467404,
          44.721359549995796,
          50.28916384272063,
          49.01020301937138,
          42.44997055358225,
          28.231188426986208,
          32.69556544854363,
          11.180339887498949,
          10.198039027185569,
          37.21558813185679,
          20.12461179749811,
          28.792360097775937,
          6.4031242374328485,
          26.68332812825267,
          32.7566787083184,
          28.30194339616981,
          11.180339887498949,
          22.20360331117452,
          59.682493245507096,
          50.15974481593781,
          44.204072210600685,
          57.706152185014034,
          34.058772731852805,
          31.906112267087632,
          24.08318915758459,
          31.400636936215164,
          35.84689665786984,
          12.041594578792296,
          0.0]
    ]
    data['demands'] = [0, 
                         7,
                         30,
                         16,
                         9,
                         21,
                         15,
                         19,
                         23,
                         11,
                         5,
                         19,
                         29,
                         23,
                         21,
                         10,
                         15,
                         3,
                         41,
                         9,
                         28,
                         8,
                         8,
                         16,
                         10,
                         28,
                         7,
                         15,
                         14,
                         6,
                         19,
                         11,
                         12,
                         23,
                         26,
                         17,
                         6,
                         9,
                         15,
                         14,
                         7,
                         27,
                         13,
                         11,
                         16,
                         10,
                         5,
                         25,
                         17,
                         18,
                         10
    ]
    data['vehicle_capacities'] = [160, 160, 160, 160, 160]
    data['num_vehicles'] = 5
    data['depot'] = 0
    return data

In [2]:
def data_CMT2():
    data = {}
    data["distance_matrix"] = [[0.0,
  25.45584412271571,
  14.560219778561036,
  19.6468827043885,
  7.0710678118654755,
  25.0,
  9.219544457292887,
  14.142135623730951,
  15.811388300841896,
  23.600847442411894,
  26.0,
  29.154759474226502,
  12.083045973594572,
  22.561028345356956,
  27.80287754891569,
  27.202941017470888,
  19.4164878389476,
  8.06225774829855,
  34.88552708502482,
  23.40939982143925,
  36.76955262170047,
  27.294688127912362,
  30.4138126514911,
  31.38470965295043,
  33.13608305156178,
  33.24154027718932,
  6.082762530298219,
  16.15549442140351,
  24.515301344262525,
  18.439088914585774,
  14.317821063276353,
  37.107950630558946,
  22.20360331117452,
  17.804493814764857,
  10.0,
  18.027756377319946,
  33.1058907144937,
  32.01562118716424,
  26.92582403567252,
  22.360679774997898,
  14.142135623730951,
  36.235341863986875,
  36.069377593742864,
  31.89043743820395,
  20.615528128088304,
  14.142135623730951,
  11.180339887498949,
  26.92582403567252,
  20.615528128088304,
  28.071337695236398,
  29.68164415931166,
  11.045361017187261,
  14.142135623730951,
  22.67156809750927,
  27.018512172212592,
  42.42640687119285,
  37.16180835212409,
  28.178005607210743,
  20.0,
  38.41874542459709,
  43.266615305567875,
  34.23448553724738,
  22.360679774997898,
  22.360679774997898,
  43.01162633521314,
  31.622776601683793,
  36.235341863986875,
  5.385164807134504,
  7.280109889280518,
  37.36308338453881,
  41.23105625617661,
  39.824615503479755,
  20.615528128088304,
  20.615528128088304,
  20.0,
  3.0],
 [25.45584412271571,
  0.0,
  14.560219778561036,
  23.021728866442675,
  26.419689627245813,
  33.06055050963308,
  16.278820596099706,
  39.59797974644666,
  40.22437072223753,
  37.21558813185679,
  47.53945729601885,
  54.20332093147061,
  31.78049716414141,
  42.05948168962618,
  53.150729063673246,
  40.049968789001575,
  14.035668847618199,
  24.596747752497688,
  36.40054944640259,
  47.70744176750625,
  44.721359549995796,
  23.769728648009426,
  9.848857801796104,
  12.529964086141668,
  25.80697580112788,
  42.2965719651132,
  30.610455730027933,
  35.11409973215888,
  14.317821063276353,
  30.265491900843113,
  21.37755832643195,
  54.74486277268398,
  31.0,
  8.06225774829855,
  33.28663395418648,
  43.278170016764804,
  34.17601498127012,
  38.63935817272331,
  50.60632371551998,
  38.8329756778952,
  29.120439557122072,
  11.180339887498949,
  10.63014581273465,
  6.708203932499369,
  26.019223662515376,
  29.120439557122072,
  35.22782990761707,
  28.861739379323623,
  26.019223662515376,
  18.867962264113206,
  34.713109915419565,
  18.384776310850235,
  35.77708763999664,
  48.104053883222775,
  48.84669896727925,
  49.47726750741192,
  16.278820596099706,
  43.289721643826724,
  42.04759208325728,
  63.7808748764079,
  45.69463863518345,
  21.2602916254693,
  8.246211251235321,
  8.246211251235321,
  18.384776310850235,
  55.569775957799216,
  61.032778078668514,
  30.479501308256342,
  19.4164878389476,
  33.28663395418648,
  46.17358552246078,
  40.718546143004666,
  40.162171256046406,
  5.385164807134504,
  18.110770276274835,
  23.430749027719962],
 [14.560219778561036,
  14.560219778561036,
  0.0,
  24.20743687382041,
  12.727922061357855,
  19.924858845171276,
  8.54400374531753,
  27.784887978899608,
  26.870057685088806,
  34.48187929913333,
  40.19950248448356,
  43.382023926967726,
  25.019992006393608,
  27.51363298439521,
  40.45985664828782,
  26.076809620810597,
  18.027756377319946,
  18.24828759089466,
  40.36087214122113,
  34.058772731852805,
  32.31098884280702,
  15.264337522473747,
  16.401219466856727,
  25.079872407968907,
  33.61547262794322,
  42.485291572496,
  20.615528128088304,
  20.615528128088304,
  10.04987562112089,
  16.0,
  7.0,
  50.24937810560445,
  30.4138126514911,
  10.44030650891055,
  19.79898987322333,
  30.610455730027933,
  24.08318915758459,
  26.40075756488817,
  41.48493702538308,
  34.52535300326414,
  24.73863375370596,
  25.632011235952593,
  24.186773244895647,
  21.18962010041709,
  26.627053911388696,
  14.560219778561036,
  21.93171219946131,
  17.804493814764857,
  13.0,
  26.832815729997478,
  36.61966684720111,
  14.7648230602334,
  21.633307652783937,
  36.359317925395686,
  34.438350715445125,
  51.10772935672255,
  30.01666203960727,
  29.017236257093817,
  34.23448553724738,
  50.99019513592785,
  35.608987629529715,
  20.0,
  8.48528137423857,
  16.492422502470642,
  29.698484809834994,
  46.17358552246078,
  50.566787519082126,
  18.35755975068582,
  7.280109889280518,
  26.076809620810597,
  34.9857113690718,
  31.144823004794873,
  34.0147027033899,
  9.219544457292887,
  7.211102550927978,
  11.704699910719626],
 [19.6468827043885,
  23.021728866442675,
  24.20743687382041,
  0.0,
  26.0,
  42.20189569201838,
  16.278820596099706,
  29.427877939124322,
  34.0,
  14.866068747318506,
  28.319604517012593,
  39.44616584663204,
  15.231546211727817,
  42.20189569201838,
  42.720018726587654,
  46.06517122512408,
  9.0,
  12.041594578792296,
  16.278820596099706,
  41.10960958218893,
  54.644304369257,
  39.408120990476064,
  32.38826948140329,
  19.72308292331602,
  14.142135623730951,
  19.4164878389476,
  20.024984394500787,
  35.73513677041127,
  32.202484376209235,
  36.359317925395686,
  29.068883707497267,
  32.57299494980466,
  8.06225774829855,
  16.76305461424021,
  29.427877939124322,
  34.36568055487916,
  48.104053883222775,
  49.20365840057018,
  33.421549934136806,
  17.4928556845359,
  10.295630140987,
  29.410882339705484,
  31.575306807693888,
  26.476404589747453,
  3.0,
  32.64965543462902,
  30.14962686336267,
  41.72529209005013,
  36.124783736376884,
  11.40175425099138,
  12.529964086141668,
  10.0,
  33.734255586866,
  36.05551275463989,
  46.17358552246078,
  27.313000567495326,
  25.0,
  47.53945729601885,
  24.20743687382041,
  52.55473337388365,
  59.413803110051795,
  41.78516483155236,
  26.570660511172846,
  15.033296378372908,
  40.44749683231337,
  38.28837943815329,
  45.0,
  24.186773244895647,
  20.808652046684813,
  50.21951811795888,
  58.25804665451803,
  55.17245689653489,
  20.518284528683193,
  21.840329667841555,
  31.400636936215164,
  20.615528128088304],
 [7.0710678118654755,
  26.419689627245813,
  12.727922061357855,
  26.0,
  0.0,
  18.027756377319946,
  12.041594578792296,
  15.811388300841896,
  14.142135623730951,
  30.610455730027933,
  31.400636936215164,
  31.622776601683793,
  18.867962264113206,
  17.0,
  27.80287754891569,
  20.248456731316587,
  24.020824298928627,
  15.0,
  41.677331968349414,
  21.400934559032695,
  29.698484809834994,
  22.02271554554524,
  29.068883707497267,
  34.713109915419565,
  38.8329756778952,
  40.311288741492746,
  11.704699910719626,
  10.04987562112089,
  21.470910553583888,
  11.40175425099138,
  9.219544457292887,
  43.32435804486894,
  29.206163733020468,
  19.924858845171276,
  7.0710678118654755,
  18.027756377319946,
  26.570660511172846,
  25.0,
  31.064449134018133,
  29.154759474226502,
  21.213203435596427,
  37.589892258425,
  36.61966684720111,
  33.12099032335839,
  27.294688127912362,
  7.0710678118654755,
  9.219544457292887,
  20.615528128088304,
  14.317821063276353,
  33.13608305156178,
  36.61966684720111,
  16.492422502470642,
  9.486832980505138,
  24.166091947189145,
  22.80350850198276,
  49.49747468305833,
  40.26164427839479,
  21.540659228538015,
  25.495097567963924,
  38.28837943815329,
  36.359317925395686,
  30.364452901377952,
  21.213203435596427,
  25.495097567963924,
  42.42640687119285,
  35.35533905932738,
  38.897300677553446,
  7.0,
  7.280109889280518,
  31.400636936215164,
  34.20526275297414,
  33.1058907144937,
  26.92582403567252,
  21.095023109728988,
  15.811388300841896,
  5.385164807134504],
 [25.0,
  33.06055050963308,
  19.924858845171276,
  42.20189569201838,
  18.027756377319946,
  0.0,
  26.076809620810597,
  30.4138126514911,
  25.0,
  48.60041152089147,
  48.38388161361178,
  45.0,
  36.89173349139343,
  16.55294535724685,
  37.656340767525464,
  8.06225774829855,
  37.57658845611187,
  32.55764119219941,
  58.412327466040935,
  28.861739379323623,
  12.529964086141668,
  13.038404810405298,
  29.832867780352597,
  44.721359549995796,
  53.225933528685054,
  58.137767414994535,
  29.5296461204668,
  14.0,
  20.396078054371138,
  6.708203932499369,
  13.416407864998739,
  60.92618484691127,
  46.66904755831214,
  30.364452901377952,
  20.615528128088304,
  30.0,
  10.04987562112089,
  7.0710678118654755,
  46.69047011971501,
  47.16990566028302,
  39.05124837953327,
  43.104524124504614,
  40.44749683231337,
  39.01281840626232,
  44.04543109109048,
  11.180339887498949,
  22.360679774997898,
  7.0710678118654755,
  7.0710678118654755,
  46.61544808322666,
  53.81449618829484,
  32.202484376209235,
  18.027756377319946,
  37.0,
  24.186773244895647,
  67.26812023536856,
  49.25444142409901,
  12.206555615733702,
  42.720018726587654,
  46.486557196677836,
  18.35755975068582,
  23.600847442411894,
  25.0,
  36.40054944640259,
  42.720018726587654,
  50.24937810560445,
  52.03844732503075,
  24.166091947189145,
  21.400934559032695,
  16.76305461424021,
  16.278820596099706,
  15.524174696260024,
  44.721359549995796,
  28.284271247461902,
  15.0,
  22.67156809750927],
 [9.219544457292887,
  16.278820596099706,
  8.54400374531753,
  16.278820596099706,
  12.041594578792296,
  26.076809620810597,
  0.0,
  23.345235059857504,
  24.596747752497688,
  25.96150997149434,
  32.7566787083184,
  38.01315561749642,
  17.11724276862369,
  29.017236257093817,
  37.013511046643494,
  30.675723300355934,
  12.165525060596439,
  10.0,
  32.55764119219941,
  32.202484376209235,
  38.58756276314948,
  23.706539182259394,
  22.135943621178654,
  22.80350850198276,
  27.51363298439521,
  34.0,
  14.422205101855956,
  22.0,
  18.110770276274835,
  20.615528128088304,
  12.806248474865697,
  42.04759208325728,
  21.95449840010015,
  8.602325267042627,
  18.027756377319946,
  27.202941017470888,
  31.89043743820395,
  33.015148038438355,
  34.92849839314596,
  26.1725046566048,
  16.278820596099706,
  27.018512172212592,
  26.90724809414742,
  22.67156809750927,
  18.439088914585774,
  17.46424919657298,
  19.697715603592208,
  25.495097567963924,
  19.849433241279208,
  21.37755832643195,
  28.42534080710379,
  6.4031242374328485,
  21.37755832643195,
  31.827660925679098,
  34.713109915419565,
  42.720018726587654,
  28.460498941515414,
  32.7566787083184,
  26.92582403567252,
  47.634021455258214,
  43.139309220245984,
  28.160255680657446,
  14.317821063276353,
  13.601470508735444,
  34.132096331752024,
  39.81205847478876,
  44.94441010848846,
  14.422205101855956,
  5.0990195135927845,
  34.48187929913333,
  42.01190307520001,
  38.948684188300895,
  26.076809620810597,
  11.661903789690601,
  15.652475842498529,
  7.615773105863909],
 [14.142135623730951,
  39.59797974644666,
  27.784887978899608,
  29.427877939124322,
  15.811388300841896,
  30.4138126514911,
  23.345235059857504,
  0.0,
  7.0710678118654755,
  25.632011235952593,
  18.867962264113206,
  15.811388300841896,
  15.033296378372908,
  19.209372712298546,
  13.892443989449804,
  28.635642126552707,
  32.202484376209235,
  18.027756377319946,
  41.43669871020132,
  12.165525060596439,
  39.395431207184416,
  37.48332962798263,
  44.10215414239989,
  44.77722635447622,
  43.56604182158393,
  35.84689665786984,
  9.848857801796104,
  16.76305461424021,
  37.16180835212409,
  24.08318915758459,
  25.0,
  32.202484376209235,
  28.160255680657446,
  31.89043743820395,
  10.0,
  5.0,
  40.19950248448356,
  36.40054944640259,
  16.278820596099706,
  22.360679774997898,
  20.0,
  50.32891812864648,
  50.20956084253277,
  46.010868281309364,
  29.068883707497267,
  20.0,
  8.06225774829855,
  35.0,
  29.068883707497267,
  39.84971769034255,
  35.510561809129406,
  23.706539182259394,
  12.649110640673518,
  8.602325267042627,
  19.235384061671343,
  44.721359549995796,
  50.60632371551998,
  27.459060435491963,
  14.142135623730951,
  24.413111231467404,
  48.08326112068523,
  46.17358552246078,
  36.05551275463989,
  36.05551275463989,
  57.0087712549569,
  20.0,
  23.08679276123039,
  9.433981132056603,
  20.808652046684813,
  46.0,
  44.94441010848846,
  45.89117562233506,
  18.027756377319946,
  34.713109915419565,
  31.622776601683793,
  16.401219466856727],
 [15.811388300841896,
  40.22437072223753,
  26.870057685088806,
  34.0,
  14.142135623730951,
  25.0,
  24.596747752497688,
  7.0710678118654755,
  0.0,
  32.202484376209235,
  25.80697580112788,
  20.0,
  20.8806130178211,
  12.206555615733702,
  13.892443989449804,
  22.135943621178654,
  35.17101079013795,
  22.02271554554524,
  47.29693436154187,
  7.615773105863909,
  32.89376840679705,
  33.83784863137726,
  43.18564576337837,
  47.16990566028302,
  48.041648597857254,
  42.485291572496,
  14.035668847618199,
  11.0,
  35.22782990761707,
  19.235384061671343,
  22.47220505424423,
  39.20459156782532,
  33.95585369269929,
  33.12099032335839,
  7.0710678118654755,
  5.0,
  35.014282800023196,
  30.4138126514911,
  22.47220505424423,
  29.154759474226502,
  25.495097567963924,
  51.31276644267,
  50.60632371551998,
  46.87216658103186,
  34.132096331752024,
  15.811388300841896,
  5.0,
  30.4138126514911,
  25.0,
  43.56604182158393,
  41.48493702538308,
  26.68332812825267,
  7.0710678118654755,
  12.0,
  12.649110640673518,
  51.478150704935004,
  52.92447448959697,
  20.591260281974,
  21.213203435596427,
  24.20743687382041,
  41.97618372363071,
  43.382023926967726,
  35.35533905932738,
  38.07886552931954,
  56.568542494923804,
  25.495097567963924,
  27.073972741361768,
  10.44030650891055,
  20.808652046684813,
  41.30375285612676,
  38.600518131237564,
  40.19950248448356,
  25.0,
  35.0,
  29.154759474226502,
  17.0],
 [23.600847442411894,
  37.21558813185679,
  34.48187929913333,
  14.866068747318506,
  30.610455730027933,
  48.60041152089147,
  25.96150997149434,
  25.632011235952593,
  32.202484376209235,
  0.0,
  15.652475842498529,
  29.614185789921695,
  12.041594578792296,
  43.266615305567875,
  36.05551275463989,
  50.20956084253277,
  23.53720459187964,
  16.55294535724685,
  17.26267650163207,
  37.64306044943742,
  60.207972893961475,
  49.39635614091387,
  46.0,
  34.438350715445125,
  24.839484696748443,
  10.295630140987,
  19.849433241279208,
  38.28837943815329,
  43.93176527297759,
  42.01190307520001,
  37.12142238654117,
  17.72004514666935,
  7.211102550927978,
  30.0,
  30.610455730027933,
  30.364452901377952,
  56.43580423808985,
  55.60575509783138,
  22.135943621178654,
  4.123105625617661,
  9.848857801796104,
  44.27188724235731,
  46.32493928760188,
  41.23105625617661,
  12.083045973594572,
  37.64306044943742,
  30.23243291566195,
  50.11985634456667,
  43.9089968002003,
  25.238858928247925,
  11.40175425099138,
  20.223748416156685,
  35.0,
  29.068883707497267,
  44.77722635447622,
  19.4164878389476,
  39.44616584663204,
  50.44799302251776,
  14.035668847618199,
  44.28317965096906,
  66.85057965343307,
  53.9351462406472,
  39.20459156782532,
  29.614185789921695,
  55.10898293381942,
  26.40075756488817,
  33.61547262794322,
  25.495097567963924,
  28.635642126552707,
  60.00833275470999,
  64.81512169239521,
  63.28506932918696,
  9.055385138137417,
  35.014282800023196,
  41.43669871020132,
  26.076809620810597],
 [26.0,
  47.53945729601885,
  40.19950248448356,
  28.319604517012593,
  31.400636936215164,
  48.38388161361178,
  32.7566787083184,
  18.867962264113206,
  25.80697580112788,
  15.652475842498529,
  0.0,
  15.033296378372908,
  15.811388300841896,
  38.01315561749642,
  23.769728648009426,
  47.41307836451879,
  35.510561809129406,
  23.08679276123039,
  32.57299494980466,
  28.42534080710379,
  58.137767414994535,
  53.150729063673246,
  54.817880294662984,
  47.80167361086848,
  40.22437072223753,
  23.08679276123039,
  20.024984394500787,
  35.34119409414458,
  50.24937810560445,
  41.7612260356422,
  40.11234224026316,
  13.45362404707371,
  22.20360331117452,
  39.56008088970496,
  27.85677655436824,
  21.93171219946131,
  57.723478758647246,
  54.78138369920935,
  7.0,
  11.661903789690601,
  18.867962264113206,
  56.43580423808985,
  57.697486947006624,
  52.773099207835045,
  26.1725046566048,
  37.36308338453881,
  26.40075756488817,
  51.97114584074513,
  45.70557952810576,
  39.59797974644666,
  26.92582403567252,
  29.154759474226502,
  31.304951684997057,
  17.4928556845359,
  36.796738985948195,
  30.265491900843113,
  53.2634959423431,
  46.32493928760188,
  6.0,
  30.066592756745816,
  66.48308055437865,
  60.13318551349163,
  47.07440918375928,
  41.182520563948,
  65.92419889539804,
  10.770329614269007,
  18.027756377319946,
  24.515301344262525,
  33.06055050963308,
  62.80127387243033,
  63.56099432828282,
  63.89053137985315,
  7.810249675906654,
  43.965895873961216,
  46.0,
  29.0],
 [29.154759474226502,
  54.20332093147061,
  43.382023926967726,
  39.44616584663204,
  31.622776601683793,
  45.0,
  38.01315561749642,
  15.811388300841896,
  20.0,
  29.614185789921695,
  15.033296378372908,
  0.0,
  24.413111231467404,
  30.805843601498726,
  10.63014581273465,
  41.593268686170845,
  44.68780594300866,
  30.4138126514911,
  46.87216658103186,
  18.384776310850235,
  52.172789842982326,
  53.150729063673246,
  59.53990258641679,
  57.48912940721924,
  52.80151512977634,
  38.01315561749642,
  23.600847442411894,
  31.0,
  52.92447448959697,
  39.11521443121589,
  40.80441152620633,
  26.40075756488817,
  35.11409973215888,
  46.22769732530488,
  25.495097567963924,
  15.0,
  55.00909015790027,
  50.24937810560445,
  8.06225774829855,
  25.495097567963924,
  29.154759474226502,
  64.4437739428721,
  64.81512169239521,
  60.30754513325841,
  38.01315561749642,
  35.35533905932738,
  23.345235059857504,
  50.24937810560445,
  44.553338819890925,
  50.774009099144415,
  41.0,
  36.76955262170047,
  27.018512172212592,
  8.0,
  26.832815729997478,
  45.27692569068709,
  63.25345840347388,
  39.293765408777,
  15.811388300841896,
  15.033296378372908,
  61.66036003787198,
  61.98386886924694,
  51.478150704935004,
  49.49747468305833,
  72.11102550927978,
  7.0710678118654755,
  7.280109889280518,
  25.079872407968907,
  36.235341863986875,
  61.204574992397426,
  58.05170109479997,
  60.13318551349163,
  20.615528128088304,
  49.64876634922564,
  47.43416490252569,
  31.76476034853718],
 [12.083045973594572,
  31.78049716414141,
  25.019992006393608,
  15.231546211727817,
  18.867962264113206,
  36.89173349139343,
  17.11724276862369,
  15.033296378372908,
  20.8806130178211,
  12.041594578792296,
  15.811388300841896,
  24.413111231467404,
  0.0,
  31.38470965295043,
  27.65863337187866,
  38.18376618407357,
  20.518284528683193,
  7.280109889280518,
  26.476404589747453,
  27.16615541441225,
  48.27007354458868,
  39.05124837953327,
  39.05124837953327,
  33.24154027718932,
  29.120439557122072,
  22.20360331117452,
  7.810249675906654,
  26.248809496813376,
  35.0,
  30.23243291566195,
  26.248809496813376,
  25.317977802344327,
  13.152946437965905,
  23.769728648009426,
  18.601075237738275,
  20.024984394500787,
  45.18849411078001,
  43.829214001622255,
  19.209372712298546,
  10.295630140987,
  5.0990195135927845,
  41.048751503547585,
  42.05948168962618,
  37.21558813185679,
  14.317821063276353,
  25.80697580112788,
  18.35755975068582,
  39.0,
  32.69556544854363,
  26.419689627245813,
  20.615528128088304,
  13.416407864998739,
  23.021728866442675,
  20.8806130178211,
  33.52610922848042,
  31.400636936215164,
  38.948684188300895,
  38.41874542459709,
  10.295630140987,
  37.33630940518894,
  55.226805085936306,
  45.0111097397076,
  31.400636936215164,
  25.80697580112788,
  50.15974481593781,
  24.20743687382041,
  30.4138126514911,
  13.45362404707371,
  18.24828759089466,
  49.33558553417604,
  53.009433122794285,
  51.884487084291386,
  9.0,
  28.160255680657446,
  31.400636936215164,
  14.866068747318506],
 [22.561028345356956,
  42.05948168962618,
  27.51363298439521,
  42.20189569201838,
  17.0,
  16.55294535724685,
  29.017236257093817,
  19.209372712298546,
  12.206555615733702,
  43.266615305567875,
  38.01315561749642,
  30.805843601498726,
  31.38470965295043,
  0.0,
  22.0,
  11.0,
  41.012193308819754,
  30.364452901377952,
  57.0087712549569,
  13.0,
  21.37755832643195,
  28.42534080710379,
  42.190046219457976,
  51.478150704935004,
  55.57877292636101,
  53.53503525729669,
  23.706539182259394,
  7.0710678118654755,
  33.015148038438355,
  13.45362404707371,
  21.02379604162864,
  51.40038910358559,
  43.86342439892262,
  36.49657518178932,
  13.0,
  16.55294535724685,
  26.248809496813376,
  20.09975124224178,
  34.438350715445125,
  40.607881008493905,
  35.34119409414458,
  53.14132102234569,
  51.478150704935004,
  48.703182647543684,
  43.01162633521314,
  13.0,
  13.038404810405298,
  23.323807579381203,
  19.79898987322333,
  50.08991914547278,
  51.478150704935004,
  33.24154027718932,
  8.54400374531753,
  23.08679276123039,
  7.810249675906654,
  62.68173577685928,
  56.88585061331157,
  8.54400374531753,
  33.301651610693426,
  30.083217912982647,
  31.064449134018133,
  38.948684188300895,
  35.34119409414458,
  42.2965719651132,
  55.758407437802596,
  37.0,
  37.33630940518894,
  18.384776310850235,
  24.08318915758459,
  33.24154027718932,
  27.294688127912362,
  30.14962686336267,
  36.796738985948195,
  36.68787265568828,
  26.627053911388696,
  22.090722034374522],
 [27.80287754891569,
  53.150729063673246,
  40.45985664828782,
  42.720018726587654,
  27.80287754891569,
  37.656340767525464,
  37.013511046643494,
  13.892443989449804,
  13.892443989449804,
  36.05551275463989,
  23.769728648009426,
  10.63014581273465,
  27.65863337187866,
  22.0,
  0.0,
  33.0,
  46.06517122512408,
  31.78049716414141,
  53.009433122794285,
  9.0,
  43.18564576337837,
  47.53945729601885,
  56.85068161420758,
  58.66856057549052,
  56.753854494650845,
  45.5411901469428,
  23.706539182259394,
  24.041630560342615,
  49.09175083453431,
  32.57299494980466,
  36.359317925395686,
  36.359317925395686,
  40.19950248448356,
  45.60701700396552,
  20.808652046684813,
  9.899494936611665,
  47.67598976424087,
  42.04759208325728,
  17.4928556845359,
  32.14031735997639,
  32.7566787083184,
  64.03124237432849,
  63.702433234531945,
  59.665735560705194,
  41.97618372363071,
  29.546573405388315,
  18.601075237738275,
  43.68065933568311,
  38.62641583165593,
  53.48831648126533,
  47.01063709417264,
  37.589892258425,
  20.615528128088304,
  7.0,
  16.76305461424021,
  53.600373133029585,
  64.4980619863884,
  30.14962686336267,
  22.20360331117452,
  10.63014581273465,
  53.03772242470448,
  57.245087125446844,
  48.91829923454004,
  49.92995093127971,
  70.09279563550022,
  17.69180601295413,
  15.811388300841896,
  22.67156809750927,
  33.94112549695428,
  54.3415126767741,
  49.16299421312742,
  52.08646657242167,
  27.16615541441225,
  48.104053883222775,
  43.04648650006177,
  29.732137494637012],
 [27.202941017470888,
  40.049968789001575,
  26.076809620810597,
  46.06517122512408,
  20.248456731316587,
  8.06225774829855,
  30.675723300355934,
  28.635642126552707,
  22.135943621178654,
  50.20956084253277,
  47.41307836451879,
  41.593268686170845,
  38.18376618407357,
  11.0,
  33.0,
  0.0,
  42.720018726587654,
  35.22782990761707,
  61.91122676865643,
  24.0,
  10.770329614269007,
  21.095023109728988,
  37.64306044943742,
  51.15662224971465,
  58.18934610390462,
  60.207972893961475,
  30.4138126514911,
  12.206555615733702,
  28.160255680657446,
  10.198039027185569,
  19.1049731745428,
  60.53924347066124,
  49.4064773081425,
  36.345563690772494,
  20.0,
  26.92582403567252,
  16.1245154965971,
  9.219544457292887,
  44.598206241955516,
  48.16637831516918,
  41.23105625617661,
  50.48762224545735,
  48.05205510693585,
  46.2709412050371,
  47.50789408087881,
  13.416407864998739,
  21.095023109728988,
  15.0,
  14.317821063276353,
  51.92301994298868,
  56.859475903318,
  36.24913792078372,
  16.1245154965971,
  33.734255586866,
  17.72004514666935,
  69.42621983083913,
  56.00892785976178,
  4.242640687119285,
  42.190046219457976,
  40.792156108742276,
  20.09975124224178,
  31.622776601683793,
  32.2490309931942,
  42.42640687119285,
  50.695167422546305,
  47.53945729601885,
  48.25971404805462,
  24.758836806279895,
  25.632011235952593,
  23.323807579381203,
  16.492422502470642,
  19.235384061671343,
  45.0,
  35.0,
  22.360679774997898,
  25.553864678361276],
 [19.4164878389476,
  14.035668847618199,
  18.027756377319946,
  9.0,
  24.020824298928627,
  37.57658845611187,
  12.165525060596439,
  32.202484376209235,
  35.17101079013795,
  23.53720459187964,
  35.510561809129406,
  44.68780594300866,
  20.518284528683193,
  41.012193308819754,
  46.06517122512408,
  42.720018726587654,
  0.0,
  14.422205101855956,
  23.323807579381203,
  42.720018726587654,
  50.08991914547278,
  32.526911934581186,
  23.53720459187964,
  12.806248474865697,
  15.652475842498529,
  28.284271247461902,
  22.360679774997898,
  34.058772731852805,
  24.413111231467404,
  32.57299494980466,
  24.166091947189145,
  41.23105625617661,
  17.029386365926403,
  8.602325267042627,
  29.274562336608895,
  36.76955262170047,
  42.01190307520001,
  44.294469180700204,
  39.698866482558415,
  25.632011235952593,
  16.64331697709324,
  21.02379604162864,
  22.80350850198276,
  17.72004514666935,
  12.0,
  29.614185789921695,
  30.59411708155671,
  35.805027579936315,
  30.886890422961002,
  9.219544457292887,
  20.8806130178211,
  8.54400374531753,
  33.06055050963308,
  39.96248240537617,
  46.2709412050371,
  35.73513677041127,
  18.601075237738275,
  44.91102314577124,
  30.610455730027933,
  56.43580423808985,
  53.600373133029585,
  33.54101966249684,
  18.35755975068582,
  6.082762530298219,
  31.575306807693888,
  44.68780594300866,
  50.91168824543142,
  24.73863375370596,
  17.26267650163207,
  43.18564576337837,
  53.0,
  49.040799340956916,
  27.784887978899608,
  13.416407864998739,
  24.839484696748443,
  19.026297590440446],
 [8.06225774829855,
  24.596747752497688,
  18.24828759089466,
  12.041594578792296,
  15.0,
  32.55764119219941,
  10.0,
  18.027756377319946,
  22.02271554554524,
  16.55294535724685,
  23.08679276123039,
  30.4138126514911,
  7.280109889280518,
  30.364452901377952,
  31.78049716414141,
  35.22782990761707,
  14.422205101855956,
  0.0,
  26.832815729997478,
  29.274562336608895,
  44.598206241955516,
  32.89376840679705,
  31.78049716414141,
  27.202941017470888,
  26.019223662515376,
  25.612496949731394,
  8.246211251235321,
  24.166091947189145,
  28.071337695236398,
  26.1725046566048,
  20.591260281974,
  32.0624390837628,
  14.212670403551895,
  16.55294535724685,
  17.46424919657298,
  22.80350850198276,
  39.96248240537617,
  39.6232255123179,
  26.076809620810597,
  16.278820596099706,
  6.708203932499369,
  34.20526275297414,
  34.9857113690718,
  30.23243291566195,
  12.649110640673518,
  22.02271554554524,
  18.110770276274835,
  33.61547262794322,
  27.459060435491963,
  21.840329667841555,
  21.633307652783937,
  6.4031242374328485,
  21.840329667841555,
  25.553864678361276,
  34.132096331752024,
  34.713109915419565,
  33.015148038438355,
  36.235341863986875,
  17.46424919657298,
  42.05948168962618,
  50.60632371551998,
  38.118237105091836,
  24.186773244895647,
  19.1049731745428,
  42.95346318982906,
  31.064449134018133,
  36.87817782917155,
  12.165525060596439,
  12.083045973594572,
  43.46262762420146,
  48.83646178829912,
  46.87216658103186,
  16.1245154965971,
  20.8806130178211,
  25.0,
  9.899494936611665],
 [34.88552708502482,
  36.40054944640259,
  40.36087214122113,
  16.278820596099706,
  41.677331968349414,
  58.412327466040935,
  32.55764119219941,
  41.43669871020132,
  47.29693436154187,
  17.26267650163207,
  32.57299494980466,
  46.87216658103186,
  26.476404589747453,
  57.0087712549569,
  53.009433122794285,
  61.91122676865643,
  23.323807579381203,
  26.832815729997478,
  0.0,
  53.600373133029585,
  70.80254232723568,
  55.44366510251645,
  46.238512086787566,
  28.071337695236398,
  13.152946437965905,
  11.313708498984761,
  33.52610922848042,
  50.99019513592785,
  47.70744176750625,
  52.43090691567332,
  45.34313619501854,
  29.732137494637012,
  13.341664064126334,
  31.906112267087632,
  44.01136216933077,
  46.389654018972806,
  64.35060217278468,
  65.43699259593154,
  39.293765408777,
  21.37755832643195,
  21.840329667841555,
  39.11521443121589,
  42.42640687119285,
  37.656340767525464,
  14.422205101855956,
  48.54894437575342,
  44.27188724235731,
  57.982756057296896,
  52.40229002629561,
  18.24828759089466,
  6.0,
  26.248809496813376,
  48.46648326421054,
  46.010868281309364,
  59.90826320300064,
  14.035668847618199,
  31.144823004794873,
  63.06346010171025,
  31.25699921617557,
  61.5223536610881,
  75.69015788066504,
  56.824290580701486,
  41.677331968349414,
  28.231188426986208,
  51.35172830587107,
  43.32435804486894,
  50.59644256269407,
  38.62641583165593,
  37.013511046643494,
  66.21933252457322,
  74.51845409024533,
  71.42128534267638,
  26.30589287593181,
  36.71511950137164,
  47.50789408087881,
  36.359317925395686],
 [23.40939982143925,
  47.70744176750625,
  34.058772731852805,
  41.10960958218893,
  21.400934559032695,
  28.861739379323623,
  32.202484376209235,
  12.165525060596439,
  7.615773105863909,
  37.64306044943742,
  28.42534080710379,
  18.384776310850235,
  27.16615541441225,
  13.0,
  9.0,
  24.0,
  42.720018726587654,
  29.274562336608895,
  53.600373133029585,
  0.0,
  34.23448553724738,
  39.35733730830886,
  50.20956084253277,
  54.78138369920935,
  55.226805085936306,
  47.75981574503821,
  21.095023109728988,
  15.652475842498529,
  41.86884283091664,
  24.166091947189145,
  29.068883707497267,
  41.773197148410844,
  40.311288741492746,
  40.70626487409524,
  14.422205101855956,
  7.280109889280518,
  38.8329756778952,
  33.06055050963308,
  23.430749027719962,
  34.17601498127012,
  32.0624390837628,
  58.83026432033091,
  58.008620049092706,
  54.378304497290095,
  41.0,
  21.633307652783937,
  12.529964086141668,
  35.11409973215888,
  30.4138126514911,
  50.99019513592785,
  47.67598976424087,
  34.20526275297414,
  12.806248474865697,
  11.40175425099138,
  8.602325267042627,
  56.462376853972415,
  60.53924347066124,
  21.213203435596427,
  25.059928172283335,
  17.88854381999832,
  44.04543109109048,
  49.39635614091387,
  42.5205832509386,
  45.69463863518345,
  63.702433234531945,
  25.059928172283335,
  24.515301344262525,
  18.027756377319946,
  28.30194339616981,
  45.60701700396552,
  40.19950248448356,
  43.104524124504614,
  29.546573405388315,
  42.43819034784589,
  35.608987629529715,
  24.596747752497688],
 [36.76955262170047,
  44.721359549995796,
  32.31098884280702,
  54.644304369257,
  29.698484809834994,
  12.529964086141668,
  38.58756276314948,
  39.395431207184416,
  32.89376840679705,
  60.207972893961475,
  58.137767414994535,
  52.172789842982326,
  48.27007354458868,
  21.37755832643195,
  43.18564576337837,
  10.770329614269007,
  50.08991914547278,
  44.598206241955516,
  70.80254232723568,
  34.23448553724738,
  0.0,
  22.02271554554524,
  40.01249804748511,
  56.753854494650845,
  65.7419196555744,
  70.00714249274856,
  40.607881008493905,
  22.825424421026653,
  31.064449134018133,
  18.439088914585774,
  25.942243542145693,
  71.19691004531025,
  58.7962583843564,
  42.720018726587654,
  30.528675044947494,
  37.64306044943742,
  12.649110640673518,
  6.082762530298219,
  55.362442142665635,
  58.412327466040935,
  50.91168824543142,
  54.08326913195984,
  51.0,
  50.24937810560445,
  56.4003546088143,
  22.627416997969522,
  31.76476034853718,
  16.0312195418814,
  19.313207915827967,
  59.09314681077663,
  66.06814663663572,
  44.654227123532216,
  26.832815729997478,
  44.384682042344295,
  27.018512172212592,
  79.19595949289332,
  61.0,
  13.038404810405298,
  52.839379254491625,
  50.15974481593781,
  10.198039027185569,
  31.04834939252005,
  36.49657518178932,
  48.703182647543684,
  51.78802950489621,
  58.240879114244144,
  58.69412236331676,
  35.0,
  33.83784863137726,
  18.867962264113206,
  6.0,
  11.40175425099138,
  55.47071299343465,
  40.26164427839479,
  26.68332812825267,
  34.713109915419565],
 [27.294688127912362,
  23.769728648009426,
  15.264337522473747,
  39.408120990476064,
  22.02271554554524,
  13.038404810405298,
  23.706539182259394,
  37.48332962798263,
  33.83784863137726,
  49.39635614091387,
  53.150729063673246,
  53.150729063673246,
  39.05124837953327,
  28.42534080710379,
  47.53945729601885,
  21.095023109728988,
  32.526911934581186,
  32.89376840679705,
  55.44366510251645,
  39.35733730830886,
  22.02271554554524,
  0.0,
  18.0,
  36.24913792078372,
  47.634021455258214,
  57.706152185014034,
  33.13608305156178,
  23.706539182259394,
  9.486832980505138,
  15.264337522473747,
  13.038404810405298,
  64.32728814430156,
  45.65084884205331,
  24.08318915758459,
  27.65863337187866,
  38.600518131237564,
  10.44030650891055,
  16.1245154965971,
  53.08483775994799,
  49.040799340956916,
  39.56008088970496,
  32.2490309931942,
  29.017236257093817,
  28.635642126552707,
  41.88078318274385,
  18.027756377319946,
  29.832867780352597,
  6.324555320336759,
  8.94427190999916,
  40.607881008493905,
  51.86520991955976,
  30.01666203960727,
  26.92582403567252,
  45.35416188179427,
  36.235341863986875,
  66.37017402418047,
  39.84971769034255,
  25.238858928247925,
  47.16990566028302,
  57.245087125446844,
  21.93171219946131,
  10.63014581273465,
  15.652475842498529,
  29.410882339705484,
  30.083217912982647,
  57.3149195236284,
  60.41522986797286,
  29.017236257093817,
  20.8806130178211,
  10.816653826391969,
  22.561028345356956,
  17.0,
  47.853944456021594,
  20.248456731316587,
  8.06225774829855,
  24.331050121192877],
 [30.4138126514911,
  9.848857801796104,
  16.401219466856727,
  32.38826948140329,
  29.068883707497267,
  29.832867780352597,
  22.135943621178654,
  44.10215414239989,
  43.18564576337837,
  46.0,
  54.817880294662984,
  59.53990258641679,
  39.05124837953327,
  42.190046219457976,
  56.85068161420758,
  37.64306044943742,
  23.53720459187964,
  31.78049716414141,
  46.238512086787566,
  50.20956084253277,
  40.01249804748511,
  18.0,
  0.0,
  21.213203435596427,
  35.510561809129406,
  51.78802950489621,
  36.24913792078372,
  35.805027579936315,
  9.486832980505138,
  29.068883707497267,
  21.400934559032695,
  63.198101237299845,
  40.19950248448356,
  16.0,
  36.124783736376884,
  47.01063709417264,
  28.160255680657446,
  34.058772731852805,
  57.0087712549569,
  47.16990566028302,
  37.21558813185679,
  14.560219778561036,
  11.045361017187261,
  11.661903789690601,
  35.35533905932738,
  29.410882339705484,
  38.28837943815329,
  24.08318915758459,
  23.40939982143925,
  28.653097563788805,
  44.384682042344295,
  26.1725046566048,
  37.53664875824692,
  52.69724850502159,
  49.64876634922564,
  59.20304046246274,
  23.323807579381203,
  41.43669871020132,
  49.040799340956916,
  67.35725647619564,
  39.05124837953327,
  12.206555615733702,
  8.06225774829855,
  18.027756377319946,
  13.601470508735444,
  61.84658438426491,
  66.64833081180653,
  34.66987164671943,
  23.323807579381203,
  25.632011235952593,
  40.311288741492746,
  33.95585369269929,
  47.853944456021594,
  11.045361017187261,
  15.652475842498529,
  27.784887978899608],
 [31.38470965295043,
  12.529964086141668,
  25.079872407968907,
  19.72308292331602,
  34.713109915419565,
  44.721359549995796,
  22.80350850198276,
  44.77722635447622,
  47.16990566028302,
  34.438350715445125,
  47.80167361086848,
  57.48912940721924,
  33.24154027718932,
  51.478150704935004,
  58.66856057549052,
  51.15662224971465,
  12.806248474865697,
  27.202941017470888,
  28.071337695236398,
  54.78138369920935,
  56.753854494650845,
  36.24913792078372,
  21.213203435596427,
  0.0,
  15.524174696260024,
  36.49657518178932,
  34.9857113690718,
  44.40720662234904,
  26.832815729997478,
  41.048751503547585,
  32.0624390837628,
  52.0,
  27.313000567495326,
  15.033296378372908,
  40.80441152620633,
  49.193495504995376,
  46.61544808322666,
  50.695167422546305,
  52.3450093132096,
  37.21558813185679,
  29.068883707497267,
  11.045361017187261,
  14.560219778561036,
  10.295630140987,
  22.360679774997898,
  39.05124837953327,
  42.37924020083418,
  41.10960958218893,
  37.656340767525464,
  10.04987562112089,
  28.284271247461902,
  21.095023109728988,
  44.14748010928823,
  52.69724850502159,
  57.48912940721924,
  42.01190307520001,
  5.830951894845301,
  54.00925846556311,
  43.18564576337837,
  69.1158447825099,
  58.180752831155424,
  33.301651610693426,
  20.615528128088304,
  9.219544457292887,
  23.345235059857504,
  57.3149195236284,
  63.655321851358195,
  36.76955262170047,
  27.459060435491963,
  45.79301256742124,
  58.52349955359813,
  53.225933528685054,
  40.0,
  16.492422502470642,
  30.083217912982647,
  30.364452901377952],
 [33.13608305156178,
  25.80697580112788,
  33.61547262794322,
  14.142135623730951,
  38.8329756778952,
  53.225933528685054,
  27.51363298439521,
  43.56604182158393,
  48.041648597857254,
  24.839484696748443,
  40.22437072223753,
  52.80151512977634,
  29.120439557122072,
  55.57877292636101,
  56.753854494650845,
  58.18934610390462,
  15.652475842498529,
  26.019223662515376,
  13.152946437965905,
  55.226805085936306,
  65.7419196555744,
  47.634021455258214,
  35.510561809129406,
  15.524174696260024,
  0.0,
  23.259406699226016,
  34.132096331752024,
  48.83646178829912,
  38.897300677553446,
  48.104053883222775,
  39.81205847478876,
  40.80441152620633,
  18.027756377319946,
  23.600847442411894,
  43.104524124504614,
  48.507731342539614,
  57.42821606144492,
  59.941638282582836,
  46.14108798023731,
  28.600699292150182,
  24.041630560342615,
  26.476404589747453,
  30.083217912982647,
  25.632011235952593,
  14.866068747318506,
  44.9221548904324,
  44.01136216933077,
  51.31276644267,
  46.52956049652737,
  7.0710678118654755,
  15.264337522473747,
  22.360679774997898,
  47.265209192385896,
  50.0,
  60.03332407921454,
  27.16615541441225,
  18.027756377319946,
  60.166435825965294,
  37.12142238654117,
  66.40783086353596,
  69.06518659932803,
  47.01063709417264,
  32.526911934581186,
  18.384776310850235,
  38.8329756778952,
  50.774009099144415,
  57.8013840664737,
  38.01315561749642,
  32.57299494980466,
  58.05170109479997,
  68.60029154456998,
  64.40496875241847,
  32.7566787083184,
  27.586228448267445,
  40.22437072223753,
  33.54101966249684],
 [33.24154027718932,
  42.2965719651132,
  42.485291572496,
  19.4164878389476,
  40.311288741492746,
  58.137767414994535,
  34.0,
  35.84689665786984,
  42.485291572496,
  10.295630140987,
  23.08679276123039,
  38.01315561749642,
  22.20360331117452,
  53.53503525729669,
  45.5411901469428,
  60.207972893961475,
  28.284271247461902,
  25.612496949731394,
  11.313708498984761,
  47.75981574503821,
  70.00714249274856,
  57.706152185014034,
  51.78802950489621,
  36.49657518178932,
  23.259406699226016,
  0.0,
  30.0,
  48.41487374764082,
  51.264022471905186,
  51.66236541235796,
  46.04345773288535,
  18.439088914585774,
  12.083045973594572,
  36.138621999185304,
  40.80441152620633,
  40.496913462633174,
  65.45991139621256,
  65.19202405202648,
  30.066592756745816,
  13.601470508735444,
  19.1049731745428,
  47.265209192385896,
  50.039984012787215,
  45.0111097397076,
  16.492422502470642,
  47.38143096192854,
  40.496913462633174,
  59.07622195096772,
  53.009433122794285,
  26.476404589747453,
  8.246211251235321,
  27.730849247724095,
  45.221676218380054,
  38.63935817272331,
  55.036351623268054,
  9.219544457292887,
  40.52159917870962,
  60.60528029800704,
  23.345235059857504,
  53.0,
  76.21679604916491,
  61.032778078668514,
  45.880278987817846,
  34.132096331752024,
  59.033888572581766,
  33.54101966249684,
  40.792156108742276,
  35.608987629529715,
  37.44329045369811,
  68.47627326307996,
  74.41102068914255,
  72.42237223399962,
  18.439088914585774,
  41.23105625617661,
  49.64876634922564,
  35.4682957019364],
 [6.082762530298219,
  30.610455730027933,
  20.615528128088304,
  20.024984394500787,
  11.704699910719626,
  29.5296461204668,
  14.422205101855956,
  9.848857801796104,
  14.035668847618199,
  19.849433241279208,
  20.024984394500787,
  23.600847442411894,
  7.810249675906654,
  23.706539182259394,
  23.706539182259394,
  30.4138126514911,
  22.360679774997898,
  8.246211251235321,
  33.52610922848042,
  21.095023109728988,
  40.607881008493905,
  33.13608305156178,
  36.24913792078372,
  34.9857113690718,
  34.132096331752024,
  30.0,
  0.0,
  18.439088914585774,
  30.59411708155671,
  22.825424421026653,
  20.09975124224178,
  31.622776601683793,
  20.248456731316587,
  22.67156809750927,
  10.816653826391969,
  14.560219778561036,
  38.27531841800928,
  36.359317925395686,
  20.8806130178211,
  17.804493814764857,
  11.704699910719626,
  41.012193308819754,
  41.23105625617661,
  36.796738985948195,
  20.09975124224178,
  18.35755975068582,
  10.770329614269007,
  32.28002478313795,
  25.96150997149434,
  30.083217912982647,
  27.85677655436824,
  13.892443989449804,
  15.264337522473747,
  17.804493814764857,
  26.476404589747453,
  39.20459156782532,
  40.8166632639171,
  30.610455730027933,
  14.035668847618199,
  34.132096331752024,
  47.885279575251516,
  40.311288741492746,
  28.231188426986208,
  26.40075756488817,
  48.54894437575342,
  25.632011235952593,
  30.528675044947494,
  5.656854249492381,
  13.341664064126334,
  42.95346318982906,
  45.48626166217664,
  44.77722635447622,
  15.231546211727817,
  26.076809620810597,
  26.019223662515376,
  9.055385138137417],
 [16.15549442140351,
  35.11409973215888,
  20.615528128088304,
  35.73513677041127,
  10.04987562112089,
  14.0,
  22.0,
  16.76305461424021,
  11.0,
  38.28837943815329,
  35.34119409414458,
  31.0,
  26.248809496813376,
  7.0710678118654755,
  24.041630560342615,
  12.206555615733702,
  34.058772731852805,
  24.166091947189145,
  50.99019513592785,
  15.652475842498529,
  22.825424421026653,
  23.706539182259394,
  35.805027579936315,
  44.40720662234904,
  48.83646178829912,
  48.41487374764082,
  18.439088914585774,
  0.0,
  26.90724809414742,
  8.54400374531753,
  14.422205101855956,
  48.373546489791295,
  38.07886552931954,
  29.427877939124322,
  7.810249675906654,
  16.0,
  24.020824298928627,
  19.6468827043885,
  32.984845004941285,
  36.069377593742864,
  29.68164415931166,
  46.238512086787566,
  44.721359549995796,
  41.78516483155236,
  36.76955262170047,
  6.4031242374328485,
  8.94427190999916,
  19.6468827043885,
  14.7648230602334,
  43.18564576337837,
  45.65084884205331,
  26.476404589747453,
  4.123105625617661,
  23.0,
  13.892443989449804,
  57.62811813689564,
  49.8196748283246,
  12.206555615733702,
  30.01666203960727,
  33.54101966249684,
  31.32091952673165,
  33.83784863137726,
  28.653097563788805,
  35.22782990761707,
  49.4064773081425,
  36.345563690772494,
  38.05259518088089,
  12.806248474865697,
  17.029386365926403,
  30.4138126514911,
  28.231188426986208,
  29.274562336608895,
  32.802438933713454,
  29.732137494637012,
  20.518284528683193,
  15.297058540778355],
 [24.515301344262525,
  14.317821063276353,
  10.04987562112089,
  32.202484376209235,
  21.470910553583888,
  20.396078054371138,
  18.110770276274835,
  37.16180835212409,
  35.22782990761707,
  43.93176527297759,
  50.24937810560445,
  52.92447448959697,
  35.0,
  33.015148038438355,
  49.09175083453431,
  28.160255680657446,
  24.413111231467404,
  28.071337695236398,
  47.70744176750625,
  41.86884283091664,
  31.064449134018133,
  9.486832980505138,
  9.486832980505138,
  26.832815729997478,
  38.897300677553446,
  51.264022471905186,
  30.59411708155671,
  26.90724809414742,
  0.0,
  19.72308292331602,
  12.806248474865697,
  60.13318551349163,
  39.21734310225516,
  15.811388300841896,
  28.30194339616981,
  39.44616584663204,
  19.924858845171276,
  25.019992006393608,
  51.419840528729765,
  44.28317965096906,
  34.36568055487916,
  23.021728866442675,
  20.09975124224178,
  19.235384061671343,
  34.92849839314596,
  20.518284528683193,
  30.528675044947494,
  15.033296378372908,
  13.92838827718412,
  31.827660925679098,
  44.721359549995796,
  23.769728648009426,
  29.068883707497267,
  45.617978911828175,
  40.607881008493905,
  59.50630218724736,
  30.364452901377952,
  31.953090617340916,
  44.28317965096906,
  59.405386961116584,
  31.38470965295043,
  10.04987562112089,
  6.4031242374328485,
  20.518284528683193,
  22.825424421026653,
  56.04462507680822,
  60.166435825965294,
  27.85677655436824,
  17.26267650163207,
  19.209372712298546,
  32.01562118716424,
  26.40075756488817,
  44.0,
  11.313708498984761,
  6.4031242374328485,
  21.587033144922902],
 [18.439088914585774,
  30.265491900843113,
  16.0,
  36.359317925395686,
  11.40175425099138,
  6.708203932499369,
  20.615528128088304,
  24.08318915758459,
  19.235384061671343,
  42.01190307520001,
  41.7612260356422,
  39.11521443121589,
  30.23243291566195,
  13.45362404707371,
  32.57299494980466,
  10.198039027185569,
  32.57299494980466,
  26.1725046566048,
  52.43090691567332,
  24.166091947189145,
  18.439088914585774,
  15.264337522473747,
  29.068883707497267,
  41.048751503547585,
  48.104053883222775,
  51.66236541235796,
  22.825424421026653,
  8.54400374531753,
  19.72308292331602,
  0.0,
  9.0,
  54.230987451824994,
  40.36087214122113,
  26.1725046566048,
  14.142135623730951,
  24.186773244895647,
  16.1245154965971,
  13.601470508735444,
  40.311288741492746,
  40.496913462633174,
  32.55764119219941,
  41.0,
  38.897300677553446,
  36.6742416417845,
  38.01315561749642,
  4.47213595499958,
  16.0312195418814,
  11.180339887498949,
  6.4031242374328485,
  41.7612260356422,
  47.634021455258214,
  26.419689627245813,
  12.165525060596439,
  31.144823004794873,
  21.213203435596427,
  60.8276253029822,
  46.010868281309364,
  13.038404810405298,
  36.05551275463989,
  42.04759208325728,
  25.059928172283335,
  25.612496949731394,
  22.80350850198276,
  32.2490309931942,
  42.5440947723653,
  44.04543109109048,
  46.2709412050371,
  17.46424919657298,
  15.652475842498529,
  22.090722034374522,
  22.80350850198276,
  22.135943621178654,
  38.01315561749642,
  25.079872407968907,
  13.416407864998739,
  16.278820596099706],
 [14.317821063276353,
  21.37755832643195,
  7.0,
  29.068883707497267,
  9.219544457292887,
  13.416407864998739,
  12.806248474865697,
  25.0,
  22.47220505424423,
  37.12142238654117,
  40.11234224026316,
  40.80441152620633,
  26.248809496813376,
  21.02379604162864,
  36.359317925395686,
  19.1049731745428,
  24.166091947189145,
  20.591260281974,
  45.34313619501854,
  29.068883707497267,
  25.942243542145693,
  13.038404810405298,
  21.400934559032695,
  32.0624390837628,
  39.81205847478876,
  46.04345773288535,
  20.09975124224178,
  14.422205101855956,
  12.806248474865697,
  9.0,
  0.0,
  51.419840528729765,
  34.20526275297414,
  17.26267650163207,
  15.652475842498529,
  26.832815729997478,
  19.4164878389476,
  20.248456731316587,
  40.19950248448356,
  36.40054944640259,
  27.294688127912362,
  32.28002478313795,
  30.463092423455635,
  27.892651361962706,
  31.11269837220809,
  8.06225774829855,
  17.88854381999832,
  13.038404810405298,
  7.0710678118654755,
  33.24154027718932,
  41.036569057366385,
  19.1049731745428,
  16.278820596099706,
  33.24154027718932,
  28.30194339616981,
  55.0,
  37.013511046643494,
  22.02271554554524,
  34.132096331752024,
  46.61544808322666,
  30.4138126514911,
  21.18962010041709,
  14.317821063276353,
  23.345235059857504,
  35.0,
  44.553338819890925,
  48.08326112068523,
  16.1245154965971,
  8.602325267042627,
  23.08679276123039,
  29.206163733020468,
  26.40075756488817,
  34.92849839314596,
  16.1245154965971,
  6.708203932499369,
  11.40175425099138],
 [37.107950630558946,
  54.74486277268398,
  50.24937810560445,
  32.57299494980466,
  43.32435804486894,
  60.92618484691127,
  42.04759208325728,
  32.202484376209235,
  39.20459156782532,
  17.72004514666935,
  13.45362404707371,
  26.40075756488817,
  25.317977802344327,
  51.40038910358559,
  36.359317925395686,
  60.53924347066124,
  41.23105625617661,
  32.0624390837628,
  29.732137494637012,
  41.773197148410844,
  71.19691004531025,
  64.32728814430156,
  63.198101237299845,
  52.0,
  40.80441152620633,
  18.439088914585774,
  31.622776601683793,
  48.373546489791295,
  60.13318551349163,
  54.230987451824994,
  51.419840528729765,
  0.0,
  24.698178070456937,
  47.265209192385896,
  40.70626487409524,
  35.38361202590826,
  69.8927750200262,
  67.54257916307313,
  18.867962264113206,
  16.0312195418814,
  26.019223662515376,
  61.98386886924694,
  64.03124237432849,
  58.9406481131655,
  29.732137494637012,
  49.76946855251722,
  39.44616584663204,
  63.89053137985315,
  57.56735185849702,
  42.485291572496,
  25.612496949731394,
  37.05401462729781,
  44.41846462902562,
  30.610455730027933,
  50.20956084253277,
  21.840329667841555,
  56.79788728465171,
  59.64059020499378,
  18.35755975068582,
  40.80441152620633,
  79.20227269466452,
  70.178344238091,
  56.00892785976178,
  47.29693436154187,
  72.78049189171504,
  19.924858845171276,
  26.30589287593181,
  36.76955262170047,
  43.56604182158393,
  74.46475676452586,
  76.47875521999558,
  76.32168761236873,
  16.492422502470642,
  52.15361924162119,
  56.71860364994893,
  40.024992192379],
 [22.20360331117452,
  31.0,
  30.4138126514911,
  8.06225774829855,
  29.206163733020468,
  46.66904755831214,
  21.95449840010015,
  28.160255680657446,
  33.95585369269929,
  7.211102550927978,
  22.20360331117452,
  35.11409973215888,
  13.152946437965905,
  43.86342439892262,
  40.19950248448356,
  49.4064773081425,
  17.029386365926403,
  14.212670403551895,
  13.341664064126334,
  40.311288741492746,
  58.7962583843564,
  45.65084884205331,
  40.19950248448356,
  27.313000567495326,
  18.027756377319946,
  12.083045973594572,
  20.248456731316587,
  38.07886552931954,
  39.21734310225516,
  40.36087214122113,
  34.20526275297414,
  24.698178070456937,
  0.0,
  24.331050121192877,
  30.870698080866262,
  33.13608305156178,
  53.600373133029585,
  53.74011537017761,
  28.178005607210743,
  10.63014581273465,
  8.54400374531753,
  37.36308338453881,
  39.6232255123179,
  34.52535300326414,
  5.0990195135927845,
  36.235341863986875,
  31.016124838541646,
  47.20169488482379,
  41.23105625617661,
  18.027756377319946,
  7.615773105863909,
  15.652475842498529,
  35.34119409414458,
  33.24154027718932,
  46.57252408878007,
  20.808652046684813,
  32.2490309931942,
  50.24937810560445,
  19.313207915827967,
  49.24428900898052,
  64.53681120105021,
  49.040799340956916,
  33.95585369269929,
  23.08679276123039,
  48.507731342539614,
  32.7566787083184,
  39.824615503479755,
  25.495097567963924,
  25.612496949731394,
  56.43580423808985,
  62.93647591023825,
  60.60528029800704,
  14.7648230602334,
  29.427877939124322,
  37.589892258425,
  24.08318915758459],
 [17.804493814764857,
  8.06225774829855,
  10.44030650891055,
  16.76305461424021,
  19.924858845171276,
  30.364452901377952,
  8.602325267042627,
  31.89043743820395,
  33.12099032335839,
  30.0,
  39.56008088970496,
  46.22769732530488,
  23.769728648009426,
  36.49657518178932,
  45.60701700396552,
  36.345563690772494,
  8.602325267042627,
  16.55294535724685,
  31.906112267087632,
  40.70626487409524,
  42.720018726587654,
  24.08318915758459,
  16.0,
  15.033296378372908,
  23.600847442411894,
  36.138621999185304,
  22.67156809750927,
  29.427877939124322,
  15.811388300841896,
  26.1725046566048,
  17.26267650163207,
  47.265209192385896,
  24.331050121192877,
  0.0,
  26.40075756488817,
  35.805027579936315,
  33.83784863137726,
  36.76955262170047,
  42.5440947723653,
  31.25699921617557,
  21.37755832643195,
  18.439088914585774,
  18.601075237738275,
  14.142135623730951,
  19.6468827043885,
  24.020824298928627,
  28.178005607210743,
  27.784887978899608,
  23.40939982143925,
  16.64331697709324,
  29.154759474226502,
  10.44030650891055,
  29.410882339705484,
  40.311288741492746,
  42.720018726587654,
  44.01136216933077,
  20.396078054371138,
  39.05124837953327,
  34.0147027033899,
  56.22277118748239,
  45.48626166217664,
  25.079872407968907,
  9.848857801796104,
  6.082762530298219,
  26.40075756488817,
  47.50789408087881,
  53.009433122794285,
  23.021728866442675,
  12.649110640673518,
  34.655446902326915,
  45.17742799230607,
  40.80441152620633,
  32.28002478313795,
  5.0990195135927845,
  16.64331697709324,
  16.1245154965971],
 [10.0,
  33.28663395418648,
  19.79898987322333,
  29.427877939124322,
  7.0710678118654755,
  20.615528128088304,
  18.027756377319946,
  10.0,
  7.0710678118654755,
  30.610455730027933,
  27.85677655436824,
  25.495097567963924,
  18.601075237738275,
  13.0,
  20.808652046684813,
  20.0,
  29.274562336608895,
  17.46424919657298,
  44.01136216933077,
  14.422205101855956,
  30.528675044947494,
  27.65863337187866,
  36.124783736376884,
  40.80441152620633,
  43.104524124504614,
  40.80441152620633,
  10.816653826391969,
  7.810249675906654,
  28.30194339616981,
  14.142135623730951,
  15.652475842498529,
  40.70626487409524,
  30.870698080866262,
  26.40075756488817,
  0.0,
  11.180339887498949,
  30.265491900843113,
  26.92582403567252,
  26.1725046566048,
  28.284271247461902,
  22.360679774997898,
  44.41846462902562,
  43.60045871318328,
  39.96248240537617,
  30.083217912982647,
  10.0,
  2.23606797749979,
  25.0,
  19.1049731745428,
  38.05259518088089,
  38.48376280978771,
  21.02379604162864,
  4.47213595499958,
  17.72004514666935,
  17.029386365926403,
  50.0,
  46.486557196677836,
  19.849433241279208,
  22.360679774997898,
  31.240998703626616,
  38.62641583165593,
  36.76955262170047,
  28.284271247461902,
  31.622776601683793,
  49.49747468305833,
  30.0,
  32.7566787083184,
  5.385164807134504,
  13.892443989449804,
  36.0,
  35.77708763999664,
  36.138621999185304,
  25.0,
  28.0178514522438,
  22.360679774997898,
  10.44030650891055],
 [18.027756377319946,
  43.278170016764804,
  30.610455730027933,
  34.36568055487916,
  18.027756377319946,
  30.0,
  27.202941017470888,
  5.0,
  5.0,
  30.364452901377952,
  21.93171219946131,
  15.0,
  20.024984394500787,
  16.55294535724685,
  9.899494936611665,
  26.92582403567252,
  36.76955262170047,
  22.80350850198276,
  46.389654018972806,
  7.280109889280518,
  37.64306044943742,
  38.600518131237564,
  47.01063709417264,
  49.193495504995376,
  48.507731342539614,
  40.496913462633174,
  14.560219778561036,
  16.0,
  39.44616584663204,
  24.186773244895647,
  26.832815729997478,
  35.38361202590826,
  33.13608305156178,
  35.805027579936315,
  11.180339887498949,
  0.0,
  40.01249804748511,
  35.35533905932738,
  17.88854381999832,
  26.92582403567252,
  25.0,
  54.20332093147061,
  53.81449618829484,
  49.8196748283246,
  34.058772731852805,
  20.615528128088304,
  8.94427190999916,
  35.35533905932738,
  29.832867780352597,
  44.64302857109943,
  40.44749683231337,
  28.231188426986208,
  12.041594578792296,
  7.0,
  15.0,
  49.24428900898052,
  55.00909015790027,
  25.079872407968907,
  18.027756377319946,
  20.518284528683193,
  46.87216658103186,
  47.92702786528704,
  39.05124837953327,
  40.311288741492746,
  60.207972893961475,
  20.615528128088304,
  22.090722034374522,
  12.806248474865697,
  24.041630560342615,
  46.2709412050371,
  43.41658669218482,
  45.17742799230607,
  22.360679774997898,
  38.2099463490856,
  33.54101966249684,
  19.849433241279208],
 [33.1058907144937,
  34.17601498127012,
  24.08318915758459,
  48.104053883222775,
  26.570660511172846,
  10.04987562112089,
  31.89043743820395,
  40.19950248448356,
  35.014282800023196,
  56.43580423808985,
  57.723478758647246,
  55.00909015790027,
  45.18849411078001,
  26.248809496813376,
  47.67598976424087,
  16.1245154965971,
  42.01190307520001,
  39.96248240537617,
  64.35060217278468,
  38.8329756778952,
  12.649110640673518,
  10.44030650891055,
  28.160255680657446,
  46.61544808322666,
  57.42821606144492,
  65.45991139621256,
  38.27531841800928,
  24.020824298928627,
  19.924858845171276,
  16.1245154965971,
  19.4164878389476,
  69.8927750200262,
  53.600373133029585,
  33.83784863137726,
  30.265491900843113,
  40.01249804748511,
  0.0,
  7.810249675906654,
  56.43580423808985,
  55.46169849544819,
  46.647615158762406,
  42.579337712087536,
  39.20459156782532,
  39.05124837953327,
  50.32891812864648,
  20.396078054371138,
  32.14031735997639,
  6.4031242374328485,
  12.529964086141668,
  50.47771785649585,
  60.30754513325841,
  38.28837943815329,
  28.0,
  47.01063709417264,
  33.61547262794322,
  74.4043009509531,
  50.28916384272063,
  20.248456731316587,
  51.92301994298868,
  56.32051136131489,
  11.661903789690601,
  18.439088914585774,
  26.0,
  39.44616584663204,
  39.319206502675,
  60.13318551349163,
  62.07253821135398,
  33.24154027718932,
  28.0178514522438,
  7.211102550927978,
  12.165525060596439,
  7.0710678118654755,
  53.48831648126533,
  30.4138126514911,
  17.204650534085253,
  30.4138126514911],
 [32.01562118716424,
  38.63935817272331,
  26.40075756488817,
  49.20365840057018,
  25.0,
  7.0710678118654755,
  33.015148038438355,
  36.40054944640259,
  30.4138126514911,
  55.60575509783138,
  54.78138369920935,
  50.24937810560445,
  43.829214001622255,
  20.09975124224178,
  42.04759208325728,
  9.219544457292887,
  44.294469180700204,
  39.6232255123179,
  65.43699259593154,
  33.06055050963308,
  6.082762530298219,
  16.1245154965971,
  34.058772731852805,
  50.695167422546305,
  59.941638282582836,
  65.19202405202648,
  36.359317925395686,
  19.6468827043885,
  25.019992006393608,
  13.601470508735444,
  20.248456731316587,
  67.54257916307313,
  53.74011537017761,
  36.76955262170047,
  26.92582403567252,
  35.35533905932738,
  7.810249675906654,
  0.0,
  52.630789467763066,
  54.08326913195984,
  46.09772228646444,
  48.041648597857254,
  45.0111097397076,
  44.181444068749045,
  51.088159097779204,
  18.027756377319946,
  28.460498941515414,
  10.0,
  13.416407864998739,
  53.225933528685054,
  60.876925020897694,
  39.20459156782532,
  23.769728648009426,
  42.2965719651132,
  26.92582403567252,
  74.33034373659252,
  54.91812087098393,
  13.0,
  49.24428900898052,
  50.00999900019995,
  11.704699910719626,
  25.632011235952593,
  30.4138126514911,
  42.720018726587654,
  46.09772228646444,
  55.90169943749474,
  57.07889277132134,
  30.886890422961002,
  28.42534080710379,
  14.866068747318506,
  9.219544457292887,
  10.04987562112089,
  51.478150704935004,
  34.20526275297414,
  20.615528128088304,
  29.732137494637012],
 [26.92582403567252,
  50.60632371551998,
  41.48493702538308,
  33.421549934136806,
  31.064449134018133,
  46.69047011971501,
  34.92849839314596,
  16.278820596099706,
  22.47220505424423,
  22.135943621178654,
  7.0,
  8.06225774829855,
  19.209372712298546,
  34.438350715445125,
  17.4928556845359,
  44.598206241955516,
  39.698866482558415,
  26.076809620810597,
  39.293765408777,
  23.430749027719962,
  55.362442142665635,
  53.08483775994799,
  57.0087712549569,
  52.3450093132096,
  46.14108798023731,
  30.066592756745816,
  20.8806130178211,
  32.984845004941285,
  51.419840528729765,
  40.311288741492746,
  40.19950248448356,
  18.867962264113206,
  28.178005607210743,
  42.5440947723653,
  26.1725046566048,
  17.88854381999832,
  56.43580423808985,
  52.630789467763066,
  0.0,
  18.027756377319946,
  23.345235059857504,
  60.21627686929839,
  61.05735008989499,
  56.302753041036986,
  31.622776601683793,
  36.124783736376884,
  24.331050121192877,
  51.088159097779204,
  45.0111097397076,
  44.82186966202994,
  33.52610922848042,
  32.449961479175904,
  28.861739379323623,
  12.041594578792296,
  32.01562118716424,
  37.21558813185679,
  57.982756057296896,
  42.95346318982906,
  9.219544457292887,
  23.08679276123039,
  64.28841264178172,
  61.0,
  49.040799340956916,
  45.0,
  68.8839603971781,
  5.0,
  11.661903789690601,
  24.08318915758459,
  34.20526275297414,
  62.07253821135398,
  61.032778078668514,
  62.16912416947821,
  13.416407864998739,
  46.51881339845203,
  46.52956049652737,
  29.832867780352597],
 [22.360679774997898,
  38.8329756778952,
  34.52535300326414,
  17.4928556845359,
  29.154759474226502,
  47.16990566028302,
  26.1725046566048,
  22.360679774997898,
  29.154759474226502,
  4.123105625617661,
  11.661903789690601,
  25.495097567963924,
  10.295630140987,
  40.607881008493905,
  32.14031735997639,
  48.16637831516918,
  25.632011235952593,
  16.278820596099706,
  21.37755832643195,
  34.17601498127012,
  58.412327466040935,
  49.040799340956916,
  47.16990566028302,
  37.21558813185679,
  28.600699292150182,
  13.601470508735444,
  17.804493814764857,
  36.069377593742864,
  44.28317965096906,
  40.496913462633174,
  36.40054944640259,
  16.0312195418814,
  10.63014581273465,
  31.25699921617557,
  28.284271247461902,
  26.92582403567252,
  55.46169849544819,
  54.08326913195984,
  18.027756377319946,
  0.0,
  10.0,
  46.61544808322666,
  48.38388161361178,
  43.32435804486894,
  15.0,
  36.05551275463989,
  27.65863337187866,
  49.24428900898052,
  42.95346318982906,
  28.42534080710379,
  15.524174696260024,
  21.02379604162864,
  32.55764119219941,
  25.179356624028344,
  41.593268686170845,
  22.360679774997898,
  42.43819034784589,
  48.104053883222775,
  10.0,
  40.19950248448356,
  65.5133574166368,
  54.3323108288245,
  40.0,
  31.622776601683793,
  57.0087712549569,
  22.360679774997898,
  29.546573405388315,
  23.430749027719962,
  28.160255680657446,
  59.464274989274024,
  63.245553203367585,
  62.177166226839255,
  5.0,
  36.124783736376884,
  41.23105625617661,
  25.079872407968907],
 [14.142135623730951,
  29.120439557122072,
  24.73863375370596,
  10.295630140987,
  21.213203435596427,
  39.05124837953327,
  16.278820596099706,
  20.0,
  25.495097567963924,
  9.848857801796104,
  18.867962264113206,
  29.154759474226502,
  5.0990195135927845,
  35.34119409414458,
  32.7566787083184,
  41.23105625617661,
  16.64331697709324,
  6.708203932499369,
  21.840329667841555,
  32.0624390837628,
  50.91168824543142,
  39.56008088970496,
  37.21558813185679,
  29.068883707497267,
  24.041630560342615,
  19.1049731745428,
  11.704699910719626,
  29.68164415931166,
  34.36568055487916,
  32.55764119219941,
  27.294688127912362,
  26.019223662515376,
  8.54400374531753,
  21.37755832643195,
  22.360679774997898,
  25.0,
  46.647615158762406,
  46.09772228646444,
  23.345235059857504,
  10.0,
  0.0,
  37.589892258425,
  39.0,
  34.0147027033899,
  9.219544457292887,
  28.284271247461902,
  22.47220505424423,
  40.311288741492746,
  34.132096331752024,
  21.633307652783937,
  16.15549442140351,
  11.045361017187261,
  26.832815729997478,
  25.96150997149434,
  38.07886552931954,
  28.284271247461902,
  34.655446902326915,
  41.88078318274385,
  14.142135623730951,
  42.37924020083418,
  57.201398584300364,
  44.40720662234904,
  30.0,
  22.360679774997898,
  47.43416490252569,
  28.284271247461902,
  34.828149534535996,
  17.0,
  18.788294228055936,
  50.15974481593781,
  55.31726674375732,
  53.53503525729669,
  11.180339887498949,
  26.1725046566048,
  31.622776601683793,
  16.401219466856727],
 [36.235341863986875,
  11.180339887498949,
  25.632011235952593,
  29.410882339705484,
  37.589892258425,
  43.104524124504614,
  27.018512172212592,
  50.32891812864648,
  51.31276644267,
  44.27188724235731,
  56.43580423808985,
  64.4437739428721,
  41.048751503547585,
  53.14132102234569,
  64.03124237432849,
  50.48762224545735,
  21.02379604162864,
  34.20526275297414,
  39.11521443121589,
  58.83026432033091,
  54.08326913195984,
  32.2490309931942,
  14.560219778561036,
  11.045361017187261,
  26.476404589747453,
  47.265209192385896,
  41.012193308819754,
  46.238512086787566,
  23.021728866442675,
  41.0,
  32.28002478313795,
  61.98386886924694,
  37.36308338453881,
  18.439088914585774,
  44.41846462902562,
  54.20332093147061,
  42.579337712087536,
  48.041648597857254,
  60.21627686929839,
  46.61544808322666,
  37.589892258425,
  0.0,
  4.242640687119285,
  4.47213595499958,
  32.28002478313795,
  40.162171256046406,
  46.32493928760188,
  38.05259518088089,
  36.22154055254967,
  21.0,
  39.11521443121589,
  27.80287754891569,
  46.95742752749558,
  58.728187440104094,
  60.00833275470999,
  53.03772242470448,
  10.0,
  53.9351462406472,
  51.31276644267,
  74.65252842335616,
  53.600373133029585,
  26.40075756488817,
  18.24828759089466,
  15.264337522473747,
  12.36931687685298,
  65.21502894272147,
  71.06335201775947,
  41.400483088968905,
  30.528675044947494,
  40.162171256046406,
  54.74486277268398,
  48.507731342539614,
  48.76474136094644,
  16.55294535724685,
  28.160255680657446,
  34.40930106817051],
 [36.069377593742864,
  10.63014581273465,
  24.186773244895647,
  31.575306807693888,
  36.61966684720111,
  40.44749683231337,
  26.90724809414742,
  50.20956084253277,
  50.60632371551998,
  46.32493928760188,
  57.697486947006624,
  64.81512169239521,
  42.05948168962618,
  51.478150704935004,
  63.702433234531945,
  48.05205510693585,
  22.80350850198276,
  34.9857113690718,
  42.42640687119285,
  58.008620049092706,
  51.0,
  29.017236257093817,
  11.045361017187261,
  14.560219778561036,
  30.083217912982647,
  50.039984012787215,
  41.23105625617661,
  44.721359549995796,
  20.09975124224178,
  38.897300677553446,
  30.463092423455635,
  64.03124237432849,
  39.6232255123179,
  18.601075237738275,
  43.60045871318328,
  53.81449618829484,
  39.20459156782532,
  45.0111097397076,
  61.05735008989499,
  48.38388161361178,
  39.0,
  4.242640687119285,
  0.0,
  5.0990195135927845,
  34.52535300326414,
  38.48376280978771,
  45.60701700396552,
  35.014282800023196,
  33.734255586866,
  24.186773244895647,
  42.0,
  28.653097563788805,
  45.79301256742124,
  58.728187440104094,
  58.59180830116101,
  56.22277118748239,
  14.212670403551895,
  51.66236541235796,
  52.354560450833695,
  74.33034373659252,
  50.00999900019995,
  22.47220505424423,
  16.15549442140351,
  16.76305461424021,
  9.0,
  66.03786792439622,
  71.61005515987263,
  41.036569057366385,
  29.832867780352597,
  36.40054944640259,
  51.35172830587107,
  44.91102314577124,
  50.15974481593781,
  15.620499351813308,
  25.709920264364882,
  33.97057550292606],
 [31.89043743820395,
  6.708203932499369,
  21.18962010041709,
  26.476404589747453,
  33.12099032335839,
  39.01281840626232,
  22.67156809750927,
  46.010868281309364,
  46.87216658103186,
  41.23105625617661,
  52.773099207835045,
  60.30754513325841,
  37.21558813185679,
  48.703182647543684,
  59.665735560705194,
  46.2709412050371,
  17.72004514666935,
  30.23243291566195,
  37.656340767525464,
  54.378304497290095,
  50.24937810560445,
  28.635642126552707,
  11.661903789690601,
  10.295630140987,
  25.632011235952593,
  45.0111097397076,
  36.796738985948195,
  41.78516483155236,
  19.235384061671343,
  36.6742416417845,
  27.892651361962706,
  58.9406481131655,
  34.52535300326414,
  14.142135623730951,
  39.96248240537617,
  49.8196748283246,
  39.05124837953327,
  44.181444068749045,
  56.302753041036986,
  43.32435804486894,
  34.0147027033899,
  4.47213595499958,
  5.0990195135927845,
  0.0,
  29.427877939124322,
  35.73513677041127,
  41.88078318274385,
  34.23448553724738,
  32.0624390837628,
  19.4164878389476,
  37.013511046643494,
  23.853720883753127,
  42.485291572496,
  54.45181356024793,
  55.54277630799526,
  51.35172830587107,
  11.661903789690601,
  49.64876634922564,
  47.50789408087881,
  70.2922470831599,
  50.28916384272063,
  23.853720883753127,
  14.035668847618199,
  11.704699910719626,
  14.035668847618199,
  61.29437168288782,
  67.00746227100382,
  37.013511046643494,
  26.076809620810597,
  37.16180835212409,
  51.19570294468082,
  45.221676218380054,
  45.18849411078001,
  12.083045973594572,
  24.020824298928627,
  30.0],
 [20.615528128088304,
  26.019223662515376,
  26.627053911388696,
  3.0,
  27.294688127912362,
  44.04543109109048,
  18.439088914585774,
  29.068883707497267,
  34.132096331752024,
  12.083045973594572,
  26.1725046566048,
  38.01315561749642,
  14.317821063276353,
  43.01162633521314,
  41.97618372363071,
  47.50789408087881,
  12.0,
  12.649110640673518,
  14.422205101855956,
  41.0,
  56.4003546088143,
  41.88078318274385,
  35.35533905932738,
  22.360679774997898,
  14.866068747318506,
  16.492422502470642,
  20.09975124224178,
  36.76955262170047,
  34.92849839314596,
  38.01315561749642,
  31.11269837220809,
  29.732137494637012,
  5.0990195135927845,
  19.6468827043885,
  30.083217912982647,
  34.058772731852805,
  50.32891812864648,
  51.088159097779204,
  31.622776601683793,
  15.0,
  9.219544457292887,
  32.28002478313795,
  34.52535300326414,
  29.427877939124322,
  0.0,
  34.132096331752024,
  30.59411708155671,
  43.93176527297759,
  38.18376618407357,
  13.45362404707371,
  10.0,
  12.041594578792296,
  34.48187929913333,
  35.17101079013795,
  46.52956049652737,
  24.596747752497688,
  27.459060435491963,
  48.75448697299562,
  22.47220505424423,
  51.54609587543949,
  61.5223536610881,
  44.598206241955516,
  29.410882339705484,
  18.027756377319946,
  43.41658669218482,
  36.40054944640259,
  43.266615305567875,
  24.73863375370596,
  22.67156809750927,
  52.69724850502159,
  60.207972893961475,
  57.38466694161429,
  18.439088914585774,
  24.73863375370596,
  33.83784863137726,
  21.95449840010015],
 [14.142135623730951,
  29.120439557122072,
  14.560219778561036,
  32.64965543462902,
  7.0710678118654755,
  11.180339887498949,
  17.46424919657298,
  20.0,
  15.811388300841896,
  37.64306044943742,
  37.36308338453881,
  35.35533905932738,
  25.80697580112788,
  13.0,
  29.546573405388315,
  13.416407864998739,
  29.614185789921695,
  22.02271554554524,
  48.54894437575342,
  21.633307652783937,
  22.627416997969522,
  18.027756377319946,
  29.410882339705484,
  39.05124837953327,
  44.9221548904324,
  47.38143096192854,
  18.35755975068582,
  6.4031242374328485,
  20.518284528683193,
  4.47213595499958,
  8.06225774829855,
  49.76946855251722,
  36.235341863986875,
  24.020824298928627,
  10.0,
  20.615528128088304,
  20.396078054371138,
  18.027756377319946,
  36.124783736376884,
  36.05551275463989,
  28.284271247461902,
  40.162171256046406,
  38.48376280978771,
  35.73513677041127,
  34.132096331752024,
  0.0,
  12.041594578792296,
  15.0,
  9.219544457292887,
  38.8329756778952,
  43.60045871318328,
  22.847319317591726,
  8.94427190999916,
  27.459060435491963,
  20.248456731316587,
  56.568542494923804,
  44.28317965096906,
  15.297058540778355,
  31.622776601683793,
  39.44616584663204,
  29.5296461204668,
  27.784887978899608,
  22.360679774997898,
  30.0,
  43.01162633521314,
  40.0,
  42.579337712087536,
  13.0,
  12.36931687685298,
  26.0,
  27.202941017470888,
  26.570660511172846,
  33.54101966249684,
  23.769728648009426,
  14.142135623730951,
  12.206555615733702],
 [11.180339887498949,
  35.22782990761707,
  21.93171219946131,
  30.14962686336267,
  9.219544457292887,
  22.360679774997898,
  19.697715603592208,
  8.06225774829855,
  5.0,
  30.23243291566195,
  26.40075756488817,
  23.345235059857504,
  18.35755975068582,
  13.038404810405298,
  18.601075237738275,
  21.095023109728988,
  30.59411708155671,
  18.110770276274835,
  44.27188724235731,
  12.529964086141668,
  31.76476034853718,
  29.832867780352597,
  38.28837943815329,
  42.37924020083418,
  44.01136216933077,
  40.496913462633174,
  10.770329614269007,
  8.94427190999916,
  30.528675044947494,
  16.0312195418814,
  17.88854381999832,
  39.44616584663204,
  31.016124838541646,
  28.178005607210743,
  2.23606797749979,
  8.94427190999916,
  32.14031735997639,
  28.460498941515414,
  24.331050121192877,
  27.65863337187866,
  22.47220505424423,
  46.32493928760188,
  45.60701700396552,
  41.88078318274385,
  30.59411708155671,
  12.041594578792296,
  0.0,
  27.018512172212592,
  21.213203435596427,
  39.20459156782532,
  38.62641583165593,
  22.20360331117452,
  5.0,
  15.524174696260024,
  16.0312195418814,
  49.64876634922564,
  48.104053883222775,
  20.518284528683193,
  21.095023109728988,
  29.068883707497267,
  40.162171256046406,
  39.0,
  30.4138126514911,
  33.24154027718932,
  51.62363799656123,
  28.0178514522438,
  30.59411708155671,
  6.0,
  15.811388300841896,
  38.01315561749642,
  37.16180835212409,
  37.8549864614954,
  24.08318915758459,
  30.0,
  24.596747752497688,
  12.083045973594572],
 [26.92582403567252,
  28.861739379323623,
  17.804493814764857,
  41.72529209005013,
  20.615528128088304,
  7.0710678118654755,
  25.495097567963924,
  35.0,
  30.4138126514911,
  50.11985634456667,
  51.97114584074513,
  50.24937810560445,
  39.0,
  23.323807579381203,
  43.68065933568311,
  15.0,
  35.805027579936315,
  33.61547262794322,
  57.982756057296896,
  35.11409973215888,
  16.0312195418814,
  6.324555320336759,
  24.08318915758459,
  41.10960958218893,
  51.31276644267,
  59.07622195096772,
  32.28002478313795,
  19.6468827043885,
  15.033296378372908,
  11.180339887498949,
  13.038404810405298,
  63.89053137985315,
  47.20169488482379,
  27.784887978899608,
  25.0,
  35.35533905932738,
  6.4031242374328485,
  10.0,
  51.088159097779204,
  49.24428900898052,
  40.311288741492746,
  38.05259518088089,
  35.014282800023196,
  34.23448553724738,
  43.93176527297759,
  15.0,
  27.018512172212592,
  0.0,
  6.324555320336759,
  44.41846462902562,
  53.907327887774215,
  31.89043743820395,
  23.345235059857504,
  42.2965719651132,
  31.064449134018133,
  68.00735254367721,
  45.12205669071391,
  19.209372712298546,
  46.09772228646444,
  52.92447448959697,
  17.804493814764857,
  16.64331697709324,
  20.615528128088304,
  33.54101966249684,
  36.40054944640259,
  55.0,
  57.42821606144492,
  27.459060435491963,
  21.633307652783937,
  11.0,
  17.46424919657298,
  13.45362404707371,
  47.43416490252569,
  24.698178070456937,
  11.180339887498949,
  24.166091947189145],
 [20.615528128088304,
  26.019223662515376,
  13.0,
  36.124783736376884,
  14.317821063276353,
  7.0710678118654755,
  19.849433241279208,
  29.068883707497267,
  25.0,
  43.9089968002003,
  45.70557952810576,
  44.553338819890925,
  32.69556544854363,
  19.79898987322333,
  38.62641583165593,
  14.317821063276353,
  30.886890422961002,
  27.459060435491963,
  52.40229002629561,
  30.4138126514911,
  19.313207915827967,
  8.94427190999916,
  23.40939982143925,
  37.656340767525464,
  46.52956049652737,
  53.009433122794285,
  25.96150997149434,
  14.7648230602334,
  13.92838827718412,
  6.4031242374328485,
  7.0710678118654755,
  57.56735185849702,
  41.23105625617661,
  23.40939982143925,
  19.1049731745428,
  29.832867780352597,
  12.529964086141668,
  13.416407864998739,
  45.0111097397076,
  42.95346318982906,
  34.132096331752024,
  36.22154055254967,
  33.734255586866,
  32.0624390837628,
  38.18376618407357,
  9.219544457292887,
  21.213203435596427,
  6.324555320336759,
  0.0,
  39.81205847478876,
  48.104053883222775,
  26.1725046566048,
  18.027756377319946,
  36.6742416417845,
  27.586228448267445,
  62.00806399170998,
  42.190046219457976,
  18.027756377319946,
  39.81205847478876,
  48.30113870293329,
  23.345235059857504,
  19.209372712298546,
  18.027756377319946,
  29.410882339705484,
  36.6742416417845,
  49.040799340956916,
  51.78802950489621,
  21.213203435596427,
  15.620499351813308,
  17.11724276862369,
  22.20360331117452,
  19.4164878389476,
  41.10960958218893,
  21.213203435596427,
  8.06225774829855,
  17.88854381999832],
 [28.071337695236398,
  18.867962264113206,
  26.832815729997478,
  11.40175425099138,
  33.13608305156178,
  46.61544808322666,
  21.37755832643195,
  39.84971769034255,
  43.56604182158393,
  25.238858928247925,
  39.59797974644666,
  50.774009099144415,
  26.419689627245813,
  50.08991914547278,
  53.48831648126533,
  51.92301994298868,
  9.219544457292887,
  21.840329667841555,
  18.24828759089466,
  50.99019513592785,
  59.09314681077663,
  40.607881008493905,
  28.653097563788805,
  10.04987562112089,
  7.0710678118654755,
  26.476404589747453,
  30.083217912982647,
  43.18564576337837,
  31.827660925679098,
  41.7612260356422,
  33.24154027718932,
  42.485291572496,
  18.027756377319946,
  16.64331697709324,
  38.05259518088089,
  44.64302857109943,
  50.47771785649585,
  53.225933528685054,
  44.82186966202994,
  28.42534080710379,
  21.633307652783937,
  21.0,
  24.186773244895647,
  19.4164878389476,
  13.45362404707371,
  38.8329756778952,
  39.20459156782532,
  44.41846462902562,
  39.81205847478876,
  0.0,
  18.24828759089466,
  17.029386365926403,
  42.0,
  47.01063709417264,
  55.08175741568164,
  32.0624390837628,
  14.317821063276353,
  54.12947441089743,
  35.608987629529715,
  63.56099432828282,
  62.12889826803627,
  40.0,
  25.45584412271571,
  11.313708498984761,
  33.13608305156178,
  49.678969393496885,
  56.4003546088143,
  33.24154027718932,
  26.476404589747453,
  50.99019513592785,
  61.773780845922005,
  57.42821606144492,
  31.827660925679098,
  20.518284528683193,
  33.28663395418648,
  28.0178514522438],
 [29.68164415931166,
  34.713109915419565,
  36.61966684720111,
  12.529964086141668,
  36.61966684720111,
  53.81449618829484,
  28.42534080710379,
  35.510561809129406,
  41.48493702538308,
  11.40175425099138,
  26.92582403567252,
  41.0,
  20.615528128088304,
  51.478150704935004,
  47.01063709417264,
  56.859475903318,
  20.8806130178211,
  21.633307652783937,
  6.0,
  47.67598976424087,
  66.06814663663572,
  51.86520991955976,
  44.384682042344295,
  28.284271247461902,
  15.264337522473747,
  8.246211251235321,
  27.85677655436824,
  45.65084884205331,
  44.721359549995796,
  47.634021455258214,
  41.036569057366385,
  25.612496949731394,
  7.615773105863909,
  29.154759474226502,
  38.48376280978771,
  40.44749683231337,
  60.30754513325841,
  60.876925020897694,
  33.52610922848042,
  15.524174696260024,
  16.15549442140351,
  39.11521443121589,
  42.0,
  37.013511046643494,
  10.0,
  43.60045871318328,
  38.62641583165593,
  53.907327887774215,
  48.104053883222775,
  18.24828759089466,
  0.0,
  22.02271554554524,
  42.95346318982906,
  40.01249804748511,
  54.120236510939236,
  14.866068747318506,
  32.28002478313795,
  57.8013840664737,
  25.317977802344327,
  55.57877292636101,
  71.449282711585,
  54.230987451824994,
  39.0,
  26.476404589747453,
  51.0,
  37.69615364994153,
  44.94441010848846,
  33.1058907144937,
  32.526911934581186,
  62.68173577685928,
  70.03570517957252,
  67.35725647619564,
  20.396078054371138,
  34.17601498127012,
  43.829214001622255,
  31.400636936215164],
 [11.045361017187261,
  18.384776310850235,
  14.7648230602334,
  10.0,
  16.492422502470642,
  32.202484376209235,
  6.4031242374328485,
  23.706539182259394,
  26.68332812825267,
  20.223748416156685,
  29.154759474226502,
  36.76955262170047,
  13.416407864998739,
  33.24154027718932,
  37.589892258425,
  36.24913792078372,
  8.54400374531753,
  6.4031242374328485,
  26.248809496813376,
  34.20526275297414,
  44.654227123532216,
  30.01666203960727,
  26.1725046566048,
  21.095023109728988,
  22.360679774997898,
  27.730849247724095,
  13.892443989449804,
  26.476404589747453,
  23.769728648009426,
  26.419689627245813,
  19.1049731745428,
  37.05401462729781,
  15.652475842498529,
  10.44030650891055,
  21.02379604162864,
  28.231188426986208,
  38.28837943815329,
  39.20459156782532,
  32.449961479175904,
  21.02379604162864,
  11.045361017187261,
  27.80287754891569,
  28.653097563788805,
  23.853720883753127,
  12.041594578792296,
  22.847319317591726,
  22.20360331117452,
  31.89043743820395,
  26.1725046566048,
  17.029386365926403,
  22.02271554554524,
  0.0,
  25.019992006393608,
  31.622776601683793,
  38.05259518088089,
  36.359317925395686,
  26.92582403567252,
  37.94733192202055,
  23.706539182259394,
  48.02082881417188,
  49.49747468305833,
  33.734255586866,
  19.026297590440446,
  12.727922061357855,
  36.76955262170047,
  37.44329045369811,
  43.278170016764804,
  16.278820596099706,
  10.816653826391969,
  40.8166632639171,
  48.27007354458868,
  45.34313619501854,
  21.840329667841555,
  15.132745950421556,
  21.95449840010015,
  11.180339887498949],
 [14.142135623730951,
  35.77708763999664,
  21.633307652783937,
  33.734255586866,
  9.486832980505138,
  18.027756377319946,
  21.37755832643195,
  12.649110640673518,
  7.0710678118654755,
  35.0,
  31.304951684997057,
  27.018512172212592,
  23.021728866442675,
  8.54400374531753,
  20.615528128088304,
  16.1245154965971,
  33.06055050963308,
  21.840329667841555,
  48.46648326421054,
  12.806248474865697,
  26.832815729997478,
  26.92582403567252,
  37.53664875824692,
  44.14748010928823,
  47.265209192385896,
  45.221676218380054,
  15.264337522473747,
  4.123105625617661,
  29.068883707497267,
  12.165525060596439,
  16.278820596099706,
  44.41846462902562,
  35.34119409414458,
  29.410882339705484,
  4.47213595499958,
  12.041594578792296,
  28.0,
  23.769728648009426,
  28.861739379323623,
  32.55764119219941,
  26.832815729997478,
  46.95742752749558,
  45.79301256742124,
  42.485291572496,
  34.48187929913333,
  8.94427190999916,
  5.0,
  23.345235059857504,
  18.027756377319946,
  42.0,
  42.95346318982906,
  25.019992006393608,
  0.0,
  19.026297590440446,
  13.341664064126334,
  54.405882034941776,
  49.72926703662542,
  15.556349186104045,
  26.076809620810597,
  30.528675044947494,
  35.4400902933387,
  36.71511950137164,
  30.0,
  34.92849839314596,
  51.088159097779204,
  32.2490309931942,
  34.132096331752024,
  9.848857801796104,
  16.76305461424021,
  34.23448553724738,
  32.31098884280702,
  33.37663853655727,
  29.068883707497267,
  30.4138126514911,
  22.80350850198276,
  14.035668847618199],
 [22.67156809750927,
  48.104053883222775,
  36.359317925395686,
  36.05551275463989,
  24.166091947189145,
  37.0,
  31.827660925679098,
  8.602325267042627,
  12.0,
  29.068883707497267,
  17.4928556845359,
  8.0,
  20.8806130178211,
  23.08679276123039,
  7.0,
  33.734255586866,
  39.96248240537617,
  25.553864678361276,
  46.010868281309364,
  11.40175425099138,
  44.384682042344295,
  45.35416188179427,
  52.69724850502159,
  52.69724850502159,
  50.0,
  38.63935817272331,
  17.804493814764857,
  23.0,
  45.617978911828175,
  31.144823004794873,
  33.24154027718932,
  30.610455730027933,
  33.24154027718932,
  40.311288741492746,
  17.72004514666935,
  7.0,
  47.01063709417264,
  42.2965719651132,
  12.041594578792296,
  25.179356624028344,
  25.96150997149434,
  58.728187440104094,
  58.728187440104094,
  54.45181356024793,
  35.17101079013795,
  27.459060435491963,
  15.524174696260024,
  42.2965719651132,
  36.6742416417845,
  47.01063709417264,
  40.01249804748511,
  31.622776601683793,
  19.026297590440446,
  0.0,
  20.0,
  46.84015371452148,
  58.52349955359813,
  31.622776601683793,
  15.297058540778355,
  16.55294535724685,
  53.75872022286245,
  54.42425929675111,
  44.654227123532216,
  44.204072210600685,
  65.60487786742691,
  13.92838827718412,
  15.132745950421556,
  18.027756377319946,
  29.410882339705484,
  53.23532661682466,
  50.21951811795888,
  52.15361924162119,
  20.223748416156685,
  43.278170016764804,
  39.92492955535426,
  25.0],
 [27.018512172212592,
  48.84669896727925,
  34.438350715445125,
  46.17358552246078,
  22.80350850198276,
  24.186773244895647,
  34.713109915419565,
  19.235384061671343,
  12.649110640673518,
  44.77722635447622,
  36.796738985948195,
  26.832815729997478,
  33.52610922848042,
  7.810249675906654,
  16.76305461424021,
  17.72004514666935,
  46.2709412050371,
  34.132096331752024,
  59.90826320300064,
  8.602325267042627,
  27.018512172212592,
  36.235341863986875,
  49.64876634922564,
  57.48912940721924,
  60.03332407921454,
  55.036351623268054,
  26.476404589747453,
  13.892443989449804,
  40.607881008493905,
  21.213203435596427,
  28.30194339616981,
  50.20956084253277,
  46.57252408878007,
  42.720018726587654,
  17.029386365926403,
  15.0,
  33.61547262794322,
  26.92582403567252,
  32.01562118716424,
  41.593268686170845,
  38.07886552931954,
  60.00833275470999,
  58.59180830116101,
  55.54277630799526,
  46.52956049652737,
  20.248456731316587,
  16.0312195418814,
  31.064449134018133,
  27.586228448267445,
  55.08175741568164,
  54.120236510939236,
  38.05259518088089,
  13.341664064126334,
  20.0,
  0.0,
  63.953107821277925,
  63.06346010171025,
  14.142135623730951,
  33.015148038438355,
  23.194827009486403,
  37.12142238654117,
  46.75467891024384,
  42.5440947723653,
  48.27007354458868,
  63.245553203367585,
  33.61547262794322,
  32.57299494980466,
  22.02271554554524,
  30.083217912982647,
  40.718546143004666,
  33.015148038438355,
  36.87817782917155,
  37.21558813185679,
  43.46262762420146,
  34.20526275297414,
  27.294688127912362],
 [42.42640687119285,
  49.47726750741192,
  51.10772935672255,
  27.313000567495326,
  49.49747468305833,
  67.26812023536856,
  42.720018726587654,
  44.721359549995796,
  51.478150704935004,
  19.4164878389476,
  30.265491900843113,
  45.27692569068709,
  31.400636936215164,
  62.68173577685928,
  53.600373133029585,
  69.42621983083913,
  35.73513677041127,
  34.713109915419565,
  14.035668847618199,
  56.462376853972415,
  79.19595949289332,
  66.37017402418047,
  59.20304046246274,
  42.01190307520001,
  27.16615541441225,
  9.219544457292887,
  39.20459156782532,
  57.62811813689564,
  59.50630218724736,
  60.8276253029822,
  55.0,
  21.840329667841555,
  20.808652046684813,
  44.01136216933077,
  50.0,
  49.24428900898052,
  74.4043009509531,
  74.33034373659252,
  37.21558813185679,
  22.360679774997898,
  28.284271247461902,
  53.03772242470448,
  56.22277118748239,
  51.35172830587107,
  24.596747752497688,
  56.568542494923804,
  49.64876634922564,
  68.00735254367721,
  62.00806399170998,
  32.0624390837628,
  14.866068747318506,
  36.359317925395686,
  54.405882034941776,
  46.84015371452148,
  63.953107821277925,
  0.0,
  45.17742799230607,
  69.81403870282824,
  31.622776601683793,
  60.29925372672534,
  85.27602242131137,
  69.07966415668217,
  53.85164807134504,
  41.23105625617661,
  65.19202405202648,
  40.0,
  47.042533945356304,
  44.82186966202994,
  46.400431032480725,
  77.17512552629896,
  83.54639429682169,
  81.40024569987489,
  26.92582403567252,
  49.040799340956916,
  58.309518948453004,
  44.598206241955516],
 [37.16180835212409,
  16.278820596099706,
  30.01666203960727,
  25.0,
  40.26164427839479,
  49.25444142409901,
  28.460498941515414,
  50.60632371551998,
  52.92447448959697,
  39.44616584663204,
  53.2634959423431,
  63.25345840347388,
  38.948684188300895,
  56.88585061331157,
  64.4980619863884,
  56.00892785976178,
  18.601075237738275,
  33.015148038438355,
  31.144823004794873,
  60.53924347066124,
  61.0,
  39.84971769034255,
  23.323807579381203,
  5.830951894845301,
  18.027756377319946,
  40.52159917870962,
  40.8166632639171,
  49.8196748283246,
  30.364452901377952,
  46.010868281309364,
  37.013511046643494,
  56.79788728465171,
  32.2490309931942,
  20.396078054371138,
  46.486557196677836,
  55.00909015790027,
  50.28916384272063,
  54.91812087098393,
  57.982756057296896,
  42.43819034784589,
  34.655446902326915,
  10.0,
  14.212670403551895,
  11.661903789690601,
  27.459060435491963,
  44.28317965096906,
  48.104053883222775,
  45.12205669071391,
  42.190046219457976,
  14.317821063276353,
  32.28002478313795,
  26.92582403567252,
  49.72926703662542,
  58.52349955359813,
  63.06346010171025,
  45.17742799230607,
  0.0,
  59.033888572581766,
  48.79549159502341,
  74.94664769020693,
  61.68468205316454,
  35.510561809129406,
  24.515301344262525,
  14.866068747318506,
  21.93171219946131,
  62.93647591023825,
  69.35416353759881,
  42.5440947723653,
  32.984845004941285,
  48.75448697299562,
  62.36184731067546,
  56.64803615307419,
  45.45327270945405,
  21.02379604162864,
  34.36568055487916,
  36.05551275463989],
 [28.178005607210743,
  43.289721643826724,
  29.017236257093817,
  47.53945729601885,
  21.540659228538015,
  12.206555615733702,
  32.7566787083184,
  27.459060435491963,
  20.591260281974,
  50.44799302251776,
  46.32493928760188,
  39.293765408777,
  38.41874542459709,
  8.54400374531753,
  30.14962686336267,
  4.242640687119285,
  44.91102314577124,
  36.235341863986875,
  63.06346010171025,
  21.213203435596427,
  13.038404810405298,
  25.238858928247925,
  41.43669871020132,
  54.00925846556311,
  60.166435825965294,
  60.60528029800704,
  30.610455730027933,
  12.206555615733702,
  31.953090617340916,
  13.038404810405298,
  22.02271554554524,
  59.64059020499378,
  50.24937810560445,
  39.05124837953327,
  19.849433241279208,
  25.079872407968907,
  20.248456731316587,
  13.0,
  42.95346318982906,
  48.104053883222775,
  41.88078318274385,
  53.9351462406472,
  51.66236541235796,
  49.64876634922564,
  48.75448697299562,
  15.297058540778355,
  20.518284528683193,
  19.209372712298546,
  18.027756377319946,
  54.12947441089743,
  57.8013840664737,
  37.94733192202055,
  15.556349186104045,
  31.622776601683793,
  14.142135623730951,
  69.81403870282824,
  59.033888572581766,
  0.0,
  41.400483088968905,
  37.33630940518894,
  23.021728866442675,
  35.805027579936315,
  35.6931365951495,
  45.09988913511872,
  54.62600113499065,
  45.5411901469428,
  45.70557952810576,
  25.0,
  27.65863337187866,
  27.459060435491963,
  19.026297590440446,
  22.80350850198276,
  44.598206241955516,
  38.118237105091836,
  25.96150997149434,
  26.92582403567252],
 [20.0,
  42.04759208325728,
  34.23448553724738,
  24.20743687382041,
  25.495097567963924,
  42.720018726587654,
  26.92582403567252,
  14.142135623730951,
  21.213203435596427,
  14.035668847618199,
  6.0,
  15.811388300841896,
  10.295630140987,
  33.301651610693426,
  22.20360331117452,
  42.190046219457976,
  30.610455730027933,
  17.46424919657298,
  31.25699921617557,
  25.059928172283335,
  52.839379254491625,
  47.16990566028302,
  49.040799340956916,
  43.18564576337837,
  37.12142238654117,
  23.345235059857504,
  14.035668847618199,
  30.01666203960727,
  44.28317965096906,
  36.05551275463989,
  34.132096331752024,
  18.35755975068582,
  19.313207915827967,
  34.0147027033899,
  22.360679774997898,
  18.027756377319946,
  51.92301994298868,
  49.24428900898052,
  9.219544457292887,
  10.0,
  14.142135623730951,
  51.31276644267,
  52.354560450833695,
  47.50789408087881,
  22.47220505424423,
  31.622776601683793,
  21.095023109728988,
  46.09772228646444,
  39.81205847478876,
  35.608987629529715,
  25.317977802344327,
  23.706539182259394,
  26.076809620810597,
  15.297058540778355,
  33.015148038438355,
  31.622776601683793,
  48.79549159502341,
  41.400483088968905,
  0.0,
  30.265491900843113,
  60.92618484691127,
  54.147945482723536,
  41.23105625617661,
  36.05551275463989,
  60.41522986797286,
  14.142135623730951,
  20.808652046684813,
  18.681541692269406,
  27.073972741361768,
  56.88585061331157,
  58.137767414994535,
  58.18934610390462,
  5.0,
  38.27531841800928,
  40.0,
  23.0],
 [38.41874542459709,
  63.7808748764079,
  50.99019513592785,
  52.55473337388365,
  38.28837943815329,
  46.486557196677836,
  47.634021455258214,
  24.413111231467404,
  24.20743687382041,
  44.28317965096906,
  30.066592756745816,
  15.033296378372908,
  37.33630940518894,
  30.083217912982647,
  10.63014581273465,
  40.792156108742276,
  56.43580423808985,
  42.05948168962618,
  61.5223536610881,
  17.88854381999832,
  50.15974481593781,
  57.245087125446844,
  67.35725647619564,
  69.1158447825099,
  66.40783086353596,
  53.0,
  34.132096331752024,
  33.54101966249684,
  59.405386961116584,
  42.04759208325728,
  46.61544808322666,
  40.80441152620633,
  49.24428900898052,
  56.22277118748239,
  31.240998703626616,
  20.518284528683193,
  56.32051136131489,
  50.00999900019995,
  23.08679276123039,
  40.19950248448356,
  42.37924020083418,
  74.65252842335616,
  74.33034373659252,
  70.2922470831599,
  51.54609587543949,
  39.44616584663204,
  29.068883707497267,
  52.92447448959697,
  48.30113870293329,
  63.56099432828282,
  55.57877292636101,
  48.02082881417188,
  30.528675044947494,
  16.55294535724685,
  23.194827009486403,
  60.29925372672534,
  74.94664769020693,
  37.33630940518894,
  30.265491900843113,
  0.0,
  60.29925372672534,
  67.23094525588644,
  59.464274989274024,
  60.4648658313239,
  80.65977932030313,
  20.8806130178211,
  15.264337522473747,
  33.301651610693426,
  44.553338819890925,
  63.245553203367585,
  56.142675390472796,
  60.01666435249463,
  35.22782990761707,
  58.728187440104094,
  53.25410782277739,
  40.36087214122113],
 [43.266615305567875,
  45.69463863518345,
  35.608987629529715,
  59.413803110051795,
  36.359317925395686,
  18.35755975068582,
  43.139309220245984,
  48.08326112068523,
  41.97618372363071,
  66.85057965343307,
  66.48308055437865,
  61.66036003787198,
  55.226805085936306,
  31.064449134018133,
  53.03772242470448,
  20.09975124224178,
  53.600373133029585,
  50.60632371551998,
  75.69015788066504,
  44.04543109109048,
  10.198039027185569,
  21.93171219946131,
  39.05124837953327,
  58.180752831155424,
  69.06518659932803,
  76.21679604916491,
  47.885279575251516,
  31.32091952673165,
  31.38470965295043,
  25.059928172283335,
  30.4138126514911,
  79.20227269466452,
  64.53681120105021,
  45.48626166217664,
  38.62641583165593,
  46.87216658103186,
  11.661903789690601,
  11.704699910719626,
  64.28841264178172,
  65.5133574166368,
  57.201398584300364,
  53.600373133029585,
  50.00999900019995,
  50.28916384272063,
  61.5223536610881,
  29.5296461204668,
  40.162171256046406,
  17.804493814764857,
  23.345235059857504,
  62.12889826803627,
  71.449282711585,
  49.49747468305833,
  35.4400902933387,
  53.75872022286245,
  37.12142238654117,
  85.27602242131137,
  61.68468205316454,
  23.021728866442675,
  60.92618484691127,
  60.29925372672534,
  0.0,
  28.071337695236398,
  37.57658845611187,
  51.10772935672255,
  49.01020301937138,
  67.468511173732,
  68.3593446428504,
  42.485291572496,
  38.948684188300895,
  14.0,
  4.47213595499958,
  5.0990195135927845,
  63.06346010171025,
  42.05948168962618,
  28.844410203711913,
  40.80441152620633],
 [34.23448553724738,
  21.2602916254693,
  20.0,
  41.78516483155236,
  30.364452901377952,
  23.600847442411894,
  28.160255680657446,
  46.17358552246078,
  43.382023926967726,
  53.9351462406472,
  60.13318551349163,
  61.98386886924694,
  45.0111097397076,
  38.948684188300895,
  57.245087125446844,
  31.622776601683793,
  33.54101966249684,
  38.118237105091836,
  56.824290580701486,
  49.39635614091387,
  31.04834939252005,
  10.63014581273465,
  12.206555615733702,
  33.301651610693426,
  47.01063709417264,
  61.032778078668514,
  40.311288741492746,
  33.83784863137726,
  10.04987562112089,
  25.612496949731394,
  21.18962010041709,
  70.178344238091,
  49.040799340956916,
  25.079872407968907,
  36.76955262170047,
  47.92702786528704,
  18.439088914585774,
  25.632011235952593,
  61.0,
  54.3323108288245,
  44.40720662234904,
  26.40075756488817,
  22.47220505424423,
  23.853720883753127,
  44.598206241955516,
  27.784887978899608,
  39.0,
  16.64331697709324,
  19.209372712298546,
  40.0,
  54.230987451824994,
  33.734255586866,
  36.71511950137164,
  54.42425929675111,
  46.75467891024384,
  69.07966415668217,
  35.510561809129406,
  35.805027579936315,
  54.147945482723536,
  67.23094525588644,
  28.071337695236398,
  0.0,
  15.231546211727817,
  28.844410203711913,
  21.02379604162864,
  65.5133574166368,
  69.26037828369117,
  37.107950630558946,
  27.073972741361768,
  14.142135623730951,
  30.066592756745816,
  23.021728866442675,
  54.00925846556311,
  20.12461179749811,
  14.560219778561036,
  31.25699921617557],
 [22.360679774997898,
  8.246211251235321,
  8.48528137423857,
  26.570660511172846,
  21.213203435596427,
  25.0,
  14.317821063276353,
  36.05551275463989,
  35.35533905932738,
  39.20459156782532,
  47.07440918375928,
  51.478150704935004,
  31.400636936215164,
  35.34119409414458,
  48.91829923454004,
  32.2490309931942,
  18.35755975068582,
  24.186773244895647,
  41.677331968349414,
  42.5205832509386,
  36.49657518178932,
  15.652475842498529,
  8.06225774829855,
  20.615528128088304,
  32.526911934581186,
  45.880278987817846,
  28.231188426986208,
  28.653097563788805,
  6.4031242374328485,
  22.80350850198276,
  14.317821063276353,
  56.00892785976178,
  33.95585369269929,
  9.848857801796104,
  28.284271247461902,
  39.05124837953327,
  26.0,
  30.4138126514911,
  49.040799340956916,
  40.0,
  30.0,
  18.24828759089466,
  16.15549442140351,
  14.035668847618199,
  29.410882339705484,
  22.360679774997898,
  30.4138126514911,
  20.615528128088304,
  18.027756377319946,
  25.45584412271571,
  39.0,
  19.026297590440446,
  30.0,
  44.654227123532216,
  42.5440947723653,
  53.85164807134504,
  24.515301344262525,
  35.6931365951495,
  41.23105625617661,
  59.464274989274024,
  37.57658845611187,
  15.231546211727817,
  0.0,
  14.142135623730951,
  21.213203435596427,
  53.85164807134504,
  58.59180830116101,
  26.627053911388696,
  15.264337522473747,
  25.612496949731394,
  37.94733192202055,
  32.64965543462902,
  40.311288741492746,
  5.0,
  10.0,
  19.72308292331602],
 [22.360679774997898,
  8.246211251235321,
  16.492422502470642,
  15.033296378372908,
  25.495097567963924,
  36.40054944640259,
  13.601470508735444,
  36.05551275463989,
  38.07886552931954,
  29.614185789921695,
  41.182520563948,
  49.49747468305833,
  25.80697580112788,
  42.2965719651132,
  49.92995093127971,
  42.42640687119285,
  6.082762530298219,
  19.1049731745428,
  28.231188426986208,
  45.69463863518345,
  48.703182647543684,
  29.410882339705484,
  18.027756377319946,
  9.219544457292887,
  18.384776310850235,
  34.132096331752024,
  26.40075756488817,
  35.22782990761707,
  20.518284528683193,
  32.2490309931942,
  23.345235059857504,
  47.29693436154187,
  23.08679276123039,
  6.082762530298219,
  31.622776601683793,
  40.311288741492746,
  39.44616584663204,
  42.720018726587654,
  45.0,
  31.622776601683793,
  22.360679774997898,
  15.264337522473747,
  16.76305461424021,
  11.704699910719626,
  18.027756377319946,
  30.0,
  33.24154027718932,
  33.54101966249684,
  29.410882339705484,
  11.313708498984761,
  26.476404589747453,
  12.727922061357855,
  34.92849839314596,
  44.204072210600685,
  48.27007354458868,
  41.23105625617661,
  14.866068747318506,
  45.09988913511872,
  36.05551275463989,
  60.4648658313239,
  51.10772935672255,
  28.844410203711913,
  14.142135623730951,
  0.0,
  25.495097567963924,
  50.0,
  55.97320787662612,
  27.730849247724095,
  18.24828759089466,
  39.698866482558415,
  50.99019513592785,
  46.32493928760188,
  33.54101966249684,
  9.219544457292887,
  22.360679774997898,
  21.18962010041709],
 [43.01162633521314,
  18.384776310850235,
  29.698484809834994,
  40.44749683231337,
  42.42640687119285,
  42.720018726587654,
  34.132096331752024,
  57.0087712549569,
  56.568542494923804,
  55.10898293381942,
  65.92419889539804,
  72.11102550927978,
  50.15974481593781,
  55.758407437802596,
  70.09279563550022,
  50.695167422546305,
  31.575306807693888,
  42.95346318982906,
  51.35172830587107,
  63.702433234531945,
  51.78802950489621,
  30.083217912982647,
  13.601470508735444,
  23.345235059857504,
  38.8329756778952,
  59.033888572581766,
  48.54894437575342,
  49.4064773081425,
  22.825424421026653,
  42.5440947723653,
  35.0,
  72.78049189171504,
  48.507731342539614,
  26.40075756488817,
  49.49747468305833,
  60.207972893961475,
  39.319206502675,
  46.09772228646444,
  68.8839603971781,
  57.0087712549569,
  47.43416490252569,
  12.36931687685298,
  9.0,
  14.035668847618199,
  43.41658669218482,
  43.01162633521314,
  51.62363799656123,
  36.40054944640259,
  36.6742416417845,
  33.13608305156178,
  51.0,
  36.76955262170047,
  51.088159097779204,
  65.60487786742691,
  63.245553203367585,
  65.19202405202648,
  21.93171219946131,
  54.62600113499065,
  60.41522986797286,
  80.65977932030313,
  49.01020301937138,
  21.02379604162864,
  21.213203435596427,
  25.495097567963924,
  0.0,
  73.824115301167,
  79.07591289387686,
  47.634021455258214,
  36.235341863986875,
  35.014282800023196,
  51.088159097779204,
  44.0,
  58.52349955359813,
  22.47220505424423,
  29.154759474226502,
  40.607881008493905],
 [31.622776601683793,
  55.569775957799216,
  46.17358552246078,
  38.28837943815329,
  35.35533905932738,
  50.24937810560445,
  39.81205847478876,
  20.0,
  25.495097567963924,
  26.40075756488817,
  10.770329614269007,
  7.0710678118654755,
  24.20743687382041,
  37.0,
  17.69180601295413,
  47.53945729601885,
  44.68780594300866,
  31.064449134018133,
  43.32435804486894,
  25.059928172283335,
  58.240879114244144,
  57.3149195236284,
  61.84658438426491,
  57.3149195236284,
  50.774009099144415,
  33.54101966249684,
  25.632011235952593,
  36.345563690772494,
  56.04462507680822,
  44.04543109109048,
  44.553338819890925,
  19.924858845171276,
  32.7566787083184,
  47.50789408087881,
  30.0,
  20.615528128088304,
  60.13318551349163,
  55.90169943749474,
  5.0,
  22.360679774997898,
  28.284271247461902,
  65.21502894272147,
  66.03786792439622,
  61.29437168288782,
  36.40054944640259,
  40.0,
  28.0178514522438,
  55.0,
  49.040799340956916,
  49.678969393496885,
  37.69615364994153,
  37.44329045369811,
  32.2490309931942,
  13.92838827718412,
  33.61547262794322,
  40.0,
  62.93647591023825,
  45.5411901469428,
  14.142135623730951,
  20.8806130178211,
  67.468511173732,
  65.5133574166368,
  53.85164807134504,
  50.0,
  73.824115301167,
  0.0,
  7.280109889280518,
  28.442925306655784,
  38.897300677553446,
  66.0,
  64.03124237432849,
  65.62011886609167,
  18.027756377319946,
  51.42956348249516,
  50.99019513592785,
  34.48187929913333],
 [36.235341863986875,
  61.032778078668514,
  50.566787519082126,
  45.0,
  38.897300677553446,
  52.03844732503075,
  44.94441010848846,
  23.08679276123039,
  27.073972741361768,
  33.61547262794322,
  18.027756377319946,
  7.280109889280518,
  30.4138126514911,
  37.33630940518894,
  15.811388300841896,
  48.25971404805462,
  50.91168824543142,
  36.87817782917155,
  50.59644256269407,
  24.515301344262525,
  58.69412236331676,
  60.41522986797286,
  66.64833081180653,
  63.655321851358195,
  57.8013840664737,
  40.792156108742276,
  30.528675044947494,
  38.05259518088089,
  60.166435825965294,
  46.2709412050371,
  48.08326112068523,
  26.30589287593181,
  39.824615503479755,
  53.009433122794285,
  32.7566787083184,
  22.090722034374522,
  62.07253821135398,
  57.07889277132134,
  11.661903789690601,
  29.546573405388315,
  34.828149534535996,
  71.06335201775947,
  71.61005515987263,
  67.00746227100382,
  43.266615305567875,
  42.579337712087536,
  30.59411708155671,
  57.42821606144492,
  51.78802950489621,
  56.4003546088143,
  44.94441010848846,
  43.278170016764804,
  34.132096331752024,
  15.132745950421556,
  32.57299494980466,
  47.042533945356304,
  69.35416353759881,
  45.70557952810576,
  20.808652046684813,
  15.264337522473747,
  68.3593446428504,
  69.26037828369117,
  58.59180830116101,
  55.97320787662612,
  79.07591289387686,
  7.280109889280518,
  0.0,
  32.31098884280702,
  43.382023926967726,
  68.3593446428504,
  64.6297145282261,
  67.02984409947557,
  25.059928172283335,
  56.60388679233962,
  54.70831746635972,
  38.91015291668744],
 [5.385164807134504,
  30.479501308256342,
  18.35755975068582,
  24.186773244895647,
  7.0,
  24.166091947189145,
  14.422205101855956,
  9.433981132056603,
  10.44030650891055,
  25.495097567963924,
  24.515301344262525,
  25.079872407968907,
  13.45362404707371,
  18.384776310850235,
  22.67156809750927,
  24.758836806279895,
  24.73863375370596,
  12.165525060596439,
  38.62641583165593,
  18.027756377319946,
  35.0,
  29.017236257093817,
  34.66987164671943,
  36.76955262170047,
  38.01315561749642,
  35.608987629529715,
  5.656854249492381,
  12.806248474865697,
  27.85677655436824,
  17.46424919657298,
  16.1245154965971,
  36.76955262170047,
  25.495097567963924,
  23.021728866442675,
  5.385164807134504,
  12.806248474865697,
  33.24154027718932,
  30.886890422961002,
  24.08318915758459,
  23.430749027719962,
  17.0,
  41.400483088968905,
  41.036569057366385,
  37.013511046643494,
  24.73863375370596,
  13.0,
  6.0,
  27.459060435491963,
  21.213203435596427,
  33.24154027718932,
  33.1058907144937,
  16.278820596099706,
  9.848857801796104,
  18.027756377319946,
  22.02271554554524,
  44.82186966202994,
  42.5440947723653,
  25.0,
  18.681541692269406,
  33.301651610693426,
  42.485291572496,
  37.107950630558946,
  26.627053911388696,
  27.730849247724095,
  47.634021455258214,
  28.442925306655784,
  32.31098884280702,
  0.0,
  11.40175425099138,
  38.3275357934736,
  39.96248240537617,
  39.56008088970496,
  20.591260281974,
  25.45584412271571,
  22.561028345356956,
  7.0710678118654755],
 [7.280109889280518,
  19.4164878389476,
  7.280109889280518,
  20.808652046684813,
  7.280109889280518,
  21.400934559032695,
  5.0990195135927845,
  20.808652046684813,
  20.808652046684813,
  28.635642126552707,
  33.06055050963308,
  36.235341863986875,
  18.24828759089466,
  24.08318915758459,
  33.94112549695428,
  25.632011235952593,
  17.26267650163207,
  12.083045973594572,
  37.013511046643494,
  28.30194339616981,
  33.83784863137726,
  20.8806130178211,
  23.323807579381203,
  27.459060435491963,
  32.57299494980466,
  37.44329045369811,
  13.341664064126334,
  17.029386365926403,
  17.26267650163207,
  15.652475842498529,
  8.602325267042627,
  43.56604182158393,
  25.612496949731394,
  12.649110640673518,
  13.892443989449804,
  24.041630560342615,
  28.0178514522438,
  28.42534080710379,
  34.20526275297414,
  28.160255680657446,
  18.788294228055936,
  30.528675044947494,
  29.832867780352597,
  26.076809620810597,
  22.67156809750927,
  12.36931687685298,
  15.811388300841896,
  21.633307652783937,
  15.620499351813308,
  26.476404589747453,
  32.526911934581186,
  10.816653826391969,
  16.76305461424021,
  29.410882339705484,
  30.083217912982647,
  46.400431032480725,
  32.984845004941285,
  27.65863337187866,
  27.073972741361768,
  44.553338819890925,
  38.948684188300895,
  27.073972741361768,
  15.264337522473747,
  18.24828759089466,
  36.235341863986875,
  38.897300677553446,
  43.382023926967726,
  11.40175425099138,
  0.0,
  31.38470965295043,
  37.53664875824692,
  35.0,
  27.16615541441225,
  14.212670403551895,
  13.152946437965905,
  4.47213595499958],
 [37.36308338453881,
  33.28663395418648,
  26.076809620810597,
  50.21951811795888,
  31.400636936215164,
  16.76305461424021,
  34.48187929913333,
  46.0,
  41.30375285612676,
  60.00833275470999,
  62.80127387243033,
  61.204574992397426,
  49.33558553417604,
  33.24154027718932,
  54.3415126767741,
  23.323807579381203,
  43.18564576337837,
  43.46262762420146,
  66.21933252457322,
  45.60701700396552,
  18.867962264113206,
  10.816653826391969,
  25.632011235952593,
  45.79301256742124,
  58.05170109479997,
  68.47627326307996,
  42.95346318982906,
  30.4138126514911,
  19.209372712298546,
  22.090722034374522,
  23.08679276123039,
  74.46475676452586,
  56.43580423808985,
  34.655446902326915,
  36.0,
  46.2709412050371,
  7.211102550927978,
  14.866068747318506,
  62.07253821135398,
  59.464274989274024,
  50.15974481593781,
  40.162171256046406,
  36.40054944640259,
  37.16180835212409,
  52.69724850502159,
  26.0,
  38.01315561749642,
  11.0,
  17.11724276862369,
  50.99019513592785,
  62.68173577685928,
  40.8166632639171,
  34.23448553724738,
  53.23532661682466,
  40.718546143004666,
  77.17512552629896,
  48.75448697299562,
  27.459060435491963,
  56.88585061331157,
  63.245553203367585,
  14.0,
  14.142135623730951,
  25.612496949731394,
  39.698866482558415,
  35.014282800023196,
  66.0,
  68.3593446428504,
  38.3275357934736,
  31.38470965295043,
  0.0,
  16.492422502470642,
  9.055385138137417,
  57.97413216254298,
  30.479501308256342,
  18.867962264113206,
  34.48187929913333],
 [41.23105625617661,
  46.17358552246078,
  34.9857113690718,
  58.25804665451803,
  34.20526275297414,
  16.278820596099706,
  42.01190307520001,
  44.94441010848846,
  38.600518131237564,
  64.81512169239521,
  63.56099432828282,
  58.05170109479997,
  53.009433122794285,
  27.294688127912362,
  49.16299421312742,
  16.492422502470642,
  53.0,
  48.83646178829912,
  74.51845409024533,
  40.19950248448356,
  6.0,
  22.561028345356956,
  40.311288741492746,
  58.52349955359813,
  68.60029154456998,
  74.41102068914255,
  45.48626166217664,
  28.231188426986208,
  32.01562118716424,
  22.80350850198276,
  29.206163733020468,
  76.47875521999558,
  62.93647591023825,
  45.17742799230607,
  35.77708763999664,
  43.41658669218482,
  12.165525060596439,
  9.219544457292887,
  61.032778078668514,
  63.245553203367585,
  55.31726674375732,
  54.74486277268398,
  51.35172830587107,
  51.19570294468082,
  60.207972893961475,
  27.202941017470888,
  37.16180835212409,
  17.46424919657298,
  22.20360331117452,
  61.773780845922005,
  70.03570517957252,
  48.27007354458868,
  32.31098884280702,
  50.21951811795888,
  33.015148038438355,
  83.54639429682169,
  62.36184731067546,
  19.026297590440446,
  58.137767414994535,
  56.142675390472796,
  4.47213595499958,
  30.066592756745816,
  37.94733192202055,
  50.99019513592785,
  51.088159097779204,
  64.03124237432849,
  64.6297145282261,
  39.96248240537617,
  37.53664875824692,
  16.492422502470642,
  0.0,
  7.615773105863909,
  60.53924347066124,
  42.15447781671598,
  28.635642126552707,
  38.948684188300895],
 [39.824615503479755,
  40.718546143004666,
  31.144823004794873,
  55.17245689653489,
  33.1058907144937,
  15.524174696260024,
  38.948684188300895,
  45.89117562233506,
  40.19950248448356,
  63.28506932918696,
  63.89053137985315,
  60.13318551349163,
  51.884487084291386,
  30.14962686336267,
  52.08646657242167,
  19.235384061671343,
  49.040799340956916,
  46.87216658103186,
  71.42128534267638,
  43.104524124504614,
  11.40175425099138,
  17.0,
  33.95585369269929,
  53.225933528685054,
  64.40496875241847,
  72.42237223399962,
  44.77722635447622,
  29.274562336608895,
  26.40075756488817,
  22.135943621178654,
  26.40075756488817,
  76.32168761236873,
  60.60528029800704,
  40.80441152620633,
  36.138621999185304,
  45.17742799230607,
  7.0710678118654755,
  10.04987562112089,
  62.16912416947821,
  62.177166226839255,
  53.53503525729669,
  48.507731342539614,
  44.91102314577124,
  45.221676218380054,
  57.38466694161429,
  26.570660511172846,
  37.8549864614954,
  13.45362404707371,
  19.4164878389476,
  57.42821606144492,
  67.35725647619564,
  45.34313619501854,
  33.37663853655727,
  52.15361924162119,
  36.87817782917155,
  81.40024569987489,
  56.64803615307419,
  22.80350850198276,
  58.18934610390462,
  60.01666435249463,
  5.0990195135927845,
  23.021728866442675,
  32.64965543462902,
  46.32493928760188,
  44.0,
  65.62011886609167,
  67.02984409947557,
  39.56008088970496,
  35.0,
  9.055385138137417,
  7.615773105863909,
  0.0,
  60.00833275470999,
  37.21558813185679,
  24.20743687382041,
  37.21558813185679],
 [20.615528128088304,
  40.162171256046406,
  34.0147027033899,
  20.518284528683193,
  26.92582403567252,
  44.721359549995796,
  26.076809620810597,
  18.027756377319946,
  25.0,
  9.055385138137417,
  7.810249675906654,
  20.615528128088304,
  9.0,
  36.796738985948195,
  27.16615541441225,
  45.0,
  27.784887978899608,
  16.1245154965971,
  26.30589287593181,
  29.546573405388315,
  55.47071299343465,
  47.853944456021594,
  47.853944456021594,
  40.0,
  32.7566787083184,
  18.439088914585774,
  15.231546211727817,
  32.802438933713454,
  44.0,
  38.01315561749642,
  34.92849839314596,
  16.492422502470642,
  14.7648230602334,
  32.28002478313795,
  25.0,
  22.360679774997898,
  53.48831648126533,
  51.478150704935004,
  13.416407864998739,
  5.0,
  11.180339887498949,
  48.76474136094644,
  50.15974481593781,
  45.18849411078001,
  18.439088914585774,
  33.54101966249684,
  24.08318915758459,
  47.43416490252569,
  41.10960958218893,
  31.827660925679098,
  20.396078054371138,
  21.840329667841555,
  29.068883707497267,
  20.223748416156685,
  37.21558813185679,
  26.92582403567252,
  45.45327270945405,
  44.598206241955516,
  5.0,
  35.22782990761707,
  63.06346010171025,
  54.00925846556311,
  40.311288741492746,
  33.54101966249684,
  58.52349955359813,
  18.027756377319946,
  25.059928172283335,
  20.591260281974,
  27.16615541441225,
  57.97413216254298,
  60.53924347066124,
  60.00833275470999,
  0.0,
  36.87817782917155,
  40.311288741492746,
  23.53720459187964],
 [20.615528128088304,
  5.385164807134504,
  9.219544457292887,
  21.840329667841555,
  21.095023109728988,
  28.284271247461902,
  11.661903789690601,
  34.713109915419565,
  35.0,
  35.014282800023196,
  43.965895873961216,
  49.64876634922564,
  28.160255680657446,
  36.68787265568828,
  48.104053883222775,
  35.0,
  13.416407864998739,
  20.8806130178211,
  36.71511950137164,
  42.43819034784589,
  40.26164427839479,
  20.248456731316587,
  11.045361017187261,
  16.492422502470642,
  27.586228448267445,
  41.23105625617661,
  26.076809620810597,
  29.732137494637012,
  11.313708498984761,
  25.079872407968907,
  16.1245154965971,
  52.15361924162119,
  29.427877939124322,
  5.0990195135927845,
  28.0178514522438,
  38.2099463490856,
  30.4138126514911,
  34.20526275297414,
  46.51881339845203,
  36.124783736376884,
  26.1725046566048,
  16.55294535724685,
  15.620499351813308,
  12.083045973594572,
  24.73863375370596,
  23.769728648009426,
  30.0,
  24.698178070456937,
  21.213203435596427,
  20.518284528683193,
  34.17601498127012,
  15.132745950421556,
  30.4138126514911,
  43.278170016764804,
  43.46262762420146,
  49.040799340956916,
  21.02379604162864,
  38.118237105091836,
  38.27531841800928,
  58.728187440104094,
  42.05948168962618,
  20.12461179749811,
  5.0,
  9.219544457292887,
  22.47220505424423,
  51.42956348249516,
  56.60388679233962,
  25.45584412271571,
  14.212670403551895,
  30.479501308256342,
  42.15447781671598,
  37.21558813185679,
  36.87817782917155,
  0.0,
  13.601470508735444,
  18.384776310850235],
 [20.0,
  18.110770276274835,
  7.211102550927978,
  31.400636936215164,
  15.811388300841896,
  15.0,
  15.652475842498529,
  31.622776601683793,
  29.154759474226502,
  41.43669871020132,
  46.0,
  47.43416490252569,
  31.400636936215164,
  26.627053911388696,
  43.04648650006177,
  22.360679774997898,
  24.839484696748443,
  25.0,
  47.50789408087881,
  35.608987629529715,
  26.68332812825267,
  8.06225774829855,
  15.652475842498529,
  30.083217912982647,
  40.22437072223753,
  49.64876634922564,
  26.019223662515376,
  20.518284528683193,
  6.4031242374328485,
  13.416407864998739,
  6.708203932499369,
  56.71860364994893,
  37.589892258425,
  16.64331697709324,
  22.360679774997898,
  33.54101966249684,
  17.204650534085253,
  20.615528128088304,
  46.52956049652737,
  41.23105625617661,
  31.622776601683793,
  28.160255680657446,
  25.709920264364882,
  24.020824298928627,
  33.83784863137726,
  14.142135623730951,
  24.596747752497688,
  11.180339887498949,
  8.06225774829855,
  33.28663395418648,
  43.829214001622255,
  21.95449840010015,
  22.80350850198276,
  39.92492955535426,
  34.20526275297414,
  58.309518948453004,
  34.36568055487916,
  25.96150997149434,
  40.0,
  53.25410782277739,
  28.844410203711913,
  14.560219778561036,
  10.0,
  22.360679774997898,
  29.154759474226502,
  50.99019513592785,
  54.70831746635972,
  22.561028345356956,
  13.152946437965905,
  18.867962264113206,
  28.635642126552707,
  24.20743687382041,
  40.311288741492746,
  13.601470508735444,
  0.0,
  17.0],
 [3.0,
  23.430749027719962,
  11.704699910719626,
  20.615528128088304,
  5.385164807134504,
  22.67156809750927,
  7.615773105863909,
  16.401219466856727,
  17.0,
  26.076809620810597,
  29.0,
  31.76476034853718,
  14.866068747318506,
  22.090722034374522,
  29.732137494637012,
  25.553864678361276,
  19.026297590440446,
  9.899494936611665,
  36.359317925395686,
  24.596747752497688,
  34.713109915419565,
  24.331050121192877,
  27.784887978899608,
  30.364452901377952,
  33.54101966249684,
  35.4682957019364,
  9.055385138137417,
  15.297058540778355,
  21.587033144922902,
  16.278820596099706,
  11.40175425099138,
  40.024992192379,
  24.08318915758459,
  16.1245154965971,
  10.44030650891055,
  19.849433241279208,
  30.4138126514911,
  29.732137494637012,
  29.832867780352597,
  25.079872407968907,
  16.401219466856727,
  34.40930106817051,
  33.97057550292606,
  30.0,
  21.95449840010015,
  12.206555615733702,
  12.083045973594572,
  24.166091947189145,
  17.88854381999832,
  28.0178514522438,
  31.400636936215164,
  11.180339887498949,
  14.035668847618199,
  25.0,
  27.294688127912362,
  44.598206241955516,
  36.05551275463989,
  26.92582403567252,
  23.0,
  40.36087214122113,
  40.80441152620633,
  31.25699921617557,
  19.72308292331602,
  21.18962010041709,
  40.607881008493905,
  34.48187929913333,
  38.91015291668744,
  7.0710678118654755,
  4.47213595499958,
  34.48187929913333,
  38.948684188300895,
  37.21558813185679,
  23.53720459187964,
  18.384776310850235,
  17.0,
  0.0]]
    data['demands'] = [0, 
                         18,
 26,
 11,
 30,
 21,
 19,
 15,
 16,
 29,
 26,
 37,
 16,
 12,
 31,
 8,
 19,
 20,
 13,
 15,
 22,
 28,
 12,
 6,
 27,
 14,
 18,
 17,
 29,
 13,
 22,
 25,
 28,
 27,
 19,
 10,
 12,
 14,
 24,
 16,
 33,
 15,
 11,
 18,
 17,
 21,
 27,
 19,
 20,
 5,
 22,
 12,
 19,
 22,
 16,
 7,
 26,
 14,
 21,
 24,
 13,
 15,
 18,
 11,
 28,
 9,
 37,
 30,
 10,
 8,
 11,
 3,
 1,
 6,
 10,
 20]
    data['num_vehicles'] = 10
    data['vehicle_capacities'] = [140]*data['num_vehicles']
    data['depot'] = 0
    return data

In [3]:
def data_CMT3():
    data = {}
    data["distance_matrix"] = [[0.0,
  15.231546211727817,
  18.0,
  22.360679774997898,
  25.0,
  20.615528128088304,
  11.180339887498949,
  21.213203435596427,
  26.248809496813376,
  32.01562118716424,
  25.495097567963924,
  33.54101966249684,
  15.0,
  11.180339887498949,
  32.01562118716424,
  30.4138126514911,
  29.154759474226502,
  30.4138126514911,
  15.811388300841896,
  32.01562118716424,
  31.622776601683793,
  18.027756377319946,
  26.92582403567252,
  36.05551275463989,
  30.0,
  33.54101966249684,
  11.180339887498949,
  5.0,
  6.324555320336759,
  29.832867780352597,
  25.495097567963924,
  17.46424919657298,
  34.0,
  24.758836806279895,
  36.05551275463989,
  41.036569057366385,
  41.400483088968905,
  21.213203435596427,
  42.42640687119285,
  33.97057550292606,
  11.180339887498949,
  28.861739379323623,
  25.495097567963924,
  34.17601498127012,
  31.89043743820395,
  29.154759474226502,
  35.4682957019364,
  34.20526275297414,
  27.80287754891569,
  43.93176527297759,
  16.97056274847714,
  26.92582403567252,
  11.313708498984761,
  4.47213595499958,
  22.80350850198276,
  30.463092423455635,
  29.206163733020468,
  23.194827009486403,
  9.055385138137417,
  17.804493814764857,
  18.027756377319946,
  25.495097567963924,
  25.495097567963924,
  34.92849839314596,
  46.51881339845203,
  49.92995093127971,
  40.496913462633174,
  43.86342439892262,
  21.37755832643195,
  12.165525060596439,
  21.095023109728988,
  39.66106403010388,
  22.47220505424423,
  20.12461179749811,
  24.596747752497688,
  27.784887978899608,
  15.652475842498529,
  19.697715603592208,
  31.064449134018133,
  25.553864678361276,
  21.095023109728988,
  27.586228448267445,
  23.323807579381203,
  21.095023109728988,
  24.331050121192877,
  23.021728866442675,
  35.35533905932738,
  18.384776310850235,
  19.235384061671343,
  9.0,
  32.2490309931942,
  25.612496949731394,
  18.384776310850235,
  20.248456731316587,
  12.041594578792296,
  14.866068747318506,
  15.264337522473747,
  17.204650534085253,
  21.2602916254693,
  17.4928556845359,
  24.041630560342615],
 [15.231546211727817,
  0.0,
  32.55764119219941,
  14.560219778561036,
  32.202484376209235,
  32.202484376209235,
  24.839484696748443,
  21.02379604162864,
  31.575306807693888,
  17.804493814764857,
  15.556349186104045,
  26.40075756488817,
  16.64331697709324,
  26.40075756488817,
  46.87216658103186,
  45.35416188179427,
  42.44997055358225,
  40.70626487409524,
  22.847319317591726,
  28.231188426986208,
  16.492422502470642,
  29.274562336608895,
  39.20459156782532,
  46.17358552246078,
  27.784887978899608,
  37.64306044943742,
  19.4164878389476,
  10.816653826391969,
  12.0,
  24.041630560342615,
  11.045361017187261,
  10.44030650891055,
  20.8806130178211,
  12.36931687685298,
  24.73863375370596,
  27.202941017470888,
  40.52159917870962,
  35.805027579936315,
  56.85068161420758,
  41.593268686170845,
  24.020824298928627,
  42.01190307520001,
  40.718546143004666,
  49.39635614091387,
  46.09772228646444,
  36.68787265568828,
  39.01281840626232,
  33.734255586866,
  28.160255680657446,
  39.824615503479755,
  6.324555320336759,
  12.041594578792296,
  15.231546211727817,
  18.439088914585774,
  25.612496949731394,
  34.058772731852805,
  38.897300677553446,
  38.07886552931954,
  23.53720459187964,
  32.01562118716424,
  28.30194339616981,
  38.28837943815329,
  19.235384061671343,
  24.413111231467404,
  38.2099463490856,
  35.0,
  25.298221281347036,
  51.10772935672255,
  18.027756377319946,
  4.47213595499958,
  8.06225774829855,
  24.839484696748443,
  33.54101966249684,
  32.14031735997639,
  36.345563690772494,
  38.8329756778952,
  10.63014581273465,
  13.416407864998739,
  20.223748416156685,
  16.0312195418814,
  19.209372712298546,
  14.866068747318506,
  26.076809620810597,
  29.546573405388315,
  34.9857113690718,
  36.796738985948195,
  48.27007354458868,
  33.61547262794322,
  15.297058540778355,
  20.518284528683193,
  20.591260281974,
  39.698866482558415,
  33.015148038438355,
  33.97057550292606,
  26.627053911388696,
  29.68164415931166,
  29.068883707497267,
  32.2490309931942,
  35.608987629529715,
  31.144823004794873,
  38.600518131237564],
 [18.0,
  32.55764119219941,
  0.0,
  34.40930106817051,
  20.223748416156685,
  23.853720883753127,
  16.401219466856727,
  36.24913792078372,
  36.069377593742864,
  47.4236228055175,
  43.289721643826724,
  50.28916384272063,
  23.430749027719962,
  9.433981132056603,
  21.18962010041709,
  13.0,
  25.179356624028344,
  32.69556544854363,
  27.459060435491963,
  47.4236228055175,
  49.03060268852505,
  10.44030650891055,
  12.206555615733702,
  23.323807579381203,
  34.9857113690718,
  30.14962686336267,
  16.401219466856727,
  23.0,
  20.8806130178211,
  38.28837943815329,
  43.289721643826724,
  35.22782990761707,
  52.0,
  39.35733730830886,
  48.41487374764082,
  55.569775957799216,
  54.20332093147061,
  15.297058540778355,
  32.31098884280702,
  25.495097567963924,
  9.433981132056603,
  12.206555615733702,
  12.083045973594572,
  18.439088914585774,
  24.186773244895647,
  35.805027579936315,
  45.27692569068709,
  47.43416490252569,
  41.340053217188775,
  58.66856057549052,
  32.31098884280702,
  43.32435804486894,
  27.202941017470888,
  14.142135623730951,
  25.059928172283335,
  28.635642126552707,
  18.681541692269406,
  5.830951894845301,
  9.055385138137417,
  15.652475842498529,
  24.758836806279895,
  24.041630560342615,
  42.44997055358225,
  52.61178575186362,
  63.245553203367585,
  65.79513659838392,
  57.723478758647246,
  34.17601498127012,
  30.4138126514911,
  30.066592756745816,
  39.05124837953327,
  55.54277630799526,
  12.041594578792296,
  9.0,
  11.704699910719626,
  15.231546211727817,
  28.653097563788805,
  31.622776601683793,
  43.60045871318328,
  38.01315561749642,
  29.0,
  42.05948168962618,
  36.05551275463989,
  29.0,
  27.784887978899608,
  19.6468827043885,
  31.016124838541646,
  7.0710678118654755,
  36.138621999185304,
  20.12461179749811,
  50.15974481593781,
  20.09975124224178,
  13.92838827718412,
  18.384776310850235,
  13.45362404707371,
  12.206555615733702,
  16.401219466856727,
  10.770329614269007,
  16.492422502470642,
  17.4928556845359,
  17.029386365926403],
 [22.360679774997898,
  14.560219778561036,
  34.40930106817051,
  0.0,
  25.0,
  42.720018726587654,
  33.54101966249684,
  35.35533905932738,
  45.044422518220834,
  15.0,
  29.154759474226502,
  40.311288741492746,
  11.180339887498949,
  32.01562118716424,
  53.150729063673246,
  47.16990566028302,
  51.478150704935004,
  52.20153254455275,
  35.35533905932738,
  42.720018726587654,
  22.360679774997898,
  26.92582403567252,
  36.40054944640259,
  40.0,
  14.142135623730951,
  26.92582403567252,
  18.027756377319946,
  20.615528128088304,
  16.1245154965971,
  9.486832980505138,
  21.213203435596427,
  25.0,
  31.240998703626616,
  7.280109889280518,
  14.142135623730951,
  21.540659228538015,
  55.08175741568164,
  43.01162633521314,
  64.03124237432849,
  33.37663853655727,
  25.0,
  40.162171256046406,
  45.27692569068709,
  52.80151512977634,
  53.823786563191554,
  49.49747468305833,
  53.08483775994799,
  48.27007354458868,
  42.579337712087536,
  54.12947441089743,
  8.246211251235321,
  14.317821063276353,
  28.071337695236398,
  22.80350850198276,
  16.1245154965971,
  23.40939982143925,
  33.06055050963308,
  40.22437072223753,
  26.870057685088806,
  39.96248240537617,
  39.56008088970496,
  47.853944456021594,
  33.61547262794322,
  36.87817782917155,
  51.22499389946279,
  32.7566787083184,
  28.635642126552707,
  41.7612260356422,
  6.082762530298219,
  18.110770276274835,
  21.095023109728988,
  23.08679276123039,
  30.083217912982647,
  30.083217912982647,
  33.24154027718932,
  34.52535300326414,
  6.708203932499369,
  2.8284271247461903,
  9.219544457292887,
  3.605551275463989,
  8.06225774829855,
  9.0,
  40.049968789001575,
  41.773197148410844,
  46.17358552246078,
  45.27692569068709,
  57.706152185014034,
  38.18376618407357,
  29.832867780352597,
  30.675723300355934,
  32.55764119219941,
  47.70744176750625,
  40.22437072223753,
  42.5440947723653,
  34.132096331752024,
  36.61966684720111,
  37.589892258425,
  38.41874542459709,
  43.266615305567875,
  39.824615503479755,
  45.803929962395145],
 [25.0,
  32.202484376209235,
  20.223748416156685,
  25.0,
  0.0,
  41.23105625617661,
  31.622776601683793,
  46.09772228646444,
  50.53711507397311,
  40.0,
  47.16990566028302,
  57.0087712549569,
  15.811388300841896,
  25.495097567963924,
  41.23105625617661,
  29.154759474226502,
  45.0,
  50.99019513592785,
  40.311288741492746,
  56.568542494923804,
  46.09772228646444,
  10.0,
  14.142135623730951,
  15.0,
  18.027756377319946,
  10.0,
  14.142135623730951,
  28.284271247461902,
  22.02271554554524,
  23.769728648009426,
  42.720018726587654,
  40.0,
  52.92447448959697,
  32.0624390837628,
  36.40054944640259,
  45.70557952810576,
  66.40030120413611,
  35.0,
  52.20153254455275,
  9.433981132056603,
  15.811388300841896,
  18.384776310850235,
  32.01562118716424,
  36.235341863986875,
  44.40720662234904,
  52.20153254455275,
  59.941638282582836,
  59.20304046246274,
  52.80151512977634,
  68.59300255857006,
  28.160255680657446,
  38.47076812334269,
  36.235341863986875,
  21.095023109728988,
  9.219544457292887,
  8.54400374531753,
  8.246211251235321,
  24.351591323771842,
  19.924858845171276,
  34.23448553724738,
  40.496913462633174,
  43.18564576337837,
  49.040799340956916,
  56.43580423808985,
  69.63476143421474,
  57.42821606144492,
  53.33854141237835,
  19.209372712298546,
  19.026297590440446,
  32.449961479175904,
  40.24922359499622,
  48.041648597857254,
  8.94427190999916,
  11.40175425099138,
  11.40175425099138,
  10.816653826391969,
  22.80350850198276,
  23.08679276123039,
  32.55764119219941,
  28.071337695236398,
  17.029386365926403,
  34.0,
  48.25971404805462,
  44.384682042344295,
  45.35416188179427,
  39.05124837953327,
  51.03920062069938,
  27.073972741361768,
  43.18564576337837,
  32.64965543462902,
  52.773099207835045,
  40.01249804748511,
  33.06055050963308,
  37.21558813185679,
  29.832867780352597,
  30.265491900843113,
  33.734255586866,
  30.01666203960727,
  36.013886210738214,
  35.510561809129406,
  37.05401462729781],
 [20.615528128088304,
  32.202484376209235,
  23.853720883753127,
  42.720018726587654,
  41.23105625617661,
  0.0,
  10.0,
  20.615528128088304,
  13.92838827718412,
  50.0,
  33.54101966249684,
  35.35533905932738,
  35.35533905932738,
  15.811388300841896,
  20.0,
  29.154759474226502,
  11.180339887498949,
  10.0,
  11.180339887498949,
  30.0,
  46.09772228646444,
  31.622776601683793,
  36.05551275463989,
  47.16990566028302,
  50.24937810560445,
  50.99019513592785,
  30.0,
  22.360679774997898,
  26.92582403567252,
  50.44799302251776,
  39.05124837953327,
  27.202941017470888,
  43.829214001622255,
  43.9089968002003,
  55.90169943749474,
  59.405386961116584,
  32.69556544854363,
  11.180339887498949,
  26.92582403567252,
  48.46648326421054,
  25.495097567963924,
  35.4682957019364,
  20.12461179749811,
  28.160255680657446,
  16.492422502470642,
  12.041594578792296,
  22.20360331117452,
  26.92582403567252,
  22.090722034374522,
  39.05124837953327,
  36.235341863986875,
  44.04543109109048,
  17.69180601295413,
  22.02271554554524,
  42.01190307520001,
  48.507731342539614,
  42.04759208325728,
  24.758836806279895,
  21.37755832643195,
  8.48528137423857,
  4.47213595499958,
  6.708203932499369,
  29.410882339705484,
  40.80441152620633,
  47.0,
  66.46803743153546,
  54.817880294662984,
  57.697486947006624,
  41.97618372363071,
  27.80287754891569,
  34.058772731852805,
  56.639209034025185,
  34.92849839314596,
  31.78049716414141,
  35.35533905932738,
  38.948684188300895,
  36.05551275463989,
  40.162171256046406,
  50.99019513592785,
  45.69463863518345,
  41.593268686170845,
  46.647615158762406,
  17.0,
  7.0710678118654755,
  4.123105625617661,
  8.06225774829855,
  16.278820596099706,
  17.69180601295413,
  24.596747752497688,
  12.083045973594572,
  40.311288741492746,
  11.0,
  10.63014581273465,
  6.708203932499369,
  11.40175425099138,
  11.661903789690601,
  7.615773105863909,
  13.45362404707371,
  9.848857801796104,
  6.4031242374328485,
  12.36931687685298],
 [11.180339887498949,
  24.839484696748443,
  16.401219466856727,
  33.54101966249684,
  31.622776601683793,
  10.0,
  0.0,
  20.615528128088304,
  19.849433241279208,
  42.42640687119285,
  30.4138126514911,
  35.35533905932738,
  25.495097567963924,
  7.0710678118654755,
  22.360679774997898,
  25.495097567963924,
  18.027756377319946,
  20.0,
  11.180339887498949,
  31.622776601683793,
  40.311288741492746,
  22.360679774997898,
  28.284271247461902,
  39.05124837953327,
  40.311288741492746,
  41.23105625617661,
  20.0,
  14.142135623730951,
  17.46424919657298,
  40.80441152620633,
  33.54101966249684,
  22.80350850198276,
  40.26164427839479,
  35.608987629529715,
  47.16990566028302,
  51.66236541235796,
  37.8021163428716,
  11.180339887498949,
  32.01562118716424,
  39.35733730830886,
  15.811388300841896,
  28.600699292150182,
  18.027756377319946,
  27.073972741361768,
  21.2602916254693,
  20.615528128088304,
  29.206163733020468,
  31.064449134018133,
  25.059928172283335,
  42.485291572496,
  27.80287754891569,
  36.87817782917155,
  13.152946437965905,
  12.041594578792296,
  32.01562118716424,
  38.63935817272331,
  33.28663395418648,
  19.313207915827967,
  11.704699910719626,
  7.211102550927978,
  8.94427190999916,
  14.317821063276353,
  28.0178514522438,
  39.05124837953327,
  48.05205510693585,
  59.81638571495272,
  49.24428900898052,
  48.877397639399746,
  32.28002478313795,
  20.808652046684813,
  28.635642126552707,
  49.678969393496885,
  26.076809620810597,
  23.021728866442675,
  27.018512172212592,
  30.610455730027933,
  26.832815729997478,
  30.870698080866262,
  42.190046219457976,
  36.71511950137164,
  31.78049716414141,
  38.41874542459709,
  19.72308292331602,
  13.038404810405298,
  14.035668847618199,
  12.041594578792296,
  24.186773244895647,
  12.36931687685298,
  22.02271554554524,
  5.0990195135927845,
  37.48332962798263,
  14.866068747318506,
  8.54400374531753,
  9.219544457292887,
  3.1622776601683795,
  6.0,
  4.242640687119285,
  9.0,
  10.816653826391969,
  6.4031242374328485,
  13.892443989449804],
 [21.213203435596427,
  21.02379604162864,
  36.24913792078372,
  35.35533905932738,
  46.09772228646444,
  20.615528128088304,
  20.615528128088304,
  0.0,
  12.206555615733702,
  36.40054944640259,
  14.142135623730951,
  15.0,
  33.54101966249684,
  26.92582403567252,
  40.311288741492746,
  46.09772228646444,
  31.622776601683793,
  25.0,
  10.0,
  11.180339887498949,
  29.154759474226502,
  39.05124837953327,
  47.16990566028302,
  57.0087712549569,
  47.43416490252569,
  54.08326913195984,
  32.01562118716424,
  18.027756377319946,
  24.698178070456937,
  44.721359549995796,
  22.360679774997898,
  11.180339887498949,
  24.20743687382041,
  33.06055050963308,
  45.27692569068709,
  45.5411901469428,
  20.591260281974,
  30.0,
  47.43416490252569,
  55.17245689653489,
  32.01562118716424,
  48.30113870293329,
  38.2099463490856,
  47.095647357266465,
  37.107950630558946,
  18.439088914585774,
  18.110770276274835,
  13.416407864998739,
  7.280109889280518,
  22.80350850198276,
  27.16615541441225,
  30.083217912982647,
  9.899494936611665,
  25.495097567963924,
  42.5440947723653,
  50.774009099144415,
  50.32891812864648,
  39.84971769034255,
  28.844410203711913,
  26.019223662515376,
  16.278820596099706,
  27.202941017470888,
  8.94427190999916,
  20.248456731316587,
  27.459060435491963,
  49.92995093127971,
  37.013511046643494,
  65.06919393998976,
  37.64306044943742,
  17.26267650163207,
  18.027756377319946,
  41.14608122288197,
  43.41658669218482,
  40.80441152620633,
  45.221676218380054,
  48.60041152089147,
  30.083217912982647,
  33.734255586866,
  41.048751503547585,
  37.05401462729781,
  38.27531841800928,
  35.22782990761707,
  5.830951894845301,
  14.317821063276353,
  21.02379604162864,
  28.284271247461902,
  35.77708763999664,
  32.984845004941285,
  6.324555320336759,
  16.15549442140351,
  20.248456731316587,
  31.400636936215164,
  28.071337695236398,
  26.076809620810597,
  23.769728648009426,
  26.476404589747453,
  23.08679276123039,
  29.427877939124322,
  29.017236257093817,
  24.0,
  32.0624390837628],
 [26.248809496813376,
  31.575306807693888,
  36.069377593742864,
  45.044422518220834,
  50.53711507397311,
  13.92838827718412,
  19.849433241279208,
  12.206555615733702,
  0.0,
  48.104053883222775,
  26.248809496813376,
  24.166091947189145,
  40.792156108742276,
  26.90724809414742,
  33.37663853655727,
  42.941821107167776,
  23.0,
  13.92838827718412,
  10.44030650891055,
  17.72004514666935,
  41.340053217188775,
  41.88078318274385,
  48.104053883222775,
  58.89821729050889,
  55.57877292636101,
  59.61543424315552,
  37.33630940518894,
  25.179356624028344,
  31.575306807693888,
  54.00925846556311,
  34.48187929913333,
  22.847319317591726,
  36.069377593742864,
  43.93176527297759,
  56.293871780150276,
  57.38466694161429,
  18.788294228055936,
  25.079872407968907,
  38.3275357934736,
  58.83026432033091,
  34.9857113690718,
  48.16637831516918,
  34.0147027033899,
  42.05948168962618,
  29.017236257093817,
  6.4031242374328485,
  9.433981132056603,
  13.152946437965905,
  9.486832980505138,
  25.317977802344327,
  37.21558813185679,
  41.78516483155236,
  17.0,
  29.546573405388315,
  49.040799340956916,
  56.64803615307419,
  53.009433122794285,
  38.01315561749642,
  31.064449134018133,
  21.95449840010015,
  11.40175425099138,
  19.1049731745428,
  20.518284528683193,
  31.064449134018133,
  34.36568055487916,
  62.12889826803627,
  49.20365840057018,
  68.5054742338158,
  46.17358552246078,
  27.294688127912362,
  29.966648127543394,
  53.23532661682466,
  45.803929962395145,
  42.80186911806539,
  46.861498055439924,
  50.44799302251776,
  39.01281840626232,
  43.0,
  51.78802950489621,
  47.265209192385896,
  46.389654018972806,
  46.32493928760188,
  6.4031242374328485,
  7.211102550927978,
  12.041594578792296,
  21.840329667841555,
  25.709920264364882,
  30.805843601498726,
  18.35755975068582,
  17.88854381999832,
  31.89043743820395,
  24.515301344262525,
  24.186773244895647,
  20.615528128088304,
  22.627416997969522,
  24.20743687382041,
  20.0,
  26.627053911388696,
  23.769728648009426,
  19.72308292331602,
  26.248809496813376],
 [32.01562118716424,
  17.804493814764857,
  47.4236228055175,
  15.0,
  40.0,
  50.0,
  42.42640687119285,
  36.40054944640259,
  48.104053883222775,
  0.0,
  25.0,
  35.35533905932738,
  25.495097567963924,
  43.01162633521314,
  64.03124237432849,
  60.41522986797286,
  60.207972893961475,
  58.309518948453004,
  40.311288741492746,
  40.0,
  11.180339887498949,
  41.23105625617661,
  50.99019513592785,
  55.0,
  26.92582403567252,
  41.23105625617661,
  31.622776601683793,
  28.284271247461902,
  26.92582403567252,
  20.12461179749811,
  15.0,
  25.298221281347036,
  21.93171219946131,
  8.246211251235321,
  11.180339887498949,
  9.433981132056603,
  53.0,
  53.150729063673246,
  74.33034373659252,
  48.25971404805462,
  38.07886552931954,
  54.57105459856901,
  57.14017850864661,
  65.36818798161687,
  63.655321851358195,
  53.71219600798314,
  54.3415126767741,
  47.16990566028302,
  42.7551166528639,
  49.64876634922564,
  15.264337522473747,
  6.324555320336759,
  32.7566787083184,
  34.132096331752024,
  31.064449134018133,
  37.8549864614954,
  48.041648597857254,
  53.225933528685054,
  38.948684188300895,
  49.51767361255979,
  46.04345773288535,
  56.08029957123981,
  31.064449134018133,
  29.410882339705484,
  43.46262762420146,
  18.384776310850235,
  14.317821063276353,
  56.293871780150276,
  21.02379604162864,
  22.20360331117452,
  18.439088914585774,
  8.246211251235321,
  44.721359549995796,
  44.384682042344295,
  47.853944456021594,
  49.36598018878993,
  18.973665961010276,
  17.11724276862369,
  10.0,
  12.165525060596439,
  23.021728866442675,
  6.0,
  42.05948168962618,
  47.01063709417264,
  52.69724850502159,
  54.45181356024793,
  66.06814663663572,
  49.92995093127971,
  30.083217912982647,
  38.28837943815329,
  25.0,
  57.28001396647874,
  50.32891812864648,
  51.62363799656123,
  43.93176527297759,
  46.861498055439924,
  46.66904755831214,
  49.20365840057018,
  53.075418038862395,
  48.79549159502341,
  55.97320787662612],
 [25.495097567963924,
  15.556349186104045,
  43.289721643826724,
  29.154759474226502,
  47.16990566028302,
  33.54101966249684,
  30.4138126514911,
  14.142135623730951,
  26.248809496813376,
  25.0,
  0.0,
  11.180339887498949,
  32.01562118716424,
  35.0,
  52.20153254455275,
  55.0,
  44.721359549995796,
  39.05124837953327,
  22.360679774997898,
  15.0,
  15.811388300841896,
  42.720018726587654,
  52.20153254455275,
  60.41522986797286,
  43.01162633521314,
  53.150729063673246,
  33.54101966249684,
  20.615528128088304,
  25.495097567963924,
  38.47076812334269,
  10.0,
  8.06225774829855,
  10.295630140987,
  24.351591323771842,
  35.35533905932738,
  33.37663853655727,
  28.0,
  41.23105625617661,
  60.41522986797286,
  56.60388679233962,
  36.40054944640259,
  54.3415126767741,
  48.373546489791295,
  57.42821606144492,
  49.76946855251722,
  32.55764119219941,
  30.463092423455635,
  22.360679774997898,
  18.788294228055936,
  25.298221281347036,
  21.400934559032695,
  19.1049731745428,
  17.26267650163207,
  29.832867780352597,
  41.10960958218893,
  49.57822102496216,
  53.225933528685054,
  48.041648597857254,
  34.52535300326414,
  37.107950630558946,
  29.068883707497267,
  40.24922359499622,
  6.324555320336759,
  9.486832980505138,
  22.67156809750927,
  36.235341863986875,
  23.021728866442675,
  66.2872536767062,
  33.421549934136806,
  14.7648230602334,
  8.06225774829855,
  28.160255680657446,
  47.16990566028302,
  45.221676218380054,
  49.64876634922564,
  52.55473337388365,
  26.1725046566048,
  28.600699292150182,
  32.01562118716424,
  29.546573405388315,
  34.713109915419565,
  25.709920264364882,
  19.849433241279208,
  28.0178514522438,
  34.66987164671943,
  40.496913462633174,
  49.39635614091387,
  42.04759208325728,
  8.94427190999916,
  25.317977802344327,
  7.0710678118654755,
  43.657759905886145,
  38.8329756778952,
  37.94733192202055,
  33.24154027718932,
  36.345563690772494,
  33.95585369269929,
  39.319206502675,
  40.52159917870962,
  35.4400902933387,
  43.68065933568311],
 [33.54101966249684,
  26.40075756488817,
  50.28916384272063,
  40.311288741492746,
  57.0087712549569,
  35.35533905932738,
  35.35533905932738,
  15.0,
  24.166091947189145,
  35.35533905932738,
  11.180339887498949,
  0.0,
  42.42640687119285,
  41.23105625617661,
  55.226805085936306,
  60.8276253029822,
  46.09772228646444,
  38.07886552931954,
  25.0,
  7.0710678118654755,
  25.0,
  51.478150704935004,
  60.41522986797286,
  69.46221994724903,
  54.08326913195984,
  63.63961030678928,
  43.01162633521314,
  29.154759474226502,
  35.0,
  49.64876634922564,
  20.615528128088304,
  17.029386365926403,
  15.524174696260024,
  35.4682957019364,
  46.09772228646444,
  43.0,
  18.681541692269406,
  45.0,
  61.84658438426491,
  66.40030120413611,
  44.721359549995796,
  62.03224967708329,
  53.150729063673246,
  62.07253821135398,
  51.78802950489621,
  30.4138126514911,
  24.758836806279895,
  15.0,
  14.7648230602334,
  14.317821063276353,
  32.449961479175904,
  29.832867780352597,
  23.08679276123039,
  38.01315561749642,
  51.62363799656123,
  60.108235708594876,
  62.433965115151864,
  54.3415126767741,
  42.15447781671598,
  41.012193308819754,
  31.144823004794873,
  41.773197148410844,
  8.06225774829855,
  8.06225774829855,
  13.0,
  43.68065933568311,
  30.083217912982647,
  76.21679604916491,
  44.40720662234904,
  24.758836806279895,
  19.235384061671343,
  37.12142238654117,
  55.94640292279746,
  53.665631459994955,
  58.137767414994535,
  61.29437168288782,
  37.013511046643494,
  39.66106403010388,
  43.01162633521314,
  40.718546143004666,
  45.60701700396552,
  36.68787265568828,
  18.681541692269406,
  28.635642126552707,
  35.17101079013795,
  43.18564576337837,
  49.64876634922564,
  47.67598976424087,
  14.317821063276353,
  30.59411708155671,
  11.180339887498949,
  46.2709412050371,
  43.04648650006177,
  41.048751503547585,
  38.47076812334269,
  41.30375285612676,
  38.05259518088089,
  44.28317965096906,
  44.01136216933077,
  39.0,
  47.042533945356304],
 [15.0,
  16.64331697709324,
  23.430749027719962,
  11.180339887498949,
  15.811388300841896,
  35.35533905932738,
  25.495097567963924,
  33.54101966249684,
  40.792156108742276,
  25.495097567963924,
  32.01562118716424,
  42.42640687119285,
  0.0,
  22.360679774997898,
  43.01162633521314,
  36.05551275463989,
  42.720018726587654,
  45.27692569068709,
  30.4138126514911,
  43.01162633521314,
  30.4138126514911,
  15.811388300841896,
  25.495097567963924,
  30.4138126514911,
  15.0,
  21.213203435596427,
  7.0710678118654755,
  15.811388300841896,
  9.219544457292887,
  15.652475842498529,
  26.92582403567252,
  25.495097567963924,
  37.16180835212409,
  17.26267650163207,
  25.0,
  32.69556544854363,
  54.120236510939236,
  33.54101966249684,
  54.08326913195984,
  25.079872407968907,
  14.142135623730951,
  29.120439557122072,
  34.713109915419565,
  41.86884283091664,
  44.294469180700204,
  44.10215414239989,
  49.72926703662542,
  46.95742752749558,
  40.718546143004666,
  55.0,
  12.36931687685298,
  23.021728866442675,
  24.351591323771842,
  13.601470508735444,
  9.219544457292887,
  17.69180601295413,
  23.194827009486403,
  29.206163733020468,
  16.64331697709324,
  31.016124838541646,
  33.015148038438355,
  39.56008088970496,
  34.713109915419565,
  41.048751503547585,
  54.67174773134658,
  43.68065933568311,
  38.01315561749642,
  34.48187929913333,
  7.211102550927978,
  17.69180601295413,
  24.698178070456937,
  33.734255586866,
  19.235384061671343,
  18.973665961010276,
  22.360679774997898,
  24.020824298928627,
  7.0710678118654755,
  8.54400374531753,
  20.248456731316587,
  14.7648230602334,
  6.324555320336759,
  19.6468827043885,
  37.0,
  36.05551275463989,
  39.20459156782532,
  36.40054944640259,
  49.040799340956916,
  27.80287754891569,
  29.410882339705484,
  24.0,
  37.21558813185679,
  38.48376280978771,
  30.870698080866262,
  33.83784863137726,
  25.298221281347036,
  27.313000567495326,
  29.120439557122072,
  28.653097563788805,
  34.0147027033899,
  31.32091952673165,
  36.235341863986875],
 [11.180339887498949,
  26.40075756488817,
  9.433981132056603,
  32.01562118716424,
  25.495097567963924,
  15.811388300841896,
  7.0710678118654755,
  26.92582403567252,
  26.90724809414742,
  43.01162633521314,
  35.0,
  41.23105625617661,
  22.360679774997898,
  0.0,
  21.213203435596427,
  20.0,
  20.615528128088304,
  25.495097567963924,
  18.027756377319946,
  38.07886552931954,
  42.720018726587654,
  15.811388300841896,
  21.213203435596427,
  32.01562118716424,
  36.40054944640259,
  35.35533905932738,
  15.811388300841896,
  15.811388300841896,
  16.278820596099706,
  38.01315561749642,
  36.40054944640259,
  27.018512172212592,
  44.28317965096906,
  35.4682957019364,
  46.09772228646444,
  51.85556864985669,
  44.82186966202994,
  11.180339887498949,
  32.01562118716424,
  32.69556544854363,
  10.0,
  21.633307652783937,
  14.317821063276353,
  23.08679276123039,
  21.95449840010015,
  27.294688127912362,
  36.235341863986875,
  38.01315561749642,
  31.906112267087632,
  49.24428900898052,
  27.80287754891569,
  38.07886552931954,
  18.24828759089466,
  9.219544457292887,
  27.294688127912362,
  33.06055050963308,
  26.419689627245813,
  13.152946437965905,
  6.082762530298219,
  9.055385138137417,
  15.811388300841896,
  18.027756377319946,
  33.54101966249684,
  44.10215414239989,
  54.120236510939236,
  61.05735008989499,
  51.62363799656123,
  42.05948168962618,
  29.5296461204668,
  23.08679276123039,
  31.78049716414141,
  50.774009099144415,
  19.235384061671343,
  16.1245154965971,
  20.0,
  23.600847442411894,
  25.495097567963924,
  29.206163733020468,
  41.10960958218893,
  35.4682957019364,
  28.635642126552707,
  38.28837943815329,
  26.627053911388696,
  20.0,
  19.924858845171276,
  14.317821063276353,
  26.92582403567252,
  7.280109889280518,
  27.294688127912362,
  10.770329614269007,
  42.01190307520001,
  16.15549442140351,
  8.54400374531753,
  12.041594578792296,
  4.47213595499958,
  5.0990195135927845,
  8.246211251235321,
  6.4031242374328485,
  11.704699910719626,
  10.04987562112089,
  13.892443989449804],
 [32.01562118716424,
  46.87216658103186,
  21.18962010041709,
  53.150729063673246,
  41.23105625617661,
  20.0,
  22.360679774997898,
  40.311288741492746,
  33.37663853655727,
  64.03124237432849,
  52.20153254455275,
  55.226805085936306,
  43.01162633521314,
  21.213203435596427,
  0.0,
  15.811388300841896,
  11.180339887498949,
  22.360679774997898,
  30.4138126514911,
  50.0,
  62.64982043070834,
  31.622776601683793,
  30.0,
  40.311288741492746,
  55.90169943749474,
  50.99019513592785,
  36.05551275463989,
  36.05551275463989,
  37.48332962798263,
  58.52349955359813,
  55.90169943749474,
  44.94441010848846,
  62.297672508690084,
  56.639209034025185,
  67.26812023536856,
  73.0,
  51.66236541235796,
  11.180339887498949,
  11.180339887498949,
  45.044422518220834,
  29.154759474226502,
  27.16615541441225,
  9.219544457292887,
  10.63014581273465,
  5.656854249492381,
  29.410882339705484,
  40.162171256046406,
  46.52956049652737,
  42.04759208325728,
  58.69412236331676,
  48.91829923454004,
  58.82176467941097,
  35.11409973215888,
  30.4138126514911,
  46.09772228646444,
  49.72926703662542,
  38.05259518088089,
  17.11724276862369,
  26.40075756488817,
  15.231546211727817,
  24.08318915758459,
  14.317821063276353,
  48.83646178829912,
  60.207972893961475,
  67.0,
  81.8413098624405,
  71.58910531638176,
  52.23983154643591,
  50.21951811795888,
  43.04648650006177,
  50.99019513592785,
  71.61005515987263,
  32.55764119219941,
  29.832867780352597,
  31.144823004794873,
  34.0147027033899,
  46.69047011971501,
  50.32891812864648,
  62.289646009589745,
  56.639209034025185,
  49.09175083453431,
  59.464274989274024,
  37.0,
  27.018512172212592,
  21.37755832643195,
  12.041594578792296,
  13.601470508735444,
  15.264337522473747,
  43.41658669218482,
  27.313000567495326,
  59.20304046246274,
  9.0,
  13.892443989449804,
  14.317821063276353,
  20.248456731316587,
  17.204650534085253,
  18.384776310850235,
  14.866068747318506,
  11.704699910719626,
  16.76305461424021,
  8.54400374531753],
 [30.4138126514911,
  45.35416188179427,
  13.0,
  47.16990566028302,
  29.154759474226502,
  29.154759474226502,
  25.495097567963924,
  46.09772228646444,
  42.941821107167776,
  60.41522986797286,
  55.0,
  60.8276253029822,
  36.05551275463989,
  20.0,
  15.811388300841896,
  0.0,
  25.0,
  35.35533905932738,
  36.40054944640259,
  57.0087712549569,
  61.84658438426491,
  21.213203435596427,
  15.811388300841896,
  25.0,
  46.09772228646444,
  38.07886552931954,
  29.154759474226502,
  35.35533905932738,
  33.83784863137726,
  50.24937810560445,
  55.90169943749474,
  47.01063709417264,
  64.19501538281614,
  52.32590180780452,
  61.032778078668514,
  68.47627326307996,
  61.71709649683789,
  18.027756377319946,
  25.0,
  30.805843601498726,
  22.360679774997898,
  12.165525060596439,
  9.219544457292887,
  7.280109889280518,
  21.02379604162864,
  40.80441152620633,
  51.31276644267,
  55.54277630799526,
  49.9799959983992,
  67.4166151627327,
  45.31004303683677,
  56.302753041036986,
  38.118237105091836,
  26.92582403567252,
  36.124783736376884,
  37.589892258425,
  24.041630560342615,
  7.280109889280518,
  21.840329667841555,
  21.02379604162864,
  31.78049716414141,
  26.1725046566048,
  53.33854141237835,
  64.07027391856539,
  73.54590403278758,
  78.79086241436883,
  70.60453243241541,
  37.0,
  42.80186911806539,
  42.579337712087536,
  51.478150704935004,
  68.54195795277518,
  20.248456731316587,
  18.439088914585774,
  17.88854381999832,
  19.924858845171276,
  41.593268686170845,
  44.41846462902562,
  56.302753041036986,
  50.774009099144415,
  41.23105625617661,
  55.00909015790027,
  44.598206241955516,
  35.77708763999664,
  32.202484376209235,
  22.02271554554524,
  29.068883707497267,
  13.152946437965905,
  47.16990566028302,
  30.265491900843113,
  62.00806399170998,
  20.518284528683193,
  18.788294228055936,
  22.47220505424423,
  22.360679774997898,
  19.6468827043885,
  23.40939982143925,
  16.76305461424021,
  19.4164878389476,
  23.259406699226016,
  17.69180601295413],
 [29.154759474226502,
  42.44997055358225,
  25.179356624028344,
  51.478150704935004,
  45.0,
  11.180339887498949,
  18.027756377319946,
  31.622776601683793,
  23.0,
  60.207972893961475,
  44.721359549995796,
  46.09772228646444,
  42.720018726587654,
  20.615528128088304,
  11.180339887498949,
  25.0,
  0.0,
  11.180339887498949,
  22.360679774997898,
  40.311288741492746,
  57.0087712549569,
  35.0,
  36.40054944640259,
  47.43416490252569,
  57.0087712549569,
  55.0,
  36.40054944640259,
  32.01562118716424,
  35.35533905932738,
  58.309518948453004,
  50.0,
  38.27531841800928,
  55.00909015790027,
  53.600373133029585,
  65.19202405202648,
  69.52697318307479,
  40.792156108742276,
  10.0,
  15.811388300841896,
  50.635955604688654,
  30.4138126514911,
  34.539832078341085,
  16.1245154965971,
  21.400934559032695,
  6.082762530298219,
  18.439088914585774,
  29.120439557122072,
  36.05551275463989,
  32.14031735997639,
  48.16637831516918,
  45.803929962395145,
  54.45181356024793,
  28.600699292150182,
  29.154759474226502,
  47.853944456021594,
  53.08483775994799,
  43.73785545725808,
  23.40939982143925,
  26.68332812825267,
  11.704699910719626,
  15.652475842498529,
  4.47213595499958,
  40.496913462633174,
  51.86520991955976,
  57.21887800367987,
  77.15568676384133,
  65.80273550544841,
  58.9406481131655,
  49.76946855251722,
  38.18376618407357,
  45.0,
  67.17886572427373,
  37.21558813185679,
  34.132096331752024,
  36.6742416417845,
  40.024992192379,
  44.77722635447622,
  48.76474136094644,
  60.207972893961475,
  54.70831746635972,
  49.040799340956916,
  56.4003546088143,
  27.459060435491963,
  17.46424919657298,
  11.045361017187261,
  6.324555320336759,
  6.324555320336759,
  18.110770276274835,
  35.77708763999664,
  21.93171219946131,
  51.478150704935004,
  5.0990195135927845,
  12.165525060596439,
  8.94427190999916,
  17.46424919657298,
  15.524174696260024,
  13.892443989449804,
  15.033296378372908,
  9.055385138137417,
  11.661903789690601,
  8.246211251235321],
 [30.4138126514911,
  40.70626487409524,
  32.69556544854363,
  52.20153254455275,
  50.99019513592785,
  10.0,
  20.0,
  25.0,
  13.92838827718412,
  58.309518948453004,
  39.05124837953327,
  38.07886552931954,
  45.27692569068709,
  25.495097567963924,
  22.360679774997898,
  35.35533905932738,
  11.180339887498949,
  0.0,
  18.027756377319946,
  31.622776601683793,
  53.150729063673246,
  41.23105625617661,
  44.721359549995796,
  55.90169943749474,
  60.207972893961475,
  60.8276253029822,
  40.0,
  31.622776601683793,
  36.6742416417845,
  60.207972893961475,
  46.09772228646444,
  34.058772731852805,
  49.20365840057018,
  52.80151512977634,
  65.0,
  67.7421582177598,
  30.14962686336267,
  18.027756377319946,
  25.0,
  57.87054518492115,
  35.35533905932738,
  43.56604182158393,
  26.1725046566048,
  32.449961479175904,
  17.08800749063506,
  8.06225774829855,
  18.24828759089466,
  26.1725046566048,
  23.40939982143925,
  38.01315561749642,
  45.31004303683677,
  52.15361924162119,
  25.553864678361276,
  32.01562118716424,
  52.009614495783374,
  58.42088667591412,
  51.264022471905186,
  32.449961479175904,
  31.25699921617557,
  17.08800749063506,
  12.649110640673518,
  9.219544457292887,
  33.83784863137726,
  44.77722635447622,
  48.05205510693585,
  73.87827826905551,
  61.5223536610881,
  66.85057965343307,
  51.78802950489621,
  36.235341863986875,
  41.23105625617661,
  64.40496875241847,
  44.27188724235731,
  41.10960958218893,
  44.384682042344295,
  47.92702786528704,
  45.60701700396552,
  49.72926703662542,
  60.166435825965294,
  55.02726596879042,
  51.478150704935004,
  55.46169849544819,
  19.72308292331602,
  11.40175425099138,
  6.082762530298219,
  13.601470508735444,
  12.041594578792296,
  25.942243542145693,
  30.4138126514911,
  21.587033144922902,
  45.221676218380054,
  14.866068747318506,
  18.788294228055936,
  14.317821063276353,
  21.213203435596427,
  20.8806130178211,
  17.26267650163207,
  21.93171219946131,
  16.64331697709324,
  15.524174696260024,
  17.69180601295413],
 [15.811388300841896,
  22.847319317591726,
  27.459060435491963,
  35.35533905932738,
  40.311288741492746,
  11.180339887498949,
  11.180339887498949,
  10.0,
  10.44030650891055,
  40.311288741492746,
  22.360679774997898,
  25.0,
  30.4138126514911,
  18.027756377319946,
  30.4138126514911,
  36.40054944640259,
  22.360679774997898,
  18.027756377319946,
  0.0,
  20.615528128088304,
  35.35533905932738,
  32.01562118716424,
  39.05124837953327,
  49.49747468305833,
  45.27692569068709,
  49.24428900898052,
  26.92582403567252,
  15.0,
  21.213203435596427,
  44.04543109109048,
  28.284271247461902,
  16.278820596099706,
  32.64965543462902,
  35.11409973215888,
  47.43416490252569,
  49.73932046178355,
  26.90724809414742,
  20.0,
  38.07886552931954,
  48.82622246293481,
  25.0,
  39.66106403010388,
  28.284271247461902,
  37.12142238654117,
  27.51363298439521,
  14.142135623730951,
  19.697715603592208,
  20.0,
  13.892443989449804,
  31.304951684997057,
  27.892651361962706,
  34.132096331752024,
  7.615773105863909,
  19.235384061671343,
  38.600518131237564,
  46.238512086787566,
  43.278170016764804,
  30.463092423455635,
  21.2602916254693,
  16.0312195418814,
  6.708203932499369,
  17.88854381999832,
  18.439088914585774,
  29.832867780352597,
  37.33630940518894,
  55.97320787662612,
  43.93176527297759,
  58.60034129593445,
  36.013886210738214,
  18.384776310850235,
  23.345235059857504,
  46.400431032480725,
  36.124783736376884,
  33.24154027718932,
  37.48332962798263,
  41.012193308819754,
  29.068883707497267,
  33.13608305156178,
  42.720018726587654,
  37.8549864614954,
  36.124783736376884,
  37.69615364994153,
  8.602325267042627,
  6.708203932499369,
  12.727922061357855,
  18.439088914585774,
  27.202941017470888,
  23.40939982143925,
  13.416407864998739,
  7.810249675906654,
  29.154759474226502,
  21.587033144922902,
  18.110770276274835,
  16.1245154965971,
  14.317821063276353,
  16.76305461424021,
  13.152946437965905,
  19.6468827043885,
  19.026297590440446,
  14.0,
  22.090722034374522],
 [32.01562118716424,
  28.231188426986208,
  47.4236228055175,
  42.720018726587654,
  56.568542494923804,
  30.0,
  31.622776601683793,
  11.180339887498949,
  17.72004514666935,
  40.0,
  15.0,
  7.0710678118654755,
  43.01162633521314,
  38.07886552931954,
  50.0,
  57.0087712549569,
  40.311288741492746,
  31.622776601683793,
  20.615528128088304,
  0.0,
  30.4138126514911,
  50.0,
  58.309518948453004,
  68.00735254367721,
  55.90169943749474,
  64.03124237432849,
  42.42640687119285,
  28.284271247461902,
  34.713109915419565,
  52.20153254455275,
  25.0,
  17.88854381999832,
  21.93171219946131,
  38.8329756778952,
  50.24937810560445,
  48.25971404805462,
  13.0,
  40.311288741492746,
  55.90169943749474,
  65.79513659838392,
  43.01162633521314,
  59.481089431852205,
  48.83646178829912,
  57.55866572463264,
  46.17358552246078,
  23.769728648009426,
  17.69180601295413,
  8.06225774829855,
  8.246211251235321,
  12.041594578792296,
  34.539832078341085,
  34.058772731852805,
  20.808652046684813,
  36.40054944640259,
  52.20153254455275,
  60.60528029800704,
  61.220911460055866,
  50.92150822589606,
  39.96248240537617,
  36.49657518178932,
  26.076809620810597,
  36.124783736376884,
  9.219544457292887,
  15.0,
  17.0,
  49.9799959983992,
  36.40054944640259,
  75.69015788066504,
  46.06517122512408,
  25.553864678361276,
  22.360679774997898,
  42.7551166528639,
  54.405882034941776,
  51.86520991955976,
  56.302753041036986,
  59.64059020499378,
  38.47076812334269,
  41.6293165929973,
  46.69047011971501,
  43.68065933568311,
  47.01063709417264,
  40.44749683231337,
  13.0,
  23.021728866442675,
  29.274562336608895,
  38.01315561749642,
  43.41658669218482,
  43.965895873961216,
  13.601470508735444,
  27.313000567495326,
  17.46424919657298,
  41.0,
  38.63935817272331,
  36.124783736376884,
  34.785054261852174,
  37.36308338453881,
  33.734255586866,
  40.26164427839479,
  39.20459156782532,
  34.36568055487916,
  42.1070065428546],
 [31.622776601683793,
  16.492422502470642,
  49.03060268852505,
  22.360679774997898,
  46.09772228646444,
  46.09772228646444,
  40.311288741492746,
  29.154759474226502,
  41.340053217188775,
  11.180339887498949,
  15.811388300841896,
  25.0,
  30.4138126514911,
  42.720018726587654,
  62.64982043070834,
  61.84658438426491,
  57.0087712549569,
  53.150729063673246,
  35.35533905932738,
  30.4138126514911,
  0.0,
  45.0,
  55.0,
  60.8276253029822,
  36.05551275463989,
  49.24428900898052,
  35.0,
  26.92582403567252,
  28.284271247461902,
  29.832867780352597,
  7.0710678118654755,
  19.1049731745428,
  10.770329614269007,
  15.264337522473747,
  22.360679774997898,
  18.0,
  43.289721643826724,
  51.478150704935004,
  72.11102550927978,
  55.08175741568164,
  40.311288741492746,
  58.077534382926416,
  57.0087712549569,
  65.7875368135941,
  61.29437168288782,
  47.43416490252569,
  46.238512086787566,
  38.07886552931954,
  34.539832078341085,
  39.11521443121589,
  18.110770276274835,
  8.06225774829855,
  28.42534080710379,
  34.92849839314596,
  37.94733192202055,
  45.69463863518345,
  53.600373133029585,
  54.57105459856901,
  40.024992192379,
  47.50789408087881,
  41.773197148410844,
  52.630789467763066,
  22.135943621178654,
  18.439088914585774,
  32.31098884280702,
  20.808652046684813,
  8.94427190999916,
  63.90618123468183,
  28.231188426986208,
  19.697715603592208,
  12.041594578792296,
  12.36931687685298,
  49.040799340956916,
  48.010415536631214,
  52.009614495783374,
  54.147945482723536,
  23.345235059857504,
  23.40939982143925,
  20.615528128088304,
  20.808652046684813,
  30.083217912982647,
  14.866068747318506,
  34.9857113690718,
  41.773197148410844,
  48.08326112068523,
  51.86520991955976,
  62.369864518050704,
  49.9799959983992,
  23.021728866442675,
  35.510561809129406,
  14.142135623730951,
  54.91812087098393,
  48.76474136094644,
  49.09175083453431,
  42.485291572496,
  45.617978911828175,
  44.41846462902562,
  48.33218389437829,
  51.10772935672255,
  46.32493928760188,
  54.20332093147061],
 [18.027756377319946,
  29.274562336608895,
  10.44030650891055,
  26.92582403567252,
  10.0,
  31.622776601683793,
  22.360679774997898,
  39.05124837953327,
  41.88078318274385,
  41.23105625617661,
  42.720018726587654,
  51.478150704935004,
  15.811388300841896,
  15.811388300841896,
  31.622776601683793,
  21.213203435596427,
  35.0,
  41.23105625617661,
  32.01562118716424,
  50.0,
  45.0,
  0.0,
  10.0,
  18.027756377319946,
  25.0,
  20.0,
  10.0,
  22.360679774997898,
  17.46424919657298,
  29.068883707497267,
  40.311288741492746,
  34.92849839314596,
  50.00999900019995,
  32.984845004941285,
  40.311288741492746,
  48.46648326421054,
  58.728187440104094,
  25.0,
  42.720018726587654,
  17.0,
  7.0710678118654755,
  13.341664064126334,
  22.47220505424423,
  27.80287754891569,
  34.52535300326414,
  42.95346318982906,
  51.31276644267,
  51.62363799656123,
  45.254833995939045,
  61.84658438426491,
  27.073972741361768,
  38.2099463490856,
  29.206163733020468,
  13.601470508735444,
  15.0,
  18.24828759089466,
  11.313708498984761,
  15.264337522473747,
  10.816653826391969,
  24.331050121192877,
  31.304951684997057,
  33.24154027718932,
  43.41658669218482,
  52.20153254455275,
  64.41273166075166,
  59.481089431852205,
  53.150729063673246,
  26.627053911388696,
  21.95449840010015,
  28.160255680657446,
  36.87817782917155,
  49.47726750741192,
  4.47213595499958,
  3.1622776601683795,
  7.0710678118654755,
  9.848857801796104,
  22.360679774997898,
  24.351591323771842,
  35.77708763999664,
  30.463092423455635,
  20.248456731316587,
  35.4400902933387,
  40.36087214122113,
  35.35533905932738,
  35.73513677041127,
  29.068883707497267,
  41.048751503547585,
  17.11724276862369,
  37.21558813185679,
  24.20743687382041,
  49.040799340956916,
  30.01666203960727,
  23.08679276123039,
  27.294688127912362,
  20.248456731316587,
  20.396078054371138,
  24.041630560342615,
  20.024984394500787,
  26.019223662515376,
  25.709920264364882,
  27.073972741361768],
 [26.92582403567252,
  39.20459156782532,
  12.206555615733702,
  36.40054944640259,
  14.142135623730951,
  36.05551275463989,
  28.284271247461902,
  47.16990566028302,
  48.104053883222775,
  50.99019513592785,
  52.20153254455275,
  60.41522986797286,
  25.495097567963924,
  21.213203435596427,
  30.0,
  15.811388300841896,
  36.40054944640259,
  44.721359549995796,
  39.05124837953327,
  58.309518948453004,
  55.0,
  10.0,
  0.0,
  11.180339887498949,
  32.01562118716424,
  22.360679774997898,
  20.0,
  31.622776601683793,
  27.294688127912362,
  37.21558813185679,
  50.24937810560445,
  44.27188724235731,
  59.841457201508724,
  42.7551166528639,
  49.24428900898052,
  57.87054518492115,
  65.94694837519019,
  26.92582403567252,
  40.311288741492746,
  15.132745950421556,
  15.811388300841896,
  4.242640687119285,
  21.095023109728988,
  23.08679276123039,
  34.23448553724738,
  48.010415536631214,
  57.38466694161429,
  59.033888572581766,
  52.80151512977634,
  69.8927750200262,
  37.05401462729781,
  48.16637831516918,
  37.589892258425,
  22.47220505424423,
  22.47220505424423,
  22.20360331117452,
  8.246211251235321,
  13.152946437965905,
  18.35755975068582,
  27.784887978899608,
  36.87817782917155,
  35.84689665786984,
  52.392747589718944,
  61.68468205316454,
  73.40980860893181,
  69.12307863514182,
  63.12685640834652,
  22.561028345356956,
  31.016124838541646,
  37.8549864614954,
  46.69047011971501,
  59.22837157984339,
  6.324555320336759,
  7.0710678118654755,
  3.1622776601683795,
  4.123105625617661,
  32.2490309931942,
  33.95585369269929,
  44.94441010848846,
  39.84971769034255,
  29.154759474226502,
  45.12205669071391,
  47.634021455258214,
  41.10960958218893,
  39.96248240537617,
  31.38470965295043,
  41.773197148410844,
  18.788294228055936,
  46.09772228646444,
  31.400636936215164,
  58.69412236331676,
  31.32091952673165,
  25.942243542145693,
  30.4138126514911,
  25.495097567963924,
  24.413111231467404,
  28.600699292150182,
  22.825424421026653,
  28.231188426986208,
  29.68164415931166,
  28.160255680657446],
 [36.05551275463989,
  46.17358552246078,
  23.323807579381203,
  40.0,
  15.0,
  47.16990566028302,
  39.05124837953327,
  57.0087712549569,
  58.89821729050889,
  55.0,
  60.41522986797286,
  69.46221994724903,
  30.4138126514911,
  32.01562118716424,
  40.311288741492746,
  25.0,
  47.43416490252569,
  55.90169943749474,
  49.49747468305833,
  68.00735254367721,
  60.8276253029822,
  18.027756377319946,
  11.180339887498949,
  0.0,
  31.622776601683793,
  18.027756377319946,
  26.92582403567252,
  40.311288741492746,
  34.92849839314596,
  38.07886552931954,
  57.0087712549569,
  52.773099207835045,
  67.05221845696084,
  47.042533945356304,
  50.99019513592785,
  60.530983801686226,
  76.38062581571324,
  38.07886552931954,
  50.0,
  8.602325267042627,
  25.0,
  13.152946437965905,
  31.78049716414141,
  32.0624390837628,
  44.91102314577124,
  59.07622195096772,
  68.24954212300622,
  69.35416353759881,
  63.0317380372777,
  79.81227975693966,
  42.7551166528639,
  53.33854141237835,
  47.20169488482379,
  31.622776601683793,
  24.08318915758459,
  19.697715603592208,
  7.280109889280518,
  24.041630560342615,
  28.319604517012593,
  38.948684188300895,
  47.80167361086848,
  47.01063709417264,
  61.40032573203501,
  69.85699678629192,
  82.365041127896,
  72.3394774656273,
  68.26419266350405,
  12.0,
  34.0147027033899,
  45.69463863518345,
  54.08326913195984,
  63.0317380372777,
  13.601470508735444,
  16.278820596099706,
  12.041594578792296,
  8.48528137423857,
  37.48332962798263,
  38.05259518088089,
  47.38143096192854,
  43.04648650006177,
  32.01562118716424,
  49.0,
  58.0,
  52.009614495783374,
  51.10772935672255,
  42.5440947723653,
  52.630789467763066,
  29.966648127543394,
  55.226805085936306,
  41.72529209005013,
  66.48308055437865,
  42.37924020083418,
  37.12142238654117,
  41.593268686170845,
  36.40054944640259,
  35.510561809129406,
  39.66106403010388,
  34.0,
  39.395431207184416,
  40.8166632639171,
  39.21734310225516],
 [30.0,
  27.784887978899608,
  34.9857113690718,
  14.142135623730951,
  18.027756377319946,
  50.24937810560445,
  40.311288741492746,
  47.43416490252569,
  55.57877292636101,
  26.92582403567252,
  43.01162633521314,
  54.08326913195984,
  15.0,
  36.40054944640259,
  55.90169943749474,
  46.09772228646444,
  57.0087712549569,
  60.207972893961475,
  45.27692569068709,
  55.90169943749474,
  36.05551275463989,
  25.0,
  32.01562118716424,
  31.622776601683793,
  0.0,
  15.0,
  20.615528128088304,
  30.4138126514911,
  24.08318915758459,
  7.0710678118654755,
  35.35533905932738,
  38.01315561749642,
  45.34313619501854,
  20.808652046684813,
  20.0,
  30.066592756745816,
  67.77905281132217,
  47.43416490252569,
  67.08203932499369,
  23.53720459187964,
  26.92582403567252,
  36.235341863986875,
  47.01063709417264,
  52.80151512977634,
  57.9396237474839,
  59.07622195096772,
  64.32728814430156,
  60.74537019394976,
  54.70831746635972,
  67.60177512462228,
  21.633307652783937,
  28.0178514522438,
  38.8329756778952,
  28.284271247461902,
  10.0,
  12.165525060596439,
  25.942243542145693,
  40.22437072223753,
  30.364452901377952,
  45.35416188179427,
  48.010415536631214,
  54.12947441089743,
  47.01063709417264,
  50.99019513592785,
  65.29931086925804,
  42.1070065428546,
  41.23105625617661,
  30.066592756745816,
  9.848857801796104,
  30.463092423455635,
  35.0,
  33.95585369269929,
  26.1725046566048,
  27.65863337187866,
  29.068883707497267,
  28.844410203711913,
  17.46424919657298,
  14.422205101855956,
  17.46424919657298,
  15.264337522473747,
  9.219544457292887,
  21.470910553583888,
  51.419840528729765,
  51.03920062069938,
  54.147945482723536,
  50.695167422546305,
  63.324560795950255,
  40.718546143004666,
  42.5440947723653,
  39.0,
  46.69047011971501,
  52.49761899362675,
  44.9221548904324,
  48.27007354458868,
  39.81205847478876,
  41.48493702538308,
  43.73785545725808,
  42.37924020083418,
  48.08326112068523,
  45.89117562233506,
  49.9799959983992],
 [33.54101966249684,
  37.64306044943742,
  30.14962686336267,
  26.92582403567252,
  10.0,
  50.99019513592785,
  41.23105625617661,
  54.08326913195984,
  59.61543424315552,
  41.23105625617661,
  53.150729063673246,
  63.63961030678928,
  21.213203435596427,
  35.35533905932738,
  50.99019513592785,
  38.07886552931954,
  55.0,
  60.8276253029822,
  49.24428900898052,
  64.03124237432849,
  49.24428900898052,
  20.0,
  22.360679774997898,
  18.027756377319946,
  15.0,
  0.0,
  22.360679774997898,
  36.05551275463989,
  29.410882339705484,
  22.02271554554524,
  47.16990566028302,
  46.69047011971501,
  57.4543296888929,
  34.17601498127012,
  35.0,
  45.044422518220834,
  74.62573282722254,
  45.0,
  61.84658438426491,
  9.433981132056603,
  25.495097567963924,
  26.419689627245813,
  41.773197148410844,
  45.31004303683677,
  54.3323108288245,
  61.68468205316454,
  68.94200461257273,
  67.4166151627327,
  61.05735008989499,
  76.05918747922567,
  32.449961479175904,
  41.23105625617661,
  44.41846462902562,
  30.083217912982647,
  12.041594578792296,
  3.605551275463989,
  14.422205101855956,
  33.95585369269929,
  29.614185789921695,
  44.181444068749045,
  50.0,
  53.150729063673246,
  55.90169943749474,
  62.00806399170998,
  75.82216034906945,
  57.07889277132134,
  55.362442142665635,
  15.132745950421556,
  21.02379604162864,
  38.897300677553446,
  45.60701700396552,
  48.662100242385755,
  18.439088914585774,
  21.213203435596427,
  20.248456731316587,
  18.35755975068582,
  27.202941017470888,
  25.942243542145693,
  32.2490309931942,
  29.120439557122072,
  19.235384061671343,
  35.4400902933387,
  56.824290580701486,
  53.75872022286245,
  55.10898293381942,
  49.040799340956916,
  61.032778078668514,
  37.05401462729781,
  50.44799302251776,
  41.78516483155236,
  58.008620049092706,
  50.00999900019995,
  43.04648650006177,
  47.16990566028302,
  39.6232255123179,
  40.19950248448356,
  43.56604182158393,
  40.01249804748511,
  46.010868281309364,
  45.39823785126467,
  47.042533945356304],
 [11.180339887498949,
  19.4164878389476,
  16.401219466856727,
  18.027756377319946,
  14.142135623730951,
  30.0,
  20.0,
  32.01562118716424,
  37.33630940518894,
  31.622776601683793,
  33.54101966249684,
  43.01162633521314,
  7.0710678118654755,
  15.811388300841896,
  36.05551275463989,
  29.154759474226502,
  36.40054944640259,
  40.0,
  26.92582403567252,
  42.42640687119285,
  35.0,
  10.0,
  20.0,
  26.92582403567252,
  20.615528128088304,
  22.360679774997898,
  0.0,
  14.142135623730951,
  8.06225774829855,
  22.47220505424423,
  30.4138126514911,
  26.076809620810597,
  40.26164427839479,
  23.40939982143925,
  32.01562118716424,
  39.35733730830886,
  52.43090691567332,
  26.92582403567252,
  47.16990566028302,
  23.430749027719962,
  7.0710678118654755,
  23.194827009486403,
  27.65863337187866,
  34.828149534535996,
  37.57658845611187,
  39.81205847478876,
  46.61544808322666,
  45.221676218380054,
  38.8329756778952,
  54.45181356024793,
  17.11724276862369,
  28.284271247461902,
  22.20360331117452,
  8.06225774829855,
  12.041594578792296,
  19.313207915827967,
  19.697715603592208,
  22.20360331117452,
  9.848857801796104,
  24.73863375370596,
  28.284271247461902,
  33.54101966249684,
  35.0,
  42.95346318982906,
  55.758407437802596,
  49.9799959983992,
  43.18564576337837,
  33.301651610693426,
  14.212670403551895,
  18.788294228055936,
  27.202941017470888,
  39.84971769034255,
  14.142135623730951,
  13.038404810405298,
  17.029386365926403,
  19.4164878389476,
  12.649110640673518,
  15.264337522473747,
  27.202941017470888,
  21.633307652783937,
  13.038404810405298,
  26.0,
  34.48187929913333,
  31.78049716414141,
  34.0147027033899,
  30.083217912982647,
  42.720018726587654,
  20.808652046684813,
  29.068883707497267,
  19.6468827043885,
  39.56008088970496,
  31.953090617340916,
  24.351591323771842,
  27.65863337187866,
  19.235384061671343,
  20.8806130178211,
  23.194827009486403,
  21.93171219946131,
  27.51363298439521,
  25.317977802344327,
  29.546573405388315],
 [5.0,
  10.816653826391969,
  23.0,
  20.615528128088304,
  28.284271247461902,
  22.360679774997898,
  14.142135623730951,
  18.027756377319946,
  25.179356624028344,
  28.284271247461902,
  20.615528128088304,
  29.154759474226502,
  15.811388300841896,
  15.811388300841896,
  36.05551275463989,
  35.35533905932738,
  32.01562118716424,
  31.622776601683793,
  15.0,
  28.284271247461902,
  26.92582403567252,
  22.360679774997898,
  31.622776601683793,
  40.311288741492746,
  30.4138126514911,
  36.05551275463989,
  14.142135623730951,
  0.0,
  6.708203932499369,
  29.068883707497267,
  20.615528128088304,
  12.649110640673518,
  29.0,
  21.633307652783937,
  33.54101966249684,
  37.53664875824692,
  38.58756276314948,
  25.0,
  46.09772228646444,
  37.53664875824692,
  15.811388300841896,
  33.734255586866,
  30.083217912982647,
  38.897300677553446,
  35.38361202590826,
  29.068883707497267,
  33.95585369269929,
  31.38470965295043,
  25.059928172283335,
  40.311288741492746,
  13.892443989449804,
  22.80350850198276,
  8.54400374531753,
  9.219544457292887,
  24.596747752497688,
  32.7566787083184,
  33.28663395418648,
  28.160255680657446,
  14.035668847618199,
  21.2602916254693,
  18.973665961010276,
  28.0178514522438,
  21.095023109728988,
  30.083217912982647,
  42.05948168962618,
  45.803929962395145,
  35.84689665786984,
  47.4236228055175,
  21.02379604162864,
  7.280109889280518,
  16.1245154965971,
  35.608987629529715,
  26.832815729997478,
  24.698178070456937,
  29.154759474226502,
  32.202484376209235,
  14.142135623730951,
  18.24828759089466,
  28.635642126552707,
  23.40939982143925,
  21.213203435596427,
  24.413111231467404,
  21.18962010041709,
  21.213203435596427,
  25.632011235952593,
  26.1725046566048,
  38.01315561749642,
  23.08679276123039,
  15.0,
  10.295630140987,
  27.294688127912362,
  29.0,
  22.20360331117452,
  23.345235059857504,
  15.811388300841896,
  18.867962264113206,
  18.384776310850235,
  21.470910553583888,
  24.839484696748443,
  20.518284528683193,
  27.80287754891569],
 [6.324555320336759,
  12.0,
  20.8806130178211,
  16.1245154965971,
  22.02271554554524,
  26.92582403567252,
  17.46424919657298,
  24.698178070456937,
  31.575306807693888,
  26.92582403567252,
  25.495097567963924,
  35.0,
  9.219544457292887,
  16.278820596099706,
  37.48332962798263,
  33.83784863137726,
  35.35533905932738,
  36.6742416417845,
  21.213203435596427,
  34.713109915419565,
  28.284271247461902,
  17.46424919657298,
  27.294688127912362,
  34.92849839314596,
  24.08318915758459,
  29.410882339705484,
  8.06225774829855,
  6.708203932499369,
  0.0,
  23.53720459187964,
  23.021728866442675,
  18.027756377319946,
  32.55764119219941,
  19.209372712298546,
  30.0,
  35.608987629529715,
  45.27692569068709,
  27.018512172212592,
  48.16637831516918,
  31.400636936215164,
  12.041594578792296,
  30.01666203960727,
  30.23243291566195,
  38.47076812334269,
  37.8021163428716,
  35.014282800023196,
  40.52159917870962,
  38.07886552931954,
  31.76476034853718,
  46.75467891024384,
  11.661903789690601,
  22.47220505424423,
  15.231546211727817,
  7.211102550927978,
  17.88854381999832,
  26.076809620810597,
  27.730849247724095,
  26.570660511172846,
  12.083045973594572,
  23.853720883753127,
  24.186773244895647,
  31.78049716414141,
  27.018512172212592,
  34.92849839314596,
  47.70744176750625,
  45.17742799230607,
  36.87817782917155,
  41.23105625617661,
  15.132745950421556,
  10.770329614269007,
  19.4164878389476,
  34.88552708502482,
  21.840329667841555,
  20.223748416156685,
  24.515301344262525,
  27.202941017470888,
  9.433981132056603,
  13.416407864998739,
  25.0,
  19.4164878389476,
  15.0,
  22.02271554554524,
  27.85677655436824,
  27.0,
  30.59411708155671,
  29.154759474226502,
  41.593268686170845,
  23.021728866442675,
  21.213203435596427,
  15.132745950421556,
  31.622776601683793,
  31.622776601683793,
  24.20743687382041,
  26.419689627245813,
  18.027756377319946,
  20.615528128088304,
  21.470910553583888,
  22.627416997969522,
  27.202941017470888,
  23.706539182259394,
  29.832867780352597],
 [29.832867780352597,
  24.041630560342615,
  38.28837943815329,
  9.486832980505138,
  23.769728648009426,
  50.44799302251776,
  40.80441152620633,
  44.721359549995796,
  54.00925846556311,
  20.12461179749811,
  38.47076812334269,
  49.64876634922564,
  15.652475842498529,
  38.01315561749642,
  58.52349955359813,
  50.24937810560445,
  58.309518948453004,
  60.207972893961475,
  44.04543109109048,
  52.20153254455275,
  29.832867780352597,
  29.068883707497267,
  37.21558813185679,
  38.07886552931954,
  7.0710678118654755,
  22.02271554554524,
  22.47220505424423,
  29.068883707497267,
  23.53720459187964,
  0.0,
  30.0,
  34.48187929913333,
  39.6232255123179,
  14.866068747318506,
  13.038404810405298,
  23.021728866442675,
  64.5600495662759,
  49.193495504995376,
  69.6419413859206,
  30.265491900843113,
  29.410882339705484,
  41.340053217188775,
  50.0,
  56.586217403180434,
  59.941638282582836,
  58.137767414994535,
  62.289646009589745,
  57.723478758647246,
  51.97114584074513,
  63.56099432828282,
  17.72004514666935,
  21.93171219946131,
  37.013511046643494,
  29.154759474226502,
  14.7648230602334,
  19.026297590440446,
  31.953090617340916,
  43.86342439892262,
  32.2490309931942,
  46.61544808322666,
  47.67598976424087,
  55.02726596879042,
  43.08131845707603,
  45.803929962395145,
  60.21627686929839,
  35.05709628591621,
  34.438350715445125,
  37.12142238654117,
  8.54400374531753,
  27.459060435491963,
  30.4138126514911,
  26.92582403567252,
  31.064449134018133,
  32.01562118716424,
  34.132096331752024,
  34.438350715445125,
  15.0,
  11.045361017187261,
  10.44030650891055,
  9.219544457292887,
  9.433981132056603,
  15.0,
  49.25444142409901,
  50.24937810560445,
  54.12947441089743,
  52.0,
  64.62197768561404,
  43.266615305567875,
  39.293765408777,
  38.63935817272331,
  41.400483088968905,
  54.12947441089743,
  46.51881339845203,
  49.39635614091387,
  40.85339643163099,
  42.95346318982906,
  44.598206241955516,
  44.294469180700204,
  49.658836071740545,
  46.8187996428785,
  51.884487084291386],
 [25.495097567963924,
  11.045361017187261,
  43.289721643826724,
  21.213203435596427,
  42.720018726587654,
  39.05124837953327,
  33.54101966249684,
  22.360679774997898,
  34.48187929913333,
  15.0,
  10.0,
  20.615528128088304,
  26.92582403567252,
  36.40054944640259,
  55.90169943749474,
  55.90169943749474,
  50.0,
  46.09772228646444,
  28.284271247461902,
  25.0,
  7.0710678118654755,
  40.311288741492746,
  50.24937810560445,
  57.0087712549569,
  35.35533905932738,
  47.16990566028302,
  30.4138126514911,
  20.615528128088304,
  23.021728866442675,
  30.0,
  0.0,
  12.041594578792296,
  10.295630140987,
  15.264337522473747,
  25.495097567963924,
  23.53720459187964,
  38.0,
  44.721359549995796,
  65.19202405202648,
  52.0,
  35.0,
  53.03772242470448,
  50.59644256269407,
  59.481089431852205,
  54.378304497290095,
  40.496913462633174,
  39.84971769034255,
  32.2490309931942,
  28.160255680657446,
  34.92849839314596,
  14.7648230602334,
  9.219544457292887,
  21.400934559032695,
  29.154759474226502,
  35.35533905932738,
  43.56604182158393,
  49.72926703662542,
  48.662100242385755,
  34.23448553724738,
  40.70626487409524,
  34.713109915419565,
  45.60701700396552,
  16.1245154965971,
  15.811388300841896,
  30.23243291566195,
  27.80287754891569,
  15.811388300841896,
  61.26989472816156,
  26.40075756488817,
  13.341664064126334,
  5.0,
  18.788294228055936,
  44.553338819890925,
  43.18564576337837,
  47.38143096192854,
  49.8196748283246,
  20.12461179749811,
  21.400934559032695,
  22.47220505424423,
  20.808652046684813,
  28.0178514522438,
  16.15549442140351,
  28.178005607210743,
  34.713109915419565,
  41.012193308819754,
  44.94441010848846,
  55.31726674375732,
  43.68065933568311,
  16.1245154965971,
  28.653097563788805,
  11.40175425099138,
  48.02082881417188,
  42.04759208325728,
  42.190046219457976,
  35.84689665786984,
  39.0,
  37.589892258425,
  41.78516483155236,
  44.294469180700204,
  39.44616584663204,
  47.41307836451879],
 [17.46424919657298,
  10.44030650891055,
  35.22782990761707,
  25.0,
  40.0,
  27.202941017470888,
  22.80350850198276,
  11.180339887498949,
  22.847319317591726,
  25.298221281347036,
  8.06225774829855,
  17.029386365926403,
  25.495097567963924,
  27.018512172212592,
  44.94441010848846,
  47.01063709417264,
  38.27531841800928,
  34.058772731852805,
  16.278820596099706,
  17.88854381999832,
  19.1049731745428,
  34.92849839314596,
  44.27188724235731,
  52.773099207835045,
  38.01315561749642,
  46.69047011971501,
  26.076809620810597,
  12.649110640673518,
  18.027756377319946,
  34.48187929913333,
  12.041594578792296,
  0.0,
  17.46424919657298,
  22.0,
  34.132096331752024,
  34.539832078341085,
  30.083217912982647,
  33.83784863137726,
  53.71219600798314,
  49.4064773081425,
  28.460498941515414,
  46.32493928760188,
  40.607881008493905,
  49.64876634922564,
  42.941821107167776,
  28.653097563788805,
  29.274562336608895,
  23.345235059857504,
  18.0,
  29.68164415931166,
  16.76305461424021,
  18.973665961010276,
  9.848857801796104,
  21.840329667841555,
  34.713109915419565,
  43.18564576337837,
  45.65084884205331,
  40.01249804748511,
  26.476404589747453,
  29.732137494637012,
  22.80350850198276,
  33.83784863137726,
  9.219544457292887,
  17.46424919657298,
  29.68164415931166,
  39.824615503479755,
  27.65863337187866,
  59.20304046246274,
  28.178005607210743,
  7.810249675906654,
  7.211102550927978,
  30.528675044947494,
  39.395431207184416,
  37.33630940518894,
  41.78516483155236,
  44.77722635447622,
  20.591260281974,
  23.769728648009426,
  30.0,
  26.30589287593181,
  29.154759474226502,
  24.08318915758459,
  16.76305461424021,
  22.67156809750927,
  29.0,
  33.54101966249684,
  43.41658669218482,
  34.132096331752024,
  5.0,
  17.72004514666935,
  15.0,
  36.6742416417845,
  31.32091952673165,
  30.870698080866262,
  25.495097567963924,
  28.635642126552707,
  26.570660511172846,
  31.575306807693888,
  33.24154027718932,
  28.231188426986208,
  36.40054944640259],
 [34.0,
  20.8806130178211,
  52.0,
  31.240998703626616,
  52.92447448959697,
  43.829214001622255,
  40.26164427839479,
  24.20743687382041,
  36.069377593742864,
  21.93171219946131,
  10.295630140987,
  15.524174696260024,
  37.16180835212409,
  44.28317965096906,
  62.297672508690084,
  64.19501538281614,
  55.00909015790027,
  49.20365840057018,
  32.64965543462902,
  21.93171219946131,
  10.770329614269007,
  50.00999900019995,
  59.841457201508724,
  67.05221845696084,
  45.34313619501854,
  57.4543296888929,
  40.26164427839479,
  29.0,
  32.55764119219941,
  39.6232255123179,
  10.295630140987,
  17.46424919657298,
  0.0,
  24.758836806279895,
  33.1058907144937,
  28.284271247461902,
  34.20526275297414,
  51.24451190127583,
  70.68238818828917,
  62.24146527838174,
  44.28317965096906,
  62.39390995922599,
  58.05170109479997,
  67.08203932499369,
  60.00833275470999,
  42.44997055358225,
  39.11521443121589,
  29.966648127543394,
  27.80287754891569,
  29.017236257093817,
  25.059928172283335,
  17.804493814764857,
  27.202941017470888,
  38.05259518088089,
  45.65084884205331,
  53.85164807134504,
  59.77457653551382,
  57.07889277132134,
  43.01162633521314,
  47.12748667179272,
  39.35733730830886,
  50.53711507397311,
  15.556349186104045,
  8.0,
  21.540659228538015,
  28.160255680657446,
  14.560219778561036,
  71.55417527999327,
  36.61966684720111,
  22.090722034374522,
  13.152946437965905,
  22.02271554554524,
  54.3415126767741,
  52.773099207835045,
  57.0701322935211,
  59.665735560705194,
  30.4138126514911,
  31.622776601683793,
  31.064449134018133,
  30.4138126514911,
  38.27531841800928,
  25.0,
  29.732137494637012,
  38.27531841800928,
  44.94441010848846,
  50.695167422546305,
  59.682493245507096,
  51.478150704935004,
  19.235384061671343,
  35.17101079013795,
  4.47213595499958,
  53.85164807134504,
  48.76474136094644,
  48.104053883222775,
  42.95346318982906,
  46.09772228646444,
  43.965895873961216,
  49.03060268852505,
  50.59644256269407,
  45.5411901469428,
  53.75872022286245],
 [24.758836806279895,
  12.36931687685298,
  39.35733730830886,
  7.280109889280518,
  32.0624390837628,
  43.9089968002003,
  35.608987629529715,
  33.06055050963308,
  43.93176527297759,
  8.246211251235321,
  24.351591323771842,
  35.4682957019364,
  17.26267650163207,
  35.4682957019364,
  56.639209034025185,
  52.32590180780452,
  53.600373133029585,
  52.80151512977634,
  35.11409973215888,
  38.8329756778952,
  15.264337522473747,
  32.984845004941285,
  42.7551166528639,
  47.042533945356304,
  20.808652046684813,
  34.17601498127012,
  23.40939982143925,
  21.633307652783937,
  19.209372712298546,
  14.866068747318506,
  15.264337522473747,
  22.0,
  24.758836806279895,
  0.0,
  12.36931687685298,
  16.401219466856727,
  51.62363799656123,
  45.967379738244816,
  67.17886572427373,
  40.607881008493905,
  29.966648127543394,
  46.32493928760188,
  49.4064773081425,
  57.4543296888929,
  56.639209034025185,
  49.040799340956916,
  51.15662224971465,
  45.17742799230607,
  40.0,
  49.64876634922564,
  7.810249675906654,
  7.211102550927978,
  27.51363298439521,
  26.40075756488817,
  23.345235059857504,
  30.675723300355934,
  40.0,
  45.17742799230607,
  31.064449134018133,
  42.5205832509386,
  40.24922359499622,
  49.64876634922564,
  29.614185789921695,
  31.064449134018133,
  45.48626166217664,
  26.570660511172846,
  21.37755832643195,
  49.040799340956916,
  13.341664064126334,
  16.76305461424021,
  16.492422502470642,
  16.492422502470642,
  36.49657518178932,
  36.138621999185304,
  39.6232255123179,
  41.19465984809196,
  10.770329614269007,
  9.0,
  8.0,
  5.656854249492381,
  15.297058540778355,
  2.8284271247461903,
  38.3275357934736,
  41.78516483155236,
  46.95742752749558,
  47.634021455258214,
  59.64059020499378,
  42.20189569201838,
  27.0,
  31.906112267087632,
  26.627053911388696,
  50.32891812864648,
  43.139309220245984,
  44.82186966202994,
  36.796738985948195,
  39.59797974644666,
  39.824615503479755,
  41.773197148410844,
  46.010868281309364,
  42.01190307520001,
  48.79549159502341],
 [36.05551275463989,
  24.73863375370596,
  48.41487374764082,
  14.142135623730951,
  36.40054944640259,
  55.90169943749474,
  47.16990566028302,
  45.27692569068709,
  56.293871780150276,
  11.180339887498949,
  35.35533905932738,
  46.09772228646444,
  25.0,
  46.09772228646444,
  67.26812023536856,
  61.032778078668514,
  65.19202405202648,
  65.0,
  47.43416490252569,
  50.24937810560445,
  22.360679774997898,
  40.311288741492746,
  49.24428900898052,
  50.99019513592785,
  20.0,
  35.0,
  32.01562118716424,
  33.54101966249684,
  30.0,
  13.038404810405298,
  25.495097567963924,
  34.132096331752024,
  33.1058907144937,
  12.36931687685298,
  0.0,
  10.198039027185569,
  63.198101237299845,
  57.0087712549569,
  78.10249675906654,
  43.289721643826724,
  39.05124837953327,
  53.225933528685054,
  59.413803110051795,
  66.84309986827361,
  67.80117993073571,
  61.40032573203501,
  63.387695966961914,
  57.0087712549569,
  52.08646657242167,
  60.41522986797286,
  19.697715603592208,
  16.278820596099706,
  39.84971769034255,
  36.87817782917155,
  27.202941017470888,
  32.0624390837628,
  44.64302857109943,
  54.20332093147061,
  41.012193308819754,
  53.823786563191554,
  52.392747589718944,
  61.40032573203501,
  41.10960958218893,
  40.496913462633174,
  54.62600113499065,
  22.20360331117452,
  24.08318915758459,
  50.039984012787215,
  18.35755975068582,
  29.120439557122072,
  28.0178514522438,
  15.264337522473747,
  42.95346318982906,
  43.41658669218482,
  46.09772228646444,
  46.8187996428785,
  20.615528128088304,
  16.97056274847714,
  5.0,
  10.63014581273465,
  20.12461179749811,
  10.04987562112089,
  50.635955604688654,
  54.08326913195984,
  59.09314681077663,
  59.07622195096772,
  71.34423592694787,
  52.32590180780452,
  39.11521443121589,
  43.829214001622255,
  36.05551275463989,
  61.61168720299745,
  54.20332093147061,
  56.302753041036986,
  48.010415536631214,
  50.60632371551998,
  51.31276644267,
  52.49761899362675,
  57.201398584300364,
  53.53503525729669,
  59.81638571495272],
 [41.036569057366385,
  27.202941017470888,
  55.569775957799216,
  21.540659228538015,
  45.70557952810576,
  59.405386961116584,
  51.66236541235796,
  45.5411901469428,
  57.38466694161429,
  9.433981132056603,
  33.37663853655727,
  43.0,
  32.69556544854363,
  51.85556864985669,
  73.0,
  68.47627326307996,
  69.52697318307479,
  67.7421582177598,
  49.73932046178355,
  48.25971404805462,
  18.0,
  48.46648326421054,
  57.87054518492115,
  60.530983801686226,
  30.066592756745816,
  45.044422518220834,
  39.35733730830886,
  37.53664875824692,
  35.608987629529715,
  23.021728866442675,
  23.53720459187964,
  34.539832078341085,
  28.284271247461902,
  16.401219466856727,
  10.198039027185569,
  0.0,
  61.204574992397426,
  62.24146527838174,
  83.45058418010026,
  53.08483775994799,
  46.14108798023731,
  61.68468205316454,
  65.80273550544841,
  73.78346698278686,
  72.83543093852057,
  63.071388124885914,
  63.324560795950255,
  55.731499172371095,
  51.66236541235796,
  57.07889277132134,
  24.08318915758459,
  15.652475842498529,
  42.190046219457976,
  42.80186911806539,
  36.49657518178932,
  42.0,
  53.9351462406472,
  61.40032573203501,
  47.43416490252569,
  58.69412236331676,
  55.47071299343465,
  65.43699259593154,
  39.6232255123179,
  36.22154055254967,
  49.47726750741192,
  12.041594578792296,
  16.1245154965971,
  60.13318551349163,
  26.92582403567252,
  31.622776601683793,
  27.51363298439521,
  6.708203932499369,
  51.54609587543949,
  51.62363799656123,
  54.70831746635972,
  55.78530272392541,
  26.92582403567252,
  24.166091947189145,
  13.152946437965905,
  18.027756377319946,
  28.861739379323623,
  13.601470508735444,
  51.264022471905186,
  56.43580423808985,
  62.12889826803627,
  63.702433234531945,
  75.43208866258443,
  58.60034129593445,
  39.21734310225516,
  47.634021455258214,
  32.0624390837628,
  66.48308055437865,
  59.413803110051795,
  60.876925020897694,
  53.03772242470448,
  55.90169943749474,
  55.90169943749474,
  58.137767414994535,
  62.22539674441618,
  58.05170109479997,
  65.06919393998976],
 [41.400483088968905,
  40.52159917870962,
  54.20332093147061,
  55.08175741568164,
  66.40030120413611,
  32.69556544854363,
  37.8021163428716,
  20.591260281974,
  18.788294228055936,
  53.0,
  28.0,
  18.681541692269406,
  54.120236510939236,
  44.82186966202994,
  51.66236541235796,
  61.71709649683789,
  40.792156108742276,
  30.14962686336267,
  26.90724809414742,
  13.0,
  43.289721643826724,
  58.728187440104094,
  65.94694837519019,
  76.38062581571324,
  67.77905281132217,
  74.62573282722254,
  52.43090691567332,
  38.58756276314948,
  45.27692569068709,
  64.5600495662759,
  38.0,
  30.083217912982647,
  34.20526275297414,
  51.62363799656123,
  63.198101237299845,
  61.204574992397426,
  0.0,
  43.86342439892262,
  55.08175741568164,
  75.28612089887484,
  51.66236541235796,
  66.40030120413611,
  52.80151512977634,
  60.74537019394976,
  46.87216658103186,
  22.360679774997898,
  12.0,
  7.211102550927978,
  13.601470508735444,
  8.94427190999916,
  46.84015371452148,
  47.042533945356304,
  30.23243291566195,
  45.45327270945405,
  63.13477647065839,
  71.34423592694787,
  70.03570517957252,
  56.60388679233962,
  48.08326112068523,
  40.70626487409524,
  30.01666203960727,
  37.36308338453881,
  22.090722034374522,
  26.570660511172846,
  21.400934559032695,
  62.36184731067546,
  48.76474136094644,
  85.14693182963201,
  57.9396237474839,
  37.33630940518894,
  35.22782990761707,
  55.57877292636101,
  62.93647591023825,
  60.108235708594876,
  64.38167441127949,
  67.89698078707183,
  50.32891812864648,
  53.75872022286245,
  59.53990258641679,
  56.293871780150276,
  58.69412236331676,
  53.33854141237835,
  18.384776310850235,
  25.942243542145693,
  30.364452901377952,
  40.496913462633174,
  42.04759208325728,
  49.39635614091387,
  25.298221281347036,
  34.655446902326915,
  29.832867780352597,
  43.01162633521314,
  42.941821107167776,
  39.395431207184416,
  40.80441152620633,
  42.720018726587654,
  38.58756276314948,
  45.27692569068709,
  42.5440947723653,
  38.47076812334269,
  44.94441010848846],
 [21.213203435596427,
  35.805027579936315,
  15.297058540778355,
  43.01162633521314,
  35.0,
  11.180339887498949,
  11.180339887498949,
  30.0,
  25.079872407968907,
  53.150729063673246,
  41.23105625617661,
  45.0,
  33.54101966249684,
  11.180339887498949,
  11.180339887498949,
  18.027756377319946,
  10.0,
  18.027756377319946,
  20.0,
  40.311288741492746,
  51.478150704935004,
  25.0,
  26.92582403567252,
  38.07886552931954,
  47.43416490252569,
  45.0,
  26.92582403567252,
  25.0,
  27.018512172212592,
  49.193495504995376,
  44.721359549995796,
  33.83784863137726,
  51.24451190127583,
  45.967379738244816,
  57.0087712549569,
  62.24146527838174,
  43.86342439892262,
  0.0,
  21.213203435596427,
  40.792156108742276,
  20.615528128088304,
  25.553864678361276,
  8.94427190999916,
  17.26267650163207,
  10.816653826391969,
  22.80350850198276,
  33.28663395418648,
  37.94733192202055,
  32.7566787083184,
  50.0,
  38.18376618407357,
  47.80167361086848,
  24.041630560342615,
  20.248456731316587,
  38.07886552931954,
  43.104524124504614,
  33.95585369269929,
  14.422205101855956,
  17.08800749063506,
  4.123105625617661,
  14.317821063276353,
  8.94427190999916,
  38.2099463490856,
  49.49747468305833,
  57.21887800367987,
  70.80254232723568,
  60.41522986797286,
  49.33558553417604,
  40.70626487409524,
  31.906112267087632,
  39.81205847478876,
  60.60528029800704,
  27.294688127912362,
  24.186773244895647,
  26.92582403567252,
  30.364452901377952,
  36.40054944640259,
  40.22437072223753,
  52.009614495783374,
  46.400431032480725,
  39.81205847478876,
  48.79549159502341,
  27.459060435491963,
  18.027756377319946,
  14.212670403551895,
  4.47213595499958,
  16.1245154965971,
  8.246211251235321,
  32.55764119219941,
  16.15549442140351,
  48.27007354458868,
  5.0990195135927845,
  2.8284271247461903,
  4.47213595499958,
  9.219544457292887,
  6.4031242374328485,
  7.280109889280518,
  5.0990195135927845,
  1.4142135623730951,
  6.0,
  2.8284271247461903],
 [42.42640687119285,
  56.85068161420758,
  32.31098884280702,
  64.03124237432849,
  52.20153254455275,
  26.92582403567252,
  32.01562118716424,
  47.43416490252569,
  38.3275357934736,
  74.33034373659252,
  60.41522986797286,
  61.84658438426491,
  54.08326913195984,
  32.01562118716424,
  11.180339887498949,
  25.0,
  15.811388300841896,
  25.0,
  38.07886552931954,
  55.90169943749474,
  72.11102550927978,
  42.720018726587654,
  40.311288741492746,
  50.0,
  67.08203932499369,
  61.84658438426491,
  47.16990566028302,
  46.09772228646444,
  48.16637831516918,
  69.6419413859206,
  65.19202405202648,
  53.71219600798314,
  70.68238818828917,
  67.17886572427373,
  78.10249675906654,
  83.45058418010026,
  55.08175741568164,
  21.213203435596427,
  0.0,
  55.44366510251645,
  40.311288741492746,
  37.05401462729781,
  20.248456731316587,
  18.110770276274835,
  10.816653826391969,
  33.015148038438355,
  43.104524124504614,
  51.088159097779204,
  47.67598976424087,
  63.00793600809346,
  59.39696961966999,
  68.8839603971781,
  43.9089968002003,
  41.23105625617661,
  57.271284253105414,
  60.728905802755904,
  48.507731342539614,
  27.892651361962706,
  37.44329045369811,
  24.839484696748443,
  31.38470965295043,
  20.248456731316587,
  56.302753041036986,
  67.67569726275453,
  72.69112738154499,
  91.8313671900838,
  80.99382692526635,
  62.0,
  61.29437168288782,
  52.80151512977634,
  60.207972893961475,
  81.68843247363729,
  43.41658669218482,
  40.80441152620633,
  41.773197148410844,
  44.40720662234904,
  57.48912940721924,
  61.220911460055866,
  73.10950690573696,
  67.47592163134935,
  60.207972893961475,
  70.00714249274856,
  43.174066289845804,
  33.24154027718932,
  26.68332812825267,
  20.248456731316587,
  13.038404810405298,
  26.419689627245813,
  51.478150704935004,
  36.61966684720111,
  67.23094525588644,
  17.204650534085253,
  24.041630560342615,
  23.021728866442675,
  30.4138126514911,
  27.586228448267445,
  27.80287754891569,
  25.612496949731394,
  21.2602916254693,
  25.80697580112788,
  18.384776310850235],
 [33.97057550292606,
  41.593268686170845,
  25.495097567963924,
  33.37663853655727,
  9.433981132056603,
  48.46648326421054,
  39.35733730830886,
  55.17245689653489,
  58.83026432033091,
  48.25971404805462,
  56.60388679233962,
  66.40030120413611,
  25.079872407968907,
  32.69556544854363,
  45.044422518220834,
  30.805843601498726,
  50.635955604688654,
  57.87054518492115,
  48.82622246293481,
  65.79513659838392,
  55.08175741568164,
  17.0,
  15.132745950421556,
  8.602325267042627,
  23.53720459187964,
  9.433981132056603,
  23.430749027719962,
  37.53664875824692,
  31.400636936215164,
  30.265491900843113,
  52.0,
  49.4064773081425,
  62.24146527838174,
  40.607881008493905,
  43.289721643826724,
  53.08483775994799,
  75.28612089887484,
  40.792156108742276,
  55.44366510251645,
  0.0,
  23.853720883753127,
  18.681541692269406,
  36.0,
  38.07886552931954,
  49.040799340956916,
  59.93329625508679,
  68.26419266350405,
  68.11754546370561,
  61.71709649683789,
  77.79460135510689,
  37.33630940518894,
  47.29693436154187,
  45.27692569068709,
  29.832867780352597,
  17.26267650163207,
  11.40175425099138,
  7.0,
  28.0,
  27.784887978899608,
  40.80441152620633,
  48.30113870293329,
  49.47726750741192,
  58.412327466040935,
  65.86349520030045,
  79.05694150420949,
  65.03076195155643,
  61.98386886924694,
  9.899494936611665,
  27.294688127912362,
  41.88078318274385,
  49.64876634922564,
  56.08029957123981,
  13.601470508735444,
  16.76305461424021,
  14.035668847618199,
  11.045361017187261,
  31.953090617340916,
  31.78049716414141,
  40.01249804748511,
  36.124783736376884,
  25.317977802344327,
  42.2965719651132,
  57.0087712549569,
  52.354560450833695,
  52.55473337388365,
  45.12205669071391,
  56.32051136131489,
  32.55764119219941,
  52.49761899362675,
  41.048751503547585,
  62.177166226839255,
  45.5411901469428,
  39.293765408777,
  43.68065933568311,
  37.16180835212409,
  37.0,
  40.85339643163099,
  36.138621999185304,
  41.97618372363071,
  42.37924020083418,
  42.42640687119285],
 [11.180339887498949,
  24.020824298928627,
  9.433981132056603,
  25.0,
  15.811388300841896,
  25.495097567963924,
  15.811388300841896,
  32.01562118716424,
  34.9857113690718,
  38.07886552931954,
  36.40054944640259,
  44.721359549995796,
  14.142135623730951,
  10.0,
  29.154759474226502,
  22.360679774997898,
  30.4138126514911,
  35.35533905932738,
  25.0,
  43.01162633521314,
  40.311288741492746,
  7.0710678118654755,
  15.811388300841896,
  25.0,
  26.92582403567252,
  25.495097567963924,
  7.0710678118654755,
  15.811388300841896,
  12.041594578792296,
  29.410882339705484,
  35.0,
  28.460498941515414,
  44.28317965096906,
  29.966648127543394,
  39.05124837953327,
  46.14108798023731,
  51.66236541235796,
  20.615528128088304,
  40.311288741492746,
  23.853720883753127,
  0.0,
  18.110770276274835,
  20.615528128088304,
  27.80287754891569,
  31.016124838541646,
  36.40054944640259,
  44.41846462902562,
  44.553338819890925,
  38.18376618407357,
  54.817880294662984,
  23.08679276123039,
  34.20526275297414,
  22.20360331117452,
  6.708203932499369,
  17.46424919657298,
  23.08679276123039,
  18.384776310850235,
  15.264337522473747,
  4.123105625617661,
  19.026297590440446,
  24.698178070456937,
  28.0178514522438,
  36.6742416417845,
  45.880278987817846,
  57.697486947006624,
  56.462376853972415,
  48.83646178829912,
  33.60059523282288,
  21.2602916254693,
  22.20360331117452,
  31.144823004794873,
  46.238512086787566,
  11.40175425099138,
  8.94427190999916,
  13.416407864998739,
  16.64331697709324,
  19.235384061671343,
  22.20360331117452,
  34.20526275297414,
  28.600699292150182,
  20.0,
  32.64965543462902,
  33.301651610693426,
  28.635642126552707,
  29.614185789921695,
  24.186773244895647,
  36.6742416417845,
  13.892443989449804,
  30.4138126514911,
  17.204650534085253,
  42.95346318982906,
  25.709920264364882,
  18.24828759089466,
  22.02271554554524,
  14.142135623730951,
  15.033296378372908,
  18.110770276274835,
  15.524174696260024,
  21.37755832643195,
  20.024984394500787,
  23.08679276123039],
 [28.861739379323623,
  42.01190307520001,
  12.206555615733702,
  40.162171256046406,
  18.384776310850235,
  35.4682957019364,
  28.600699292150182,
  48.30113870293329,
  48.16637831516918,
  54.57105459856901,
  54.3415126767741,
  62.03224967708329,
  29.120439557122072,
  21.633307652783937,
  27.16615541441225,
  12.165525060596439,
  34.539832078341085,
  43.56604182158393,
  39.66106403010388,
  59.481089431852205,
  58.077534382926416,
  13.341664064126334,
  4.242640687119285,
  13.152946437965905,
  36.235341863986875,
  26.419689627245813,
  23.194827009486403,
  33.734255586866,
  30.01666203960727,
  41.340053217188775,
  53.03772242470448,
  46.32493928760188,
  62.39390995922599,
  46.32493928760188,
  53.225933528685054,
  61.68468205316454,
  66.40030120413611,
  25.553864678361276,
  37.05401462729781,
  18.681541692269406,
  18.110770276274835,
  0.0,
  18.681541692269406,
  19.4164878389476,
  31.78049716414141,
  47.50789408087881,
  57.28001396647874,
  59.64059020499378,
  53.53503525729669,
  70.83078426785913,
  40.311288741492746,
  51.478150704935004,
  39.0,
  24.515301344262525,
  26.627053911388696,
  26.40075756488817,
  12.083045973594572,
  11.180339887498949,
  19.924858845171276,
  27.018512172212592,
  36.796738985948195,
  34.48187929913333,
  54.08326913195984,
  63.788713735268246,
  75.02666192761077,
  72.80109889280519,
  66.37017402418047,
  25.079872407968907,
  34.92849839314596,
  40.311288741492746,
  49.25444142409901,
  62.81719509815764,
  10.295630140987,
  10.198039027185569,
  7.211102550927978,
  8.06225774829855,
  35.6931365951495,
  37.64306044943742,
  48.84669896727925,
  43.657759905886145,
  33.1058907144937,
  48.76474136094644,
  48.25971404805462,
  41.036569057366385,
  39.20459156782532,
  30.01666203960727,
  39.56008088970496,
  17.804493814764857,
  47.75981574503821,
  32.2490309931942,
  61.0,
  29.546573405388315,
  25.0,
  29.410882339705484,
  25.612496949731394,
  24.041630560342615,
  28.284271247461902,
  22.02271554554524,
  26.92582403567252,
  29.068883707497267,
  26.40075756488817],
 [25.495097567963924,
  40.718546143004666,
  12.083045973594572,
  45.27692569068709,
  32.01562118716424,
  20.12461179749811,
  18.027756377319946,
  38.2099463490856,
  34.0147027033899,
  57.14017850864661,
  48.373546489791295,
  53.150729063673246,
  34.713109915419565,
  14.317821063276353,
  9.219544457292887,
  9.219544457292887,
  16.1245154965971,
  26.1725046566048,
  28.284271247461902,
  48.83646178829912,
  57.0087712549569,
  22.47220505424423,
  21.095023109728988,
  31.78049716414141,
  47.01063709417264,
  41.773197148410844,
  27.65863337187866,
  30.083217912982647,
  30.23243291566195,
  50.0,
  50.59644256269407,
  40.607881008493905,
  58.05170109479997,
  49.4064773081425,
  59.413803110051795,
  65.80273550544841,
  52.80151512977634,
  8.94427190999916,
  20.248456731316587,
  36.0,
  20.615528128088304,
  18.681541692269406,
  0.0,
  9.055385138137417,
  13.152946437965905,
  31.622776601683793,
  42.190046219457976,
  46.8187996428785,
  41.48493702538308,
  58.82176467941097,
  41.88078318274385,
  52.354560450833695,
  31.144823004794873,
  23.021728866442675,
  37.12142238654117,
  40.52159917870962,
  29.0,
  8.0,
  18.439088914585774,
  12.36931687685298,
  23.08679276123039,
  16.97056274847714,
  46.0,
  57.07889277132134,
  65.62011886609167,
  75.29276193632427,
  65.92419889539804,
  43.56604182158393,
  41.86884283091664,
  37.33630940518894,
  45.880278987817846,
  65.0,
  23.345235059857504,
  20.615528128088304,
  22.02271554554524,
  25.019992006393608,
  39.05124837953327,
  42.44997055358225,
  54.48853090330111,
  48.83646178829912,
  40.607881008493905,
  52.20153254455275,
  36.138621999185304,
  26.92582403567252,
  23.021728866442675,
  12.806248474865697,
  20.8806130178211,
  7.211102550927978,
  40.049968789001575,
  23.08679276123039,
  55.44366510251645,
  11.40175425099138,
  10.198039027185569,
  13.416407864998739,
  15.132745950421556,
  12.041594578792296,
  15.132745950421556,
  9.055385138137417,
  10.295630140987,
  14.560219778561036,
  8.48528137423857],
 [34.17601498127012,
  49.39635614091387,
  18.439088914585774,
  52.80151512977634,
  36.235341863986875,
  28.160255680657446,
  27.073972741361768,
  47.095647357266465,
  42.05948168962618,
  65.36818798161687,
  57.42821606144492,
  62.07253821135398,
  41.86884283091664,
  23.08679276123039,
  10.63014581273465,
  7.280109889280518,
  21.400934559032695,
  32.449961479175904,
  37.12142238654117,
  57.55866572463264,
  65.7875368135941,
  27.80287754891569,
  23.08679276123039,
  32.0624390837628,
  52.80151512977634,
  45.31004303683677,
  34.828149534535996,
  38.897300677553446,
  38.47076812334269,
  56.586217403180434,
  59.481089431852205,
  49.64876634922564,
  67.08203932499369,
  57.4543296888929,
  66.84309986827361,
  73.78346698278686,
  60.74537019394976,
  17.26267650163207,
  18.110770276274835,
  38.07886552931954,
  27.80287754891569,
  19.4164878389476,
  9.055385138137417,
  0.0,
  16.278820596099706,
  38.91015291668744,
  49.658836071740545,
  55.08175741568164,
  50.00999900019995,
  67.1863081289633,
  50.11985634456667,
  60.83584469702052,
  40.19950248448356,
  31.304951684997057,
  42.80186911806539,
  44.721359549995796,
  31.32091952673165,
  12.727922061357855,
  26.419689627245813,
  21.095023109728988,
  31.575306807693888,
  23.706539182259394,
  55.00909015790027,
  66.12110101926616,
  74.43117626371358,
  83.64807230295268,
  74.67261881037788,
  44.04543109109048,
  48.83646178829912,
  46.17358552246078,
  54.817880294662984,
  73.35529974037323,
  27.294688127912362,
  25.238858928247925,
  25.079872407968907,
  27.202941017470888,
  46.87216658103186,
  50.0,
  62.00806399170998,
  56.4003546088143,
  47.38143096192854,
  60.207972893961475,
  44.721359549995796,
  35.17101079013795,
  30.463092423455635,
  20.248456731316587,
  24.20743687382041,
  15.811388300841896,
  49.09175083453431,
  32.14031735997639,
  64.4980619863884,
  17.88854381999832,
  19.026297590440446,
  21.587033144922902,
  24.186773244895647,
  21.095023109728988,
  24.020824298928627,
  18.110770276274835,
  18.439088914585774,
  23.194827009486403,
  15.811388300841896],
 [31.89043743820395,
  46.09772228646444,
  24.186773244895647,
  53.823786563191554,
  44.40720662234904,
  16.492422502470642,
  21.2602916254693,
  37.107950630558946,
  29.017236257093817,
  63.655321851358195,
  49.76946855251722,
  51.78802950489621,
  44.294469180700204,
  21.95449840010015,
  5.656854249492381,
  21.02379604162864,
  6.082762530298219,
  17.08800749063506,
  27.51363298439521,
  46.17358552246078,
  61.29437168288782,
  34.52535300326414,
  34.23448553724738,
  44.91102314577124,
  57.9396237474839,
  54.3323108288245,
  37.57658845611187,
  35.38361202590826,
  37.8021163428716,
  59.941638282582836,
  54.378304497290095,
  42.941821107167776,
  60.00833275470999,
  56.639209034025185,
  67.80117993073571,
  72.83543093852057,
  46.87216658103186,
  10.816653826391969,
  10.816653826391969,
  49.040799340956916,
  31.016124838541646,
  31.78049716414141,
  13.152946437965905,
  16.278820596099706,
  0.0,
  24.515301344262525,
  35.17101079013795,
  42.1070065428546,
  38.05259518088089,
  54.230987451824994,
  48.83646178829912,
  58.137767414994535,
  33.12099032335839,
  31.064449134018133,
  48.38388161361178,
  52.773099207835045,
  42.04759208325728,
  21.095023109728988,
  27.730849247724095,
  14.142135623730951,
  20.8806130178211,
  10.04987562112089,
  45.880278987817846,
  57.28001396647874,
  63.12685640834652,
  81.05553651663777,
  70.178344238091,
  56.71860364994893,
  51.478150704935004,
  42.01190307520001,
  49.39635614091387,
  70.9365914038728,
  36.05551275463989,
  33.13608305156178,
  35.014282800023196,
  38.118237105091836,
  47.20169488482379,
  51.03920062069938,
  62.80127387243033,
  57.201398584300364,
  50.53711507397311,
  59.464274989274024,
  33.24154027718932,
  23.194827009486403,
  17.0,
  9.433981132056603,
  8.06225774829855,
  17.46424919657298,
  40.85339643163099,
  25.80697580112788,
  56.64803615307419,
  6.4031242374328485,
  13.601470508735444,
  12.206555615733702,
  19.849433241279208,
  17.204650534085253,
  17.029386365926403,
  15.652475842498529,
  10.63014581273465,
  15.0,
  8.06225774829855],
 [29.154759474226502,
  36.68787265568828,
  35.805027579936315,
  49.49747468305833,
  52.20153254455275,
  12.041594578792296,
  20.615528128088304,
  18.439088914585774,
  6.4031242374328485,
  53.71219600798314,
  32.55764119219941,
  30.4138126514911,
  44.10215414239989,
  27.294688127912362,
  29.410882339705484,
  40.80441152620633,
  18.439088914585774,
  8.06225774829855,
  14.142135623730951,
  23.769728648009426,
  47.43416490252569,
  42.95346318982906,
  48.010415536631214,
  59.07622195096772,
  59.07622195096772,
  61.68468205316454,
  39.81205847478876,
  29.068883707497267,
  35.014282800023196,
  58.137767414994535,
  40.496913462633174,
  28.653097563788805,
  42.44997055358225,
  49.040799340956916,
  61.40032573203501,
  63.071388124885914,
  22.360679774997898,
  22.80350850198276,
  33.015148038438355,
  59.93329625508679,
  36.40054944640259,
  47.50789408087881,
  31.622776601683793,
  38.91015291668744,
  24.515301344262525,
  0.0,
  10.770329614269007,
  18.110770276274835,
  15.652475842498529,
  30.0,
  41.97618372363071,
  47.4236228055175,
  21.587033144922902,
  31.78049716414141,
  51.78802950489621,
  58.9406481131655,
  53.71219600798314,
  36.76955262170047,
  32.31098884280702,
  20.518284528683193,
  11.704699910719626,
  15.231546211727817,
  26.90724809414742,
  37.44329045369811,
  40.024992192379,
  68.24221567329127,
  55.44366510251645,
  69.35416353759881,
  50.00999900019995,
  32.28002478313795,
  35.84689665786984,
  59.16924876994806,
  46.52956049652737,
  43.41658669218482,
  47.16990566028302,
  50.774009099144415,
  43.18564576337837,
  47.265209192385896,
  56.753854494650845,
  51.97114584074513,
  50.00999900019995,
  51.54609587543949,
  12.727922061357855,
  8.06225774829855,
  8.602325267042627,
  18.867962264113206,
  20.09975124224178,
  29.732137494637012,
  24.413111231467404,
  20.223748416156685,
  38.28837943815329,
  21.02379604162864,
  22.627416997969522,
  18.439088914585774,
  22.825424421026653,
  23.600847442411894,
  19.4164878389476,
  25.495097567963924,
  21.400934559032695,
  18.439088914585774,
  23.323807579381203],
 [35.4682957019364,
  39.01281840626232,
  45.27692569068709,
  53.08483775994799,
  59.941638282582836,
  22.20360331117452,
  29.206163733020468,
  18.110770276274835,
  9.433981132056603,
  54.3415126767741,
  30.463092423455635,
  24.758836806279895,
  49.72926703662542,
  36.235341863986875,
  40.162171256046406,
  51.31276644267,
  29.120439557122072,
  18.24828759089466,
  19.697715603592208,
  17.69180601295413,
  46.238512086787566,
  51.31276644267,
  57.38466694161429,
  68.24954212300622,
  64.32728814430156,
  68.94200461257273,
  46.61544808322666,
  33.95585369269929,
  40.52159917870962,
  62.289646009589745,
  39.84971769034255,
  29.274562336608895,
  39.11521443121589,
  51.15662224971465,
  63.387695966961914,
  63.324560795950255,
  12.0,
  33.28663395418648,
  43.104524124504614,
  68.26419266350405,
  44.41846462902562,
  57.28001396647874,
  42.190046219457976,
  49.658836071740545,
  35.17101079013795,
  10.770329614269007,
  0.0,
  10.0,
  11.704699910719626,
  20.396078054371138,
  45.0111097397076,
  48.05205510693585,
  25.495097567963924,
  38.91015291668744,
  58.18934610390462,
  65.92419889539804,
  62.42595614005443,
  46.861498055439924,
  40.496913462633174,
  30.610455730027933,
  20.518284528683193,
  26.0,
  24.166091947189145,
  32.64965543462902,
  31.78049716414141,
  66.64082832618455,
  53.23532661682466,
  77.93587107359485,
  54.74486277268398,
  35.014282800023196,
  35.90264614203248,
  58.52349955359813,
  55.21775076911409,
  52.20153254455275,
  56.22277118748239,
  59.81638571495272,
  47.38143096192854,
  51.24451190127583,
  59.135437767890075,
  55.0,
  55.10898293381942,
  53.33854141237835,
  13.038404810405298,
  16.278820596099706,
  19.235384061671343,
  29.5296461204668,
  30.066592756745816,
  39.698866482558415,
  24.331050121192877,
  27.294688127912362,
  34.66987164671943,
  31.78049716414141,
  32.802438933713454,
  28.844410203711913,
  31.89043743820395,
  33.24154027718932,
  29.0,
  35.4682957019364,
  31.906112267087632,
  28.42534080710379,
  34.0],
 [34.20526275297414,
  33.734255586866,
  47.43416490252569,
  48.27007354458868,
  59.20304046246274,
  26.92582403567252,
  31.064449134018133,
  13.416407864998739,
  13.152946437965905,
  47.16990566028302,
  22.360679774997898,
  15.0,
  46.95742752749558,
  38.01315561749642,
  46.52956049652737,
  55.54277630799526,
  36.05551275463989,
  26.1725046566048,
  20.0,
  8.06225774829855,
  38.07886552931954,
  51.62363799656123,
  59.033888572581766,
  69.35416353759881,
  60.74537019394976,
  67.4166151627327,
  45.221676218380054,
  31.38470965295043,
  38.07886552931954,
  57.723478758647246,
  32.2490309931942,
  23.345235059857504,
  29.966648127543394,
  45.17742799230607,
  57.0087712549569,
  55.731499172371095,
  7.211102550927978,
  37.94733192202055,
  51.088159097779204,
  68.11754546370561,
  44.553338819890925,
  59.64059020499378,
  46.8187996428785,
  55.08175741568164,
  42.1070065428546,
  18.110770276274835,
  10.0,
  0.0,
  6.4031242374328485,
  12.165525060596439,
  40.024992192379,
  41.048751503547585,
  23.021728866442675,
  38.28837943815329,
  55.94640292279746,
  64.1404708432983,
  62.93647591023825,
  50.11985634456667,
  41.036569057366385,
  34.539832078341085,
  23.769728648009426,
  32.2490309931942,
  16.1245154965971,
  23.021728866442675,
  22.135943621178654,
  57.9396237474839,
  44.384682042344295,
  77.98717843338096,
  50.92150822589606,
  30.364452901377952,
  29.0,
  50.44799302251776,
  55.86591089385369,
  53.075418038862395,
  57.38466694161429,
  60.876925020897694,
  43.32435804486894,
  46.84015371452148,
  53.150729063673246,
  49.64876634922564,
  51.62363799656123,
  47.042533945356304,
  11.40175425099138,
  19.924858845171276,
  25.179356624028344,
  34.92849839314596,
  38.2099463490856,
  42.941821107167776,
  18.439088914585774,
  27.65863337187866,
  25.495097567963924,
  37.656340767525464,
  36.76955262170047,
  33.52610922848042,
  34.132096331752024,
  36.235341863986875,
  32.202484376209235,
  38.91015291668744,
  36.68787265568828,
  32.31098884280702,
  39.293765408777],
 [27.80287754891569,
  28.160255680657446,
  41.340053217188775,
  42.579337712087536,
  52.80151512977634,
  22.090722034374522,
  25.059928172283335,
  7.280109889280518,
  9.486832980505138,
  42.7551166528639,
  18.788294228055936,
  14.7648230602334,
  40.718546143004666,
  31.906112267087632,
  42.04759208325728,
  49.9799959983992,
  32.14031735997639,
  23.40939982143925,
  13.892443989449804,
  8.246211251235321,
  34.539832078341085,
  45.254833995939045,
  52.80151512977634,
  63.0317380372777,
  54.70831746635972,
  61.05735008989499,
  38.8329756778952,
  25.059928172283335,
  31.76476034853718,
  51.97114584074513,
  28.160255680657446,
  18.0,
  27.80287754891569,
  40.0,
  52.08646657242167,
  51.66236541235796,
  13.601470508735444,
  32.7566787083184,
  47.67598976424087,
  61.71709649683789,
  38.18376618407357,
  53.53503525729669,
  41.48493702538308,
  50.00999900019995,
  38.05259518088089,
  15.652475842498529,
  11.704699910719626,
  6.4031242374328485,
  0.0,
  17.46424919657298,
  34.36568055487916,
  36.49657518178932,
  16.64331697709324,
  31.89043743820395,
  49.64876634922564,
  57.8013840664737,
  56.568542494923804,
  44.28317965096906,
  34.713109915419565,
  29.120439557122072,
  18.439088914585774,
  28.0178514522438,
  12.529964086141668,
  22.02271554554524,
  25.079872407968907,
  55.00909015790027,
  41.677331968349414,
  71.58910531638176,
  44.9221548904324,
  24.515301344262525,
  24.331050121192877,
  46.8187996428785,
  49.51767361255979,
  46.75467891024384,
  51.088159097779204,
  54.56189146281496,
  37.36308338453881,
  41.0,
  48.0,
  44.181444068749045,
  45.5411901469428,
  42.04759208325728,
  5.385164807134504,
  15.033296378372908,
  21.095023109728988,
  30.14962686336267,
  35.17101079013795,
  37.16180835212409,
  13.0,
  21.400934559032695,
  23.430749027719962,
  33.06055050963308,
  31.32091952673165,
  28.442925306655784,
  28.178005607210743,
  30.463092423455635,
  26.570660511172846,
  33.24154027718932,
  31.575306807693888,
  26.92582403567252,
  34.36568055487916],
 [43.93176527297759,
  39.824615503479755,
  58.66856057549052,
  54.12947441089743,
  68.59300255857006,
  39.05124837953327,
  42.485291572496,
  22.80350850198276,
  25.317977802344327,
  49.64876634922564,
  25.298221281347036,
  14.317821063276353,
  55.0,
  49.24428900898052,
  58.69412236331676,
  67.4166151627327,
  48.16637831516918,
  38.01315561749642,
  31.304951684997057,
  12.041594578792296,
  39.11521443121589,
  61.84658438426491,
  69.8927750200262,
  79.81227975693966,
  67.60177512462228,
  76.05918747922567,
  54.45181356024793,
  40.311288741492746,
  46.75467891024384,
  63.56099432828282,
  34.92849839314596,
  29.68164415931166,
  29.017236257093817,
  49.64876634922564,
  60.41522986797286,
  57.07889277132134,
  8.94427190999916,
  50.0,
  63.00793600809346,
  77.79460135510689,
  54.817880294662984,
  70.83078426785913,
  58.82176467941097,
  67.1863081289633,
  54.230987451824994,
  30.0,
  20.396078054371138,
  12.165525060596439,
  17.46424919657298,
  0.0,
  46.06517122512408,
  44.14748010928823,
  32.64965543462902,
  48.27007354458868,
  64.20280367709809,
  72.62231062146122,
  73.10950690573696,
  61.741396161732524,
  51.61395160225576,
  46.486557196677836,
  35.73513677041127,
  44.40720662234904,
  20.591260281974,
  21.02379604162864,
  12.727922061357855,
  56.71860364994893,
  43.289721643826724,
  87.69264507357501,
  57.8013840664737,
  37.44329045369811,
  33.24154027718932,
  51.0,
  66.21933252457322,
  63.60031446463138,
  68.00735254367721,
  71.40028011149536,
  50.24937810560445,
  53.23532661682466,
  57.28001396647874,
  54.78138369920935,
  58.83026432033091,
  50.96076922496363,
  22.847319317591726,
  32.01562118716424,
  37.33630940518894,
  47.07440918375928,
  50.039984012787215,
  54.62600113499065,
  25.612496949731394,
  38.58756276314948,
  25.019992006393608,
  49.8196748283246,
  48.703182647543684,
  45.60701700396552,
  45.617978911828175,
  47.92702786528704,
  44.01136216933077,
  50.695167422546305,
  48.76474136094644,
  44.27188724235731,
  51.419840528729765],
 [16.97056274847714,
  6.324555320336759,
  32.31098884280702,
  8.246211251235321,
  28.160255680657446,
  36.235341863986875,
  27.80287754891569,
  27.16615541441225,
  37.21558813185679,
  15.264337522473747,
  21.400934559032695,
  32.449961479175904,
  12.36931687685298,
  27.80287754891569,
  48.91829923454004,
  45.31004303683677,
  45.803929962395145,
  45.31004303683677,
  27.892651361962706,
  34.539832078341085,
  18.110770276274835,
  27.073972741361768,
  37.05401462729781,
  42.7551166528639,
  21.633307652783937,
  32.449961479175904,
  17.11724276862369,
  13.892443989449804,
  11.661903789690601,
  17.72004514666935,
  14.7648230602334,
  16.76305461424021,
  25.059928172283335,
  7.810249675906654,
  19.697715603592208,
  24.08318915758459,
  46.84015371452148,
  38.18376618407357,
  59.39696961966999,
  37.33630940518894,
  23.08679276123039,
  40.311288741492746,
  41.88078318274385,
  50.11985634456667,
  48.83646178829912,
  41.97618372363071,
  45.0111097397076,
  40.024992192379,
  34.36568055487916,
  46.06517122512408,
  0.0,
  11.180339887498949,
  20.396078054371138,
  18.867962264113206,
  20.591260281974,
  28.844410203711913,
  35.510561809129406,
  38.07886552931954,
  23.706539182259394,
  34.713109915419565,
  32.69556544854363,
  41.88078318274385,
  25.495097567963924,
  29.732137494637012,
  43.86342439892262,
  33.54101966249684,
  26.076809620810597,
  46.51881339845203,
  12.041594578792296,
  10.0,
  13.45362404707371,
  23.259406699226016,
  31.0,
  30.14962686336267,
  34.0147027033899,
  36.05551275463989,
  5.385164807134504,
  7.211102550927978,
  14.866068747318506,
  10.04987562112089,
  13.45362404707371,
  10.63014581273465,
  32.0,
  34.48187929913333,
  39.395431207184416,
  39.824615503479755,
  51.86520991955976,
  34.66987164671943,
  21.587033144922902,
  24.186773244895647,
  25.612496949731394,
  42.5205832509386,
  35.35533905932738,
  37.013511046643494,
  29.0,
  31.827660925679098,
  32.01562118716424,
  34.058772731852805,
  38.2099463490856,
  34.20526275297414,
  41.012193308819754],
 [26.92582403567252,
  12.041594578792296,
  43.32435804486894,
  14.317821063276353,
  38.47076812334269,
  44.04543109109048,
  36.87817782917155,
  30.083217912982647,
  41.78516483155236,
  6.324555320336759,
  19.1049731745428,
  29.832867780352597,
  23.021728866442675,
  38.07886552931954,
  58.82176467941097,
  56.302753041036986,
  54.45181356024793,
  52.15361924162119,
  34.132096331752024,
  34.058772731852805,
  8.06225774829855,
  38.2099463490856,
  48.16637831516918,
  53.33854141237835,
  28.0178514522438,
  41.23105625617661,
  28.284271247461902,
  22.80350850198276,
  22.47220505424423,
  21.93171219946131,
  9.219544457292887,
  18.973665961010276,
  17.804493814764857,
  7.211102550927978,
  16.278820596099706,
  15.652475842498529,
  47.042533945356304,
  47.80167361086848,
  68.8839603971781,
  47.29693436154187,
  34.20526275297414,
  51.478150704935004,
  52.354560450833695,
  60.83584469702052,
  58.137767414994535,
  47.4236228055175,
  48.05205510693585,
  41.048751503547585,
  36.49657518178932,
  44.14748010928823,
  11.180339887498949,
  0.0,
  26.627053911388696,
  29.546573405388315,
  30.083217912982647,
  37.69615364994153,
  46.17358552246078,
  49.040799340956916,
  34.539832078341085,
  44.04543109109048,
  40.0,
  50.24937810560445,
  25.0,
  24.596747752497688,
  38.948684188300895,
  23.021728866442675,
  15.0,
  55.97320787662612,
  20.248456731316587,
  16.278820596099706,
  12.165525060596439,
  12.806248474865697,
  42.04759208325728,
  41.30375285612676,
  45.09988913511872,
  47.0,
  16.0,
  15.524174696260024,
  13.416407864998739,
  12.806248474865697,
  22.135943621178654,
  7.211102550927978,
  35.73513677041127,
  40.8166632639171,
  46.61544808322666,
  48.83646178829912,
  60.207972893961475,
  45.17742799230607,
  23.769728648009426,
  32.526911934581186,
  20.12461179749811,
  51.73973328110612,
  45.0,
  46.010868281309364,
  38.600518131237564,
  41.617304093369626,
  41.10960958218893,
  44.10215414239989,
  47.634021455258214,
  43.18564576337837,
  50.60632371551998],
 [11.313708498984761,
  15.231546211727817,
  27.202941017470888,
  28.071337695236398,
  36.235341863986875,
  17.69180601295413,
  13.152946437965905,
  9.899494936611665,
  17.0,
  32.7566787083184,
  17.26267650163207,
  23.08679276123039,
  24.351591323771842,
  18.24828759089466,
  35.11409973215888,
  38.118237105091836,
  28.600699292150182,
  25.553864678361276,
  7.615773105863909,
  20.808652046684813,
  28.42534080710379,
  29.206163733020468,
  37.589892258425,
  47.20169488482379,
  38.8329756778952,
  44.41846462902562,
  22.20360331117452,
  8.54400374531753,
  15.231546211727817,
  37.013511046643494,
  21.400934559032695,
  9.848857801796104,
  27.202941017470888,
  27.51363298439521,
  39.84971769034255,
  42.190046219457976,
  30.23243291566195,
  24.041630560342615,
  43.9089968002003,
  45.27692569068709,
  22.20360331117452,
  39.0,
  31.144823004794873,
  40.19950248448356,
  33.12099032335839,
  21.587033144922902,
  25.495097567963924,
  23.021728866442675,
  16.64331697709324,
  32.64965543462902,
  20.396078054371138,
  26.627053911388696,
  0.0,
  15.620499351813308,
  33.1058907144937,
  41.182520563948,
  40.45985664828782,
  31.400636936215164,
  19.235384061671343,
  19.924858845171276,
  13.45362404707371,
  24.20743687382041,
  15.297058540778355,
  26.0,
  36.05551275463989,
  48.79549159502341,
  37.20215047547655,
  55.17245689653489,
  29.274562336608895,
  10.770329614269007,
  16.401219466856727,
  39.05124837953327,
  33.60059523282288,
  31.064449134018133,
  35.510561809129406,
  38.8329756778952,
  22.02271554554524,
  26.0,
  35.17101079013795,
  30.4138126514911,
  29.614185789921695,
  30.083217912982647,
  12.649110640673518,
  14.317821063276353,
  20.0,
  23.706539182259394,
  33.97057550292606,
  25.019992006393608,
  9.055385138137417,
  8.06225774829855,
  24.331050121192877,
  26.832815729997478,
  21.587033144922902,
  21.02379604162864,
  16.0312195418814,
  19.1049731745428,
  16.76305461424021,
  22.090722034374522,
  23.40939982143925,
  18.384776310850235,
  26.570660511172846],
 [4.47213595499958,
  18.439088914585774,
  14.142135623730951,
  22.80350850198276,
  21.095023109728988,
  22.02271554554524,
  12.041594578792296,
  25.495097567963924,
  29.546573405388315,
  34.132096331752024,
  29.832867780352597,
  38.01315561749642,
  13.601470508735444,
  9.219544457292887,
  30.4138126514911,
  26.92582403567252,
  29.154759474226502,
  32.01562118716424,
  19.235384061671343,
  36.40054944640259,
  34.92849839314596,
  13.601470508735444,
  22.47220505424423,
  31.622776601683793,
  28.284271247461902,
  30.083217912982647,
  8.06225774829855,
  9.219544457292887,
  7.211102550927978,
  29.154759474226502,
  29.154759474226502,
  21.840329667841555,
  38.05259518088089,
  26.40075756488817,
  36.87817782917155,
  42.80186911806539,
  45.45327270945405,
  20.248456731316587,
  41.23105625617661,
  29.832867780352597,
  6.708203932499369,
  24.515301344262525,
  23.021728866442675,
  31.304951684997057,
  31.064449134018133,
  31.78049716414141,
  38.91015291668744,
  38.28837943815329,
  31.89043743820395,
  48.27007354458868,
  18.867962264113206,
  29.546573405388315,
  15.620499351813308,
  0.0,
  20.09975124224178,
  27.202941017470888,
  24.839484696748443,
  19.6468827043885,
  5.0990195135927845,
  17.46424919657298,
  20.223748416156685,
  25.96150997149434,
  29.966648127543394,
  39.293765408777,
  50.99019513592785,
  52.354560450833695,
  43.68065933568311,
  39.698866482558415,
  20.615528128088304,
  16.0,
  25.0,
  42.05948168962618,
  18.027756377319946,
  15.652475842498529,
  20.12461179749811,
  23.323807579381203,
  16.278820596099706,
  20.0,
  31.89043743820395,
  26.248809496813376,
  19.924858845171276,
  29.206163733020468,
  27.202941017470888,
  23.769728648009426,
  26.0,
  22.847319317591726,
  35.4682957019364,
  15.811388300841896,
  23.706539182259394,
  11.704699910719626,
  36.49657518178932,
  25.059928172283335,
  17.4928556845359,
  20.248456731316587,
  11.704699910719626,
  13.892443989449804,
  15.524174696260024,
  15.620499351813308,
  20.591260281974,
  17.72004514666935,
  23.021728866442675],
 [22.80350850198276,
  25.612496949731394,
  25.059928172283335,
  16.1245154965971,
  9.219544457292887,
  42.01190307520001,
  32.01562118716424,
  42.5440947723653,
  49.040799340956916,
  31.064449134018133,
  41.10960958218893,
  51.62363799656123,
  9.219544457292887,
  27.294688127912362,
  46.09772228646444,
  36.124783736376884,
  47.853944456021594,
  52.009614495783374,
  38.600518131237564,
  52.20153254455275,
  37.94733192202055,
  15.0,
  22.47220505424423,
  24.08318915758459,
  10.0,
  12.041594578792296,
  12.041594578792296,
  24.596747752497688,
  17.88854381999832,
  14.7648230602334,
  35.35533905932738,
  34.713109915419565,
  45.65084884205331,
  23.345235059857504,
  27.202941017470888,
  36.49657518178932,
  63.13477647065839,
  38.07886552931954,
  57.271284253105414,
  17.26267650163207,
  17.46424919657298,
  26.627053911388696,
  37.12142238654117,
  42.80186911806539,
  48.38388161361178,
  51.78802950489621,
  58.18934610390462,
  55.94640292279746,
  49.64876634922564,
  64.20280367709809,
  20.591260281974,
  30.083217912982647,
  33.1058907144937,
  20.09975124224178,
  0.0,
  8.48528137423857,
  17.46424919657298,
  30.23243291566195,
  21.213203435596427,
  36.345563690772494,
  40.311288741492746,
  45.27692569068709,
  43.93176527297759,
  50.0,
  63.7808748764079,
  48.25971404805462,
  44.721359549995796,
  26.0,
  10.04987562112089,
  26.90724809414742,
  33.60059523282288,
  39.0,
  16.401219466856727,
  17.69180601295413,
  19.4164878389476,
  19.697715603592208,
  15.264337522473747,
  14.560219778561036,
  23.345235059857504,
  19.0,
  8.06225774829855,
  25.079872407968907,
  45.69463863518345,
  43.73785545725808,
  46.04345773288535,
  41.593268686170845,
  54.12947441089743,
  31.016124838541646,
  38.600518131237564,
  31.575306807693888,
  46.04345773288535,
  43.174066289845804,
  35.6931365951495,
  39.319206502675,
  31.064449134018133,
  32.38826948140329,
  35.05709628591621,
  32.984845004941285,
  38.8329756778952,
  37.12142238654117,
  40.52159917870962],
 [30.463092423455635,
  34.058772731852805,
  28.635642126552707,
  23.40939982143925,
  8.54400374531753,
  48.507731342539614,
  38.63935817272331,
  50.774009099144415,
  56.64803615307419,
  37.8549864614954,
  49.57822102496216,
  60.108235708594876,
  17.69180601295413,
  33.06055050963308,
  49.72926703662542,
  37.589892258425,
  53.08483775994799,
  58.42088667591412,
  46.238512086787566,
  60.60528029800704,
  45.69463863518345,
  18.24828759089466,
  22.20360331117452,
  19.697715603592208,
  12.165525060596439,
  3.605551275463989,
  19.313207915827967,
  32.7566787083184,
  26.076809620810597,
  19.026297590440446,
  43.56604182158393,
  43.18564576337837,
  53.85164807134504,
  30.675723300355934,
  32.0624390837628,
  42.0,
  71.34423592694787,
  43.104524124504614,
  60.728905802755904,
  11.40175425099138,
  23.08679276123039,
  26.40075756488817,
  40.52159917870962,
  44.721359549995796,
  52.773099207835045,
  58.9406481131655,
  65.92419889539804,
  64.1404708432983,
  57.8013840664737,
  72.62231062146122,
  28.844410203711913,
  37.69615364994153,
  41.182520563948,
  27.202941017470888,
  8.48528137423857,
  0.0,
  14.866068747318506,
  32.89376840679705,
  27.16615541441225,
  42.01190307520001,
  47.29693436154187,
  51.0098029794274,
  52.40229002629561,
  58.412327466040935,
  72.24956747275377,
  54.00925846556311,
  51.92301994298868,
  18.439088914585774,
  17.46424919657298,
  35.38361202590826,
  42.01190307520001,
  45.39823785126467,
  17.46424919657298,
  19.924858845171276,
  19.72308292331602,
  18.439088914585774,
  23.600847442411894,
  22.360679774997898,
  29.068883707497267,
  25.709920264364882,
  15.652475842498529,
  32.01562118716424,
  53.665631459994955,
  50.96076922496363,
  52.61178575186362,
  47.01063709417264,
  59.21148537234985,
  35.35533905932738,
  47.01063709417264,
  38.897300677553446,
  54.405882034941776,
  48.16637831516918,
  41.012193308819754,
  45.0111097397076,
  37.21558813185679,
  38.01315561749642,
  41.19465984809196,
  38.05259518088089,
  44.04543109109048,
  43.104524124504614,
  45.27692569068709],
 [29.206163733020468,
  38.897300677553446,
  18.681541692269406,
  33.06055050963308,
  8.246211251235321,
  42.04759208325728,
  33.28663395418648,
  50.32891812864648,
  53.009433122794285,
  48.041648597857254,
  53.225933528685054,
  62.433965115151864,
  23.194827009486403,
  26.419689627245813,
  38.05259518088089,
  24.041630560342615,
  43.73785545725808,
  51.264022471905186,
  43.278170016764804,
  61.220911460055866,
  53.600373133029585,
  11.313708498984761,
  8.246211251235321,
  7.280109889280518,
  25.942243542145693,
  14.422205101855956,
  19.697715603592208,
  33.28663395418648,
  27.730849247724095,
  31.953090617340916,
  49.72926703662542,
  45.65084884205331,
  59.77457653551382,
  40.0,
  44.64302857109943,
  53.9351462406472,
  70.03570517957252,
  33.95585369269929,
  48.507731342539614,
  7.0,
  18.384776310850235,
  12.083045973594572,
  29.0,
  31.32091952673165,
  42.04759208325728,
  53.71219600798314,
  62.42595614005443,
  62.93647591023825,
  56.568542494923804,
  73.10950690573696,
  35.510561809129406,
  46.17358552246078,
  40.45985664828782,
  24.839484696748443,
  17.46424919657298,
  14.866068747318506,
  0.0,
  21.0,
  22.02271554554524,
  34.17601498127012,
  42.190046219457976,
  42.720018726587654,
  54.378304497290095,
  62.64982043070834,
  75.29276193632427,
  65.62011886609167,
  61.1310068623117,
  15.652475842498529,
  27.16615541441225,
  38.48376280978771,
  46.8187996428785,
  56.142675390472796,
  7.211102550927978,
  10.295630140987,
  7.0710678118654755,
  4.123105625617661,
  30.265491900843113,
  31.0,
  40.792156108742276,
  36.22154055254967,
  25.179356624028344,
  42.04759208325728,
  51.66236541235796,
  46.32493928760188,
  46.09772228646444,
  38.3275357934736,
  49.36598018878993,
  25.709920264364882,
  48.25971404805462,
  35.4682957019364,
  59.23681287847955,
  38.63935817272331,
  32.57299494980466,
  37.0,
  30.886890422961002,
  30.463092423455635,
  34.438350715445125,
  29.410882339705484,
  35.17101079013795,
  35.84689665786984,
  35.510561809129406],
 [23.194827009486403,
  38.07886552931954,
  5.830951894845301,
  40.22437072223753,
  24.351591323771842,
  24.758836806279895,
  19.313207915827967,
  39.84971769034255,
  38.01315561749642,
  53.225933528685054,
  48.041648597857254,
  54.3415126767741,
  29.206163733020468,
  13.152946437965905,
  17.11724276862369,
  7.280109889280518,
  23.40939982143925,
  32.449961479175904,
  30.463092423455635,
  50.92150822589606,
  54.57105459856901,
  15.264337522473747,
  13.152946437965905,
  24.041630560342615,
  40.22437072223753,
  33.95585369269929,
  22.20360331117452,
  28.160255680657446,
  26.570660511172846,
  43.86342439892262,
  48.662100242385755,
  40.01249804748511,
  57.07889277132134,
  45.17742799230607,
  54.20332093147061,
  61.40032573203501,
  56.60388679233962,
  14.422205101855956,
  27.892651361962706,
  28.0,
  15.264337522473747,
  11.180339887498949,
  8.0,
  12.727922061357855,
  21.095023109728988,
  36.76955262170047,
  46.861498055439924,
  50.11985634456667,
  44.28317965096906,
  61.741396161732524,
  38.07886552931954,
  49.040799340956916,
  31.400636936215164,
  19.6468827043885,
  30.23243291566195,
  32.89376840679705,
  21.0,
  0.0,
  14.560219778561036,
  16.278820596099706,
  26.627053911388696,
  23.323807579381203,
  46.69047011971501,
  57.21887800367987,
  67.1863081289633,
  71.58910531638176,
  63.324560795950255,
  35.6931365951495,
  36.124783736376884,
  35.35533905932738,
  44.28317965096906,
  61.326992425847855,
  15.524174696260024,
  13.0,
  14.035668847618199,
  17.029386365926403,
  34.48187929913333,
  37.44329045369811,
  49.4064773081425,
  43.829214001622255,
  34.655446902326915,
  47.885279575251516,
  38.91015291668744,
  30.805843601498726,
  28.319604517012593,
  18.867962264113206,
  28.635642126552707,
  7.211102550927978,
  40.44749683231337,
  23.769728648009426,
  55.00909015790027,
  18.384776310850235,
  14.142135623730951,
  18.439088914585774,
  16.15549442140351,
  13.892443989449804,
  18.027756377319946,
  11.40175425099138,
  15.811388300841896,
  18.439088914585774,
  15.231546211727817],
 [9.055385138137417,
  23.53720459187964,
  9.055385138137417,
  26.870057685088806,
  19.924858845171276,
  21.37755832643195,
  11.704699910719626,
  28.844410203711913,
  31.064449134018133,
  38.948684188300895,
  34.52535300326414,
  42.15447781671598,
  16.64331697709324,
  6.082762530298219,
  26.40075756488817,
  21.840329667841555,
  26.68332812825267,
  31.25699921617557,
  21.2602916254693,
  39.96248240537617,
  40.024992192379,
  10.816653826391969,
  18.35755975068582,
  28.319604517012593,
  30.364452901377952,
  29.614185789921695,
  9.848857801796104,
  14.035668847618199,
  12.083045973594572,
  32.2490309931942,
  34.23448553724738,
  26.476404589747453,
  43.01162633521314,
  31.064449134018133,
  41.012193308819754,
  47.43416490252569,
  48.08326112068523,
  17.08800749063506,
  37.44329045369811,
  27.784887978899608,
  4.123105625617661,
  19.924858845171276,
  18.439088914585774,
  26.419689627245813,
  27.730849247724095,
  32.31098884280702,
  40.496913462633174,
  41.036569057366385,
  34.713109915419565,
  51.61395160225576,
  23.706539182259394,
  34.539832078341085,
  19.235384061671343,
  5.0990195135927845,
  21.213203435596427,
  27.16615541441225,
  22.02271554554524,
  14.560219778561036,
  0.0,
  15.132745950421556,
  20.615528128088304,
  24.08318915758459,
  34.17601498127012,
  43.93176527297759,
  55.154328932550705,
  57.245087125446844,
  48.76474136094644,
  37.44329045369811,
  23.853720883753127,
  21.02379604162864,
  30.01666203960727,
  46.95742752749558,
  14.866068747318506,
  12.041594578792296,
  16.401219466856727,
  19.849433241279208,
  20.615528128088304,
  24.041630560342615,
  36.069377593742864,
  30.4138126514911,
  22.825424421026653,
  33.83784863137726,
  29.698484809834994,
  24.596747752497688,
  25.495097567963924,
  20.396078054371138,
  32.984845004941285,
  11.313708498984761,
  27.85677655436824,
  13.45362404707371,
  41.30375285612676,
  22.135943621178654,
  14.560219778561036,
  18.110770276274835,
  10.04987562112089,
  11.180339887498949,
  14.035668847618199,
  12.083045973594572,
  17.72004514666935,
  16.0,
  19.697715603592208],
 [17.804493814764857,
  32.01562118716424,
  15.652475842498529,
  39.96248240537617,
  34.23448553724738,
  8.48528137423857,
  7.211102550927978,
  26.019223662515376,
  21.95449840010015,
  49.51767361255979,
  37.107950630558946,
  41.012193308819754,
  31.016124838541646,
  9.055385138137417,
  15.231546211727817,
  21.02379604162864,
  11.704699910719626,
  17.08800749063506,
  16.0312195418814,
  36.49657518178932,
  47.50789408087881,
  24.331050121192877,
  27.784887978899608,
  38.948684188300895,
  45.35416188179427,
  44.181444068749045,
  24.73863375370596,
  21.2602916254693,
  23.853720883753127,
  46.61544808322666,
  40.70626487409524,
  29.732137494637012,
  47.12748667179272,
  42.5205832509386,
  53.823786563191554,
  58.69412236331676,
  40.70626487409524,
  4.123105625617661,
  24.839484696748443,
  40.80441152620633,
  19.026297590440446,
  27.018512172212592,
  12.36931687685298,
  21.095023109728988,
  14.142135623730951,
  20.518284528683193,
  30.610455730027933,
  34.539832078341085,
  29.120439557122072,
  46.486557196677836,
  34.713109915419565,
  44.04543109109048,
  19.924858845171276,
  17.46424919657298,
  36.345563690772494,
  42.01190307520001,
  34.17601498127012,
  16.278820596099706,
  15.132745950421556,
  0.0,
  10.770329614269007,
  9.0,
  34.132096331752024,
  45.39823785126467,
  53.33854141237835,
  67.00746227100382,
  56.43580423808985,
  49.76946855251722,
  38.07886552931954,
  28.0178514522438,
  35.77708763999664,
  56.85068161420758,
  27.202941017470888,
  24.041630560342615,
  27.313000567495326,
  30.870698080866262,
  33.28663395418648,
  37.21558813185679,
  48.82622246293481,
  43.266615305567875,
  37.33630940518894,
  45.34313619501854,
  23.769728648009426,
  14.7648230602334,
  12.206555615733702,
  5.385164807134504,
  18.027756377319946,
  9.219544457292887,
  28.442925306655784,
  12.083045973594572,
  44.14748010928823,
  7.810249675906654,
  2.23606797749979,
  3.0,
  5.830951894845301,
  4.0,
  3.1622776601683795,
  5.0,
  3.605551275463989,
  2.23606797749979,
  6.708203932499369],
 [18.027756377319946,
  28.30194339616981,
  24.758836806279895,
  39.56008088970496,
  40.496913462633174,
  4.47213595499958,
  8.94427190999916,
  16.278820596099706,
  11.40175425099138,
  46.04345773288535,
  29.068883707497267,
  31.144823004794873,
  33.015148038438355,
  15.811388300841896,
  24.08318915758459,
  31.78049716414141,
  15.652475842498529,
  12.649110640673518,
  6.708203932499369,
  26.076809620810597,
  41.773197148410844,
  31.304951684997057,
  36.87817782917155,
  47.80167361086848,
  48.010415536631214,
  50.0,
  28.284271247461902,
  18.973665961010276,
  24.186773244895647,
  47.67598976424087,
  34.713109915419565,
  22.80350850198276,
  39.35733730830886,
  40.24922359499622,
  52.392747589718944,
  55.47071299343465,
  30.01666203960727,
  14.317821063276353,
  31.38470965295043,
  48.30113870293329,
  24.698178070456937,
  36.796738985948195,
  23.08679276123039,
  31.575306807693888,
  20.8806130178211,
  11.704699910719626,
  20.518284528683193,
  23.769728648009426,
  18.439088914585774,
  35.73513677041127,
  32.69556544854363,
  40.0,
  13.45362404707371,
  20.223748416156685,
  40.311288741492746,
  47.29693436154187,
  42.190046219457976,
  26.627053911388696,
  20.615528128088304,
  10.770329614269007,
  0.0,
  11.180339887498949,
  25.0,
  36.40054944640259,
  43.04648650006177,
  62.24146527838174,
  50.44799302251776,
  57.8013840664737,
  39.319206502675,
  23.853720883753127,
  29.732137494637012,
  52.49761899362675,
  34.9857113690718,
  31.906112267087632,
  35.805027579936315,
  39.408120990476064,
  32.984845004941285,
  37.107950630558946,
  47.53945729601885,
  42.37924020083418,
  39.11521443121589,
  42.941821107167776,
  13.152946437965905,
  4.242640687119285,
  6.708203932499369,
  12.041594578792296,
  20.615528128088304,
  19.4164878389476,
  20.12461179749811,
  9.055385138137417,
  35.84689665786984,
  15.132745950421556,
  13.0,
  10.04987562112089,
  11.40175425099138,
  12.806248474865697,
  8.602325267042627,
  15.264337522473747,
  13.152946437965905,
  8.54400374531753,
  16.0312195418814],
 [25.495097567963924,
  38.28837943815329,
  24.041630560342615,
  47.853944456021594,
  43.18564576337837,
  6.708203932499369,
  14.317821063276353,
  27.202941017470888,
  19.1049731745428,
  56.08029957123981,
  40.24922359499622,
  41.773197148410844,
  39.56008088970496,
  18.027756377319946,
  14.317821063276353,
  26.1725046566048,
  4.47213595499958,
  9.219544457292887,
  17.88854381999832,
  36.124783736376884,
  52.630789467763066,
  33.24154027718932,
  35.84689665786984,
  47.01063709417264,
  54.12947441089743,
  53.150729063673246,
  33.54101966249684,
  28.0178514522438,
  31.78049716414141,
  55.02726596879042,
  45.60701700396552,
  33.83784863137726,
  50.53711507397311,
  49.64876634922564,
  61.40032573203501,
  65.43699259593154,
  37.36308338453881,
  8.94427190999916,
  20.248456731316587,
  49.47726750741192,
  28.0178514522438,
  34.48187929913333,
  16.97056274847714,
  23.706539182259394,
  10.04987562112089,
  15.231546211727817,
  26.0,
  32.2490309931942,
  28.0178514522438,
  44.40720662234904,
  41.88078318274385,
  50.24937810560445,
  24.20743687382041,
  25.96150997149434,
  45.27692569068709,
  51.0098029794274,
  42.720018726587654,
  23.323807579381203,
  24.08318915758459,
  9.0,
  11.180339887498949,
  0.0,
  36.05551275463989,
  47.43416490252569,
  53.08483775994799,
  72.86288492778748,
  61.40032573203501,
  58.18934610390462,
  46.486557196677836,
  33.97057550292606,
  40.607881008493905,
  62.93647591023825,
  35.90264614203248,
  32.7566787083184,
  35.73513677041127,
  39.21734310225516,
  41.14608122288197,
  45.18849411078001,
  56.43580423808985,
  51.0,
  45.880278987817846,
  52.43090691567332,
  23.194827009486403,
  13.152946437965905,
  7.0710678118654755,
  4.47213595499958,
  10.0,
  17.08800749063506,
  31.304951684997057,
  17.804493814764857,
  47.01063709417264,
  5.830951894845301,
  10.198039027185569,
  6.0,
  14.317821063276353,
  13.0,
  10.44030650891055,
  13.341664064126334,
  7.615773105863909,
  8.246211251235321,
  8.48528137423857],
 [25.495097567963924,
  19.235384061671343,
  42.44997055358225,
  33.61547262794322,
  49.040799340956916,
  29.410882339705484,
  28.0178514522438,
  8.94427190999916,
  20.518284528683193,
  31.064449134018133,
  6.324555320336759,
  8.06225774829855,
  34.713109915419565,
  33.54101966249684,
  48.83646178829912,
  53.33854141237835,
  40.496913462633174,
  33.83784863137726,
  18.439088914585774,
  9.219544457292887,
  22.135943621178654,
  43.41658669218482,
  52.392747589718944,
  61.40032573203501,
  47.01063709417264,
  55.90169943749474,
  35.0,
  21.095023109728988,
  27.018512172212592,
  43.08131845707603,
  16.1245154965971,
  9.219544457292887,
  15.556349186104045,
  29.614185789921695,
  41.10960958218893,
  39.6232255123179,
  22.090722034374522,
  38.2099463490856,
  56.302753041036986,
  58.412327466040935,
  36.6742416417845,
  54.08326913195984,
  46.0,
  55.00909015790027,
  45.880278987817846,
  26.90724809414742,
  24.166091947189145,
  16.1245154965971,
  12.529964086141668,
  20.591260281974,
  25.495097567963924,
  25.0,
  15.297058540778355,
  29.966648127543394,
  43.93176527297759,
  52.40229002629561,
  54.378304497290095,
  46.69047011971501,
  34.17601498127012,
  34.132096331752024,
  25.0,
  36.05551275463989,
  0.0,
  11.40175425099138,
  21.02379604162864,
  42.485291572496,
  29.154759474226502,
  68.24954212300622,
  37.21558813185679,
  17.029386365926403,
  13.152946437965905,
  34.48187929913333,
  47.885279575251516,
  45.617978911828175,
  50.08991914547278,
  53.23532661682466,
  29.68164415931166,
  32.64965543462902,
  37.48332962798263,
  34.48187929913333,
  38.27531841800928,
  31.25699921617557,
  14.212670403551895,
  23.259406699226016,
  29.966648127543394,
  36.87817782917155,
  44.721359549995796,
  40.19950248448356,
  6.324555320336759,
  23.08679276123039,
  11.40175425099138,
  40.024992192379,
  36.05551275463989,
  34.52535300326414,
  31.064449134018133,
  34.0147027033899,
  31.064449134018133,
  37.013511046643494,
  37.33630940518894,
  32.2490309931942,
  40.44749683231337],
 [34.92849839314596,
  24.413111231467404,
  52.61178575186362,
  36.87817782917155,
  56.43580423808985,
  40.80441152620633,
  39.05124837953327,
  20.248456731316587,
  31.064449134018133,
  29.410882339705484,
  9.486832980505138,
  8.06225774829855,
  41.048751503547585,
  44.10215414239989,
  60.207972893961475,
  64.07027391856539,
  51.86520991955976,
  44.77722635447622,
  29.832867780352597,
  15.0,
  18.439088914585774,
  52.20153254455275,
  61.68468205316454,
  69.85699678629192,
  50.99019513592785,
  62.00806399170998,
  42.95346318982906,
  30.083217912982647,
  34.92849839314596,
  45.803929962395145,
  15.811388300841896,
  17.46424919657298,
  8.0,
  31.064449134018133,
  40.496913462633174,
  36.22154055254967,
  26.570660511172846,
  49.49747468305833,
  67.67569726275453,
  65.86349520030045,
  45.880278987817846,
  63.788713735268246,
  57.07889277132134,
  66.12110101926616,
  57.28001396647874,
  37.44329045369811,
  32.64965543462902,
  23.021728866442675,
  22.02271554554524,
  21.02379604162864,
  29.732137494637012,
  24.596747752497688,
  26.0,
  39.293765408777,
  50.0,
  58.412327466040935,
  62.64982043070834,
  57.21887800367987,
  43.93176527297759,
  45.39823785126467,
  36.40054944640259,
  47.43416490252569,
  11.40175425099138,
  0.0,
  14.422205101855956,
  35.90264614203248,
  22.360679774997898,
  75.47184905645283,
  41.72529209005013,
  24.166091947189145,
  16.401219466856727,
  30.01666203960727,
  56.64803615307419,
  54.70831746635972,
  59.135437767890075,
  62.03224967708329,
  34.828149534535996,
  36.76955262170047,
  38.01315561749642,
  36.61966684720111,
  43.18564576337837,
  31.76476034853718,
  25.059928172283335,
  34.539832078341085,
  41.23105625617661,
  48.27007354458868,
  55.94640292279746,
  51.0098029794274,
  17.029386365926403,
  34.0147027033899,
  4.47213595499958,
  51.419840528729765,
  47.265209192385896,
  45.89117562233506,
  42.01190307520001,
  45.044422518220834,
  42.2965719651132,
  48.041648597857254,
  48.662100242385755,
  43.56604182158393,
  51.78802950489621],
 [46.51881339845203,
  38.2099463490856,
  63.245553203367585,
  51.22499389946279,
  69.63476143421474,
  47.0,
  48.05205510693585,
  27.459060435491963,
  34.36568055487916,
  43.46262762420146,
  22.67156809750927,
  13.0,
  54.67174773134658,
  54.120236510939236,
  67.0,
  73.54590403278758,
  57.21887800367987,
  48.05205510693585,
  37.33630940518894,
  17.0,
  32.31098884280702,
  64.41273166075166,
  73.40980860893181,
  82.365041127896,
  65.29931086925804,
  75.82216034906945,
  55.758407437802596,
  42.05948168962618,
  47.70744176750625,
  60.21627686929839,
  30.23243291566195,
  29.68164415931166,
  21.540659228538015,
  45.48626166217664,
  54.62600113499065,
  49.47726750741192,
  21.400934559032695,
  57.21887800367987,
  72.69112738154499,
  79.05694150420949,
  57.697486947006624,
  75.02666192761077,
  65.62011886609167,
  74.43117626371358,
  63.12685640834652,
  40.024992192379,
  31.78049716414141,
  22.135943621178654,
  25.079872407968907,
  12.727922061357855,
  43.86342439892262,
  38.948684188300895,
  36.05551275463989,
  50.99019513592785,
  63.7808748764079,
  72.24956747275377,
  75.29276193632427,
  67.1863081289633,
  55.154328932550705,
  53.33854141237835,
  43.04648650006177,
  53.08483775994799,
  21.02379604162864,
  14.422205101855956,
  0.0,
  47.0,
  34.23448553724738,
  88.81441324469807,
  55.90169943749474,
  37.20215047547655,
  30.4138126514911,
  42.95346318982906,
  68.8839603971781,
  66.64082832618455,
  71.11258679024411,
  74.24284477308234,
  48.79549159502341,
  50.99019513592785,
  52.354560450833695,
  51.03920062069938,
  57.28001396647874,
  46.14108798023731,
  30.0,
  40.01249804748511,
  46.17358552246078,
  55.00909015790027,
  60.01666435249463,
  60.41522986797286,
  27.313000567495326,
  43.41658669218482,
  18.867962264113206,
  58.0,
  55.44366510251645,
  53.08483775994799,
  51.19570294468082,
  53.9351462406472,
  50.48762224545735,
  56.88585061331157,
  56.142675390472796,
  51.24451190127583,
  59.07622195096772],
 [49.92995093127971,
  35.0,
  65.79513659838392,
  32.7566787083184,
  57.42821606144492,
  66.46803743153546,
  59.81638571495272,
  49.92995093127971,
  62.12889826803627,
  18.384776310850235,
  36.235341863986875,
  43.68065933568311,
  43.68065933568311,
  61.05735008989499,
  81.8413098624405,
  78.79086241436883,
  77.15568676384133,
  73.87827826905551,
  55.97320787662612,
  49.9799959983992,
  20.808652046684813,
  59.481089431852205,
  69.12307863514182,
  72.3394774656273,
  42.1070065428546,
  57.07889277132134,
  49.9799959983992,
  45.803929962395145,
  45.17742799230607,
  35.05709628591621,
  27.80287754891569,
  39.824615503479755,
  28.160255680657446,
  26.570660511172846,
  22.20360331117452,
  12.041594578792296,
  62.36184731067546,
  70.80254232723568,
  91.8313671900838,
  65.03076195155643,
  56.462376853972415,
  72.80109889280519,
  75.29276193632427,
  83.64807230295268,
  81.05553651663777,
  68.24221567329127,
  66.64082832618455,
  57.9396237474839,
  55.00909015790027,
  56.71860364994893,
  33.54101966249684,
  23.021728866442675,
  48.79549159502341,
  52.354560450833695,
  48.25971404805462,
  54.00925846556311,
  65.62011886609167,
  71.58910531638176,
  57.245087125446844,
  67.00746227100382,
  62.24146527838174,
  72.86288492778748,
  42.485291572496,
  35.90264614203248,
  47.0,
  0.0,
  13.601470508735444,
  72.17340230306452,
  38.47076812334269,
  39.05124837953327,
  32.64965543462902,
  10.295630140987,
  62.81719509815764,
  62.6418390534633,
  65.96969000988257,
  67.26812023536856,
  37.33630940518894,
  35.17101079013795,
  25.019992006393608,
  29.427877939124322,
  40.44749683231337,
  24.041630560342615,
  55.758407437802596,
  62.48199740725323,
  68.6804193347711,
  71.7007670809734,
  82.73451516749222,
  68.09552114493287,
  43.829214001622255,
  55.31726674375732,
  32.57299494980466,
  74.65252842335616,
  68.00735254367721,
  68.8839603971781,
  61.61168720299745,
  64.63745044476924,
  64.03124237432849,
  67.11929677819934,
  70.60453243241541,
  66.06814663663572,
  73.60027173862879],
 [40.496913462633174,
  25.298221281347036,
  57.723478758647246,
  28.635642126552707,
  53.33854141237835,
  54.817880294662984,
  49.24428900898052,
  37.013511046643494,
  49.20365840057018,
  14.317821063276353,
  23.021728866442675,
  30.083217912982647,
  38.01315561749642,
  51.62363799656123,
  71.58910531638176,
  70.60453243241541,
  65.80273550544841,
  61.5223536610881,
  43.93176527297759,
  36.40054944640259,
  8.94427190999916,
  53.150729063673246,
  63.12685640834652,
  68.26419266350405,
  41.23105625617661,
  55.362442142665635,
  43.18564576337837,
  35.84689665786984,
  36.87817782917155,
  34.438350715445125,
  15.811388300841896,
  27.65863337187866,
  14.560219778561036,
  21.37755832643195,
  24.08318915758459,
  16.1245154965971,
  48.76474136094644,
  60.41522986797286,
  80.99382692526635,
  61.98386886924694,
  48.83646178829912,
  66.37017402418047,
  65.92419889539804,
  74.67261881037788,
  70.178344238091,
  55.44366510251645,
  53.23532661682466,
  44.384682042344295,
  41.677331968349414,
  43.289721643826724,
  26.076809620810597,
  15.0,
  37.20215047547655,
  43.68065933568311,
  44.721359549995796,
  51.92301994298868,
  61.1310068623117,
  63.324560795950255,
  48.76474136094644,
  56.43580423808985,
  50.44799302251776,
  61.40032573203501,
  29.154759474226502,
  22.360679774997898,
  34.23448553724738,
  13.601470508735444,
  0.0,
  70.3420215802759,
  34.713109915419565,
  28.635642126552707,
  20.808652046684813,
  9.433981132056603,
  57.0350769263968,
  56.22277118748239,
  60.07495318350236,
  62.0,
  31.0,
  30.265491900843113,
  24.186773244895647,
  26.248809496813376,
  36.6742416417845,
  19.924858845171276,
  42.80186911806539,
  50.20956084253277,
  56.639209034025185,
  60.74537019394976,
  71.06335201775947,
  58.872744118140105,
  31.144823004794873,
  44.41846462902562,
  18.973665961010276,
  63.812224534175265,
  57.706152185014034,
  57.982756057296896,
  51.42956348249516,
  54.56189146281496,
  53.33854141237835,
  57.271284253105414,
  60.03332407921454,
  55.226805085936306,
  63.13477647065839],
 [43.86342439892262,
  51.10772935672255,
  34.17601498127012,
  41.7612260356422,
  19.209372712298546,
  57.697486947006624,
  48.877397639399746,
  65.06919393998976,
  68.5054742338158,
  56.293871780150276,
  66.2872536767062,
  76.21679604916491,
  34.48187929913333,
  42.05948168962618,
  52.23983154643591,
  37.0,
  58.9406481131655,
  66.85057965343307,
  58.60034129593445,
  75.69015788066504,
  63.90618123468183,
  26.627053911388696,
  22.561028345356956,
  12.0,
  30.066592756745816,
  15.132745950421556,
  33.301651610693426,
  47.4236228055175,
  41.23105625617661,
  37.12142238654117,
  61.26989472816156,
  59.20304046246274,
  71.55417527999327,
  49.040799340956916,
  50.039984012787215,
  60.13318551349163,
  85.14693182963201,
  49.33558553417604,
  62.0,
  9.899494936611665,
  33.60059523282288,
  25.079872407968907,
  43.56604182158393,
  44.04543109109048,
  56.71860364994893,
  69.35416353759881,
  77.93587107359485,
  77.98717843338096,
  71.58910531638176,
  87.69264507357501,
  46.51881339845203,
  55.97320787662612,
  55.17245689653489,
  39.698866482558415,
  26.0,
  18.439088914585774,
  15.652475842498529,
  35.6931365951495,
  37.44329045369811,
  49.76946855251722,
  57.8013840664737,
  58.18934610390462,
  68.24954212300622,
  75.47184905645283,
  88.81441324469807,
  72.17340230306452,
  70.3420215802759,
  0.0,
  35.73513677041127,
  51.61395160225576,
  59.16924876994806,
  63.788713735268246,
  22.825424421026653,
  25.942243542145693,
  22.47220505424423,
  18.973665961010276,
  41.14608122288197,
  40.496913462633174,
  47.38143096192854,
  44.14748010928823,
  33.83784863137726,
  50.44799302251776,
  66.84309986827361,
  61.91122676865643,
  61.741396161732524,
  53.75872022286245,
  64.32728814430156,
  41.10960958218893,
  62.369864518050704,
  50.80354318352215,
  71.69379331573968,
  53.85164807134504,
  48.104053883222775,
  52.55473337388365,
  46.52956049652737,
  46.09772228646444,
  50.08991914547278,
  44.94441010848846,
  50.59644256269407,
  51.478150704935004,
  50.695167422546305],
 [21.37755832643195,
  18.027756377319946,
  30.4138126514911,
  6.082762530298219,
  19.026297590440446,
  41.97618372363071,
  32.28002478313795,
  37.64306044943742,
  46.17358552246078,
  21.02379604162864,
  33.421549934136806,
  44.40720662234904,
  7.211102550927978,
  29.5296461204668,
  50.21951811795888,
  42.80186911806539,
  49.76946855251722,
  51.78802950489621,
  36.013886210738214,
  46.06517122512408,
  28.231188426986208,
  21.95449840010015,
  31.016124838541646,
  34.0147027033899,
  9.848857801796104,
  21.02379604162864,
  14.212670403551895,
  21.02379604162864,
  15.132745950421556,
  8.54400374531753,
  26.40075756488817,
  28.178005607210743,
  36.61966684720111,
  13.341664064126334,
  18.35755975068582,
  26.92582403567252,
  57.9396237474839,
  40.70626487409524,
  61.29437168288782,
  27.294688127912362,
  21.2602916254693,
  34.92849839314596,
  41.86884283091664,
  48.83646178829912,
  51.478150704935004,
  50.00999900019995,
  54.74486277268398,
  50.92150822589606,
  44.9221548904324,
  57.8013840664737,
  12.041594578792296,
  20.248456731316587,
  29.274562336608895,
  20.615528128088304,
  10.04987562112089,
  17.46424919657298,
  27.16615541441225,
  36.124783736376884,
  23.853720883753127,
  38.07886552931954,
  39.319206502675,
  46.486557196677836,
  37.21558813185679,
  41.72529209005013,
  55.90169943749474,
  38.47076812334269,
  34.713109915419565,
  35.73513677041127,
  0.0,
  20.615528128088304,
  25.495097567963924,
  29.017236257093817,
  24.698178070456937,
  25.059928172283335,
  27.85677655436824,
  28.861739379323623,
  7.615773105863909,
  5.0,
  13.92838827718412,
  9.055385138137417,
  2.0,
  15.033296378372908,
  41.773197148410844,
  42.04759208325728,
  45.70557952810576,
  43.46262762420146,
  56.08029957123981,
  35.0,
  32.69556544854363,
  30.265491900843113,
  37.53664875824692,
  45.617978911828175,
  38.01315561749642,
  40.85339643163099,
  32.31098884280702,
  34.438350715445125,
  36.05551275463989,
  35.84689665786984,
  41.14608122288197,
  38.27531841800928,
  43.41658669218482],
 [12.165525060596439,
  4.47213595499958,
  30.066592756745816,
  18.110770276274835,
  32.449961479175904,
  27.80287754891569,
  20.808652046684813,
  17.26267650163207,
  27.294688127912362,
  22.20360331117452,
  14.7648230602334,
  24.758836806279895,
  17.69180601295413,
  23.08679276123039,
  43.04648650006177,
  42.579337712087536,
  38.18376618407357,
  36.235341863986875,
  18.384776310850235,
  25.553864678361276,
  19.697715603592208,
  28.160255680657446,
  37.8549864614954,
  45.69463863518345,
  30.463092423455635,
  38.897300677553446,
  18.788294228055936,
  7.280109889280518,
  10.770329614269007,
  27.459060435491963,
  13.341664064126334,
  7.810249675906654,
  22.090722034374522,
  16.76305461424021,
  29.120439557122072,
  31.622776601683793,
  37.33630940518894,
  31.906112267087632,
  52.80151512977634,
  41.88078318274385,
  22.20360331117452,
  40.311288741492746,
  37.33630940518894,
  46.17358552246078,
  42.01190307520001,
  32.28002478313795,
  35.014282800023196,
  30.364452901377952,
  24.515301344262525,
  37.44329045369811,
  10.0,
  16.278820596099706,
  10.770329614269007,
  16.0,
  26.90724809414742,
  35.38361202590826,
  38.48376280978771,
  35.35533905932738,
  21.02379604162864,
  28.0178514522438,
  23.853720883753127,
  33.97057550292606,
  17.029386365926403,
  24.166091947189145,
  37.20215047547655,
  39.05124837953327,
  28.635642126552707,
  51.61395160225576,
  20.615528128088304,
  0.0,
  9.0,
  29.0,
  32.57299494980466,
  30.805843601498726,
  35.17101079013795,
  37.94733192202055,
  13.0,
  16.492422502470642,
  24.515301344262525,
  20.024984394500787,
  21.470910553583888,
  19.313207915827967,
  22.0,
  25.079872407968907,
  30.528675044947494,
  32.64965543462902,
  43.93176527297759,
  30.364452901377952,
  12.083045973594572,
  16.278820596099706,
  20.8806130178211,
  35.608987629529715,
  29.154759474226502,
  29.832867780352597,
  22.825424421026653,
  25.942243542145693,
  25.0,
  28.635642126552707,
  31.622776601683793,
  27.018512172212592,
  34.66987164671943],
 [21.095023109728988,
  8.06225774829855,
  39.05124837953327,
  21.095023109728988,
  40.24922359499622,
  34.058772731852805,
  28.635642126552707,
  18.027756377319946,
  29.966648127543394,
  18.439088914585774,
  8.06225774829855,
  19.235384061671343,
  24.698178070456937,
  31.78049716414141,
  50.99019513592785,
  51.478150704935004,
  45.0,
  41.23105625617661,
  23.345235059857504,
  22.360679774997898,
  12.041594578792296,
  36.87817782917155,
  46.69047011971501,
  54.08326913195984,
  35.0,
  45.60701700396552,
  27.202941017470888,
  16.1245154965971,
  19.4164878389476,
  30.4138126514911,
  5.0,
  7.211102550927978,
  13.152946437965905,
  16.492422502470642,
  28.0178514522438,
  27.51363298439521,
  35.22782990761707,
  39.81205847478876,
  60.207972893961475,
  49.64876634922564,
  31.144823004794873,
  49.25444142409901,
  45.880278987817846,
  54.817880294662984,
  49.39635614091387,
  35.84689665786984,
  35.90264614203248,
  29.0,
  24.331050121192877,
  33.24154027718932,
  13.45362404707371,
  12.165525060596439,
  16.401219466856727,
  25.0,
  33.60059523282288,
  42.01190307520001,
  46.8187996428785,
  44.28317965096906,
  30.01666203960727,
  35.77708763999664,
  29.732137494637012,
  40.607881008493905,
  13.152946437965905,
  16.401219466856727,
  30.4138126514911,
  32.64965543462902,
  20.808652046684813,
  59.16924876994806,
  25.495097567963924,
  9.0,
  0.0,
  23.323807579381203,
  41.23105625617661,
  39.6232255123179,
  43.93176527297759,
  46.57252408878007,
  18.439088914585774,
  20.615528128088304,
  24.331050121192877,
  21.540659228538015,
  26.870057685088806,
  18.110770276274835,
  23.769728648009426,
  29.832867780352597,
  36.069377593742864,
  39.96248240537617,
  50.32891812864648,
  39.05124837953327,
  11.704699910719626,
  23.706539182259394,
  12.529964086141668,
  43.04648650006177,
  37.16180835212409,
  37.21558813185679,
  31.016124838541646,
  34.17601498127012,
  32.64965543462902,
  37.0,
  39.35733730830886,
  34.48187929913333,
  42.485291572496],
 [39.66106403010388,
  24.839484696748443,
  55.54277630799526,
  23.08679276123039,
  48.041648597857254,
  56.639209034025185,
  49.678969393496885,
  41.14608122288197,
  53.23532661682466,
  8.246211251235321,
  28.160255680657446,
  37.12142238654117,
  33.734255586866,
  50.774009099144415,
  71.61005515987263,
  68.54195795277518,
  67.17886572427373,
  64.40496875241847,
  46.400431032480725,
  42.7551166528639,
  12.36931687685298,
  49.47726750741192,
  59.22837157984339,
  63.0317380372777,
  33.95585369269929,
  48.662100242385755,
  39.84971769034255,
  35.608987629529715,
  34.88552708502482,
  26.92582403567252,
  18.788294228055936,
  30.528675044947494,
  22.02271554554524,
  16.492422502470642,
  15.264337522473747,
  6.708203932499369,
  55.57877292636101,
  60.60528029800704,
  81.68843247363729,
  56.08029957123981,
  46.238512086787566,
  62.81719509815764,
  65.0,
  73.35529974037323,
  70.9365914038728,
  59.16924876994806,
  58.52349955359813,
  50.44799302251776,
  46.8187996428785,
  51.0,
  23.259406699226016,
  12.806248474865697,
  39.05124837953327,
  42.05948168962618,
  39.0,
  45.39823785126467,
  56.142675390472796,
  61.326992425847855,
  46.95742752749558,
  56.85068161420758,
  52.49761899362675,
  62.93647591023825,
  34.48187929913333,
  30.01666203960727,
  42.95346318982906,
  10.295630140987,
  9.433981132056603,
  63.788713735268246,
  29.017236257093817,
  29.0,
  23.323807579381203,
  0.0,
  52.952809179494906,
  52.630789467763066,
  56.089214649520635,
  57.55866572463264,
  27.202941017470888,
  25.317977802344327,
  16.492422502470642,
  20.0,
  31.016124838541646,
  14.142135623730951,
  46.95742752749558,
  53.009433122794285,
  59.033888572581766,
  61.61980201201558,
  72.86288492778748,
  57.8013840664737,
  34.88552708502482,
  45.27692569068709,
  26.019223662515376,
  64.53681120105021,
  57.8013840664737,
  58.7962583843564,
  51.40038910358559,
  54.405882034941776,
  53.907327887774215,
  56.859475903318,
  60.440052945046304,
  55.97320787662612,
  63.41135544995076],
 [22.47220505424423,
  33.54101966249684,
  12.041594578792296,
  30.083217912982647,
  8.94427190999916,
  34.92849839314596,
  26.076809620810597,
  43.41658669218482,
  45.803929962395145,
  44.721359549995796,
  47.16990566028302,
  55.94640292279746,
  19.235384061671343,
  19.235384061671343,
  32.55764119219941,
  20.248456731316587,
  37.21558813185679,
  44.27188724235731,
  36.124783736376884,
  54.405882034941776,
  49.040799340956916,
  4.47213595499958,
  6.324555320336759,
  13.601470508735444,
  26.1725046566048,
  18.439088914585774,
  14.142135623730951,
  26.832815729997478,
  21.840329667841555,
  31.064449134018133,
  44.553338819890925,
  39.395431207184416,
  54.3415126767741,
  36.49657518178932,
  42.95346318982906,
  51.54609587543949,
  62.93647591023825,
  27.294688127912362,
  43.41658669218482,
  13.601470508735444,
  11.40175425099138,
  10.295630140987,
  23.345235059857504,
  27.294688127912362,
  36.05551275463989,
  46.52956049652737,
  55.21775076911409,
  55.86591089385369,
  49.51767361255979,
  66.21933252457322,
  31.0,
  42.04759208325728,
  33.60059523282288,
  18.027756377319946,
  16.401219466856727,
  17.46424919657298,
  7.211102550927978,
  15.524174696260024,
  14.866068747318506,
  27.202941017470888,
  34.9857113690718,
  35.90264614203248,
  47.885279575251516,
  56.64803615307419,
  68.8839603971781,
  62.81719509815764,
  57.0350769263968,
  22.825424421026653,
  24.698178070456937,
  32.57299494980466,
  41.23105625617661,
  52.952809179494906,
  0.0,
  3.1622776601683795,
  3.1622776601683795,
  5.385164807134504,
  26.076809620810597,
  27.65863337187866,
  38.62641583165593,
  33.52610922848042,
  22.847319317591726,
  38.8329756778952,
  44.553338819890925,
  39.11521443121589,
  39.0,
  31.575306807693888,
  43.04648650006177,
  19.1049731745428,
  41.677331968349414,
  28.319604517012593,
  53.45091205957107,
  32.14031735997639,
  25.709920264364882,
  30.083217912982647,
  23.706539182259394,
  23.40939982143925,
  27.313000567495326,
  22.561028345356956,
  28.442925306655784,
  28.792360097775937,
  29.068883707497267],
 [20.12461179749811,
  32.14031735997639,
  9.0,
  30.083217912982647,
  11.40175425099138,
  31.78049716414141,
  23.021728866442675,
  40.80441152620633,
  42.80186911806539,
  44.384682042344295,
  45.221676218380054,
  53.665631459994955,
  18.973665961010276,
  16.1245154965971,
  29.832867780352597,
  18.439088914585774,
  34.132096331752024,
  41.10960958218893,
  33.24154027718932,
  51.86520991955976,
  48.010415536631214,
  3.1622776601683795,
  7.0710678118654755,
  16.278820596099706,
  27.65863337187866,
  21.213203435596427,
  13.038404810405298,
  24.698178070456937,
  20.223748416156685,
  32.01562118716424,
  43.18564576337837,
  37.33630940518894,
  52.773099207835045,
  36.138621999185304,
  43.41658669218482,
  51.62363799656123,
  60.108235708594876,
  24.186773244895647,
  40.80441152620633,
  16.76305461424021,
  8.94427190999916,
  10.198039027185569,
  20.615528128088304,
  25.238858928247925,
  33.13608305156178,
  43.41658669218482,
  52.20153254455275,
  53.075418038862395,
  46.75467891024384,
  63.60031446463138,
  30.14962686336267,
  41.30375285612676,
  31.064449134018133,
  15.652475842498529,
  17.69180601295413,
  19.924858845171276,
  10.295630140987,
  13.0,
  12.041594578792296,
  24.041630560342615,
  31.906112267087632,
  32.7566787083184,
  45.617978911828175,
  54.70831746635972,
  66.64082832618455,
  62.6418390534633,
  56.22277118748239,
  25.942243542145693,
  25.059928172283335,
  30.805843601498726,
  39.6232255123179,
  52.630789467763066,
  3.1622776601683795,
  0.0,
  4.47213595499958,
  7.810249675906654,
  25.495097567963924,
  27.51363298439521,
  38.91015291668744,
  33.61547262794322,
  23.323807579381203,
  38.600518131237564,
  41.72529209005013,
  36.05551275463989,
  35.84689665786984,
  28.442925306655784,
  40.01249804748511,
  16.0312195418814,
  39.35733730830886,
  25.45584412271571,
  51.66236541235796,
  29.068883707497267,
  22.561028345356956,
  26.92582403567252,
  20.591260281974,
  20.248456731316587,
  24.166091947189145,
  19.4164878389476,
  25.317977802344327,
  25.632011235952593,
  26.019223662515376],
 [24.596747752497688,
  36.345563690772494,
  11.704699910719626,
  33.24154027718932,
  11.40175425099138,
  35.35533905932738,
  27.018512172212592,
  45.221676218380054,
  46.861498055439924,
  47.853944456021594,
  49.64876634922564,
  58.137767414994535,
  22.360679774997898,
  20.0,
  31.144823004794873,
  17.88854381999832,
  36.6742416417845,
  44.384682042344295,
  37.48332962798263,
  56.302753041036986,
  52.009614495783374,
  7.0710678118654755,
  3.1622776601683795,
  12.041594578792296,
  29.068883707497267,
  20.248456731316587,
  17.029386365926403,
  29.154759474226502,
  24.515301344262525,
  34.132096331752024,
  47.38143096192854,
  41.78516483155236,
  57.0701322935211,
  39.6232255123179,
  46.09772228646444,
  54.70831746635972,
  64.38167441127949,
  26.92582403567252,
  41.773197148410844,
  14.035668847618199,
  13.416407864998739,
  7.211102550927978,
  22.02271554554524,
  25.079872407968907,
  35.014282800023196,
  47.16990566028302,
  56.22277118748239,
  57.38466694161429,
  51.088159097779204,
  68.00735254367721,
  34.0147027033899,
  45.09988913511872,
  35.510561809129406,
  20.12461179749811,
  19.4164878389476,
  19.72308292331602,
  7.0710678118654755,
  14.035668847618199,
  16.401219466856727,
  27.313000567495326,
  35.805027579936315,
  35.73513677041127,
  50.08991914547278,
  59.135437767890075,
  71.11258679024411,
  65.96969000988257,
  60.07495318350236,
  22.47220505424423,
  27.85677655436824,
  35.17101079013795,
  43.93176527297759,
  56.089214649520635,
  3.1622776601683795,
  4.47213595499958,
  0.0,
  3.605551275463989,
  29.154759474226502,
  30.805843601498726,
  41.78516483155236,
  36.68787265568828,
  26.0,
  41.97618372363071,
  46.010868281309364,
  40.0,
  39.35733730830886,
  31.32091952673165,
  42.2965719651132,
  18.681541692269406,
  43.829214001622255,
  29.732137494637012,
  56.04462507680822,
  31.575306807693888,
  25.632011235952593,
  30.083217912982647,
  24.413111231467404,
  23.706539182259394,
  27.784887978899608,
  22.47220505424423,
  28.160255680657446,
  29.068883707497267,
  28.442925306655784],
 [27.784887978899608,
  38.8329756778952,
  15.231546211727817,
  34.52535300326414,
  10.816653826391969,
  38.948684188300895,
  30.610455730027933,
  48.60041152089147,
  50.44799302251776,
  49.36598018878993,
  52.55473337388365,
  61.29437168288782,
  24.020824298928627,
  23.600847442411894,
  34.0147027033899,
  19.924858845171276,
  40.024992192379,
  47.92702786528704,
  41.012193308819754,
  59.64059020499378,
  54.147945482723536,
  9.848857801796104,
  4.123105625617661,
  8.48528137423857,
  28.844410203711913,
  18.35755975068582,
  19.4164878389476,
  32.202484376209235,
  27.202941017470888,
  34.438350715445125,
  49.8196748283246,
  44.77722635447622,
  59.665735560705194,
  41.19465984809196,
  46.8187996428785,
  55.78530272392541,
  67.89698078707183,
  30.364452901377952,
  44.40720662234904,
  11.045361017187261,
  16.64331697709324,
  8.06225774829855,
  25.019992006393608,
  27.202941017470888,
  38.118237105091836,
  50.774009099144415,
  59.81638571495272,
  60.876925020897694,
  54.56189146281496,
  71.40028011149536,
  36.05551275463989,
  47.0,
  38.8329756778952,
  23.323807579381203,
  19.697715603592208,
  18.439088914585774,
  4.123105625617661,
  17.029386365926403,
  19.849433241279208,
  30.870698080866262,
  39.408120990476064,
  39.21734310225516,
  53.23532661682466,
  62.03224967708329,
  74.24284477308234,
  67.26812023536856,
  62.0,
  18.973665961010276,
  28.861739379323623,
  37.94733192202055,
  46.57252408878007,
  57.55866572463264,
  5.385164807134504,
  7.810249675906654,
  3.605551275463989,
  0.0,
  31.0,
  32.2490309931942,
  42.720018726587654,
  37.8549864614954,
  26.92582403567252,
  43.41658669218482,
  49.51767361255979,
  43.60045871318328,
  42.941821107167776,
  34.785054261852174,
  45.5411901469428,
  22.135943621178654,
  47.01063709417264,
  33.24154027718932,
  58.82176467941097,
  34.92849839314596,
  29.154759474226502,
  33.61547262794322,
  28.0178514522438,
  27.294688127912362,
  31.38470965295043,
  26.0,
  31.622776601683793,
  32.64965543462902,
  31.78049716414141],
 [15.652475842498529,
  10.63014581273465,
  28.653097563788805,
  6.708203932499369,
  22.80350850198276,
  36.05551275463989,
  26.832815729997478,
  30.083217912982647,
  39.01281840626232,
  18.973665961010276,
  26.1725046566048,
  37.013511046643494,
  7.0710678118654755,
  25.495097567963924,
  46.69047011971501,
  41.593268686170845,
  44.77722635447622,
  45.60701700396552,
  29.068883707497267,
  38.47076812334269,
  23.345235059857504,
  22.360679774997898,
  32.2490309931942,
  37.48332962798263,
  17.46424919657298,
  27.202941017470888,
  12.649110640673518,
  14.142135623730951,
  9.433981132056603,
  15.0,
  20.12461179749811,
  20.591260281974,
  30.4138126514911,
  10.770329614269007,
  20.615528128088304,
  26.92582403567252,
  50.32891812864648,
  36.40054944640259,
  57.48912940721924,
  31.953090617340916,
  19.235384061671343,
  35.6931365951495,
  39.05124837953327,
  46.87216658103186,
  47.20169488482379,
  43.18564576337837,
  47.38143096192854,
  43.32435804486894,
  37.36308338453881,
  50.24937810560445,
  5.385164807134504,
  16.0,
  22.02271554554524,
  16.278820596099706,
  15.264337522473747,
  23.600847442411894,
  30.265491900843113,
  34.48187929913333,
  20.615528128088304,
  33.28663395418648,
  32.984845004941285,
  41.14608122288197,
  29.68164415931166,
  34.828149534535996,
  48.79549159502341,
  37.33630940518894,
  31.0,
  41.14608122288197,
  7.615773105863909,
  13.0,
  18.439088914585774,
  27.202941017470888,
  26.076809620810597,
  25.495097567963924,
  29.154759474226502,
  31.0,
  0.0,
  4.123105625617661,
  15.620499351813308,
  10.0,
  8.602325267042627,
  13.416407864998739,
  34.36568055487916,
  35.35533905932738,
  39.56008088970496,
  38.58756276314948,
  51.0,
  31.89043743820395,
  25.079872407968907,
  24.041630560342615,
  30.805843601498726,
  41.048751503547585,
  33.60059523282288,
  35.84689665786984,
  27.459060435491963,
  30.0,
  30.886890422961002,
  31.89043743820395,
  36.61966684720111,
  33.12099032335839,
  39.20459156782532],
 [19.697715603592208,
  13.416407864998739,
  31.622776601683793,
  2.8284271247461903,
  23.08679276123039,
  40.162171256046406,
  30.870698080866262,
  33.734255586866,
  43.0,
  17.11724276862369,
  28.600699292150182,
  39.66106403010388,
  8.54400374531753,
  29.206163733020468,
  50.32891812864648,
  44.41846462902562,
  48.76474136094644,
  49.72926703662542,
  33.13608305156178,
  41.6293165929973,
  23.40939982143925,
  24.351591323771842,
  33.95585369269929,
  38.05259518088089,
  14.422205101855956,
  25.942243542145693,
  15.264337522473747,
  18.24828759089466,
  13.416407864998739,
  11.045361017187261,
  21.400934559032695,
  23.769728648009426,
  31.622776601683793,
  9.0,
  16.97056274847714,
  24.166091947189145,
  53.75872022286245,
  40.22437072223753,
  61.220911460055866,
  31.78049716414141,
  22.20360331117452,
  37.64306044943742,
  42.44997055358225,
  50.0,
  51.03920062069938,
  47.265209192385896,
  51.24451190127583,
  46.84015371452148,
  41.0,
  53.23532661682466,
  7.211102550927978,
  15.524174696260024,
  26.0,
  20.0,
  14.560219778561036,
  22.360679774997898,
  31.0,
  37.44329045369811,
  24.041630560342615,
  37.21558813185679,
  37.107950630558946,
  45.18849411078001,
  32.64965543462902,
  36.76955262170047,
  50.99019513592785,
  35.17101079013795,
  30.265491900843113,
  40.496913462633174,
  5.0,
  16.492422502470642,
  20.615528128088304,
  25.317977802344327,
  27.65863337187866,
  27.51363298439521,
  30.805843601498726,
  32.2490309931942,
  4.123105625617661,
  0.0,
  12.041594578792296,
  6.4031242374328485,
  6.708203932499369,
  11.180339887498949,
  38.2099463490856,
  39.45883931389772,
  43.68065933568311,
  42.5440947723653,
  55.00909015790027,
  35.35533905932738,
  28.460498941515414,
  28.160255680657446,
  32.55764119219941,
  44.94441010848846,
  37.44329045369811,
  39.824615503479755,
  31.38470965295043,
  33.83784863137726,
  34.88552708502482,
  35.608987629529715,
  40.496913462633174,
  37.12142238654117,
  43.01162633521314],
 [31.064449134018133,
  20.223748416156685,
  43.60045871318328,
  9.219544457292887,
  32.55764119219941,
  50.99019513592785,
  42.190046219457976,
  41.048751503547585,
  51.78802950489621,
  10.0,
  32.01562118716424,
  43.01162633521314,
  20.248456731316587,
  41.10960958218893,
  62.289646009589745,
  56.302753041036986,
  60.207972893961475,
  60.166435825965294,
  42.720018726587654,
  46.69047011971501,
  20.615528128088304,
  35.77708763999664,
  44.94441010848846,
  47.38143096192854,
  17.46424919657298,
  32.2490309931942,
  27.202941017470888,
  28.635642126552707,
  25.0,
  10.44030650891055,
  22.47220505424423,
  30.0,
  31.064449134018133,
  8.0,
  5.0,
  13.152946437965905,
  59.53990258641679,
  52.009614495783374,
  73.10950690573696,
  40.01249804748511,
  34.20526275297414,
  48.84669896727925,
  54.48853090330111,
  62.00806399170998,
  62.80127387243033,
  56.753854494650845,
  59.135437767890075,
  53.150729063673246,
  48.0,
  57.28001396647874,
  14.866068747318506,
  13.416407864998739,
  35.17101079013795,
  31.89043743820395,
  23.345235059857504,
  29.068883707497267,
  40.792156108742276,
  49.4064773081425,
  36.069377593742864,
  48.82622246293481,
  47.53945729601885,
  56.43580423808985,
  37.48332962798263,
  38.01315561749642,
  52.354560450833695,
  25.019992006393608,
  24.186773244895647,
  47.38143096192854,
  13.92838827718412,
  24.515301344262525,
  24.331050121192877,
  16.492422502470642,
  38.62641583165593,
  38.91015291668744,
  41.78516483155236,
  42.720018726587654,
  15.620499351813308,
  12.041594578792296,
  0.0,
  5.656854249492381,
  15.811388300841896,
  6.324555320336759,
  46.2709412050371,
  49.33558553417604,
  54.230987451824994,
  54.08326913195984,
  66.37017402418047,
  47.38143096192854,
  35.0,
  38.91015291668744,
  33.54101966249684,
  56.61271941887264,
  49.20365840057018,
  51.31276644267,
  43.01162633521314,
  45.60701700396552,
  46.32493928760188,
  47.50789408087881,
  52.20153254455275,
  48.54894437575342,
  54.817880294662984],
 [25.553864678361276,
  16.0312195418814,
  38.01315561749642,
  3.605551275463989,
  28.071337695236398,
  45.69463863518345,
  36.71511950137164,
  37.05401462729781,
  47.265209192385896,
  12.165525060596439,
  29.546573405388315,
  40.718546143004666,
  14.7648230602334,
  35.4682957019364,
  56.639209034025185,
  50.774009099144415,
  54.70831746635972,
  55.02726596879042,
  37.8549864614954,
  43.68065933568311,
  20.808652046684813,
  30.463092423455635,
  39.84971769034255,
  43.04648650006177,
  15.264337522473747,
  29.120439557122072,
  21.633307652783937,
  23.40939982143925,
  19.4164878389476,
  9.219544457292887,
  20.808652046684813,
  26.30589287593181,
  30.4138126514911,
  5.656854249492381,
  10.63014581273465,
  18.027756377319946,
  56.293871780150276,
  46.400431032480725,
  67.47592163134935,
  36.124783736376884,
  28.600699292150182,
  43.657759905886145,
  48.83646178829912,
  56.4003546088143,
  57.201398584300364,
  51.97114584074513,
  55.0,
  49.64876634922564,
  44.181444068749045,
  54.78138369920935,
  10.04987562112089,
  12.806248474865697,
  30.4138126514911,
  26.248809496813376,
  19.0,
  25.709920264364882,
  36.22154055254967,
  43.829214001622255,
  30.4138126514911,
  43.266615305567875,
  42.37924020083418,
  51.0,
  34.48187929913333,
  36.61966684720111,
  51.03920062069938,
  29.427877939124322,
  26.248809496813376,
  44.14748010928823,
  9.055385138137417,
  20.024984394500787,
  21.540659228538015,
  20.0,
  33.52610922848042,
  33.61547262794322,
  36.68787265568828,
  37.8549864614954,
  10.0,
  6.4031242374328485,
  5.656854249492381,
  0.0,
  11.045361017187261,
  6.324555320336759,
  42.01190307520001,
  44.384682042344295,
  49.040799340956916,
  48.54894437575342,
  60.90155991434045,
  41.72529209005013,
  31.25699921617557,
  33.61547262794322,
  32.202484376209235,
  51.03920062069938,
  43.60045871318328,
  45.79301256742124,
  37.44329045369811,
  40.0,
  40.8166632639171,
  41.86884283091664,
  46.61544808322666,
  43.04648650006177,
  49.20365840057018],
 [21.095023109728988,
  19.209372712298546,
  29.0,
  8.06225774829855,
  17.029386365926403,
  41.593268686170845,
  31.78049716414141,
  38.27531841800928,
  46.389654018972806,
  23.021728866442675,
  34.713109915419565,
  45.60701700396552,
  6.324555320336759,
  28.635642126552707,
  49.09175083453431,
  41.23105625617661,
  49.040799340956916,
  51.478150704935004,
  36.124783736376884,
  47.01063709417264,
  30.083217912982647,
  20.248456731316587,
  29.154759474226502,
  32.01562118716424,
  9.219544457292887,
  19.235384061671343,
  13.038404810405298,
  21.213203435596427,
  15.0,
  9.433981132056603,
  28.0178514522438,
  29.154759474226502,
  38.27531841800928,
  15.297058540778355,
  20.12461179749811,
  28.861739379323623,
  58.69412236331676,
  39.81205847478876,
  60.207972893961475,
  25.317977802344327,
  20.0,
  33.1058907144937,
  40.607881008493905,
  47.38143096192854,
  50.53711507397311,
  50.00999900019995,
  55.10898293381942,
  51.62363799656123,
  45.5411901469428,
  58.83026432033091,
  13.45362404707371,
  22.135943621178654,
  29.614185789921695,
  19.924858845171276,
  8.06225774829855,
  15.652475842498529,
  25.179356624028344,
  34.655446902326915,
  22.825424421026653,
  37.33630940518894,
  39.11521443121589,
  45.880278987817846,
  38.27531841800928,
  43.18564576337837,
  57.28001396647874,
  40.44749683231337,
  36.6742416417845,
  33.83784863137726,
  2.0,
  21.470910553583888,
  26.870057685088806,
  31.016124838541646,
  22.847319317591726,
  23.323807579381203,
  26.0,
  26.92582403567252,
  8.602325267042627,
  6.708203932499369,
  15.811388300841896,
  11.045361017187261,
  0.0,
  17.029386365926403,
  42.20189569201838,
  42.0,
  45.39823785126467,
  42.720018726587654,
  55.362442142665635,
  33.83784863137726,
  33.54101966249684,
  30.066592756745816,
  39.05124837953327,
  44.77722635447622,
  37.16180835212409,
  40.162171256046406,
  31.622776601683793,
  33.61547262794322,
  35.4400902933387,
  34.88552708502482,
  40.311288741492746,
  37.64306044943742,
  42.485291572496],
 [27.586228448267445,
  14.866068747318506,
  42.05948168962618,
  9.0,
  34.0,
  46.647615158762406,
  38.41874542459709,
  35.22782990761707,
  46.32493928760188,
  6.0,
  25.709920264364882,
  36.68787265568828,
  19.6468827043885,
  38.28837943815329,
  59.464274989274024,
  55.00909015790027,
  56.4003546088143,
  55.46169849544819,
  37.69615364994153,
  40.44749683231337,
  14.866068747318506,
  35.4400902933387,
  45.12205669071391,
  49.0,
  21.470910553583888,
  35.4400902933387,
  26.0,
  24.413111231467404,
  22.02271554554524,
  15.0,
  16.15549442140351,
  24.08318915758459,
  25.0,
  2.8284271247461903,
  10.04987562112089,
  13.601470508735444,
  53.33854141237835,
  48.79549159502341,
  70.00714249274856,
  42.2965719651132,
  32.64965543462902,
  48.76474136094644,
  52.20153254455275,
  60.207972893961475,
  59.464274989274024,
  51.54609587543949,
  53.33854141237835,
  47.042533945356304,
  42.04759208325728,
  50.96076922496363,
  10.63014581273465,
  7.211102550927978,
  30.083217912982647,
  29.206163733020468,
  25.079872407968907,
  32.01562118716424,
  42.04759208325728,
  47.885279575251516,
  33.83784863137726,
  45.34313619501854,
  42.941821107167776,
  52.43090691567332,
  31.25699921617557,
  31.76476034853718,
  46.14108798023731,
  24.041630560342615,
  19.924858845171276,
  50.44799302251776,
  15.033296378372908,
  19.313207915827967,
  18.110770276274835,
  14.142135623730951,
  38.8329756778952,
  38.600518131237564,
  41.97618372363071,
  43.41658669218482,
  13.416407864998739,
  11.180339887498949,
  6.324555320336759,
  6.324555320336759,
  17.029386365926403,
  0.0,
  40.607881008493905,
  44.384682042344295,
  49.64876634922564,
  50.44799302251776,
  62.42595614005443,
  45.0,
  29.068883707497267,
  34.66987164671943,
  27.294688127912362,
  53.150729063673246,
  45.967379738244816,
  47.634021455258214,
  39.6232255123179,
  42.42640687119285,
  42.638011210655684,
  44.598206241955516,
  48.83646178829912,
  44.82186966202994,
  51.62363799656123],
 [23.323807579381203,
  26.076809620810597,
  36.05551275463989,
  40.049968789001575,
  48.25971404805462,
  17.0,
  19.72308292331602,
  5.830951894845301,
  6.4031242374328485,
  42.05948168962618,
  19.849433241279208,
  18.681541692269406,
  37.0,
  26.627053911388696,
  37.0,
  44.598206241955516,
  27.459060435491963,
  19.72308292331602,
  8.602325267042627,
  13.0,
  34.9857113690718,
  40.36087214122113,
  47.634021455258214,
  58.0,
  51.419840528729765,
  56.824290580701486,
  34.48187929913333,
  21.18962010041709,
  27.85677655436824,
  49.25444142409901,
  28.178005607210743,
  16.76305461424021,
  29.732137494637012,
  38.3275357934736,
  50.635955604688654,
  51.264022471905186,
  18.384776310850235,
  27.459060435491963,
  43.174066289845804,
  57.0087712549569,
  33.301651610693426,
  48.25971404805462,
  36.138621999185304,
  44.721359549995796,
  33.24154027718932,
  12.727922061357855,
  13.038404810405298,
  11.40175425099138,
  5.385164807134504,
  22.847319317591726,
  32.0,
  35.73513677041127,
  12.649110640673518,
  27.202941017470888,
  45.69463863518345,
  53.665631459994955,
  51.66236541235796,
  38.91015291668744,
  29.698484809834994,
  23.769728648009426,
  13.152946437965905,
  23.194827009486403,
  14.212670403551895,
  25.059928172283335,
  30.0,
  55.758407437802596,
  42.80186911806539,
  66.84309986827361,
  41.773197148410844,
  22.0,
  23.769728648009426,
  46.95742752749558,
  44.553338819890925,
  41.72529209005013,
  46.010868281309364,
  49.51767361255979,
  34.36568055487916,
  38.2099463490856,
  46.2709412050371,
  42.01190307520001,
  42.20189569201838,
  40.607881008493905,
  0.0,
  10.04987562112089,
  16.492422502470642,
  25.019992006393608,
  31.016124838541646,
  31.78049716414141,
  12.083045973594572,
  16.278820596099706,
  25.612496949731394,
  28.0,
  25.96150997149434,
  23.194827009486403,
  22.825424421026653,
  25.079872407968907,
  21.18962010041709,
  27.85677655436824,
  26.30589287593181,
  21.587033144922902,
  29.154759474226502],
 [21.095023109728988,
  29.546573405388315,
  29.0,
  41.773197148410844,
  44.384682042344295,
  7.0710678118654755,
  13.038404810405298,
  14.317821063276353,
  7.211102550927978,
  47.01063709417264,
  28.0178514522438,
  28.635642126552707,
  36.05551275463989,
  20.0,
  27.018512172212592,
  35.77708763999664,
  17.46424919657298,
  11.40175425099138,
  6.708203932499369,
  23.021728866442675,
  41.773197148410844,
  35.35533905932738,
  41.10960958218893,
  52.009614495783374,
  51.03920062069938,
  53.75872022286245,
  31.78049716414141,
  21.213203435596427,
  27.0,
  50.24937810560445,
  34.713109915419565,
  22.67156809750927,
  38.27531841800928,
  41.78516483155236,
  54.08326913195984,
  56.43580423808985,
  25.942243542145693,
  18.027756377319946,
  33.24154027718932,
  52.354560450833695,
  28.635642126552707,
  41.036569057366385,
  26.92582403567252,
  35.17101079013795,
  23.194827009486403,
  8.06225774829855,
  16.278820596099706,
  19.924858845171276,
  15.033296378372908,
  32.01562118716424,
  34.48187929913333,
  40.8166632639171,
  14.317821063276353,
  23.769728648009426,
  43.73785545725808,
  50.96076922496363,
  46.32493928760188,
  30.805843601498726,
  24.596747752497688,
  14.7648230602334,
  4.242640687119285,
  13.152946437965905,
  23.259406699226016,
  34.539832078341085,
  40.01249804748511,
  62.48199740725323,
  50.20956084253277,
  61.91122676865643,
  42.04759208325728,
  25.079872407968907,
  29.832867780352597,
  53.009433122794285,
  39.11521443121589,
  36.05551275463989,
  40.0,
  43.60045871318328,
  35.35533905932738,
  39.45883931389772,
  49.33558553417604,
  44.384682042344295,
  42.0,
  44.384682042344295,
  10.04987562112089,
  0.0,
  6.708203932499369,
  15.132745950421556,
  21.470910553583888,
  23.600847442411894,
  19.209372712298546,
  12.165525060596439,
  34.48187929913333,
  18.027756377319946,
  17.0,
  13.601470508735444,
  15.620499351813308,
  17.029386365926403,
  12.806248474865697,
  19.4164878389476,
  16.76305461424021,
  12.529964086141668,
  19.4164878389476],
 [24.331050121192877,
  34.9857113690718,
  27.784887978899608,
  46.17358552246078,
  45.35416188179427,
  4.123105625617661,
  14.035668847618199,
  21.02379604162864,
  12.041594578792296,
  52.69724850502159,
  34.66987164671943,
  35.17101079013795,
  39.20459156782532,
  19.924858845171276,
  21.37755832643195,
  32.202484376209235,
  11.045361017187261,
  6.082762530298219,
  12.727922061357855,
  29.274562336608895,
  48.08326112068523,
  35.73513677041127,
  39.96248240537617,
  51.10772935672255,
  54.147945482723536,
  55.10898293381942,
  34.0147027033899,
  25.632011235952593,
  30.59411708155671,
  54.12947441089743,
  41.012193308819754,
  29.0,
  44.94441010848846,
  46.95742752749558,
  59.09314681077663,
  62.12889826803627,
  30.364452901377952,
  14.212670403551895,
  26.68332812825267,
  52.55473337388365,
  29.614185789921695,
  39.20459156782532,
  23.021728866442675,
  30.463092423455635,
  17.0,
  8.602325267042627,
  19.235384061671343,
  25.179356624028344,
  21.095023109728988,
  37.33630940518894,
  39.395431207184416,
  46.61544808322666,
  20.0,
  26.0,
  46.04345773288535,
  52.61178575186362,
  46.09772228646444,
  28.319604517012593,
  25.495097567963924,
  12.206555615733702,
  6.708203932499369,
  7.0710678118654755,
  29.966648127543394,
  41.23105625617661,
  46.17358552246078,
  68.6804193347711,
  56.639209034025185,
  61.741396161732524,
  45.70557952810576,
  30.528675044947494,
  36.069377593742864,
  59.033888572581766,
  39.0,
  35.84689665786984,
  39.35733730830886,
  42.941821107167776,
  39.56008088970496,
  43.68065933568311,
  54.230987451824994,
  49.040799340956916,
  45.39823785126467,
  49.64876634922564,
  16.492422502470642,
  6.708203932499369,
  0.0,
  10.295630140987,
  14.7648230602334,
  21.400934559032695,
  25.80697580112788,
  15.524174696260024,
  41.182520563948,
  12.649110640673518,
  14.212670403551895,
  9.899494936611665,
  15.524174696260024,
  15.652475842498529,
  11.704699910719626,
  17.204650534085253,
  12.806248474865697,
  10.295630140987,
  14.7648230602334],
 [23.021728866442675,
  36.796738985948195,
  19.6468827043885,
  45.27692569068709,
  39.05124837953327,
  8.06225774829855,
  12.041594578792296,
  28.284271247461902,
  21.840329667841555,
  54.45181356024793,
  40.496913462633174,
  43.18564576337837,
  36.40054944640259,
  14.317821063276353,
  12.041594578792296,
  22.02271554554524,
  6.324555320336759,
  13.601470508735444,
  18.439088914585774,
  38.01315561749642,
  51.86520991955976,
  29.068883707497267,
  31.38470965295043,
  42.5440947723653,
  50.695167422546305,
  49.040799340956916,
  30.083217912982647,
  26.1725046566048,
  29.154759474226502,
  52.0,
  44.94441010848846,
  33.54101966249684,
  50.695167422546305,
  47.634021455258214,
  59.07622195096772,
  63.702433234531945,
  40.496913462633174,
  4.47213595499958,
  20.248456731316587,
  45.12205669071391,
  24.186773244895647,
  30.01666203960727,
  12.806248474865697,
  20.248456731316587,
  9.433981132056603,
  18.867962264113206,
  29.5296461204668,
  34.92849839314596,
  30.14962686336267,
  47.07440918375928,
  39.824615503479755,
  48.83646178829912,
  23.706539182259394,
  22.847319317591726,
  41.593268686170845,
  47.01063709417264,
  38.3275357934736,
  18.867962264113206,
  20.396078054371138,
  5.385164807134504,
  12.041594578792296,
  4.47213595499958,
  36.87817782917155,
  48.27007354458868,
  55.00909015790027,
  71.7007670809734,
  60.74537019394976,
  53.75872022286245,
  43.46262762420146,
  32.64965543462902,
  39.96248240537617,
  61.61980201201558,
  31.575306807693888,
  28.442925306655784,
  31.32091952673165,
  34.785054261852174,
  38.58756276314948,
  42.5440947723653,
  54.08326913195984,
  48.54894437575342,
  42.720018726587654,
  50.44799302251776,
  25.019992006393608,
  15.132745950421556,
  10.295630140987,
  0.0,
  12.649110640673518,
  12.649110640673518,
  31.622776601683793,
  16.401219466856727,
  47.43416490252569,
  3.1622776601683795,
  6.0,
  2.8284271247461903,
  11.180339887498949,
  9.219544457292887,
  7.810249675906654,
  9.055385138137417,
  3.1622776601683795,
  5.656854249492381,
  4.47213595499958],
 [35.35533905932738,
  48.27007354458868,
  31.016124838541646,
  57.706152185014034,
  51.03920062069938,
  16.278820596099706,
  24.186773244895647,
  35.77708763999664,
  25.709920264364882,
  66.06814663663572,
  49.39635614091387,
  49.64876634922564,
  49.040799340956916,
  26.92582403567252,
  13.601470508735444,
  29.068883707497267,
  6.324555320336759,
  12.041594578792296,
  27.202941017470888,
  43.41658669218482,
  62.369864518050704,
  41.048751503547585,
  41.773197148410844,
  52.630789467763066,
  63.324560795950255,
  61.032778078668514,
  42.720018726587654,
  38.01315561749642,
  41.593268686170845,
  64.62197768561404,
  55.31726674375732,
  43.41658669218482,
  59.682493245507096,
  59.64059020499378,
  71.34423592694787,
  75.43208866258443,
  42.04759208325728,
  16.1245154965971,
  13.038404810405298,
  56.32051136131489,
  36.6742416417845,
  39.56008088970496,
  20.8806130178211,
  24.20743687382041,
  8.06225774829855,
  20.09975124224178,
  30.066592756745816,
  38.2099463490856,
  35.17101079013795,
  50.039984012787215,
  51.86520991955976,
  60.207972893961475,
  33.97057550292606,
  35.4682957019364,
  54.12947441089743,
  59.21148537234985,
  49.36598018878993,
  28.635642126552707,
  32.984845004941285,
  18.027756377319946,
  20.615528128088304,
  10.0,
  44.721359549995796,
  55.94640292279746,
  60.01666435249463,
  82.73451516749222,
  71.06335201775947,
  64.32728814430156,
  56.08029957123981,
  43.93176527297759,
  50.32891812864648,
  72.86288492778748,
  43.04648650006177,
  40.01249804748511,
  42.2965719651132,
  45.5411901469428,
  51.0,
  55.00909015790027,
  66.37017402418047,
  60.90155991434045,
  55.362442142665635,
  62.42595614005443,
  31.016124838541646,
  21.470910553583888,
  14.7648230602334,
  12.649110640673518,
  0.0,
  24.0,
  40.496913462633174,
  27.80287754891569,
  55.94640292279746,
  11.045361017187261,
  18.439088914585774,
  15.231546211727817,
  23.769728648009426,
  21.840329667841555,
  20.12461179749811,
  21.213203435596427,
  15.297058540778355,
  17.88854381999832,
  14.0],
 [18.384776310850235,
  33.61547262794322,
  7.0710678118654755,
  38.18376618407357,
  27.073972741361768,
  17.69180601295413,
  12.36931687685298,
  32.984845004941285,
  30.805843601498726,
  49.92995093127971,
  42.04759208325728,
  47.67598976424087,
  27.80287754891569,
  7.280109889280518,
  15.264337522473747,
  13.152946437965905,
  18.110770276274835,
  25.942243542145693,
  23.40939982143925,
  43.965895873961216,
  49.9799959983992,
  17.11724276862369,
  18.788294228055936,
  29.966648127543394,
  40.718546143004666,
  37.05401462729781,
  20.808652046684813,
  23.08679276123039,
  23.021728866442675,
  43.266615305567875,
  43.68065933568311,
  34.132096331752024,
  51.478150704935004,
  42.20189569201838,
  52.32590180780452,
  58.60034129593445,
  49.39635614091387,
  8.246211251235321,
  26.419689627245813,
  32.55764119219941,
  13.892443989449804,
  17.804493814764857,
  7.211102550927978,
  15.811388300841896,
  17.46424919657298,
  29.732137494637012,
  39.698866482558415,
  42.941821107167776,
  37.16180835212409,
  54.62600113499065,
  34.66987164671943,
  45.17742799230607,
  25.019992006393608,
  15.811388300841896,
  31.016124838541646,
  35.35533905932738,
  25.709920264364882,
  7.211102550927978,
  11.313708498984761,
  9.219544457292887,
  19.4164878389476,
  17.08800749063506,
  40.19950248448356,
  51.0098029794274,
  60.41522986797286,
  68.09552114493287,
  58.872744118140105,
  41.10960958218893,
  35.0,
  30.364452901377952,
  39.05124837953327,
  57.8013840664737,
  19.1049731745428,
  16.0312195418814,
  18.681541692269406,
  22.135943621178654,
  31.89043743820395,
  35.35533905932738,
  47.38143096192854,
  41.72529209005013,
  33.83784863137726,
  45.0,
  31.78049716414141,
  23.600847442411894,
  21.400934559032695,
  12.649110640673518,
  24.0,
  0.0,
  34.058772731852805,
  17.11724276862369,
  49.09175083453431,
  13.038404810405298,
  7.211102550927978,
  11.661903789690601,
  9.219544457292887,
  6.708203932499369,
  10.816653826391969,
  4.242640687119285,
  9.486832980505138,
  11.313708498984761,
  10.0],
 [19.235384061671343,
  15.297058540778355,
  36.138621999185304,
  29.832867780352597,
  43.18564576337837,
  24.596747752497688,
  22.02271554554524,
  6.324555320336759,
  18.35755975068582,
  30.083217912982647,
  8.94427190999916,
  14.317821063276353,
  29.410882339705484,
  27.294688127912362,
  43.41658669218482,
  47.16990566028302,
  35.77708763999664,
  30.4138126514911,
  13.416407864998739,
  13.601470508735444,
  23.021728866442675,
  37.21558813185679,
  46.09772228646444,
  55.226805085936306,
  42.5440947723653,
  50.44799302251776,
  29.068883707497267,
  15.0,
  21.213203435596427,
  39.293765408777,
  16.1245154965971,
  5.0,
  19.235384061671343,
  27.0,
  39.11521443121589,
  39.21734310225516,
  25.298221281347036,
  32.55764119219941,
  51.478150704935004,
  52.49761899362675,
  30.4138126514911,
  47.75981574503821,
  40.049968789001575,
  49.09175083453431,
  40.85339643163099,
  24.413111231467404,
  24.331050121192877,
  18.439088914585774,
  13.0,
  25.612496949731394,
  21.587033144922902,
  23.769728648009426,
  9.055385138137417,
  23.706539182259394,
  38.600518131237564,
  47.01063709417264,
  48.25971404805462,
  40.44749683231337,
  27.85677655436824,
  28.442925306655784,
  20.12461179749811,
  31.304951684997057,
  6.324555320336759,
  17.029386365926403,
  27.313000567495326,
  43.829214001622255,
  31.144823004794873,
  62.369864518050704,
  32.69556544854363,
  12.083045973594572,
  11.704699910719626,
  34.88552708502482,
  41.677331968349414,
  39.35733730830886,
  43.829214001622255,
  47.01063709417264,
  25.079872407968907,
  28.460498941515414,
  35.0,
  31.25699921617557,
  33.54101966249684,
  29.068883707497267,
  12.083045973594572,
  19.209372712298546,
  25.80697580112788,
  31.622776601683793,
  40.496913462633174,
  34.058772731852805,
  0.0,
  17.0,
  15.811388300841896,
  34.785054261852174,
  30.265491900843113,
  29.120439557122072,
  25.0,
  28.0178514522438,
  25.317977802344327,
  31.016124838541646,
  31.78049716414141,
  26.68332812825267,
  34.92849839314596],
 [9.0,
  20.518284528683193,
  20.12461179749811,
  30.675723300355934,
  32.64965543462902,
  12.083045973594572,
  5.0990195135927845,
  16.15549442140351,
  17.88854381999832,
  38.28837943815329,
  25.317977802344327,
  30.59411708155671,
  24.0,
  10.770329614269007,
  27.313000567495326,
  30.265491900843113,
  21.93171219946131,
  21.587033144922902,
  7.810249675906654,
  27.313000567495326,
  35.510561809129406,
  24.20743687382041,
  31.400636936215164,
  41.72529209005013,
  39.0,
  41.78516483155236,
  19.6468827043885,
  10.295630140987,
  15.132745950421556,
  38.63935817272331,
  28.653097563788805,
  17.72004514666935,
  35.17101079013795,
  31.906112267087632,
  43.829214001622255,
  47.634021455258214,
  34.655446902326915,
  16.15549442140351,
  36.61966684720111,
  41.048751503547585,
  17.204650534085253,
  32.2490309931942,
  23.08679276123039,
  32.14031735997639,
  25.80697580112788,
  20.223748416156685,
  27.294688127912362,
  27.65863337187866,
  21.400934559032695,
  38.58756276314948,
  24.186773244895647,
  32.526911934581186,
  8.06225774829855,
  11.704699910719626,
  31.575306807693888,
  38.897300677553446,
  35.4682957019364,
  23.769728648009426,
  13.45362404707371,
  12.083045973594572,
  9.055385138137417,
  17.804493814764857,
  23.08679276123039,
  34.0147027033899,
  43.41658669218482,
  55.31726674375732,
  44.41846462902562,
  50.80354318352215,
  30.265491900843113,
  16.278820596099706,
  23.706539182259394,
  45.27692569068709,
  28.319604517012593,
  25.45584412271571,
  29.732137494637012,
  33.24154027718932,
  24.041630560342615,
  28.160255680657446,
  38.91015291668744,
  33.61547262794322,
  30.066592756745816,
  34.66987164671943,
  16.278820596099706,
  12.165525060596439,
  15.524174696260024,
  16.401219466856727,
  27.80287754891569,
  17.11724276862369,
  17.0,
  0.0,
  32.38826948140329,
  19.4164878389476,
  13.601470508735444,
  13.601470508735444,
  8.0,
  11.045361017187261,
  8.94427190999916,
  14.035668847618199,
  15.652475842498529,
  10.816653826391969,
  18.788294228055936],
 [32.2490309931942,
  20.591260281974,
  50.15974481593781,
  32.55764119219941,
  52.773099207835045,
  40.311288741492746,
  37.48332962798263,
  20.248456731316587,
  31.89043743820395,
  25.0,
  7.0710678118654755,
  11.180339887498949,
  37.21558813185679,
  42.01190307520001,
  59.20304046246274,
  62.00806399170998,
  51.478150704935004,
  45.221676218380054,
  29.154759474226502,
  17.46424919657298,
  14.142135623730951,
  49.040799340956916,
  58.69412236331676,
  66.48308055437865,
  46.69047011971501,
  58.008620049092706,
  39.56008088970496,
  27.294688127912362,
  31.622776601683793,
  41.400483088968905,
  11.40175425099138,
  15.0,
  4.47213595499958,
  26.627053911388696,
  36.05551275463989,
  32.0624390837628,
  29.832867780352597,
  48.27007354458868,
  67.23094525588644,
  62.177166226839255,
  42.95346318982906,
  61.0,
  55.44366510251645,
  64.4980619863884,
  56.64803615307419,
  38.28837943815329,
  34.66987164671943,
  25.495097567963924,
  23.430749027719962,
  25.019992006393608,
  25.612496949731394,
  20.12461179749811,
  24.331050121192877,
  36.49657518178932,
  46.04345773288535,
  54.405882034941776,
  59.23681287847955,
  55.00909015790027,
  41.30375285612676,
  44.14748010928823,
  35.84689665786984,
  47.01063709417264,
  11.40175425099138,
  4.47213595499958,
  18.867962264113206,
  32.57299494980466,
  18.973665961010276,
  71.69379331573968,
  37.53664875824692,
  20.8806130178211,
  12.529964086141668,
  26.019223662515376,
  53.45091205957107,
  51.66236541235796,
  56.04462507680822,
  58.82176467941097,
  30.805843601498726,
  32.55764119219941,
  33.54101966249684,
  32.202484376209235,
  39.05124837953327,
  27.294688127912362,
  25.612496949731394,
  34.48187929913333,
  41.182520563948,
  47.43416490252569,
  55.94640292279746,
  49.09175083453431,
  15.811388300841896,
  32.38826948140329,
  0.0,
  50.59644256269407,
  45.89117562233506,
  44.9221548904324,
  40.311288741492746,
  43.41658669218482,
  41.0,
  46.389654018972806,
  47.53945729601885,
  42.44997055358225,
  50.695167422546305],
 [25.612496949731394,
  39.698866482558415,
  20.09975124224178,
  47.70744176750625,
  40.01249804748511,
  11.0,
  14.866068747318506,
  31.400636936215164,
  24.515301344262525,
  57.28001396647874,
  43.657759905886145,
  46.2709412050371,
  38.48376280978771,
  16.15549442140351,
  9.0,
  20.518284528683193,
  5.0990195135927845,
  14.866068747318506,
  21.587033144922902,
  41.0,
  54.91812087098393,
  30.01666203960727,
  31.32091952673165,
  42.37924020083418,
  52.49761899362675,
  50.00999900019995,
  31.953090617340916,
  29.0,
  31.622776601683793,
  54.12947441089743,
  48.02082881417188,
  36.6742416417845,
  53.85164807134504,
  50.32891812864648,
  61.61168720299745,
  66.48308055437865,
  43.01162633521314,
  5.0990195135927845,
  17.204650534085253,
  45.5411901469428,
  25.709920264364882,
  29.546573405388315,
  11.40175425099138,
  17.88854381999832,
  6.4031242374328485,
  21.02379604162864,
  31.78049716414141,
  37.656340767525464,
  33.06055050963308,
  49.8196748283246,
  42.5205832509386,
  51.73973328110612,
  26.832815729997478,
  25.059928172283335,
  43.174066289845804,
  48.16637831516918,
  38.63935817272331,
  18.384776310850235,
  22.135943621178654,
  7.810249675906654,
  15.132745950421556,
  5.830951894845301,
  40.024992192379,
  51.419840528729765,
  58.0,
  74.65252842335616,
  63.812224534175265,
  53.85164807134504,
  45.617978911828175,
  35.608987629529715,
  43.04648650006177,
  64.53681120105021,
  32.14031735997639,
  29.068883707497267,
  31.575306807693888,
  34.92849839314596,
  41.048751503547585,
  44.94441010848846,
  56.61271941887264,
  51.03920062069938,
  44.77722635447622,
  53.150729063673246,
  28.0,
  18.027756377319946,
  12.649110640673518,
  3.1622776601683795,
  11.045361017187261,
  13.038404810405298,
  34.785054261852174,
  19.4164878389476,
  50.59644256269407,
  0.0,
  7.615773105863909,
  5.830951894845301,
  13.601470508735444,
  11.180339887498949,
  10.63014581273465,
  10.198039027185569,
  4.47213595499958,
  8.602325267042627,
  3.1622776601683795],
 [18.384776310850235,
  33.015148038438355,
  13.92838827718412,
  40.22437072223753,
  33.06055050963308,
  10.63014581273465,
  8.54400374531753,
  28.071337695236398,
  24.186773244895647,
  50.32891812864648,
  38.8329756778952,
  43.04648650006177,
  30.870698080866262,
  8.54400374531753,
  13.892443989449804,
  18.788294228055936,
  12.165525060596439,
  18.788294228055936,
  18.110770276274835,
  38.63935817272331,
  48.76474136094644,
  23.08679276123039,
  25.942243542145693,
  37.12142238654117,
  44.9221548904324,
  43.04648650006177,
  24.351591323771842,
  22.20360331117452,
  24.20743687382041,
  46.51881339845203,
  42.04759208325728,
  31.32091952673165,
  48.76474136094644,
  43.139309220245984,
  54.20332093147061,
  59.413803110051795,
  42.941821107167776,
  2.8284271247461903,
  24.041630560342615,
  39.293765408777,
  18.24828759089466,
  25.0,
  10.198039027185569,
  19.026297590440446,
  13.601470508735444,
  22.627416997969522,
  32.802438933713454,
  36.76955262170047,
  31.32091952673165,
  48.703182647543684,
  35.35533905932738,
  45.0,
  21.587033144922902,
  17.4928556845359,
  35.6931365951495,
  41.012193308819754,
  32.57299494980466,
  14.142135623730951,
  14.560219778561036,
  2.23606797749979,
  13.0,
  10.198039027185569,
  36.05551275463989,
  47.265209192385896,
  55.44366510251645,
  68.00735254367721,
  57.706152185014034,
  48.104053883222775,
  38.01315561749642,
  29.154759474226502,
  37.16180835212409,
  57.8013840664737,
  25.709920264364882,
  22.561028345356956,
  25.632011235952593,
  29.154759474226502,
  33.60059523282288,
  37.44329045369811,
  49.20365840057018,
  43.60045871318328,
  37.16180835212409,
  45.967379738244816,
  25.96150997149434,
  17.0,
  14.212670403551895,
  6.0,
  18.439088914585774,
  7.211102550927978,
  30.265491900843113,
  13.601470508735444,
  45.89117562233506,
  7.615773105863909,
  0.0,
  4.47213595499958,
  6.4031242374328485,
  3.605551275463989,
  5.0,
  3.1622776601683795,
  3.1622776601683795,
  4.47213595499958,
  5.656854249492381],
 [20.248456731316587,
  33.97057550292606,
  18.384776310850235,
  42.5440947723653,
  37.21558813185679,
  6.708203932499369,
  9.219544457292887,
  26.076809620810597,
  20.615528128088304,
  51.62363799656123,
  37.94733192202055,
  41.048751503547585,
  33.83784863137726,
  12.041594578792296,
  14.317821063276353,
  22.47220505424423,
  8.94427190999916,
  14.317821063276353,
  16.1245154965971,
  36.124783736376884,
  49.09175083453431,
  27.294688127912362,
  30.4138126514911,
  41.593268686170845,
  48.27007354458868,
  47.16990566028302,
  27.65863337187866,
  23.345235059857504,
  26.419689627245813,
  49.39635614091387,
  42.190046219457976,
  30.870698080866262,
  48.104053883222775,
  44.82186966202994,
  56.302753041036986,
  60.876925020897694,
  39.395431207184416,
  4.47213595499958,
  23.021728866442675,
  43.68065933568311,
  22.02271554554524,
  29.410882339705484,
  13.416407864998739,
  21.587033144922902,
  12.206555615733702,
  18.439088914585774,
  28.844410203711913,
  33.52610922848042,
  28.442925306655784,
  45.60701700396552,
  37.013511046643494,
  46.010868281309364,
  21.02379604162864,
  20.248456731316587,
  39.319206502675,
  45.0111097397076,
  37.0,
  18.439088914585774,
  18.110770276274835,
  3.0,
  10.04987562112089,
  6.0,
  34.52535300326414,
  45.89117562233506,
  53.08483775994799,
  68.8839603971781,
  57.982756057296896,
  52.55473337388365,
  40.85339643163099,
  29.832867780352597,
  37.21558813185679,
  58.7962583843564,
  30.083217912982647,
  26.92582403567252,
  30.083217912982647,
  33.61547262794322,
  35.84689665786984,
  39.824615503479755,
  51.31276644267,
  45.79301256742124,
  40.162171256046406,
  47.634021455258214,
  23.194827009486403,
  13.601470508735444,
  9.899494936611665,
  2.8284271247461903,
  15.231546211727817,
  11.661903789690601,
  29.120439557122072,
  13.601470508735444,
  44.9221548904324,
  5.830951894845301,
  4.47213595499958,
  0.0,
  8.54400374531753,
  7.0,
  5.0,
  7.615773105863909,
  3.1622776601683795,
  2.8284271247461903,
  6.0],
 [12.041594578792296,
  26.627053911388696,
  13.45362404707371,
  34.132096331752024,
  29.832867780352597,
  11.40175425099138,
  3.1622776601683795,
  23.769728648009426,
  22.627416997969522,
  43.93176527297759,
  33.24154027718932,
  38.47076812334269,
  25.298221281347036,
  4.47213595499958,
  20.248456731316587,
  22.360679774997898,
  17.46424919657298,
  21.213203435596427,
  14.317821063276353,
  34.785054261852174,
  42.485291572496,
  20.248456731316587,
  25.495097567963924,
  36.40054944640259,
  39.81205847478876,
  39.6232255123179,
  19.235384061671343,
  15.811388300841896,
  18.027756377319946,
  40.85339643163099,
  35.84689665786984,
  25.495097567963924,
  42.95346318982906,
  36.796738985948195,
  48.010415536631214,
  53.03772242470448,
  40.80441152620633,
  9.219544457292887,
  30.4138126514911,
  37.16180835212409,
  14.142135623730951,
  25.612496949731394,
  15.132745950421556,
  24.186773244895647,
  19.849433241279208,
  22.825424421026653,
  31.89043743820395,
  34.132096331752024,
  28.178005607210743,
  45.617978911828175,
  29.0,
  38.600518131237564,
  16.0312195418814,
  11.704699910719626,
  31.064449134018133,
  37.21558813185679,
  30.886890422961002,
  16.15549442140351,
  10.04987562112089,
  5.830951894845301,
  11.40175425099138,
  14.317821063276353,
  31.064449134018133,
  42.01190307520001,
  51.19570294468082,
  61.61168720299745,
  51.42956348249516,
  46.52956049652737,
  32.31098884280702,
  22.825424421026653,
  31.016124838541646,
  51.40038910358559,
  23.706539182259394,
  20.591260281974,
  24.413111231467404,
  28.0178514522438,
  27.459060435491963,
  31.38470965295043,
  43.01162633521314,
  37.44329045369811,
  31.622776601683793,
  39.6232255123179,
  22.825424421026653,
  15.620499351813308,
  15.524174696260024,
  11.180339887498949,
  23.769728648009426,
  9.219544457292887,
  25.0,
  8.0,
  40.311288741492746,
  13.601470508735444,
  6.4031242374328485,
  8.54400374531753,
  0.0,
  3.1622776601683795,
  4.0,
  6.082762530298219,
  9.219544457292887,
  6.082762530298219,
  12.041594578792296],
 [14.866068747318506,
  29.68164415931166,
  12.206555615733702,
  36.61966684720111,
  30.265491900843113,
  11.661903789690601,
  6.0,
  26.476404589747453,
  24.20743687382041,
  46.861498055439924,
  36.345563690772494,
  41.30375285612676,
  27.313000567495326,
  5.0990195135927845,
  17.204650534085253,
  19.6468827043885,
  15.524174696260024,
  20.8806130178211,
  16.76305461424021,
  37.36308338453881,
  45.617978911828175,
  20.396078054371138,
  24.413111231467404,
  35.510561809129406,
  41.48493702538308,
  40.19950248448356,
  20.8806130178211,
  18.867962264113206,
  20.615528128088304,
  42.95346318982906,
  39.0,
  28.635642126552707,
  46.09772228646444,
  39.59797974644666,
  50.60632371551998,
  55.90169943749474,
  42.720018726587654,
  6.4031242374328485,
  27.586228448267445,
  37.0,
  15.033296378372908,
  24.041630560342615,
  12.041594578792296,
  21.095023109728988,
  17.204650534085253,
  23.600847442411894,
  33.24154027718932,
  36.235341863986875,
  30.463092423455635,
  47.92702786528704,
  31.827660925679098,
  41.617304093369626,
  19.1049731745428,
  13.892443989449804,
  32.38826948140329,
  38.01315561749642,
  30.463092423455635,
  13.892443989449804,
  11.180339887498949,
  4.0,
  12.806248474865697,
  13.0,
  34.0147027033899,
  45.044422518220834,
  53.9351462406472,
  64.63745044476924,
  54.56189146281496,
  46.09772228646444,
  34.438350715445125,
  25.942243542145693,
  34.17601498127012,
  54.405882034941776,
  23.40939982143925,
  20.248456731316587,
  23.706539182259394,
  27.294688127912362,
  30.0,
  33.83784863137726,
  45.60701700396552,
  40.0,
  33.61547262794322,
  42.42640687119285,
  25.079872407968907,
  17.029386365926403,
  15.652475842498529,
  9.219544457292887,
  21.840329667841555,
  6.708203932499369,
  28.0178514522438,
  11.045361017187261,
  43.41658669218482,
  11.180339887498949,
  3.605551275463989,
  7.0,
  3.1622776601683795,
  0.0,
  4.242640687119285,
  3.0,
  6.708203932499369,
  5.385164807134504,
  9.219544457292887],
 [15.264337522473747,
  29.068883707497267,
  16.401219466856727,
  37.589892258425,
  33.734255586866,
  7.615773105863909,
  4.242640687119285,
  23.08679276123039,
  20.0,
  46.66904755831214,
  33.95585369269929,
  38.05259518088089,
  29.120439557122072,
  8.246211251235321,
  18.384776310850235,
  23.40939982143925,
  13.892443989449804,
  17.26267650163207,
  13.152946437965905,
  33.734255586866,
  44.41846462902562,
  24.041630560342615,
  28.600699292150182,
  39.66106403010388,
  43.73785545725808,
  43.56604182158393,
  23.194827009486403,
  18.384776310850235,
  21.470910553583888,
  44.598206241955516,
  37.589892258425,
  26.570660511172846,
  43.965895873961216,
  39.824615503479755,
  51.31276644267,
  55.90169943749474,
  38.58756276314948,
  7.280109889280518,
  27.80287754891569,
  40.85339643163099,
  18.110770276274835,
  28.284271247461902,
  15.132745950421556,
  24.020824298928627,
  17.029386365926403,
  19.4164878389476,
  29.0,
  32.202484376209235,
  26.570660511172846,
  44.01136216933077,
  32.01562118716424,
  41.10960958218893,
  16.76305461424021,
  15.524174696260024,
  35.05709628591621,
  41.19465984809196,
  34.438350715445125,
  18.027756377319946,
  14.035668847618199,
  3.1622776601683795,
  8.602325267042627,
  10.44030650891055,
  31.064449134018133,
  42.2965719651132,
  50.48762224545735,
  64.03124237432849,
  53.33854141237835,
  50.08991914547278,
  36.05551275463989,
  25.0,
  32.64965543462902,
  53.907327887774215,
  27.313000567495326,
  24.166091947189145,
  27.784887978899608,
  31.38470965295043,
  30.886890422961002,
  34.88552708502482,
  46.32493928760188,
  40.8166632639171,
  35.4400902933387,
  42.638011210655684,
  21.18962010041709,
  12.806248474865697,
  11.704699910719626,
  7.810249675906654,
  20.12461179749811,
  10.816653826391969,
  25.317977802344327,
  8.94427190999916,
  41.0,
  10.63014581273465,
  5.0,
  5.0,
  4.0,
  4.242640687119285,
  0.0,
  6.708203932499369,
  6.708203932499369,
  2.23606797749979,
  9.848857801796104],
 [17.204650534085253,
  32.2490309931942,
  10.770329614269007,
  38.41874542459709,
  30.01666203960727,
  13.45362404707371,
  9.0,
  29.427877939124322,
  26.627053911388696,
  49.20365840057018,
  39.319206502675,
  44.28317965096906,
  28.653097563788805,
  6.4031242374328485,
  14.866068747318506,
  16.76305461424021,
  15.033296378372908,
  21.93171219946131,
  19.6468827043885,
  40.26164427839479,
  48.33218389437829,
  20.024984394500787,
  22.825424421026653,
  34.0,
  42.37924020083418,
  40.01249804748511,
  21.93171219946131,
  21.470910553583888,
  22.627416997969522,
  44.294469180700204,
  41.78516483155236,
  31.575306807693888,
  49.03060268852505,
  41.773197148410844,
  52.49761899362675,
  58.137767414994535,
  45.27692569068709,
  5.0990195135927845,
  25.612496949731394,
  36.138621999185304,
  15.524174696260024,
  22.02271554554524,
  9.055385138137417,
  18.110770276274835,
  15.652475842498529,
  25.495097567963924,
  35.4682957019364,
  38.91015291668744,
  33.24154027718932,
  50.695167422546305,
  34.058772731852805,
  44.10215414239989,
  22.090722034374522,
  15.620499351813308,
  32.984845004941285,
  38.05259518088089,
  29.410882339705484,
  11.40175425099138,
  12.083045973594572,
  5.0,
  15.264337522473747,
  13.341664064126334,
  37.013511046643494,
  48.041648597857254,
  56.88585061331157,
  67.11929677819934,
  57.271284253105414,
  44.94441010848846,
  35.84689665786984,
  28.635642126552707,
  37.0,
  56.859475903318,
  22.561028345356956,
  19.4164878389476,
  22.47220505424423,
  26.0,
  31.89043743820395,
  35.608987629529715,
  47.50789408087881,
  41.86884283091664,
  34.88552708502482,
  44.598206241955516,
  27.85677655436824,
  19.4164878389476,
  17.204650534085253,
  9.055385138137417,
  21.213203435596427,
  4.242640687119285,
  31.016124838541646,
  14.035668847618199,
  46.389654018972806,
  10.198039027185569,
  3.1622776601683795,
  7.615773105863909,
  6.082762530298219,
  3.0,
  6.708203932499369,
  0.0,
  6.0,
  7.0710678118654755,
  7.615773105863909],
 [21.2602916254693,
  35.608987629529715,
  16.492422502470642,
  43.266615305567875,
  36.013886210738214,
  9.848857801796104,
  10.816653826391969,
  29.017236257093817,
  23.769728648009426,
  53.075418038862395,
  40.52159917870962,
  44.01136216933077,
  34.0147027033899,
  11.704699910719626,
  11.704699910719626,
  19.4164878389476,
  9.055385138137417,
  16.64331697709324,
  19.026297590440446,
  39.20459156782532,
  51.10772935672255,
  26.019223662515376,
  28.231188426986208,
  39.395431207184416,
  48.08326112068523,
  46.010868281309364,
  27.51363298439521,
  24.839484696748443,
  27.202941017470888,
  49.658836071740545,
  44.294469180700204,
  33.24154027718932,
  50.59644256269407,
  46.010868281309364,
  57.201398584300364,
  62.22539674441618,
  42.5440947723653,
  1.4142135623730951,
  21.2602916254693,
  41.97618372363071,
  21.37755832643195,
  26.92582403567252,
  10.295630140987,
  18.439088914585774,
  10.63014581273465,
  21.400934559032695,
  31.906112267087632,
  36.68787265568828,
  31.575306807693888,
  48.76474136094644,
  38.2099463490856,
  47.634021455258214,
  23.40939982143925,
  20.591260281974,
  38.8329756778952,
  44.04543109109048,
  35.17101079013795,
  15.811388300841896,
  17.72004514666935,
  3.605551275463989,
  13.152946437965905,
  7.615773105863909,
  37.33630940518894,
  48.662100242385755,
  56.142675390472796,
  70.60453243241541,
  60.03332407921454,
  50.59644256269407,
  41.14608122288197,
  31.622776601683793,
  39.35733730830886,
  60.440052945046304,
  28.442925306655784,
  25.317977802344327,
  28.160255680657446,
  31.622776601683793,
  36.61966684720111,
  40.496913462633174,
  52.20153254455275,
  46.61544808322666,
  40.311288741492746,
  48.83646178829912,
  26.30589287593181,
  16.76305461424021,
  12.806248474865697,
  3.1622776601683795,
  15.297058540778355,
  9.486832980505138,
  31.78049716414141,
  15.652475842498529,
  47.53945729601885,
  4.47213595499958,
  3.1622776601683795,
  3.1622776601683795,
  9.219544457292887,
  6.708203932499369,
  6.708203932499369,
  6.0,
  0.0,
  5.0990195135927845,
  3.1622776601683795],
 [17.4928556845359,
  31.144823004794873,
  17.4928556845359,
  39.824615503479755,
  35.510561809129406,
  6.4031242374328485,
  6.4031242374328485,
  24.0,
  19.72308292331602,
  48.79549159502341,
  35.4400902933387,
  39.0,
  31.32091952673165,
  10.04987562112089,
  16.76305461424021,
  23.259406699226016,
  11.661903789690601,
  15.524174696260024,
  14.0,
  34.36568055487916,
  46.32493928760188,
  25.709920264364882,
  29.68164415931166,
  40.8166632639171,
  45.89117562233506,
  45.39823785126467,
  25.317977802344327,
  20.518284528683193,
  23.706539182259394,
  46.8187996428785,
  39.44616584663204,
  28.231188426986208,
  45.5411901469428,
  42.01190307520001,
  53.53503525729669,
  58.05170109479997,
  38.47076812334269,
  6.0,
  25.80697580112788,
  42.37924020083418,
  20.024984394500787,
  29.068883707497267,
  14.560219778561036,
  23.194827009486403,
  15.0,
  18.439088914585774,
  28.42534080710379,
  32.31098884280702,
  26.92582403567252,
  44.27188724235731,
  34.20526275297414,
  43.18564576337837,
  18.384776310850235,
  17.72004514666935,
  37.12142238654117,
  43.104524124504614,
  35.84689665786984,
  18.439088914585774,
  16.0,
  2.23606797749979,
  8.54400374531753,
  8.246211251235321,
  32.2490309931942,
  43.56604182158393,
  51.24451190127583,
  66.06814663663572,
  55.226805085936306,
  51.478150704935004,
  38.27531841800928,
  27.018512172212592,
  34.48187929913333,
  55.97320787662612,
  28.792360097775937,
  25.632011235952593,
  29.068883707497267,
  32.64965543462902,
  33.12099032335839,
  37.12142238654117,
  48.54894437575342,
  43.04648650006177,
  37.64306044943742,
  44.82186966202994,
  21.587033144922902,
  12.529964086141668,
  10.295630140987,
  5.656854249492381,
  17.88854381999832,
  11.313708498984761,
  26.68332812825267,
  10.816653826391969,
  42.44997055358225,
  8.602325267042627,
  4.47213595499958,
  2.8284271247461903,
  6.082762530298219,
  5.385164807134504,
  2.23606797749979,
  7.0710678118654755,
  5.0990195135927845,
  0.0,
  8.246211251235321],
 [24.041630560342615,
  38.600518131237564,
  17.029386365926403,
  45.803929962395145,
  37.05401462729781,
  12.36931687685298,
  13.892443989449804,
  32.0624390837628,
  26.248809496813376,
  55.97320787662612,
  43.68065933568311,
  47.042533945356304,
  36.235341863986875,
  13.892443989449804,
  8.54400374531753,
  17.69180601295413,
  8.246211251235321,
  17.69180601295413,
  22.090722034374522,
  42.1070065428546,
  54.20332093147061,
  27.073972741361768,
  28.160255680657446,
  39.21734310225516,
  49.9799959983992,
  47.042533945356304,
  29.546573405388315,
  27.80287754891569,
  29.832867780352597,
  51.884487084291386,
  47.41307836451879,
  36.40054944640259,
  53.75872022286245,
  48.79549159502341,
  59.81638571495272,
  65.06919393998976,
  44.94441010848846,
  2.8284271247461903,
  18.384776310850235,
  42.42640687119285,
  23.08679276123039,
  26.40075756488817,
  8.48528137423857,
  15.811388300841896,
  8.06225774829855,
  23.323807579381203,
  34.0,
  39.293765408777,
  34.36568055487916,
  51.419840528729765,
  41.012193308819754,
  50.60632371551998,
  26.570660511172846,
  23.021728866442675,
  40.52159917870962,
  45.27692569068709,
  35.510561809129406,
  15.231546211727817,
  19.697715603592208,
  6.708203932499369,
  16.0312195418814,
  8.48528137423857,
  40.44749683231337,
  51.78802950489621,
  59.07622195096772,
  73.60027173862879,
  63.13477647065839,
  50.695167422546305,
  43.41658669218482,
  34.66987164671943,
  42.485291572496,
  63.41135544995076,
  29.068883707497267,
  26.019223662515376,
  28.442925306655784,
  31.78049716414141,
  39.20459156782532,
  43.01162633521314,
  54.817880294662984,
  49.20365840057018,
  42.485291572496,
  51.62363799656123,
  29.154759474226502,
  19.4164878389476,
  14.7648230602334,
  4.47213595499958,
  14.0,
  10.0,
  34.92849839314596,
  18.788294228055936,
  50.695167422546305,
  3.1622776601683795,
  5.656854249492381,
  6.0,
  12.041594578792296,
  9.219544457292887,
  9.848857801796104,
  7.615773105863909,
  3.1622776601683795,
  8.246211251235321,
  0.0]]
    data['demands'] = [0,
 10,
 7,
 13,
 19,
 26,
 3,
 5,
 9,
 16,
 16,
 12,
 19,
 23,
 20,
 8,
 19,
 2,
 12,
 17,
 9,
 11,
 18,
 29,
 3,
 6,
 17,
 16,
 16,
 9,
 21,
 27,
 23,
 11,
 14,
 8,
 5,
 8,
 16,
 31,
 9,
 5,
 5,
 7,
 18,
 16,
 1,
 27,
 36,
 30,
 13,
 10,
 9,
 14,
 18,
 2,
 6,
 7,
 18,
 28,
 3,
 13,
 19,
 10,
 9,
 20,
 25,
 25,
 36,
 6,
 5,
 15,
 25,
 9,
 8,
 18,
 13,
 14,
 3,
 23,
 6,
 26,
 16,
 11,
 7,
 41,
 35,
 26,
 9,
 15,
 3,
 1,
 2,
 22,
 27,
 20,
 11,
 12,
 10,
 9,
 17]
    data['num_vehicles'] = 8
    data['vehicle_capacities'] = [200]*data['num_vehicles']
    data['depot'] = 0
    return data

In [4]:
def data_CMT11():
    data = {}
    data["distance_matrix"] = [[0.0,
  46.486557196677836,
  44.598206241955516,
  45.17742799230607,
  45.65084884205331,
  43.41658669218482,
  42.190046219457976,
  43.266615305567875,
  50.91168824543142,
  45.48626166217664,
  45.79301256742124,
  47.16990566028302,
  53.75872022286245,
  50.0,
  51.03920062069938,
  49.39635614091387,
  74.09453421137081,
  73.06161783043132,
  16.64331697709324,
  76.66159403508382,
  74.70609078247904,
  73.348483283569,
  79.92496481075234,
  76.94153624668537,
  80.77747210701756,
  79.75587752636166,
  76.92203845452875,
  83.35466393669883,
  81.41252974819048,
  85.44003745317531,
  85.0881895447306,
  84.11896337925236,
  82.29216244576394,
  86.95401083331349,
  86.76981041814025,
  83.6301381082203,
  85.86617494683223,
  76.10519036176179,
  76.00657866263946,
  76.00657866263946,
  75.66372975210778,
  79.02531240052139,
  79.00632886041473,
  79.30952023559341,
  82.05485969764375,
  82.29823813423954,
  84.0535543567314,
  84.00595217006948,
  84.0535543567314,
  86.05230967266364,
  89.00561780022652,
  89.1403387922662,
  80.95677859203639,
  82.29823813423954,
  83.19254774317228,
  85.0,
  86.95401083331349,
  84.581321815162,
  87.23531395025755,
  86.16263691415206,
  90.82400563727631,
  90.87353850269065,
  91.706052144883,
  93.94147114027967,
  93.47192091746055,
  91.92388155425118,
  99.28242543370907,
  46.61544808322666,
  53.150729063673246,
  47.16990566028302,
  48.877397639399746,
  53.2634959423431,
  56.85068161420758,
  52.3450093132096,
  55.31726674375732,
  56.85068161420758,
  53.74011537017761,
  56.568542494923804,
  58.728187440104094,
  60.14149981501958,
  62.93647591023825,
  10.0,
  5.0,
  17.0,
  12.206555615733702,
  9.219544457292887,
  6.4031242374328485,
  5.830951894845301,
  3.1622776601683795,
  9.433981132056603,
  12.529964086141668,
  11.180339887498949,
  8.94427190999916,
  10.04987562112089,
  12.041594578792296,
  6.0,
  10.0,
  15.0,
  22.360679774997898,
  11.180339887498949,
  13.416407864998739,
  8.94427190999916,
  6.708203932499369,
  14.142135623730951,
  11.313708498984761,
  6.4031242374328485,
  7.810249675906654,
  10.816653826391969,
  21.633307652783937,
  24.041630560342615,
  20.615528128088304,
  5.830951894845301,
  10.295630140987,
  16.1245154965971,
  17.0,
  20.024984394500787,
  16.55294535724685,
  13.416407864998739,
  18.027756377319946,
  7.0710678118654755,
  7.0710678118654755],
 [46.486557196677836,
  0.0,
  2.0,
  7.211102550927978,
  8.06225774829855,
  8.48528137423857,
  10.63014581273465,
  12.041594578792296,
  22.47220505424423,
  11.661903789690601,
  10.295630140987,
  10.770329614269007,
  22.561028345356956,
  15.524174696260024,
  14.142135623730951,
  11.180339887498949,
  48.25971404805462,
  48.507731342539614,
  35.014282800023196,
  51.24451190127583,
  51.78802950489621,
  52.392747589718944,
  53.03772242470448,
  53.600373133029585,
  54.037024344425184,
  54.147945482723536,
  54.91812087098393,
  57.0350769263968,
  57.3149195236284,
  66.49060083951716,
  59.033888572581766,
  59.135437767890075,
  59.53990258641679,
  60.0,
  62.12889826803627,
  60.40695324215582,
  62.289646009589745,
  72.94518489934754,
  74.63243262818116,
  75.8023746329889,
  80.72174428244226,
  76.55063683601855,
  78.2368199762746,
  81.83520025025906,
  78.54934754662193,
  84.20213774008353,
  80.26207074328447,
  81.30190649671138,
  83.48652585896721,
  81.98780397107853,
  86.6083136886985,
  88.75246475450696,
  98.00510190801293,
  100.43903623591775,
  100.0,
  103.22790320451152,
  106.50821564555478,
  100.42410069301094,
  105.20931517693668,
  100.00499987500625,
  108.60018416190647,
  105.75916035975324,
  107.29864864013899,
  111.21151019566275,
  109.48059188732951,
  105.55093557141026,
  114.97825881443848,
  82.87339742040264,
  82.86132994346639,
  84.5931439302264,
  86.57944328765345,
  87.0976463516667,
  90.47098982546837,
  84.64632301523794,
  88.52683209061533,
  90.47098982546837,
  85.16454661418682,
  87.64131445842195,
  90.52071586106685,
  89.81091247727082,
  93.60555539069249,
  37.16180835212409,
  41.78516483155236,
  29.832867780352597,
  34.92849839314596,
  38.07886552931954,
  40.52159917870962,
  42.20189569201838,
  43.32435804486894,
  39.6232255123179,
  38.2099463490856,
  39.319206502675,
  40.607881008493905,
  43.289721643826724,
  43.104524124504614,
  44.91102314577124,
  44.28317965096906,
  44.0,
  54.230987451824994,
  49.25444142409901,
  50.08991914547278,
  48.507731342539614,
  47.853944456021594,
  54.230987451824994,
  52.46903848937962,
  50.21951811795888,
  50.99019513592785,
  53.75872022286245,
  32.14031735997639,
  37.8549864614954,
  49.25444142409901,
  40.80441152620633,
  36.40054944640259,
  30.805843601498726,
  36.0,
  45.27692569068709,
  51.0,
  33.24154027718932,
  34.0,
  43.829214001622255,
  52.92447448959697],
 [44.598206241955516,
  2.0,
  0.0,
  6.324555320336759,
  7.280109889280518,
  7.211102550927978,
  9.219544457292887,
  10.816653826391969,
  21.840329667841555,
  10.770329614269007,
  9.486832980505138,
  10.198039027185569,
  22.20360331117452,
  15.132745950421556,
  14.0,
  11.0,
  48.093658625644196,
  48.25971404805462,
  33.015148038438355,
  51.088159097779204,
  51.478150704935004,
  51.97114584074513,
  53.0,
  53.33854141237835,
  54.0,
  54.037024344425184,
  54.589376255824725,
  57.0,
  57.14017850864661,
  66.09841147864296,
  59.0,
  59.033888572581766,
  59.30430001273095,
  60.03332407921454,
  62.03224967708329,
  60.207972893961475,
  62.12889826803627,
  71.8679344353238,
  73.49829930005184,
  74.63243262818116,
  79.39773296511683,
  75.47184905645283,
  77.10382610480494,
  80.60397012554654,
  77.52418977325722,
  83.00602387778854,
  79.25906888173743,
  80.26207074328447,
  82.37718130647589,
  81.00617260431454,
  85.58621384311844,
  87.66413177577246,
  96.40020746865642,
  98.812954616285,
  98.40731680114035,
  101.6070863670443,
  104.86181383134664,
  98.85848471426213,
  103.60019305001319,
  98.49365461794989,
  107.00467279516349,
  104.23531071570709,
  105.75916035975324,
  109.63576058932597,
  107.93516572461452,
  104.06248123122954,
  113.45483682946268,
  80.89499366462674,
  80.95677859203639,
  82.60750571225353,
  84.5931439302264,
  85.14693182963201,
  88.52683209061533,
  82.71033792700887,
  86.58521813797087,
  88.52683209061533,
  83.24061508662703,
  85.72630868059116,
  88.60022573334675,
  87.92041856133307,
  91.706052144883,
  35.34119409414458,
  39.92492955535426,
  27.892651361962706,
  32.984845004941285,
  36.138621999185304,
  38.600518131237564,
  40.26164427839479,
  41.43669871020132,
  37.656340767525464,
  36.22154055254967,
  37.33630940518894,
  38.63935817272331,
  41.30375285612676,
  41.10960958218893,
  42.95346318982906,
  42.2965719651132,
  42.0,
  52.23983154643591,
  47.265209192385896,
  48.093658625644196,
  46.52956049652737,
  45.89117562233506,
  52.23983154643591,
  50.48762224545735,
  48.27007354458868,
  49.03060268852505,
  51.78802950489621,
  30.14962686336267,
  35.90264614203248,
  47.265209192385896,
  38.897300677553446,
  34.48187929913333,
  28.861739379323623,
  34.0,
  43.289721643826724,
  49.0,
  31.32091952673165,
  32.0,
  42.05948168962618,
  51.07837115648854],
 [45.17742799230607,
  7.211102550927978,
  6.324555320336759,
  0.0,
  1.0,
  2.0,
  4.123105625617661,
  5.0,
  15.524174696260024,
  4.47213595499958,
  3.1622776601683795,
  4.0,
  16.0312195418814,
  9.0,
  8.246211251235321,
  5.385164807134504,
  42.01190307520001,
  42.1070065428546,
  31.78049716414141,
  45.0111097397076,
  45.27692569068709,
  45.70557952810576,
  47.042533945356304,
  47.16990566028302,
  48.041648597857254,
  48.0,
  48.373546489791295,
  51.03920062069938,
  51.03920062069938,
  59.841457201508724,
  53.03772242470448,
  53.0,
  53.150729063673246,
  54.147945482723536,
  56.0,
  54.08326913195984,
  56.0357029044876,
  65.73431371817918,
  67.42403132415029,
  68.60029154456998,
  73.59347797189639,
  69.33974329343886,
  71.02816342831905,
  74.65252842335616,
  71.34423592694787,
  77.00649323271382,
  73.06161783043132,
  74.09453421137081,
  76.27581530209953,
  74.79304780526061,
  79.40403012442127,
  81.54140052758476,
  91.2633551870629,
  93.74433316206373,
  93.23089616645332,
  96.51942809610924,
  99.85990186255943,
  93.60021367496978,
  98.4733466477097,
  93.08598175880189,
  101.83319694480774,
  98.85848471426213,
  100.42410069301094,
  104.4030650891055,
  102.61578825892242,
  98.59513172565875,
  108.07404868885037,
  78.23042886243178,
  77.3692445355388,
  80.09993757800315,
  82.09750300709517,
  82.03657720797473,
  85.32877591996736,
  79.42921376924235,
  83.3606621854697,
  85.32877591996736,
  79.83107164506812,
  82.22530024268686,
  85.14693182963201,
  84.20213774008353,
  88.05679985100527,
  36.61966684720111,
  40.8166632639171,
  28.178005607210743,
  33.1058907144937,
  36.24913792078372,
  38.91015291668744,
  40.311288741492746,
  42.05948168962618,
  37.33630940518894,
  35.4400902933387,
  36.68787265568828,
  38.27531841800928,
  40.52159917870962,
  40.024992192379,
  42.720018726587654,
  41.48493702538308,
  40.44749683231337,
  50.00999900019995,
  46.32493928760188,
  46.87216658103186,
  45.880278987817846,
  45.5411901469428,
  51.19570294468082,
  49.72926703662542,
  48.104053883222775,
  48.703182647543684,
  51.24451190127583,
  28.160255680657446,
  33.06055050963308,
  45.0111097397076,
  39.35733730830886,
  34.88552708502482,
  29.068883707497267,
  32.55764119219941,
  41.012193308819754,
  47.38143096192854,
  31.76476034853718,
  30.59411708155671,
  43.60045871318328,
  51.97114584074513],
 [45.65084884205331,
  8.06225774829855,
  7.280109889280518,
  1.0,
  0.0,
  2.23606797749979,
  4.0,
  4.47213595499958,
  14.560219778561036,
  3.605551275463989,
  2.23606797749979,
  3.0,
  15.033296378372908,
  8.0,
  7.280109889280518,
  4.47213595499958,
  41.012193308819754,
  41.10960958218893,
  32.01562118716424,
  44.01136216933077,
  44.28317965096906,
  44.721359549995796,
  46.04345773288535,
  46.17358552246078,
  47.042533945356304,
  47.0,
  47.38143096192854,
  50.039984012787215,
  50.039984012787215,
  58.855755878248644,
  52.03844732503075,
  52.0,
  52.15361924162119,
  53.150729063673246,
  55.0,
  53.08483775994799,
  55.036351623268054,
  64.89992295835181,
  66.61080993352356,
  67.80117993073571,
  72.86288492778748,
  68.5054742338158,
  70.21395872616783,
  73.87827826905551,
  70.49113419430843,
  76.21679604916491,
  72.20110802473879,
  73.24616030891995,
  75.45197147855052,
  73.92563831310488,
  78.54934754662193,
  80.7093550959243,
  90.69729874698585,
  93.1933474020544,
  92.65527507918802,
  95.96353474106714,
  99.32270636667126,
  93.00537618869137,
  97.9081201943945,
  92.45539465060976,
  101.25709851659784,
  98.2344135219425,
  99.80981915623332,
  103.81233067415451,
  102.00490184299969,
  97.94896630388705,
  107.44766167767449,
  78.16009211867653,
  77.12976079309465,
  80.05623023850174,
  82.05485969764375,
  81.88406438373708,
  85.15867542417507,
  79.2464510246358,
  83.18653737234169,
  85.15867542417507,
  79.62411694957753,
  82.0,
  84.92938243034621,
  83.93449827097318,
  87.80091115700337,
  37.20215047547655,
  41.340053217188775,
  28.653097563788805,
  33.54101966249684,
  36.6742416417845,
  39.35733730830886,
  40.718546143004666,
  42.5440947723653,
  37.69615364994153,
  35.73513677041127,
  37.0,
  38.62641583165593,
  40.80441152620633,
  40.26164427839479,
  43.08131845707603,
  41.7612260356422,
  40.607881008493905,
  50.039984012787215,
  46.57252408878007,
  47.07440918375928,
  46.17358552246078,
  45.880278987817846,
  51.419840528729765,
  50.0,
  48.46648326421054,
  49.040799340956916,
  51.54609587543949,
  28.284271247461902,
  33.015148038438355,
  45.044422518220834,
  39.824615503479755,
  35.35533905932738,
  29.5296461204668,
  32.7566787083184,
  41.048751503547585,
  47.51841748206689,
  32.2490309931942,
  30.805843601498726,
  44.204072210600685,
  52.478567053607705],
 [43.41658669218482,
  8.48528137423857,
  7.211102550927978,
  2.0,
  2.23606797749979,
  0.0,
  2.23606797749979,
  3.605551275463989,
  15.132745950421556,
  4.0,
  3.1622776601683795,
  4.47213595499958,
  16.0312195418814,
  9.219544457292887,
  8.94427190999916,
  6.4031242374328485,
  42.01190307520001,
  42.01190307520001,
  29.832867780352597,
  45.0111097397076,
  45.09988913511872,
  45.39823785126467,
  47.16990566028302,
  47.042533945356304,
  48.16637831516918,
  48.041648597857254,
  48.16637831516918,
  51.15662224971465,
  51.0,
  59.53990258641679,
  53.150729063673246,
  53.03772242470448,
  53.03772242470448,
  54.3323108288245,
  56.0357029044876,
  54.00925846556311,
  56.0,
  64.66065264130884,
  66.2872536767062,
  67.42403132415029,
  72.24956747275377,
  68.26419266350405,
  69.8927750200262,
  73.40980860893181,
  70.32780389006896,
  75.8023746329889,
  72.069410986909,
  73.06161783043132,
  75.16648189186454,
  73.824115301167,
  78.39005038906404,
  80.45495634204272,
  89.62700485902673,
  92.0869154657707,
  91.60785992479029,
  94.86832980505137,
  98.18350166906862,
  92.00543462209176,
  96.83491106000976,
  91.54780172128658,
  100.20977996183805,
  97.30878685915265,
  98.85848471426213,
  102.80077820717118,
  101.04454463255303,
  97.082439194738,
  106.5269918846862,
  76.23647421018367,
  75.43208866258443,
  78.10249675906654,
  80.09993757800315,
  80.06247560499239,
  83.3606621854697,
  77.46612162745726,
  81.39410298049853,
  83.3606621854697,
  77.87810988975015,
  80.2807573457052,
  83.19855767980596,
  82.28000972289685,
  86.1278120005379,
  35.0,
  39.11521443121589,
  26.419689627245813,
  31.304951684997057,
  34.438350715445125,
  37.12142238654117,
  38.48376280978771,
  40.311288741492746,
  35.4682957019364,
  33.52610922848042,
  34.785054261852174,
  36.40054944640259,
  38.600518131237564,
  38.07886552931954,
  40.85339643163099,
  39.56008088970496,
  38.47076812334269,
  48.010415536631214,
  44.384682042344295,
  44.91102314577124,
  43.965895873961216,
  43.657759905886145,
  49.24428900898052,
  47.80167361086848,
  46.238512086787566,
  46.8187996428785,
  49.33558553417604,
  26.1725046566048,
  31.064449134018133,
  43.01162633521314,
  37.589892258425,
  33.12099032335839,
  27.294688127912362,
  30.59411708155671,
  39.01281840626232,
  45.39823785126467,
  30.01666203960727,
  28.635642126552707,
  42.01190307520001,
  50.24937810560445],
 [42.190046219457976,
  10.63014581273465,
  9.219544457292887,
  4.123105625617661,
  4.0,
  2.23606797749979,
  0.0,
  2.0,
  14.0,
  3.605551275463989,
  3.605551275463989,
  5.0,
  15.297058540778355,
  8.94427190999916,
  9.219544457292887,
  7.211102550927978,
  41.10960958218893,
  41.012193308819754,
  28.160255680657446,
  44.10215414239989,
  44.01136216933077,
  44.181444068749045,
  46.389654018972806,
  46.0,
  47.38143096192854,
  47.16990566028302,
  47.042533945356304,
  50.35871324805669,
  50.039984012787215,
  58.309518948453004,
  52.3450093132096,
  52.15361924162119,
  52.0,
  53.600373133029585,
  55.14526271584895,
  53.009433122794285,
  55.036351623268054,
  62.76941930590086,
  64.35060217278468,
  65.45991139621256,
  70.178344238091,
  66.37017402418047,
  67.95586803212802,
  71.40028011149536,
  68.47627326307996,
  73.81734213584231,
  70.2353187506115,
  71.19691004531025,
  73.24616030891995,
  72.00694410957877,
  76.53757247260981,
  78.54934754662193,
  87.41853350405736,
  89.87213138676528,
  89.40357934669059,
  92.65527507918802,
  95.96353474106714,
  89.81091247727082,
  94.62557793746889,
  89.37561188601732,
  98.00510190801293,
  95.13148795220224,
  96.67471230885562,
  100.60318086422517,
  98.85848471426213,
  94.92101980067429,
  104.35037134576953,
  74.16872656315464,
  73.24616030891995,
  76.05918747922567,
  78.05767098754612,
  77.92945527847606,
  81.21576201698781,
  75.31268153505093,
  79.2464510246358,
  81.21576201698781,
  75.7099729229908,
  78.10249675906654,
  81.02468759581859,
  80.0812087820857,
  83.93449827097318,
  34.058772731852805,
  38.01315561749642,
  25.238858928247925,
  30.01666203960727,
  33.12099032335839,
  35.84689665786984,
  37.12142238654117,
  39.11521443121589,
  34.0147027033899,
  31.953090617340916,
  33.24154027718932,
  34.92849839314596,
  37.0,
  36.40054944640259,
  39.395431207184416,
  37.94733192202055,
  36.6742416417845,
  46.04345773288535,
  42.720018726587654,
  43.174066289845804,
  42.37924020083418,
  42.15447781671598,
  47.53945729601885,
  46.17358552246078,
  44.77722635447622,
  45.31004303683677,
  47.75981574503821,
  24.331050121192877,
  29.017236257093817,
  41.048751503547585,
  36.359317925395686,
  31.906112267087632,
  26.076809620810597,
  28.861739379323623,
  37.05401462729781,
  43.56604182158393,
  28.844410203711913,
  26.92582403567252,
  41.10960958218893,
  49.09175083453431],
 [43.266615305567875,
  12.041594578792296,
  10.816653826391969,
  5.0,
  4.47213595499958,
  3.605551275463989,
  2.0,
  0.0,
  12.0,
  2.23606797749979,
  3.0,
  4.123105625617661,
  13.341664064126334,
  7.211102550927978,
  7.810249675906654,
  6.324555320336759,
  39.11521443121589,
  39.01281840626232,
  28.792360097775937,
  42.1070065428546,
  42.01190307520001,
  42.190046219457976,
  44.40720662234904,
  44.0,
  45.39823785126467,
  45.17742799230607,
  45.044422518220834,
  48.373546489791295,
  48.041648597857254,
  56.32051136131489,
  50.35871324805669,
  50.15974481593781,
  50.0,
  51.62363799656123,
  53.150729063673246,
  51.0098029794274,
  53.03772242470448,
  61.05735008989499,
  62.68173577685928,
  63.8200595424354,
  68.6804193347711,
  64.66065264130884,
  66.2872536767062,
  69.81403870282824,
  66.73080248281148,
  72.20110802473879,
  68.47627326307996,
  69.46221994724903,
  71.56116265125938,
  70.2353187506115,
  74.79304780526061,
  76.8505042273634,
  86.2670273047588,
  88.75246475450696,
  88.23264701911646,
  91.52595260361949,
  94.87360012142472,
  88.60022573334675,
  93.47726996441435,
  88.09086218218096,
  96.83491106000976,
  93.86160024205851,
  95.42536350467836,
  99.40321926376429,
  97.6165969494942,
  93.60555539069249,
  103.07764064044152,
  74.06078584514209,
  72.78049189171504,
  76.00657866263946,
  78.00640999302557,
  77.64663547121665,
  80.89499366462674,
  74.96665925596525,
  78.91767862779544,
  80.89499366462674,
  75.31268153505093,
  77.6659513557904,
  80.60397012554654,
  79.55501241279521,
  83.43260753446461,
  35.38361202590826,
  39.20459156782532,
  26.40075756488817,
  31.064449134018133,
  34.132096331752024,
  36.89173349139343,
  38.07886552931954,
  40.22437072223753,
  34.88552708502482,
  32.69556544854363,
  34.0147027033899,
  35.77708763999664,
  37.69615364994153,
  37.0,
  40.24922359499622,
  38.62641583165593,
  37.107950630558946,
  46.17358552246078,
  43.32435804486894,
  43.68065933568311,
  43.08131845707603,
  42.95346318982906,
  48.08326112068523,
  46.8187996428785,
  45.617978911828175,
  46.09772228646444,
  48.46648326421054,
  24.73863375370596,
  29.017236257093817,
  41.19465984809196,
  37.44329045369811,
  33.015148038438355,
  27.202941017470888,
  29.410882339705484,
  37.21558813185679,
  43.93176527297759,
  30.0,
  27.51363298439521,
  42.44997055358225,
  50.21951811795888],
 [50.91168824543142,
  22.47220505424423,
  21.840329667841555,
  15.524174696260024,
  14.560219778561036,
  15.132745950421556,
  14.0,
  12.0,
  0.0,
  11.180339887498949,
  12.36931687685298,
  11.704699910719626,
  3.1622776601683795,
  7.211102550927978,
  9.219544457292887,
  11.661903789690601,
  27.16615541441225,
  27.018512172212592,
  34.828149534535996,
  30.14962686336267,
  30.01666203960727,
  30.265491900843113,
  32.55764119219941,
  32.0,
  33.54101966249684,
  33.24154027718932,
  33.06055050963308,
  36.49657518178932,
  36.05551275463989,
  44.40720662234904,
  38.47076812334269,
  38.2099463490856,
  38.0,
  39.81205847478876,
  41.19465984809196,
  39.01281840626232,
  41.048751503547585,
  51.22499389946279,
  53.150729063673246,
  54.48853090330111,
  60.30754513325841,
  54.817880294662984,
  56.72741841473134,
  60.81118318204309,
  56.61271941887264,
  62.96824596572466,
  58.249463516842795,
  59.405386961116584,
  61.84658438426491,
  59.90826320300064,
  64.63745044476924,
  67.00746227100382,
  80.06247560499239,
  82.73451516749222,
  81.88406438373708,
  85.42247947700886,
  89.0,
  81.98780397107853,
  87.23531395025755,
  80.99382692526635,
  90.4267659490264,
  86.82165628459296,
  88.50988645343524,
  92.78469701410896,
  90.7358804442873,
  86.2670273047588,
  95.96353474106714,
  74.54528824815154,
  71.11258679024411,
  76.79192665899195,
  78.77182237323191,
  77.02596964660685,
  80.0,
  74.0,
  78.0,
  80.0,
  74.02702209328699,
  76.10519036176179,
  79.1012010022604,
  77.41446893184762,
  81.39410298049853,
  44.40720662234904,
  47.50789408087881,
  35.0,
  38.948684188300895,
  41.72529209005013,
  44.553338819890925,
  45.27692569068709,
  48.104053883222775,
  41.773197148410844,
  39.05124837953327,
  40.45985664828782,
  42.5205832509386,
  43.60045871318328,
  42.43819034784589,
  46.861498055439924,
  44.40720662234904,
  41.677331968349414,
  48.703182647543684,
  48.54894437575342,
  48.373546489791295,
  48.82622246293481,
  49.20365840057018,
  52.839379254491625,
  52.15361924162119,
  52.009614495783374,
  52.20153254455275,
  54.08326913195984,
  30.0,
  31.78049716414141,
  44.01136216933077,
  45.27692569068709,
  41.10960958218893,
  35.608987629529715,
  35.0,
  40.311288741492746,
  47.853944456021594,
  38.41874542459709,
  33.421549934136806,
  51.40038910358559,
  57.982756057296896],
 [45.48626166217664,
  11.661903789690601,
  10.770329614269007,
  4.47213595499958,
  3.605551275463989,
  4.0,
  3.605551275463989,
  2.23606797749979,
  11.180339887498949,
  0.0,
  1.4142135623730951,
  2.0,
  12.041594578792296,
  5.385164807134504,
  5.656854249492381,
  4.123105625617661,
  38.01315561749642,
  38.01315561749642,
  31.016124838541646,
  41.012193308819754,
  41.10960958218893,
  41.43669871020132,
  43.18564576337837,
  43.04648650006177,
  44.181444068749045,
  44.04543109109048,
  44.181444068749045,
  47.16990566028302,
  47.0,
  55.57877292636101,
  49.16299421312742,
  49.040799340956916,
  49.040799340956916,
  50.35871324805669,
  52.03844732503075,
  50.00999900019995,
  52.0,
  61.29437168288782,
  63.00793600809346,
  64.20280367709809,
  69.31089380465383,
  64.89992295835181,
  66.61080993352356,
  70.2922470831599,
  66.88796603276258,
  72.62231062146122,
  68.60029154456998,
  69.6419413859206,
  71.84705978674423,
  70.32780389006896,
  74.94664769020693,
  77.10382610480494,
  87.36704184073076,
  89.88882021697692,
  89.30845424706443,
  92.64987857520376,
  96.04165762834376,
  89.62700485902673,
  94.57801012920498,
  89.02246907382428,
  97.9081201943945,
  94.81033698917012,
  96.40020746865642,
  100.43903623591775,
  98.60020283954795,
  94.49338601193207,
  104.01922899156675,
  76.02631123499285,
  74.52516353554684,
  78.0,
  80.0,
  79.51100552753688,
  82.73451516749222,
  76.79192665899195,
  80.75270893288967,
  82.73451516749222,
  77.10382610480494,
  79.42921376924235,
  82.37718130647589,
  81.25269226308751,
  85.14693182963201,
  37.53664875824692,
  41.400483088968905,
  28.600699292150182,
  33.28663395418648,
  36.359317925395686,
  39.11521443121589,
  40.311288741492746,
  42.43819034784589,
  37.12142238654117,
  34.92849839314596,
  36.24913792078372,
  38.01315561749642,
  39.92492955535426,
  39.21734310225516,
  42.485291572496,
  40.85339643163099,
  39.293765408777,
  48.25971404805462,
  45.5411901469428,
  45.880278987817846,
  45.31004303683677,
  45.18849411078001,
  50.28916384272063,
  49.040799340956916,
  47.853944456021594,
  48.33218389437829,
  50.695167422546305,
  26.92582403567252,
  31.064449134018133,
  43.289721643826724,
  39.66106403010388,
  35.22782990761707,
  29.410882339705484,
  31.622776601683793,
  39.319206502675,
  46.09772228646444,
  32.202484376209235,
  29.732137494637012,
  44.598206241955516,
  52.43090691567332],
 [45.79301256742124,
  10.295630140987,
  9.486832980505138,
  3.1622776601683795,
  2.23606797749979,
  3.1622776601683795,
  3.605551275463989,
  3.0,
  12.36931687685298,
  1.4142135623730951,
  0.0,
  1.4142135623730951,
  13.0,
  6.082762530298219,
  5.830951894845301,
  3.605551275463989,
  39.0,
  39.05124837953327,
  31.622776601683793,
  42.0,
  42.190046219457976,
  42.579337712087536,
  44.10215414239989,
  44.10215414239989,
  45.09988913511872,
  45.0111097397076,
  45.27692569068709,
  48.093658625644196,
  48.010415536631214,
  56.71860364994893,
  50.08991914547278,
  50.00999900019995,
  50.08991914547278,
  51.24451190127583,
  53.009433122794285,
  51.03920062069938,
  53.009433122794285,
  62.68173577685928,
  64.40496875241847,
  65.60487786742691,
  70.7248188403477,
  66.2872536767062,
  68.00735254367721,
  71.7007670809734,
  68.26419266350405,
  74.02702209328699,
  69.9714227381436,
  71.02112361825881,
  73.23933369440222,
  71.69379331573968,
  76.32168761236873,
  78.49203781276162,
  88.75246475450696,
  91.26883367283708,
  90.69729874698585,
  94.03190947758107,
  97.41663102366043,
  91.02197536858887,
  95.96353474106714,
  90.4267659490264,
  99.29753269845128,
  96.21330469326995,
  97.8008179924892,
  101.83319694480774,
  100.0,
  95.90099061010788,
  105.4229576515476,
  77.05841939723393,
  75.7099729229908,
  79.00632886041473,
  81.00617260431454,
  80.62257748298549,
  83.86298349093002,
  77.92945527847606,
  81.88406438373708,
  83.86298349093002,
  78.26237921249263,
  80.60397012554654,
  83.54639429682169,
  82.46211251235322,
  86.34813257969161,
  37.64306044943742,
  41.617304093369626,
  28.844410203711913,
  33.61547262794322,
  36.71511950137164,
  39.44616584663204,
  40.70626487409524,
  42.720018726587654,
  37.57658845611187,
  35.4682957019364,
  36.76955262170047,
  38.48376280978771,
  40.496913462633174,
  39.84971769034255,
  42.95346318982906,
  41.43669871020132,
  40.024992192379,
  49.16299421312742,
  46.17358552246078,
  46.57252408878007,
  45.880278987817846,
  45.69463863518345,
  50.96076922496363,
  49.64876634922564,
  48.33218389437829,
  48.84669896727925,
  51.264022471905186,
  27.65863337187866,
  32.01562118716424,
  44.181444068749045,
  39.96248240537617,
  35.510561809129406,
  29.68164415931166,
  32.28002478313795,
  40.19950248448356,
  46.87216658103186,
  32.449961479175904,
  30.364452901377952,
  44.68780594300866,
  52.69724850502159],
 [47.16990566028302,
  10.770329614269007,
  10.198039027185569,
  4.0,
  3.0,
  4.47213595499958,
  5.0,
  4.123105625617661,
  11.704699910719626,
  2.0,
  1.4142135623730951,
  0.0,
  12.041594578792296,
  5.0,
  4.47213595499958,
  2.23606797749979,
  38.01315561749642,
  38.118237105091836,
  32.89376840679705,
  41.012193308819754,
  41.30375285612676,
  41.773197148410844,
  43.04648650006177,
  43.18564576337837,
  44.04543109109048,
  44.0,
  44.40720662234904,
  47.042533945356304,
  47.042533945356304,
  55.90169943749474,
  49.040799340956916,
  49.0,
  49.16299421312742,
  50.15974481593781,
  52.0,
  50.08991914547278,
  52.03844732503075,
  62.42595614005443,
  64.20280367709809,
  65.43699259593154,
  70.71067811865476,
  66.03029607687671,
  67.80117993073571,
  71.58910531638176,
  67.95586803212802,
  73.87827826905551,
  69.6419413859206,
  70.7248188403477,
  73.00684899377592,
  71.34423592694787,
  76.00657866263946,
  78.2368199762746,
  89.04493247793498,
  91.58602513484249,
  90.97252332435328,
  94.33981132056604,
  97.75479527879949,
  91.2633551870629,
  96.25487000666512,
  90.60353193998566,
  99.56907150315303,
  96.40020746865642,
  98.00510190801293,
  102.07840124139877,
  100.20977996183805,
  96.04686356149273,
  105.60303025955268,
  78.02563681252464,
  76.48529270389177,
  80.0,
  82.0,
  81.49846624323675,
  84.71717653463199,
  78.77182237323191,
  82.73451516749222,
  84.71717653463199,
  79.07591289387686,
  81.39410298049853,
  84.34453153583817,
  83.19855767980596,
  87.0976463516667,
  39.05124837953327,
  43.01162633521314,
  30.23243291566195,
  34.9857113690718,
  38.07886552931954,
  40.8166632639171,
  42.05948168962618,
  44.10215414239989,
  38.91015291668744,
  36.76955262170047,
  38.07886552931954,
  39.81205847478876,
  41.78516483155236,
  41.10960958218893,
  44.28317965096906,
  42.720018726587654,
  41.23105625617661,
  50.24937810560445,
  47.43416490252569,
  47.80167361086848,
  47.16990566028302,
  47.01063709417264,
  52.20153254455275,
  50.92150822589606,
  49.658836071740545,
  50.15974481593781,
  52.55473337388365,
  28.861739379323623,
  33.06055050963308,
  45.27692569068709,
  41.340053217188775,
  36.89173349139343,
  31.064449134018133,
  33.52610922848042,
  41.30375285612676,
  48.05205510693585,
  33.83784863137726,
  31.622776601683793,
  46.09772228646444,
  54.08326913195984],
 [53.75872022286245,
  22.561028345356956,
  22.20360331117452,
  16.0312195418814,
  15.033296378372908,
  16.0312195418814,
  15.297058540778355,
  13.341664064126334,
  3.1622776601683795,
  12.041594578792296,
  13.0,
  12.041594578792296,
  0.0,
  7.0710678118654755,
  8.54400374531753,
  11.40175425099138,
  26.0,
  26.076809620810597,
  37.8021163428716,
  29.0,
  29.274562336608895,
  29.832867780352597,
  31.144823004794873,
  31.144823004794873,
  32.14031735997639,
  32.01562118716424,
  32.38826948140329,
  35.12833614050059,
  35.014282800023196,
  43.93176527297759,
  37.12142238654117,
  37.013511046643494,
  37.12142238654117,
  38.3275357934736,
  40.01249804748511,
  38.05259518088089,
  40.01249804748511,
  52.40229002629561,
  54.45181356024793,
  55.86591089385369,
  62.00806399170998,
  55.97320787662612,
  58.0,
  62.289646009589745,
  57.62811813689564,
  64.35060217278468,
  59.20304046246274,
  60.440052945046304,
  63.0317380372777,
  60.802960454241045,
  65.60487786742691,
  68.11754546370561,
  82.29216244576394,
  85.0,
  84.07734534343957,
  87.66413177577246,
  91.28526715741155,
  84.11896337925236,
  89.44271909999159,
  83.00602387778854,
  92.59049627256569,
  88.83692925805124,
  90.55385138137417,
  94.89467845985885,
  92.78469701410896,
  88.20430828479978,
  97.94386147176351,
  77.64663547121665,
  74.06078584514209,
  79.90619500389191,
  81.88406438373708,
  80.05623023850174,
  83.00602387778854,
  77.00649323271382,
  81.00617260431454,
  83.00602387778854,
  77.00649323271382,
  79.05694150420949,
  82.05485969764375,
  80.30566605165541,
  84.29116205154607,
  47.01063709417264,
  50.24937810560445,
  37.64306044943742,
  41.72529209005013,
  44.553338819890925,
  47.38143096192854,
  48.16637831516918,
  50.91168824543142,
  44.68780594300866,
  42.01190307520001,
  43.41658669218482,
  45.45327270945405,
  46.61544808322666,
  45.48626166217664,
  49.8196748283246,
  47.43416490252569,
  44.77722635447622,
  51.86520991955976,
  51.62363799656123,
  51.478150704935004,
  51.86520991955976,
  52.20153254455275,
  55.94640292279746,
  55.226805085936306,
  55.0,
  55.21775076911409,
  57.14017850864661,
  33.015148038438355,
  34.92849839314596,
  47.16990566028302,
  48.08326112068523,
  43.86342439892262,
  38.28837943815329,
  38.01315561749642,
  43.46262762420146,
  50.99019513592785,
  41.10960958218893,
  36.40054944640259,
  54.037024344425184,
  60.8276253029822],
 [50.0,
  15.524174696260024,
  15.132745950421556,
  9.0,
  8.0,
  9.219544457292887,
  8.94427190999916,
  7.211102550927978,
  7.211102550927978,
  5.385164807134504,
  6.082762530298219,
  5.0,
  7.0710678118654755,
  0.0,
  2.23606797749979,
  4.47213595499958,
  33.015148038438355,
  33.13608305156178,
  34.88552708502482,
  36.013886210738214,
  36.345563690772494,
  36.87817782917155,
  38.05259518088089,
  38.2099463490856,
  39.05124837953327,
  39.0,
  39.45883931389772,
  42.04759208325728,
  42.04759208325728,
  50.99019513592785,
  44.04543109109048,
  44.0,
  44.181444068749045,
  45.17742799230607,
  47.0,
  45.09988913511872,
  47.042533945356304,
  58.412327466040935,
  60.30754513325841,
  61.61980201201558,
  67.26812023536856,
  62.00806399170998,
  63.89053137985315,
  67.89698078707183,
  63.8200595424354,
  70.09279563550022,
  65.45991139621256,
  66.61080993352356,
  69.02897942168927,
  67.11929677819934,
  71.84705978674423,
  74.2024258363566,
  86.45229898620394,
  89.06739021662193,
  88.32326986700618,
  91.7877987534291,
  95.29428104561154,
  88.50988645343524,
  93.64827814754524,
  87.65842800324451,
  96.89685237405806,
  93.47726996441435,
  95.13148795220224,
  99.32270636667126,
  97.3498844375277,
  93.00537618869137,
  102.6498904042279,
  78.05767098754612,
  75.66372975210778,
  80.15609770940699,
  82.15229759416349,
  81.09870529176159,
  84.2140130857092,
  78.23042886243178,
  82.21921916437786,
  84.2140130857092,
  78.40918313565064,
  80.62257748298549,
  83.60023923410746,
  82.20097323024831,
  86.14522621712709,
  42.42640687119285,
  46.09772228646444,
  33.301651610693426,
  37.8021163428716,
  40.80441152620633,
  43.60045871318328,
  44.654227123532216,
  47.01063709417264,
  41.340053217188775,
  38.948684188300895,
  40.311288741492746,
  42.190046219457976,
  43.829214001622255,
  42.95346318982906,
  46.647615158762406,
  44.721359549995796,
  42.720018726587654,
  50.99019513592785,
  49.24428900898052,
  49.39635614091387,
  49.193495504995376,
  49.24428900898052,
  53.85164807134504,
  52.80151512977634,
  51.97114584074513,
  52.354560450833695,
  54.56189146281496,
  30.463092423455635,
  33.734255586866,
  46.09772228646444,
  44.204072210600685,
  39.824615503479755,
  34.058772731852805,
  35.34119409414458,
  42.20189569201838,
  49.33558553417604,
  36.87817782917155,
  33.54101966249684,
  49.49747468305833,
  57.0087712549569],
 [51.03920062069938,
  14.142135623730951,
  14.0,
  8.246211251235321,
  7.280109889280518,
  8.94427190999916,
  9.219544457292887,
  7.810249675906654,
  9.219544457292887,
  5.656854249492381,
  5.830951894845301,
  4.47213595499958,
  8.54400374531753,
  2.23606797749979,
  0.0,
  3.0,
  34.132096331752024,
  34.36568055487916,
  36.24913792078372,
  37.12142238654117,
  37.656340767525464,
  38.3275357934736,
  39.0,
  39.45883931389772,
  40.0,
  40.049968789001575,
  40.792156108742276,
  43.0,
  43.18564576337837,
  52.392747589718944,
  45.0,
  45.044422518220834,
  45.39823785126467,
  46.04345773288535,
  48.041648597857254,
  46.2709412050371,
  48.16637831516918,
  60.440052945046304,
  62.369864518050704,
  63.702433234531945,
  69.42621983083913,
  64.03124237432849,
  65.94694837519019,
  70.00714249274856,
  65.80273550544841,
  72.18032973047436,
  67.42403132415029,
  68.60029154456998,
  71.06335201775947,
  69.06518659932803,
  73.81734213584231,
  76.21679604916491,
  88.68483523128404,
  91.30169768410661,
  90.55385138137417,
  94.02127418834527,
  97.52948272189288,
  90.7358804442873,
  95.88013350011565,
  89.87213138676528,
  99.12618221237011,
  95.69221493935649,
  97.3498844375277,
  101.54801819828883,
  99.56907150315303,
  95.2102935611481,
  104.86181383134664,
  80.02499609497022,
  77.78174593052023,
  82.09750300709517,
  84.09518416651456,
  83.15046602394962,
  86.28441342444185,
  80.30566605165541,
  84.29116205154607,
  86.28441342444185,
  80.50465825031493,
  82.73451516749222,
  85.70880934886448,
  84.34453153583817,
  88.28363381737297,
  43.18564576337837,
  47.01063709417264,
  34.20526275297414,
  38.8329756778952,
  41.88078318274385,
  44.654227123532216,
  45.79301256742124,
  48.010415536631214,
  42.5440947723653,
  40.24922359499622,
  41.593268686170845,
  43.41658669218482,
  45.18849411078001,
  44.384682042344295,
  47.885279575251516,
  46.09772228646444,
  44.27188724235731,
  52.773099207835045,
  50.695167422546305,
  50.92150822589606,
  50.566787519082126,
  50.53711507397311,
  55.362442142665635,
  54.230987451824994,
  53.23532661682466,
  53.665631459994955,
  55.94640292279746,
  31.953090617340916,
  35.510561809129406,
  47.853944456021594,
  45.221676218380054,
  40.80441152620633,
  35.0,
  36.76955262170047,
  43.93176527297759,
  50.96076922496363,
  37.8021163428716,
  34.92849839314596,
  50.24937810560445,
  58.008620049092706],
 [49.39635614091387,
  11.180339887498949,
  11.0,
  5.385164807134504,
  4.47213595499958,
  6.4031242374328485,
  7.211102550927978,
  6.324555320336759,
  11.661903789690601,
  4.123105625617661,
  3.605551275463989,
  2.23606797749979,
  11.40175425099138,
  4.47213595499958,
  3.0,
  0.0,
  37.12142238654117,
  37.33630940518894,
  35.11409973215888,
  40.11234224026316,
  40.607881008493905,
  41.23105625617661,
  42.0,
  42.42640687119285,
  43.0,
  43.04648650006177,
  43.73785545725808,
  46.0,
  46.17358552246078,
  55.31726674375732,
  48.0,
  48.041648597857254,
  48.373546489791295,
  49.040799340956916,
  51.03920062069938,
  49.25444142409901,
  51.15662224971465,
  62.80127387243033,
  64.66065264130884,
  65.94694837519019,
  71.449282711585,
  66.40030120413611,
  68.24954212300622,
  72.18032973047436,
  68.24221567329127,
  74.41102068914255,
  69.8927750200262,
  71.02816342831905,
  73.40980860893181,
  71.56116265125938,
  76.27581530209953,
  78.60025445251433,
  90.21086409075129,
  92.78469701410896,
  92.11405973031478,
  95.524865872714,
  98.97979591815695,
  92.35799911215054,
  97.41663102366043,
  91.60785992479029,
  100.70253224224304,
  97.41663102366043,
  99.04544411531506,
  103.1746092796091,
  101.25709851659784,
  97.0051545022222,
  106.60675400742676,
  80.00624975587844,
  78.26237921249263,
  82.00609733428362,
  84.00595217006948,
  83.38465086573187,
  86.57944328765345,
  80.62257748298549,
  84.5931439302264,
  86.57944328765345,
  80.89499366462674,
  83.18653737234169,
  86.14522621712709,
  84.92938243034621,
  88.84255736976509,
  41.23105625617661,
  45.221676218380054,
  32.449961479175904,
  37.21558813185679,
  40.311288741492746,
  43.04648650006177,
  44.294469180700204,
  46.32493928760188,
  41.14608122288197,
  39.0,
  40.311288741492746,
  42.04759208325728,
  44.01136216933077,
  43.32435804486894,
  46.51881339845203,
  44.94441010848846,
  43.41658669218482,
  52.3450093132096,
  49.64876634922564,
  50.0,
  49.39635614091387,
  49.24428900898052,
  54.405882034941776,
  53.14132102234569,
  51.894122981316485,
  52.392747589718944,
  54.78138369920935,
  31.04834939252005,
  35.12833614050059,
  47.38143096192854,
  43.56604182158393,
  39.11521443121589,
  33.28663395418648,
  35.73513677041127,
  43.41658669218482,
  50.21951811795888,
  36.05551275463989,
  33.83784863137726,
  48.27007354458868,
  56.302753041036986],
 [74.09453421137081,
  48.25971404805462,
  48.093658625644196,
  42.01190307520001,
  41.012193308819754,
  42.01190307520001,
  41.10960958218893,
  39.11521443121589,
  27.16615541441225,
  38.01315561749642,
  39.0,
  38.01315561749642,
  26.0,
  33.015148038438355,
  34.132096331752024,
  37.12142238654117,
  0.0,
  2.0,
  57.4543296888929,
  3.0,
  5.0,
  7.615773105863909,
  5.830951894845301,
  5.830951894845301,
  6.708203932499369,
  6.082762530298219,
  7.810249675906654,
  9.486832980505138,
  9.055385138137417,
  19.235384061671343,
  11.40175425099138,
  11.045361017187261,
  11.40175425099138,
  13.0,
  14.035668847618199,
  12.165525060596439,
  14.035668847618199,
  37.33630940518894,
  40.162171256046406,
  42.05948168962618,
  50.44799302251776,
  40.311288741492746,
  43.08131845707603,
  48.703182647543684,
  40.70626487409524,
  49.76946855251722,
  41.677331968349414,
  43.41658669218482,
  46.95742752749558,
  42.720018726587654,
  47.70744176750625,
  51.10772935672255,
  74.67261881037788,
  77.64663547121665,
  75.95393340703298,
  79.90619500389191,
  83.86298349093002,
  75.31268153505093,
  81.21576201698781,
  73.00684899377592,
  83.74365647617735,
  78.587530817554,
  80.52328855678958,
  85.37564055396598,
  82.71033792700887,
  77.20103626247513,
  86.97700845625813,
  85.0,
  77.4919350642375,
  87.66413177577246,
  89.47066558375433,
  85.09406559801923,
  87.28115489611719,
  81.59656855530139,
  85.38149682454625,
  87.28115489611719,
  80.95677859203639,
  82.28000972289685,
  85.16454661418682,
  82.22530024268686,
  86.12200647918046,
  69.35416353759881,
  71.58910531638176,
  60.00833275470999,
  63.06346010171025,
  65.36818798161687,
  68.09552114493287,
  68.26419266350405,
  71.69379331573968,
  64.66065264130884,
  61.587336360651285,
  62.96824596572466,
  65.19202405202648,
  65.21502894272147,
  63.60031446463138,
  69.06518659932803,
  65.80273550544841,
  61.84658438426491,
  65.19202405202648,
  68.8839603971781,
  68.01470429252781,
  69.81403870282824,
  70.80254232723568,
  72.18032973047436,
  72.34638899074369,
  73.60027173862879,
  73.40980860893181,
  74.51845409024533,
  52.478567053607705,
  51.22499389946279,
  61.5223536610881,
  68.96375859826666,
  65.29931086925804,
  60.41522986797286,
  57.14017850864661,
  58.728187440104094,
  66.48308055437865,
  63.071388124885914,
  56.08029957123981,
  76.02631123499285,
  80.99382692526635],
 [73.06161783043132,
  48.507731342539614,
  48.25971404805462,
  42.1070065428546,
  41.10960958218893,
  42.01190307520001,
  41.012193308819754,
  39.01281840626232,
  27.018512172212592,
  38.01315561749642,
  39.05124837953327,
  38.118237105091836,
  26.076809620810597,
  33.13608305156178,
  34.36568055487916,
  37.33630940518894,
  2.0,
  0.0,
  56.43580423808985,
  3.605551275463989,
  3.605551275463989,
  5.830951894845301,
  7.0710678118654755,
  5.0990195135927845,
  7.810249675906654,
  6.708203932499369,
  6.708203932499369,
  10.295630140987,
  9.055385138137417,
  18.384776310850235,
  12.083045973594572,
  11.40175425099138,
  11.045361017187261,
  13.892443989449804,
  14.317821063276353,
  12.0,
  14.035668847618199,
  35.4682957019364,
  38.27531841800928,
  40.162171256046406,
  48.507731342539614,
  38.48376280978771,
  41.23105625617661,
  46.8187996428785,
  38.948684188300895,
  47.92702786528704,
  39.96248240537617,
  41.677331968349414,
  45.17742799230607,
  41.048751503547585,
  46.04345773288535,
  49.39635614091387,
  72.69112738154499,
  75.66372975210778,
  73.97972695272672,
  77.92945527847606,
  81.88406438373708,
  73.348483283569,
  79.2464510246358,
  71.06335201775947,
  81.78630692236935,
  76.6550715869472,
  78.587530817554,
  83.43260753446461,
  80.77747210701756,
  75.28612089887484,
  85.07055894961546,
  83.19254774317228,
  75.58438992278762,
  85.86617494683223,
  87.66413177577246,
  83.21658488546619,
  85.38149682454625,
  79.71198153351854,
  83.48652585896721,
  85.38149682454625,
  79.05694150420949,
  80.36168241145776,
  83.24061508662703,
  80.2807573457052,
  84.17244204607586,
  68.54195795277518,
  70.66116330771806,
  59.23681287847955,
  62.16912416947821,
  64.41273166075166,
  67.11929677819934,
  67.23094525588644,
  70.71067811865476,
  63.631753079732135,
  60.53924347066124,
  61.91122676865643,
  64.1404708432983,
  64.07027391856539,
  62.42595614005443,
  67.95586803212802,
  64.63745044476924,
  60.60528029800704,
  63.702433234531945,
  67.62396025078685,
  66.70832032063167,
  68.60029154456998,
  69.63476143421474,
  70.83784299369935,
  71.06335201775947,
  72.42237223399962,
  72.20110802473879,
  73.24616030891995,
  51.478150704935004,
  50.0,
  60.108235708594876,
  68.0,
  64.40496875241847,
  59.61543424315552,
  56.08029957123981,
  57.38466694161429,
  65.11528238439882,
  62.24146527838174,
  55.072679252057455,
  75.15317691222374,
  79.92496481075234],
 [16.64331697709324,
  35.014282800023196,
  33.015148038438355,
  31.78049716414141,
  32.01562118716424,
  29.832867780352597,
  28.160255680657446,
  28.792360097775937,
  34.828149534535996,
  31.016124838541646,
  31.622776601683793,
  32.89376840679705,
  37.8021163428716,
  34.88552708502482,
  36.24913792078372,
  35.11409973215888,
  57.4543296888929,
  56.43580423808985,
  0.0,
  60.03332407921454,
  58.137767414994535,
  56.859475903318,
  63.28506932918696,
  60.37383539249432,
  64.1404708432983,
  63.13477647065839,
  60.41522986797286,
  66.73080248281148,
  64.8459713474939,
  69.26037828369117,
  68.47627326307996,
  67.53517601961218,
  65.79513659838392,
  70.32780389006896,
  70.21395872616783,
  67.11929677819934,
  69.35416353759881,
  62.20128616033595,
  62.51399843235114,
  62.80127387243033,
  63.89053137985315,
  65.37583651472461,
  65.76473218982953,
  66.94027188471824,
  68.26419266350405,
  69.85699678629192,
  70.25667228100119,
  70.45565981523414,
  71.02112361825881,
  72.24956747275377,
  75.66372975210778,
  76.2954782408499,
  73.60027173862879,
  75.43208866258443,
  75.8023746329889,
  78.24321056807422,
  80.8084154033477,
  76.84399781375251,
  80.43009387039157,
  77.69813382572325,
  84.02380615040002,
  82.92767933567175,
  84.0535543567314,
  87.00574693662483,
  86.02325267042627,
  83.52843827104634,
  91.83681179135085,
  48.76474136094644,
  51.10772935672255,
  50.21951811795888,
  52.172789842982326,
  53.85164807134504,
  57.38466694161429,
  51.894122981316485,
  55.54277630799526,
  57.38466694161429,
  52.773099207835045,
  55.47071299343465,
  58.137767414994535,
  58.309518948453004,
  61.773780845922005,
  14.035668847618199,
  14.560219778561036,
  8.48528137423857,
  7.0710678118654755,
  8.246211251235321,
  10.770329614269007,
  10.816653826391969,
  14.317821063276353,
  7.211102550927978,
  4.242640687119285,
  5.656854249492381,
  7.810249675906654,
  8.94427190999916,
  8.246211251235321,
  12.041594578792296,
  9.848857801796104,
  9.055385138137417,
  19.924858845171276,
  14.560219778561036,
  15.132745950421556,
  14.317821063276353,
  14.422205101855956,
  19.4164878389476,
  18.027756377319946,
  17.204650534085253,
  17.4928556845359,
  19.697715603592208,
  5.0,
  9.219544457292887,
  15.231546211727817,
  11.704699910719626,
  9.0,
  7.810249675906654,
  1.4142135623730951,
  11.661903789690601,
  16.0312195418814,
  8.54400374531753,
  1.4142135623730951,
  19.4164878389476,
  23.600847442411894],
 [76.66159403508382,
  51.24451190127583,
  51.088159097779204,
  45.0111097397076,
  44.01136216933077,
  45.0111097397076,
  44.10215414239989,
  42.1070065428546,
  30.14962686336267,
  41.012193308819754,
  42.0,
  41.012193308819754,
  29.0,
  36.013886210738214,
  37.12142238654117,
  40.11234224026316,
  3.0,
  3.605551275463989,
  60.03332407921454,
  0.0,
  4.0,
  7.0,
  3.605551275463989,
  3.605551275463989,
  4.242640687119285,
  3.1622776601683795,
  5.830951894845301,
  6.708203932499369,
  6.082762530298219,
  16.64331697709324,
  8.54400374531753,
  8.06225774829855,
  8.54400374531753,
  10.295630140987,
  11.045361017187261,
  9.219544457292887,
  11.045361017187261,
  36.40054944640259,
  39.293765408777,
  41.23105625617661,
  49.8196748283246,
  39.21734310225516,
  42.05948168962618,
  47.80167361086848,
  39.395431207184416,
  48.703182647543684,
  40.24922359499622,
  42.04759208325728,
  45.69463863518345,
  41.182520563948,
  46.14108798023731,
  49.64876634922564,
  74.33034373659252,
  77.31752712031083,
  75.53806987208503,
  79.51100552753688,
  83.48652585896721,
  74.81310045707235,
  80.75270893288967,
  72.3671196055225,
  83.18653737234169,
  77.87810988975015,
  79.83107164506812,
  84.72307831990054,
  82.0,
  76.40026177965623,
  86.1278120005379,
  86.31338250816034,
  78.43468620451031,
  89.00561780022652,
  90.78546139112804,
  86.16263691415206,
  88.25531145489205,
  82.63776376451628,
  86.37708029332781,
  88.25531145489205,
  81.93289937503738,
  83.16850365372699,
  86.02325267042627,
  82.96987405076617,
  86.83317338436964,
  72.09022125087424,
  74.24284477308234,
  62.76941930590086,
  65.7419196555744,
  68.0,
  70.71067811865476,
  70.83078426785913,
  74.30343195303969,
  67.23094525588644,
  64.1404708432983,
  65.5133574166368,
  67.7421582177598,
  67.67569726275453,
  66.03029607687671,
  71.56116265125938,
  68.24221567329127,
  64.20280367709809,
  67.20863039818622,
  71.21797525905943,
  70.2922470831599,
  72.20110802473879,
  73.23933369440222,
  74.41102068914255,
  74.65252842335616,
  76.02631123499285,
  75.8023746329889,
  76.83749084919418,
  55.072679252057455,
  53.600373133029585,
  63.655321851358195,
  71.58910531638176,
  67.97793759742936,
  63.15853069855251,
  59.682493245507096,
  60.959002616512684,
  68.6804193347711,
  65.79513659838392,
  58.66856057549052,
  78.7210264160726,
  83.52843827104634],
 [74.70609078247904,
  51.78802950489621,
  51.478150704935004,
  45.27692569068709,
  44.28317965096906,
  45.09988913511872,
  44.01136216933077,
  42.01190307520001,
  30.01666203960727,
  41.10960958218893,
  42.190046219457976,
  41.30375285612676,
  29.274562336608895,
  36.345563690772494,
  37.656340767525464,
  40.607881008493905,
  5.0,
  3.605551275463989,
  58.137767414994535,
  4.0,
  0.0,
  3.0,
  7.280109889280518,
  2.23606797749979,
  7.615773105863909,
  5.830951894845301,
  3.1622776601683795,
  9.219544457292887,
  6.708203932499369,
  14.866068747318506,
  10.63014581273465,
  9.433981132056603,
  8.06225774829855,
  12.727922061357855,
  12.083045973594572,
  9.219544457292887,
  11.40175425099138,
  32.57299494980466,
  35.4400902933387,
  37.36308338453881,
  45.89117562233506,
  35.4682957019364,
  38.27531841800928,
  43.965895873961216,
  35.77708763999664,
  44.94441010848846,
  36.71511950137164,
  38.47076812334269,
  42.04759208325728,
  37.73592452822641,
  42.720018726587654,
  46.14108798023731,
  70.34912934784623,
  73.33484846919642,
  71.56814934033156,
  75.53806987208503,
  79.51100552753688,
  70.8590149522275,
  76.79192665899195,
  68.4470598345904,
  79.2464510246358,
  73.97972695272672,
  75.92759709091287,
  80.8084154033477,
  78.10249675906654,
  72.53275122315436,
  82.28000972289685,
  82.76472678623425,
  74.67261881037788,
  85.47514258543241,
  87.23531395025755,
  82.46211251235322,
  84.50443775329198,
  78.92401408950256,
  82.63776376451628,
  84.50443775329198,
  78.18567643756751,
  79.37883848986453,
  82.21921916437786,
  79.12016177940993,
  82.96987405076617,
  70.57619995437555,
  72.49827584156743,
  61.35144660071187,
  64.07807737440318,
  66.2117814289874,
  68.87670143089025,
  68.8839603971781,
  72.44998274671983,
  65.29931086925804,
  62.177166226839255,
  63.52952069707436,
  65.76473218982953,
  65.5133574166368,
  63.812224534175265,
  69.46221994724903,
  66.03786792439622,
  61.85466837676846,
  64.35060217278468,
  68.81860213634101,
  67.80117993073571,
  69.8927750200262,
  71.02112361825881,
  71.84010022264724,
  72.20110802473879,
  73.78346698278686,
  73.49829930005184,
  74.4043009509531,
  53.225933528685054,
  51.31276644267,
  60.959002616512684,
  69.77822009767804,
  66.30987860040162,
  61.68468205316454,
  57.706152185014034,
  58.412327466040935,
  66.06814663663572,
  64.25729530566937,
  56.79788728465171,
  77.07788269017254,
  81.49233092751734],
 [73.348483283569,
  52.392747589718944,
  51.97114584074513,
  45.70557952810576,
  44.721359549995796,
  45.39823785126467,
  44.181444068749045,
  42.190046219457976,
  30.265491900843113,
  41.43669871020132,
  42.579337712087536,
  41.773197148410844,
  29.832867780352597,
  36.87817782917155,
  38.3275357934736,
  41.23105625617661,
  7.615773105863909,
  5.830951894845301,
  56.859475903318,
  7.0,
  3.0,
  0.0,
  10.198039027185569,
  4.47213595499958,
  10.44030650891055,
  8.54400374531753,
  3.605551275463989,
  11.661903789690601,
  8.48528137423857,
  14.142135623730951,
  12.806248474865697,
  11.313708498984761,
  8.94427190999916,
  15.0,
  13.601470508735444,
  10.295630140987,
  12.529964086141668,
  29.732137494637012,
  32.57299494980466,
  34.48187929913333,
  42.95346318982906,
  32.69556544854363,
  35.4682957019364,
  41.10960958218893,
  33.12099032335839,
  42.15447781671598,
  34.132096331752024,
  35.84689665786984,
  39.35733730830886,
  35.22782990761707,
  40.22437072223753,
  43.56604182158393,
  67.36467917239716,
  70.34912934784623,
  68.59300255857006,
  72.56031973468694,
  76.53103945458993,
  67.89698078707183,
  73.824115301167,
  65.5133574166368,
  76.2954782408499,
  71.06335201775947,
  73.00684899377592,
  77.87810988975015,
  75.18643494673756,
  69.6419413859206,
  79.40403012442127,
  80.1311425102625,
  71.8679344353238,
  82.85529554590944,
  84.59905436823747,
  79.7057087039567,
  81.70679286326198,
  76.15773105863909,
  79.84985911070852,
  81.70679286326198,
  75.39230729988306,
  76.55063683601855,
  79.37883848986453,
  76.24303246854758,
  80.0812087820857,
  69.57010852370435,
  71.30918594402827,
  60.440052945046304,
  62.96824596572466,
  65.0,
  67.62396025078685,
  67.54257916307313,
  71.17583859709698,
  63.97655820689325,
  60.83584469702052,
  62.16912416947821,
  64.40496875241847,
  64.00781202322104,
  62.26556030423239,
  68.0,
  64.4980619863884,
  60.207972893961475,
  62.289646009589745,
  67.11929677819934,
  66.03029607687671,
  68.26419266350405,
  69.46221994724903,
  70.0,
  70.45565981523414,
  72.20110802473879,
  71.8679344353238,
  72.67048919609665,
  52.0,
  49.73932046178355,
  59.033888572581766,
  68.54195795277518,
  65.19202405202648,
  60.728905802755904,
  56.36488268416781,
  56.61271941887264,
  64.20280367709809,
  63.245553203367585,
  55.54277630799526,
  75.96051605933177,
  80.06247560499239],
 [79.92496481075234,
  53.03772242470448,
  53.0,
  47.042533945356304,
  46.04345773288535,
  47.16990566028302,
  46.389654018972806,
  44.40720662234904,
  32.55764119219941,
  43.18564576337837,
  44.10215414239989,
  43.04648650006177,
  31.144823004794873,
  38.05259518088089,
  39.0,
  42.0,
  5.830951894845301,
  7.0710678118654755,
  63.28506932918696,
  3.605551275463989,
  7.280109889280518,
  10.198039027185569,
  0.0,
  6.0,
  1.0,
  2.23606797749979,
  8.06225774829855,
  4.0,
  5.656854249492381,
  16.97056274847714,
  6.0,
  6.324555320336759,
  8.48528137423857,
  7.280109889280518,
  9.219544457292887,
  8.602325267042627,
  9.848857801796104,
  38.8329756778952,
  41.773197148410844,
  43.73785545725808,
  52.46903848937962,
  41.48493702538308,
  44.384682042344295,
  50.21951811795888,
  41.43669871020132,
  50.96076922496363,
  42.15447781671598,
  44.01136216933077,
  47.75981574503821,
  42.95346318982906,
  47.853944456021594,
  51.478150704935004,
  77.1621668954417,
  80.15609770940699,
  78.31347265956222,
  82.29823813423954,
  86.28441342444185,
  77.52418977325722,
  83.48652585896721,
  74.96665925596525,
  85.84287972802403,
  80.4114419718985,
  82.37718130647589,
  87.29833904490967,
  84.52810183601665,
  78.85429601486528,
  88.52683209061533,
  89.89438247187641,
  81.93289937503738,
  92.59049627256569,
  94.36630754670864,
  89.69392398596462,
  91.76055797563569,
  86.16263691415206,
  89.88882021697692,
  91.76055797563569,
  85.44003745317531,
  86.64871608973787,
  89.49301648732151,
  86.40023148117139,
  90.24965373894794,
  75.15317691222374,
  77.41446893184762,
  65.79513659838392,
  68.8839603971781,
  71.19691004531025,
  73.92563831310488,
  74.09453421137081,
  77.52418977325722,
  70.49113419430843,
  67.4166151627327,
  68.79680225126747,
  71.02112361825881,
  71.02816342831905,
  69.40461079784254,
  74.88658090739622,
  71.61005515987263,
  67.62396025078685,
  70.76722405181653,
  74.65252842335616,
  73.7563556583431,
  75.6042326857432,
  76.60939890117922,
  77.89736837660179,
  78.10249675906654,
  79.40403012442127,
  79.20227269466452,
  80.2807573457052,
  58.309518948453004,
  57.0087712549569,
  67.17886572427373,
  74.79304780526061,
  71.11961754677819,
  66.2117814289874,
  62.96824596572466,
  64.4437739428721,
  72.18032973047436,
  68.87670143089025,
  61.91122676865643,
  81.8413098624405,
  86.82165628459296],
 [76.94153624668537,
  53.600373133029585,
  53.33854141237835,
  47.16990566028302,
  46.17358552246078,
  47.042533945356304,
  46.0,
  44.0,
  32.0,
  43.04648650006177,
  44.10215414239989,
  43.18564576337837,
  31.144823004794873,
  38.2099463490856,
  39.45883931389772,
  42.42640687119285,
  5.830951894845301,
  5.0990195135927845,
  60.37383539249432,
  3.605551275463989,
  2.23606797749979,
  4.47213595499958,
  6.0,
  0.0,
  6.082762530298219,
  4.123105625617661,
  2.23606797749979,
  7.211102550927978,
  4.47213595499958,
  13.416407864998739,
  8.48528137423857,
  7.211102550927978,
  6.0,
  10.63014581273465,
  9.848857801796104,
  7.0710678118654755,
  9.219544457292887,
  32.984845004941285,
  35.90264614203248,
  37.8549864614954,
  46.52956049652737,
  35.73513677041127,
  38.600518131237564,
  44.384682042344295,
  35.84689665786984,
  45.221676218380054,
  36.6742416417845,
  38.48376280978771,
  42.15447781671598,
  37.589892258425,
  42.5440947723653,
  46.06517122512408,
  71.17583859709698,
  74.16872656315464,
  72.3394774656273,
  76.32168761236873,
  80.30566605165541,
  71.56814934033156,
  77.52418977325722,
  69.05070600652827,
  79.90619500389191,
  74.52516353554684,
  76.48529270389177,
  81.39410298049853,
  78.64477096412705,
  73.00684899377592,
  82.71033792700887,
  84.59905436823747,
  76.32168761236873,
  87.32124598286491,
  89.06739021662193,
  84.17244204607586,
  86.16263691415206,
  80.62257748298549,
  84.30895563343196,
  86.16263691415206,
  79.84985911070852,
  80.99382692526635,
  83.81527307120105,
  80.65358020571685,
  84.48076704197234,
  72.80109889280519,
  74.73285756613352,
  63.56886030125127,
  66.30987860040162,
  68.4470598345904,
  71.11258679024411,
  71.11961754677819,
  74.68600939935136,
  67.53517601961218,
  64.41273166075166,
  65.76473218982953,
  68.0,
  67.7421582177598,
  66.03786792439622,
  71.69379331573968,
  68.26419266350405,
  64.07027391856539,
  66.48308055437865,
  71.02816342831905,
  70.0,
  72.11102550927978,
  73.24616030891995,
  74.02702209328699,
  74.4043009509531,
  76.00657866263946,
  75.71657678474378,
  76.60939890117922,
  55.46169849544819,
  53.53503525729669,
  63.12685640834652,
  72.01388754955533,
  68.54195795277518,
  63.90618123468183,
  59.941638282582836,
  60.60528029800704,
  68.24954212300622,
  66.48308055437865,
  59.033888572581766,
  79.30952023559341,
  83.72574275573791],
 [80.77747210701756,
  54.037024344425184,
  54.0,
  48.041648597857254,
  47.042533945356304,
  48.16637831516918,
  47.38143096192854,
  45.39823785126467,
  33.54101966249684,
  44.181444068749045,
  45.09988913511872,
  44.04543109109048,
  32.14031735997639,
  39.05124837953327,
  40.0,
  43.0,
  6.708203932499369,
  7.810249675906654,
  64.1404708432983,
  4.242640687119285,
  7.615773105863909,
  10.44030650891055,
  1.0,
  6.082762530298219,
  0.0,
  2.0,
  8.0,
  3.0,
  5.0,
  16.278820596099706,
  5.0,
  5.385164807134504,
  7.810249675906654,
  6.324555320336759,
  8.246211251235321,
  7.810249675906654,
  8.94427190999916,
  38.63935817272331,
  41.593268686170845,
  43.56604182158393,
  52.3450093132096,
  41.23105625617661,
  44.14748010928823,
  50.00999900019995,
  41.10960958218893,
  50.695167422546305,
  41.78516483155236,
  43.657759905886145,
  47.43416490252569,
  42.5440947723653,
  47.4236228055175,
  51.07837115648854,
  77.10382610480494,
  80.09993757800315,
  78.23042886243178,
  82.21921916437786,
  86.20904824900923,
  77.41446893184762,
  83.38465086573187,
  74.81310045707235,
  85.70880934886448,
  80.23091673413684,
  82.20097323024831,
  87.13208364316786,
  84.34453153583817,
  78.64477096412705,
  88.29496021857646,
  90.35485598461214,
  82.28000972289685,
  93.05912099305473,
  94.82615672903758,
  90.07774419910837,
  92.11405973031478,
  86.53900854527974,
  90.24965373894794,
  92.11405973031478,
  85.7962703152066,
  86.97700845625813,
  89.81091247727082,
  86.68333173107735,
  90.52071586106685,
  76.05918747922567,
  78.29431652425353,
  66.70832032063167,
  69.77105417004964,
  72.069410986909,
  74.79304780526061,
  74.94664769020693,
  78.39005038906404,
  71.34423592694787,
  68.26419266350405,
  69.6419413859206,
  71.8679344353238,
  71.84705978674423,
  70.21395872616783,
  75.71657678474378,
  72.42237223399962,
  68.41052550594829,
  71.449282711585,
  75.43208866258443,
  74.51845409024533,
  76.40026177965623,
  77.42092740338364,
  78.64477096412705,
  78.87331614684399,
  80.2122185206219,
  80.0,
  81.05553651663777,
  59.16924876994806,
  57.8013840664737,
  67.89698078707183,
  75.66372975210778,
  72.00694410957877,
  67.11929677819934,
  63.812224534175265,
  65.19202405202648,
  72.91776189653656,
  69.77822009767804,
  62.76941930590086,
  82.73451516749222,
  87.66413177577246],
 [79.75587752636166,
  54.147945482723536,
  54.037024344425184,
  48.0,
  47.0,
  48.041648597857254,
  47.16990566028302,
  45.17742799230607,
  33.24154027718932,
  44.04543109109048,
  45.0111097397076,
  44.0,
  32.01562118716424,
  39.0,
  40.049968789001575,
  43.04648650006177,
  6.082762530298219,
  6.708203932499369,
  63.13477647065839,
  3.1622776601683795,
  5.830951894845301,
  8.54400374531753,
  2.23606797749979,
  4.123105625617661,
  2.0,
  0.0,
  6.0,
  3.605551275463989,
  3.605551275463989,
  14.866068747318506,
  5.385164807134504,
  5.0,
  6.4031242374328485,
  7.211102550927978,
  8.0,
  6.708203932499369,
  8.246211251235321,
  36.6742416417845,
  39.6232255123179,
  41.593268686170845,
  50.35871324805669,
  39.293765408777,
  42.20189569201838,
  48.05205510693585,
  39.21734310225516,
  48.76474136094644,
  39.92492955535426,
  41.78516483155236,
  45.5411901469428,
  40.718546143004666,
  45.617978911828175,
  49.24428900898052,
  75.1065909225016,
  78.10249675906654,
  76.23647421018367,
  80.22468448052632,
  84.2140130857092,
  75.4254598925323,
  81.39410298049853,
  72.83543093852057,
  83.72574275573791,
  78.26237921249263,
  80.23091673413684,
  85.15867542417507,
  82.37718130647589,
  76.68767828015137,
  86.34813257969161,
  88.58893836140041,
  80.4114419718985,
  91.30169768410661,
  93.05912099305473,
  88.23831367382311,
  90.24965373894794,
  84.69356528095862,
  88.3911760301898,
  90.24965373894794,
  83.93449827097318,
  85.09406559801923,
  87.92041856133307,
  84.77027781009096,
  88.60022573334675,
  75.23961722390672,
  77.3692445355388,
  65.92419889539804,
  68.87670143089025,
  71.11961754677819,
  73.824115301167,
  73.92563831310488,
  77.41446893184762,
  70.32780389006896,
  67.23094525588644,
  68.60029154456998,
  70.83078426785913,
  70.7248188403477,
  69.06518659932803,
  74.62573282722254,
  71.28113354878694,
  67.20119046564577,
  70.00714249274856,
  74.2024258363566,
  73.24616030891995,
  75.21303078589507,
  76.27581530209953,
  77.33692520394123,
  77.62087348130012,
  79.05694150420949,
  78.81624198095213,
  79.81227975693966,
  58.180752831155424,
  56.61271941887264,
  66.52818951391959,
  74.70609078247904,
  71.11258679024411,
  66.30987860040162,
  62.76941930590086,
  63.89053137985315,
  71.58910531638176,
  68.94200461257273,
  61.773780845922005,
  81.85963596303125,
  86.6083136886985],
 [76.92203845452875,
  54.91812087098393,
  54.589376255824725,
  48.373546489791295,
  47.38143096192854,
  48.16637831516918,
  47.042533945356304,
  45.044422518220834,
  33.06055050963308,
  44.181444068749045,
  45.27692569068709,
  44.40720662234904,
  32.38826948140329,
  39.45883931389772,
  40.792156108742276,
  43.73785545725808,
  7.810249675906654,
  6.708203932499369,
  60.41522986797286,
  5.830951894845301,
  3.1622776601683795,
  3.605551275463989,
  8.06225774829855,
  2.23606797749979,
  8.0,
  6.0,
  0.0,
  8.54400374531753,
  5.0,
  11.704699910719626,
  9.433981132056603,
  7.810249675906654,
  5.385164807134504,
  11.661903789690601,
  10.0,
  6.708203932499369,
  8.94427190999916,
  30.805843601498726,
  33.734255586866,
  35.6931365951495,
  44.40720662234904,
  33.52610922848042,
  36.40054944640259,
  42.20189569201838,
  33.61547262794322,
  43.01162633521314,
  34.438350715445125,
  36.24913792078372,
  39.92492955535426,
  35.35533905932738,
  40.311288741492746,
  43.829214001622255,
  69.1158447825099,
  72.11102550927978,
  70.25667228100119,
  74.24284477308234,
  78.23042886243178,
  69.46221994724903,
  75.4254598925323,
  66.91038783328041,
  77.78174593052023,
  72.3671196055225,
  74.33034373659252,
  79.2464510246358,
  76.48529270389177,
  70.83078426785913,
  80.52328855678958,
  83.35466393669883,
  74.84650960465692,
  86.09297300012354,
  87.8179936004006,
  82.76472678623425,
  84.69356528095862,
  79.20227269466452,
  82.85529554590944,
  84.69356528095862,
  78.39005038906404,
  79.47955712005447,
  82.28000972289685,
  79.05694150420949,
  82.86132994346639,
  73.05477397131553,
  74.84650960465692,
  63.89053137985315,
  66.48308055437865,
  68.54195795277518,
  71.17583859709698,
  71.11258679024411,
  74.73285756613352,
  67.54257916307313,
  64.40496875241847,
  65.7419196555744,
  67.97793759742936,
  67.60177512462228,
  65.86349520030045,
  71.58910531638176,
  68.09552114493287,
  63.812224534175265,
  65.85590330410783,
  70.7248188403477,
  69.63476143421474,
  71.8679344353238,
  73.06161783043132,
  73.60027173862879,
  74.06078584514209,
  75.8023746329889,
  75.47184905645283,
  76.27581530209953,
  55.54277630799526,
  53.33854141237835,
  62.625873247404705,
  72.09022125087424,
  68.70953354520753,
  64.19501538281614,
  59.93329625508679,
  60.21627686929839,
  67.80117993073571,
  66.73080248281148,
  59.09314681077663,
  79.47955712005447,
  83.64807230295268],
 [83.35466393669883,
  57.0350769263968,
  57.0,
  51.03920062069938,
  50.039984012787215,
  51.15662224971465,
  50.35871324805669,
  48.373546489791295,
  36.49657518178932,
  47.16990566028302,
  48.093658625644196,
  47.042533945356304,
  35.12833614050059,
  42.04759208325728,
  43.0,
  46.0,
  9.486832980505138,
  10.295630140987,
  66.73080248281148,
  6.708203932499369,
  9.219544457292887,
  11.661903789690601,
  4.0,
  7.211102550927978,
  3.0,
  3.605551275463989,
  8.54400374531753,
  0.0,
  4.0,
  14.422205101855956,
  2.0,
  2.8284271247461903,
  6.324555320336759,
  3.605551275463989,
  5.385164807134504,
  5.830951894845301,
  6.4031242374328485,
  38.2099463490856,
  41.19465984809196,
  43.18564576337837,
  52.08646657242167,
  40.607881008493905,
  43.56604182158393,
  49.49747468305833,
  40.26164427839479,
  50.00999900019995,
  40.80441152620633,
  42.720018726587654,
  46.57252408878007,
  41.43669871020132,
  46.238512086787566,
  49.9799959983992,
  77.00649323271382,
  80.00624975587844,
  78.05767098754612,
  82.05485969764375,
  86.05230967266364,
  77.1621668954417,
  83.15046602394962,
  74.43117626371358,
  85.37564055396598,
  79.76214641043707,
  81.74350127074322,
  86.70063436907483,
  83.86298349093002,
  78.08969202141856,
  87.66413177577246,
  91.7877987534291,
  83.38465086573187,
  94.51454914456292,
  96.25487000666512,
  91.28526715741155,
  93.23089616645332,
  87.72684879784524,
  91.3892772703669,
  93.23089616645332,
  86.92525524840292,
  88.02272433866155,
  90.82400563727631,
  87.5956619930462,
  91.39474820797966,
  78.79086241436883,
  80.95060222135473,
  69.46221994724903,
  72.44998274671983,
  74.70609078247904,
  77.41446893184762,
  77.52418977325722,
  81.00617260431454,
  73.92563831310488,
  70.83078426785913,
  72.20110802473879,
  74.43117626371358,
  74.33034373659252,
  72.67048919609665,
  78.23042886243178,
  74.88658090739622,
  70.80254232723568,
  73.53910524340094,
  77.80102827083971,
  76.83749084919418,
  78.81624198095213,
  79.88116173416608,
  80.91971329657564,
  81.21576201698781,
  82.66196223173026,
  82.41965784932621,
  83.40863264674707,
  61.773780845922005,
  60.21627686929839,
  70.09279563550022,
  78.29431652425353,
  74.68600939935136,
  69.85699678629192,
  66.37017402418047,
  67.47592163134935,
  75.16648189186454,
  72.49827584156743,
  65.36818798161687,
  85.42833253669417,
  90.21086409075129],
 [81.41252974819048,
  57.3149195236284,
  57.14017850864661,
  51.03920062069938,
  50.039984012787215,
  51.0,
  50.039984012787215,
  48.041648597857254,
  36.05551275463989,
  47.0,
  48.010415536631214,
  47.042533945356304,
  35.014282800023196,
  42.04759208325728,
  43.18564576337837,
  46.17358552246078,
  9.055385138137417,
  9.055385138137417,
  64.8459713474939,
  6.082762530298219,
  6.708203932499369,
  8.48528137423857,
  5.656854249492381,
  4.47213595499958,
  5.0,
  3.605551275463989,
  5.0,
  4.0,
  0.0,
  11.313708498984761,
  4.47213595499958,
  2.8284271247461903,
  2.8284271247461903,
  6.708203932499369,
  5.385164807134504,
  3.1622776601683795,
  5.0,
  34.23448553724738,
  37.21558813185679,
  39.20459156782532,
  48.093658625644196,
  36.6742416417845,
  39.6232255123179,
  45.5411901469428,
  36.40054944640259,
  46.09772228646444,
  37.0,
  38.897300677553446,
  42.720018726587654,
  37.69615364994153,
  42.5440947723653,
  46.238512086787566,
  73.00684899377592,
  76.00657866263946,
  74.06078584514209,
  78.05767098754612,
  82.05485969764375,
  73.17103251970687,
  79.1580697086532,
  70.45565981523414,
  81.39410298049853,
  75.8023746329889,
  77.78174593052023,
  82.73451516749222,
  79.90619500389191,
  74.14849964766651,
  83.74365647617735,
  88.32326986700618,
  79.7057087039567,
  91.0659101969557,
  92.78469701410896,
  87.66413177577246,
  89.55445270895244,
  84.09518416651456,
  87.72684879784524,
  89.55445270895244,
  83.2586331859946,
  84.30895563343196,
  87.09190547921202,
  83.81527307120105,
  87.5956619930462,
  77.25283166331187,
  79.20227269466452,
  68.00735254367721,
  70.77428911688199,
  72.91776189653656,
  75.58438992278762,
  75.59100475585703,
  79.1580697086532,
  72.00694410957877,
  68.8839603971781,
  70.2353187506115,
  72.47068372797375,
  72.20110802473879,
  70.49113419430843,
  76.15773105863909,
  72.71863585079137,
  68.5054742338158,
  70.76722405181653,
  75.45197147855052,
  74.4043009509531,
  76.55063683601855,
  77.69813382572325,
  78.40918313565064,
  78.81624198095213,
  80.45495634204272,
  80.15609770940699,
  81.02468759581859,
  59.93329625508679,
  57.982756057296896,
  67.47592163134935,
  76.48529270389177,
  73.00684899377592,
  68.35202996254024,
  64.41273166075166,
  65.0,
  72.62231062146122,
  70.9365914038728,
  63.50590523722971,
  83.773504164503,
  88.19297024139736],
 [85.44003745317531,
  66.49060083951716,
  66.09841147864296,
  59.841457201508724,
  58.855755878248644,
  59.53990258641679,
  58.309518948453004,
  56.32051136131489,
  44.40720662234904,
  55.57877292636101,
  56.71860364994893,
  55.90169943749474,
  43.93176527297759,
  50.99019513592785,
  52.392747589718944,
  55.31726674375732,
  19.235384061671343,
  18.384776310850235,
  69.26037828369117,
  16.64331697709324,
  14.866068747318506,
  14.142135623730951,
  16.97056274847714,
  13.416407864998739,
  16.278820596099706,
  14.866068747318506,
  11.704699910719626,
  14.422205101855956,
  11.313708498984761,
  0.0,
  13.416407864998739,
  11.661903789690601,
  8.48528137423857,
  14.866068747318506,
  10.44030650891055,
  8.602325267042627,
  8.54400374531753,
  26.30589287593181,
  29.274562336608895,
  31.25699921617557,
  40.311288741492746,
  28.0178514522438,
  31.016124838541646,
  37.013511046643494,
  27.073972741361768,
  37.05401462729781,
  27.294688127912362,
  29.274562336608895,
  33.24154027718932,
  27.65863337187866,
  32.28002478313795,
  36.138621999185304,
  65.37583651472461,
  68.3593446428504,
  66.18912297349165,
  70.178344238091,
  74.16872656315464,
  65.06919393998976,
  71.06335201775947,
  62.0,
  73.0,
  67.06713054842886,
  69.06518659932803,
  74.06078584514209,
  71.11258679024411,
  65.19202405202648,
  74.54528824815154,
  86.21484790916237,
  76.32168761236873,
  89.02246907382428,
  90.60353193998566,
  84.59905436823747,
  86.09297300012354,
  80.99382692526635,
  84.38009243891595,
  86.09297300012354,
  79.92496481075234,
  80.62257748298549,
  83.24061508662703,
  79.60527620704548,
  83.19254774317228,
  82.46211251235322,
  83.81527307120105,
  73.54590403278758,
  75.69015788066504,
  77.4919350642375,
  80.00624975587844,
  79.71198153351854,
  83.48652585896721,
  76.21679604916491,
  73.05477397131553,
  74.33034373659252,
  76.55063683601855,
  75.76938695805846,
  73.92563831310488,
  79.84985911070852,
  76.15773105863909,
  71.58910531638176,
  72.11102550927978,
  78.26237921249263,
  76.94153624668537,
  79.62411694957753,
  81.02468759581859,
  80.62257748298549,
  81.41252974819048,
  83.6719785830358,
  83.19254774317228,
  83.64807230295268,
  64.5600495662759,
  61.465437442517235,
  69.46221994724903,
  80.95677859203639,
  77.88452991448301,
  73.7563556583431,
  68.62215385719105,
  67.53517601961218,
  74.79304780526061,
  76.15773105863909,
  68.00735254367721,
  88.60022573334675,
  91.92388155425118],
 [85.0881895447306,
  59.033888572581766,
  59.0,
  53.03772242470448,
  52.03844732503075,
  53.150729063673246,
  52.3450093132096,
  50.35871324805669,
  38.47076812334269,
  49.16299421312742,
  50.08991914547278,
  49.040799340956916,
  37.12142238654117,
  44.04543109109048,
  45.0,
  48.0,
  11.40175425099138,
  12.083045973594572,
  68.47627326307996,
  8.54400374531753,
  10.63014581273465,
  12.806248474865697,
  6.0,
  8.48528137423857,
  5.0,
  5.385164807134504,
  9.433981132056603,
  2.0,
  4.47213595499958,
  13.416407864998739,
  0.0,
  2.0,
  6.0,
  2.23606797749979,
  3.605551275463989,
  5.0990195135927845,
  5.0,
  38.05259518088089,
  41.048751503547585,
  43.04648650006177,
  52.009614495783374,
  40.311288741492746,
  43.289721643826724,
  49.25444142409901,
  39.81205847478876,
  49.64876634922564,
  40.26164427839479,
  42.20189569201838,
  46.09772228646444,
  40.80441152620633,
  45.5411901469428,
  49.33558553417604,
  77.00649323271382,
  80.00624975587844,
  78.00640999302557,
  82.00609733428362,
  86.00581375697809,
  77.05841939723393,
  83.05419917138447,
  74.24284477308234,
  85.21150157109074,
  79.51100552753688,
  81.49846624323675,
  86.46964785403026,
  83.60023923410746,
  77.78174593052023,
  87.29833904490967,
  92.78469701410896,
  84.17244204607586,
  95.524865872714,
  97.24710792614863,
  92.13576938409967,
  94.02127418834527,
  88.56635930193812,
  92.19544457292888,
  94.02127418834527,
  87.72684879784524,
  88.76936408468859,
  91.54780172128658,
  88.25531145489205,
  92.02716990106781,
  80.62257748298549,
  82.73451516749222,
  71.30918594402827,
  74.24957912338628,
  76.47875521999558,
  79.17701686727027,
  79.25906888173743,
  82.76472678623425,
  75.66372975210778,
  72.56031973468694,
  73.92563831310488,
  76.15773105863909,
  76.00657866263946,
  74.33034373659252,
  79.92496481075234,
  76.55063683601855,
  72.42237223399962,
  74.96665925596525,
  79.40403012442127,
  78.40918313565064,
  80.44874144447506,
  81.54140052758476,
  82.46211251235322,
  82.80096617793781,
  84.31488599292535,
  84.0535543567314,
  85.0,
  63.52952069707436,
  61.85466837676846,
  71.58910531638176,
  80.06247560499239,
  76.48529270389177,
  71.69379331573968,
  68.09552114493287,
  69.02897942168927,
  76.69419795525604,
  74.32361670424818,
  67.11929677819934,
  87.23531395025755,
  91.92388155425118],
 [84.11896337925236,
  59.135437767890075,
  59.033888572581766,
  53.0,
  52.0,
  53.03772242470448,
  52.15361924162119,
  50.15974481593781,
  38.2099463490856,
  49.040799340956916,
  50.00999900019995,
  49.0,
  37.013511046643494,
  44.0,
  45.044422518220834,
  48.041648597857254,
  11.045361017187261,
  11.40175425099138,
  67.53517601961218,
  8.06225774829855,
  9.433981132056603,
  11.313708498984761,
  6.324555320336759,
  7.211102550927978,
  5.385164807134504,
  5.0,
  7.810249675906654,
  2.8284271247461903,
  2.8284271247461903,
  11.661903789690601,
  2.0,
  0.0,
  4.0,
  4.123105625617661,
  3.0,
  3.1622776601683795,
  3.605551275463989,
  36.05551275463989,
  39.05124837953327,
  41.048751503547585,
  50.00999900019995,
  38.3275357934736,
  41.30375285612676,
  47.265209192385896,
  37.8549864614954,
  47.67598976424087,
  38.3275357934736,
  40.26164427839479,
  44.14748010928823,
  38.897300677553446,
  43.657759905886145,
  47.43416490252569,
  75.0066663703967,
  78.00640999302557,
  76.00657866263946,
  80.00624975587844,
  84.00595217006948,
  75.05997601918082,
  81.05553651663777,
  72.24956747275377,
  83.21658488546619,
  77.52418977325722,
  79.51100552753688,
  84.48076704197234,
  81.61494961096282,
  75.8023746329889,
  85.32877591996736,
  91.0659101969557,
  82.34682750416071,
  93.81364506296512,
  95.524865872714,
  90.33825324855468,
  92.19544457292888,
  86.76404785393545,
  90.37698822156003,
  92.19544457292888,
  85.90692637965812,
  86.92525524840292,
  89.69392398596462,
  86.37708029332781,
  90.13878188659973,
  79.84985911070852,
  81.85963596303125,
  70.57619995437555,
  73.40980860893181,
  75.58438992278762,
  78.26237921249263,
  78.29431652425353,
  81.8413098624405,
  74.70609078247904,
  71.58910531638176,
  72.94518489934754,
  75.17978451685,
  74.94664769020693,
  73.24616030891995,
  78.89233169326408,
  75.47184905645283,
  71.28113354878694,
  73.59347797189639,
  78.2368199762746,
  77.20103626247513,
  79.32212806020776,
  80.45495634204272,
  81.21576201698781,
  81.60882305241266,
  83.21658488546619,
  82.92767933567175,
  83.81527307120105,
  62.609903369994115,
  60.74537019394976,
  70.2922470831599,
  79.1580697086532,
  75.64390259630977,
  70.9365914038728,
  67.11929677819934,
  67.80117993073571,
  75.43208866258443,
  73.53910524340094,
  66.18912297349165,
  86.4060183089118,
  90.91754506144565],
 [82.29216244576394,
  59.53990258641679,
  59.30430001273095,
  53.150729063673246,
  52.15361924162119,
  53.03772242470448,
  52.0,
  50.0,
  38.0,
  49.040799340956916,
  50.08991914547278,
  49.16299421312742,
  37.12142238654117,
  44.181444068749045,
  45.39823785126467,
  48.373546489791295,
  11.40175425099138,
  11.045361017187261,
  65.79513659838392,
  8.54400374531753,
  8.06225774829855,
  8.94427190999916,
  8.48528137423857,
  6.0,
  7.810249675906654,
  6.4031242374328485,
  5.385164807134504,
  6.324555320336759,
  2.8284271247461903,
  8.48528137423857,
  6.0,
  4.0,
  0.0,
  8.06225774829855,
  5.0,
  1.4142135623730951,
  3.605551275463989,
  32.0624390837628,
  35.05709628591621,
  37.05401462729781,
  46.010868281309364,
  34.36568055487916,
  37.33630940518894,
  43.289721643826724,
  33.95585369269929,
  43.73785545725808,
  34.48187929913333,
  36.40054944640259,
  40.26164427839479,
  35.11409973215888,
  39.92492955535426,
  43.657759905886145,
  71.00704190430693,
  74.00675644831355,
  72.00694410957877,
  76.00657866263946,
  80.00624975587844,
  71.06335201775947,
  77.05841939723393,
  68.26419266350405,
  79.22752047110903,
  73.55270219373317,
  75.53806987208503,
  80.50465825031493,
  77.64663547121665,
  71.84705978674423,
  81.39410298049853,
  87.66413177577246,
  78.7210264160726,
  90.4267659490264,
  92.11405973031478,
  86.76981041814025,
  88.56635930193812,
  83.18653737234169,
  86.76404785393545,
  88.56635930193812,
  82.29216244576394,
  83.2586331859946,
  86.00581375697809,
  82.63776376451628,
  86.37708029332781,
  78.43468620451031,
  80.23091673413684,
  69.26037828369117,
  71.8679344353238,
  73.92563831310488,
  76.55716818169282,
  76.48529270389177,
  80.11242100947892,
  72.91776189653656,
  69.77822009767804,
  71.11258679024411,
  73.348483283569,
  72.94518489934754,
  71.19691004531025,
  76.94153624668537,
  73.43023900274328,
  69.1158447825099,
  70.9365914038728,
  76.00657866263946,
  74.88658090739622,
  77.17512552629896,
  78.39005038906404,
  78.81624198095213,
  79.32212806020776,
  81.12336284942828,
  80.77747210701756,
  81.54140052758476,
  60.92618484691127,
  58.66856057549052,
  67.80117993073571,
  77.47257579298626,
  74.09453421137081,
  69.57010852370435,
  65.30696746902278,
  65.45991139621256,
  73.00684899377592,
  72.11102550927978,
  64.47480127925948,
  84.86459803710851,
  89.00561780022652],
 [86.95401083331349,
  60.0,
  60.03332407921454,
  54.147945482723536,
  53.150729063673246,
  54.3323108288245,
  53.600373133029585,
  51.62363799656123,
  39.81205847478876,
  50.35871324805669,
  51.24451190127583,
  50.15974481593781,
  38.3275357934736,
  45.17742799230607,
  46.04345773288535,
  49.040799340956916,
  13.0,
  13.892443989449804,
  70.32780389006896,
  10.295630140987,
  12.727922061357855,
  15.0,
  7.280109889280518,
  10.63014581273465,
  6.324555320336759,
  7.211102550927978,
  11.661903789690601,
  3.605551275463989,
  6.708203932499369,
  14.866068747318506,
  2.23606797749979,
  4.123105625617661,
  8.06225774829855,
  0.0,
  4.47213595499958,
  7.0,
  6.324555320336759,
  40.01249804748511,
  43.01162633521314,
  45.0111097397076,
  54.0,
  42.190046219457976,
  45.17742799230607,
  51.15662224971465,
  41.593268686170845,
  51.478150704935004,
  41.97618372363071,
  43.93176527297759,
  47.853944456021594,
  42.44997055358225,
  47.12748667179272,
  50.96076922496363,
  79.02531240052139,
  82.02438661763951,
  80.0,
  84.0,
  88.0,
  79.02531240052139,
  85.0235261559999,
  76.16429609731846,
  87.14355971613737,
  81.39410298049853,
  83.38465086573187,
  88.36288813749809,
  85.47514258543241,
  79.63039620647382,
  89.10667763978185,
  95.01578816175763,
  86.4060183089118,
  97.75479527879949,
  99.4786409235671,
  94.37160589923221,
  96.25487000666512,
  90.80198235721508,
  94.42986815621421,
  96.25487000666512,
  89.96110270555825,
  91.0,
  93.77632963600144,
  90.47651629014017,
  94.2443632266673,
  82.34682750416071,
  84.53401682163222,
  73.00684899377592,
  76.02631123499285,
  78.29431652425353,
  81.00617260431454,
  81.12336284942828,
  84.59905436823747,
  77.52418977325722,
  74.43117626371358,
  75.8023746329889,
  78.03204469959762,
  77.93587107359485,
  76.27581530209953,
  81.83520025025906,
  78.49203781276162,
  74.4043009509531,
  77.07788269017254,
  81.40024569987489,
  80.43009387039157,
  82.41965784932621,
  83.48652585896721,
  84.50443775329198,
  84.81155581640984,
  86.2670273047588,
  86.02325267042627,
  87.00574693662483,
  65.36818798161687,
  63.8200595424354,
  73.66138744281159,
  81.88406438373708,
  78.26237921249263,
  73.40980860893181,
  69.9714227381436,
  71.06335201775947,
  78.74642849044012,
  76.05918747922567,
  68.96375859826666,
  89.0,
  93.81364506296512],
 [86.76981041814025,
  62.12889826803627,
  62.03224967708329,
  56.0,
  55.0,
  56.0357029044876,
  55.14526271584895,
  53.150729063673246,
  41.19465984809196,
  52.03844732503075,
  53.009433122794285,
  52.0,
  40.01249804748511,
  47.0,
  48.041648597857254,
  51.03920062069938,
  14.035668847618199,
  14.317821063276353,
  70.21395872616783,
  11.045361017187261,
  12.083045973594572,
  13.601470508735444,
  9.219544457292887,
  9.848857801796104,
  8.246211251235321,
  8.0,
  10.0,
  5.385164807134504,
  5.385164807134504,
  10.44030650891055,
  3.605551275463989,
  3.0,
  5.0,
  4.47213595499958,
  0.0,
  3.605551275463989,
  2.0,
  36.013886210738214,
  39.01281840626232,
  41.012193308819754,
  50.039984012787215,
  38.05259518088089,
  41.048751503547585,
  47.042533945356304,
  37.33630940518894,
  47.265209192385896,
  37.656340767525464,
  39.6232255123179,
  43.56604182158393,
  38.07886552931954,
  42.720018726587654,
  46.57252408878007,
  75.1065909225016,
  78.10249675906654,
  76.02631123499285,
  80.02499609497022,
  84.02380615040002,
  75.0,
  81.0,
  72.06247289678588,
  83.05419917138447,
  77.23341245859851,
  79.22752047110903,
  84.2140130857092,
  81.30190649671138,
  75.4254598925323,
  84.8528137423857,
  92.64987857520376,
  83.6301381082203,
  95.4148835350125,
  97.09788875150684,
  91.706052144883,
  93.47192091746055,
  88.11923740024082,
  91.67878707749138,
  93.47192091746055,
  87.20665112249179,
  88.14193099768123,
  90.87353850269065,
  87.46427842267951,
  91.18113840043894,
  82.63776376451628,
  84.581321815162,
  73.38937252763509,
  76.15773105863909,
  78.29431652425353,
  80.95677859203639,
  80.95060222135473,
  84.52810183601665,
  77.3692445355388,
  74.24284477308234,
  75.59100475585703,
  77.82673062643708,
  77.52418977325722,
  75.8023746329889,
  81.49233092751734,
  78.03204469959762,
  73.78346698278686,
  75.82216034906945,
  80.7093550959243,
  79.63039620647382,
  81.83520025025906,
  83.00602387778854,
  83.60023923410746,
  84.0535543567314,
  85.75546629807339,
  85.44003745317531,
  86.2670273047588,
  65.30696746902278,
  63.28506932918696,
  72.62231062146122,
  81.85963596303125,
  78.39005038906404,
  73.7360156233031,
  69.77105417004964,
  70.21395872616783,
  77.80102827083971,
  76.32168761236873,
  68.87670143089025,
  89.15716460273958,
  93.53608929178085],
 [83.6301381082203,
  60.40695324215582,
  60.207972893961475,
  54.08326913195984,
  53.08483775994799,
  54.00925846556311,
  53.009433122794285,
  51.0098029794274,
  39.01281840626232,
  50.00999900019995,
  51.03920062069938,
  50.08991914547278,
  38.05259518088089,
  45.09988913511872,
  46.2709412050371,
  49.25444142409901,
  12.165525060596439,
  12.0,
  67.11929677819934,
  9.219544457292887,
  9.219544457292887,
  10.295630140987,
  8.602325267042627,
  7.0710678118654755,
  7.810249675906654,
  6.708203932499369,
  6.708203932499369,
  5.830951894845301,
  3.1622776601683795,
  8.602325267042627,
  5.0990195135927845,
  3.1622776601683795,
  1.4142135623730951,
  7.0,
  3.605551275463989,
  0.0,
  2.23606797749979,
  33.015148038438355,
  36.013886210738214,
  38.01315561749642,
  47.0,
  35.22782990761707,
  38.2099463490856,
  44.181444068749045,
  34.713109915419565,
  44.553338819890925,
  35.17101079013795,
  37.107950630558946,
  41.0,
  35.73513677041127,
  40.496913462633174,
  44.27188724235731,
  72.02777242147643,
  75.02666192761077,
  73.0,
  77.0,
  81.0,
  72.02777242147643,
  78.02563681252464,
  69.18092222571191,
  80.15609770940699,
  74.43117626371358,
  76.4198926981712,
  81.39410298049853,
  78.5175139698144,
  72.69112738154499,
  82.20097323024831,
  89.04493247793498,
  80.05623023850174,
  91.80958555619343,
  93.49331526906082,
  88.11923740024082,
  89.89994438263018,
  84.53401682163222,
  88.10221336606703,
  89.89994438263018,
  83.6301381082203,
  84.581321815162,
  87.32124598286491,
  83.93449827097318,
  87.66413177577246,
  79.71198153351854,
  81.54140052758476,
  70.51950084905593,
  73.16419889536138,
  75.23961722390672,
  77.87810988975015,
  77.82030583337487,
  81.43709228600933,
  74.24957912338628,
  71.11258679024411,
  72.44998274671983,
  74.68600939935136,
  74.30343195303969,
  72.56031973468694,
  78.29431652425353,
  74.79304780526061,
  70.49113419430843,
  72.34638899074369,
  77.38862965578342,
  76.27581530209953,
  78.54934754662193,
  79.75587752636166,
  80.2122185206219,
  80.7093550959243,
  82.49242389456137,
  82.15229759416349,
  82.92767933567175,
  62.24146527838174,
  60.03332407921454,
  69.20260110718382,
  78.79086241436883,
  75.39230729988306,
  70.83784299369935,
  66.64082832618455,
  66.85057965343307,
  74.4043009509531,
  73.38937252763509,
  65.79513659838392,
  86.16263691415206,
  90.35485598461214],
 [85.86617494683223,
  62.289646009589745,
  62.12889826803627,
  56.0357029044876,
  55.036351623268054,
  56.0,
  55.036351623268054,
  53.03772242470448,
  41.048751503547585,
  52.0,
  53.009433122794285,
  52.03844732503075,
  40.01249804748511,
  47.042533945356304,
  48.16637831516918,
  51.15662224971465,
  14.035668847618199,
  14.035668847618199,
  69.35416353759881,
  11.045361017187261,
  11.40175425099138,
  12.529964086141668,
  9.848857801796104,
  9.219544457292887,
  8.94427190999916,
  8.246211251235321,
  8.94427190999916,
  6.4031242374328485,
  5.0,
  8.54400374531753,
  5.0,
  3.605551275463989,
  3.605551275463989,
  6.324555320336759,
  2.0,
  2.23606797749979,
  0.0,
  34.0147027033899,
  37.013511046643494,
  39.01281840626232,
  48.041648597857254,
  36.05551275463989,
  39.05124837953327,
  45.044422518220834,
  35.35533905932738,
  45.27692569068709,
  35.6931365951495,
  37.656340767525464,
  41.593268686170845,
  36.138621999185304,
  40.80441152620633,
  44.64302857109943,
  73.10950690573696,
  76.10519036176179,
  74.02702209328699,
  78.02563681252464,
  82.02438661763951,
  73.0,
  79.0,
  70.06425622241343,
  81.05553651663777,
  75.23961722390672,
  77.23341245859851,
  82.21921916437786,
  79.30952023559341,
  73.43704787094863,
  82.87339742040264,
  90.97252332435328,
  81.8413098624405,
  93.74433316206373,
  95.4148835350125,
  89.94442728707543,
  91.67878707749138,
  86.35392289873113,
  89.89438247187641,
  91.67878707749138,
  85.42247947700886,
  86.33075929238663,
  89.05054744357274,
  85.61541917201598,
  89.31965069345043,
  81.93289937503738,
  83.773504164503,
  72.73238618387272,
  75.39230729988306,
  77.47257579298626,
  80.11242100947892,
  80.05623023850174,
  83.6719785830358,
  76.48529270389177,
  73.348483283569,
  74.68600939935136,
  76.92203845452875,
  76.53757247260981,
  74.79304780526061,
  80.52949770115296,
  77.02596964660685,
  72.71863585079137,
  74.51845409024533,
  79.61155694998057,
  78.49203781276162,
  80.77747210701756,
  81.98780397107853,
  82.41965784932621,
  82.92767933567175,
  84.72307831990054,
  84.38009243891595,
  85.14693182963201,
  64.47480127925948,
  62.26556030423239,
  71.40028011149536,
  81.02468759581859,
  77.62087348130012,
  73.05477397131553,
  68.87670143089025,
  69.06518659932803,
  76.60939890117922,
  75.61084578286372,
  68.0294054067798,
  88.3911760301898,
  92.59049627256569],
 [76.10519036176179,
  72.94518489934754,
  71.8679344353238,
  65.73431371817918,
  64.89992295835181,
  64.66065264130884,
  62.76941930590086,
  61.05735008989499,
  51.22499389946279,
  61.29437168288782,
  62.68173577685928,
  62.42595614005443,
  52.40229002629561,
  58.412327466040935,
  60.440052945046304,
  62.80127387243033,
  37.33630940518894,
  35.4682957019364,
  62.20128616033595,
  36.40054944640259,
  32.57299494980466,
  29.732137494637012,
  38.8329756778952,
  32.984845004941285,
  38.63935817272331,
  36.6742416417845,
  30.805843601498726,
  38.2099463490856,
  34.23448553724738,
  26.30589287593181,
  38.05259518088089,
  36.05551275463989,
  32.0624390837628,
  40.01249804748511,
  36.013886210738214,
  33.015148038438355,
  34.0147027033899,
  0.0,
  3.0,
  5.0,
  14.035668847618199,
  3.605551275463989,
  5.830951894845301,
  11.40175425099138,
  6.082762530298219,
  12.529964086141668,
  8.06225774829855,
  8.54400374531753,
  10.63014581273465,
  10.04987562112089,
  13.92838827718412,
  15.811388300841896,
  39.11521443121589,
  42.1070065428546,
  40.01249804748511,
  44.01136216933077,
  48.010415536631214,
  39.01281840626232,
  45.0111097397076,
  36.22154055254967,
  47.16990566028302,
  41.593268686170845,
  43.56604182158393,
  48.507731342539614,
  45.70557952810576,
  40.024992192379,
  49.72926703662542,
  64.53681120105021,
  53.075418038862395,
  67.35725647619564,
  68.6804193347711,
  61.554853586049575,
  62.48199740725323,
  58.0,
  60.959002616512684,
  62.48199740725323,
  56.639209034025185,
  56.85068161420758,
  59.20304046246274,
  55.21775076911409,
  58.52349955359813,
  76.23647421018367,
  76.00657866263946,
  68.8839603971781,
  69.26037828369117,
  70.06425622241343,
  72.00694410957877,
  71.00704190430693,
  75.0066663703967,
  68.00735254367721,
  65.03076195155643,
  66.007575322837,
  68.0,
  66.06814663663572,
  64.07027391856539,
  70.11419257183242,
  66.12110101926616,
  61.1310068623117,
  57.723478758647246,
  66.61080993352356,
  64.77653896280658,
  68.46897107449476,
  70.34912934784623,
  67.468511173732,
  69.05070600652827,
  72.56031973468694,
  71.7007670809734,
  71.19691004531025,
  58.54912467321779,
  53.08483775994799,
  56.71860364994893,
  73.00684899377592,
  71.17583859709698,
  68.73136110975832,
  61.1310068623117,
  56.22277118748239,
  61.98386886924694,
  70.45565981523414,
  61.29437168288782,
  81.00617260431454,
  81.49846624323675],
 [76.00657866263946,
  74.63243262818116,
  73.49829930005184,
  67.42403132415029,
  66.61080993352356,
  66.2872536767062,
  64.35060217278468,
  62.68173577685928,
  53.150729063673246,
  63.00793600809346,
  64.40496875241847,
  64.20280367709809,
  54.45181356024793,
  60.30754513325841,
  62.369864518050704,
  64.66065264130884,
  40.162171256046406,
  38.27531841800928,
  62.51399843235114,
  39.293765408777,
  35.4400902933387,
  32.57299494980466,
  41.773197148410844,
  35.90264614203248,
  41.593268686170845,
  39.6232255123179,
  33.734255586866,
  41.19465984809196,
  37.21558813185679,
  29.274562336608895,
  41.048751503547585,
  39.05124837953327,
  35.05709628591621,
  43.01162633521314,
  39.01281840626232,
  36.013886210738214,
  37.013511046643494,
  3.0,
  0.0,
  2.0,
  11.045361017187261,
  3.1622776601683795,
  3.605551275463989,
  8.54400374531753,
  6.324555320336759,
  10.0,
  8.246211251235321,
  8.0,
  8.94427190999916,
  10.198039027185569,
  13.152946437965905,
  14.317821063276353,
  36.124783736376884,
  39.11521443121589,
  37.013511046643494,
  41.012193308819754,
  45.0111097397076,
  36.013886210738214,
  42.01190307520001,
  33.24154027718932,
  44.181444068749045,
  38.63935817272331,
  40.607881008493905,
  45.5411901469428,
  42.7551166528639,
  37.107950630558946,
  46.84015371452148,
  62.625873247404705,
  50.91168824543142,
  65.43699259593154,
  66.70832032063167,
  59.39696961966999,
  60.207972893961475,
  55.86591089385369,
  58.728187440104094,
  60.207972893961475,
  54.45181356024793,
  54.56189146281496,
  56.85068161420758,
  52.80151512977634,
  56.0357029044876,
  76.53103945458993,
  76.10519036176179,
  69.42621983083913,
  69.58448102845921,
  70.25667228100119,
  72.11102550927978,
  71.02816342831905,
  75.02666192761077,
  68.11754546370561,
  65.19202405202648,
  66.12110101926616,
  68.06614430096654,
  66.0,
  64.0,
  70.00714249274856,
  66.007575322837,
  61.00819617067857,
  57.0701322935211,
  66.27216610312357,
  64.38167441127949,
  68.18357573492314,
  70.11419257183242,
  66.91038783328041,
  68.59300255857006,
  72.24956747275377,
  71.34423592694787,
  70.71067811865476,
  59.033888572581766,
  53.33854141237835,
  56.32051136131489,
  73.10950690573696,
  71.449282711585,
  69.2314957226839,
  61.40032573203501,
  56.0357029044876,
  61.5223536610881,
  70.8590149522275,
  61.66036003787198,
  81.09870529176159,
  81.2219182240853],
 [76.00657866263946,
  75.8023746329889,
  74.63243262818116,
  68.60029154456998,
  67.80117993073571,
  67.42403132415029,
  65.45991139621256,
  63.8200595424354,
  54.48853090330111,
  64.20280367709809,
  65.60487786742691,
  65.43699259593154,
  55.86591089385369,
  61.61980201201558,
  63.702433234531945,
  65.94694837519019,
  42.05948168962618,
  40.162171256046406,
  62.80127387243033,
  41.23105625617661,
  37.36308338453881,
  34.48187929913333,
  43.73785545725808,
  37.8549864614954,
  43.56604182158393,
  41.593268686170845,
  35.6931365951495,
  43.18564576337837,
  39.20459156782532,
  31.25699921617557,
  43.04648650006177,
  41.048751503547585,
  37.05401462729781,
  45.0111097397076,
  41.012193308819754,
  38.01315561749642,
  39.01281840626232,
  5.0,
  2.0,
  0.0,
  9.055385138137417,
  4.242640687119285,
  3.0,
  6.708203932499369,
  7.211102550927978,
  8.48528137423857,
  8.94427190999916,
  8.246211251235321,
  8.246211251235321,
  10.770329614269007,
  13.0,
  13.601470508735444,
  34.132096331752024,
  37.12142238654117,
  35.014282800023196,
  39.01281840626232,
  43.01162633521314,
  34.0147027033899,
  40.01249804748511,
  31.25699921617557,
  42.190046219457976,
  36.6742416417845,
  38.63935817272331,
  43.56604182158393,
  40.792156108742276,
  35.17101079013795,
  44.9221548904324,
  61.40032573203501,
  49.51767361255979,
  64.20280367709809,
  65.43699259593154,
  58.0,
  58.728187440104094,
  54.48853090330111,
  57.28001396647874,
  58.728187440104094,
  53.03772242470448,
  53.075418038862395,
  55.31726674375732,
  51.22499389946279,
  54.405882034941776,
  76.79192665899195,
  76.23647421018367,
  69.85699678629192,
  69.87131027825369,
  70.45565981523414,
  72.24956747275377,
  71.11258679024411,
  75.1065909225016,
  68.26419266350405,
  65.37583651472461,
  66.27216610312357,
  68.18357573492314,
  66.03029607687671,
  64.03124237432849,
  70.00714249274856,
  66.007575322837,
  61.00819617067857,
  56.71860364994893,
  66.12110101926616,
  64.19501538281614,
  68.06614430096654,
  70.02856560004639,
  66.61080993352356,
  68.3593446428504,
  72.11102550927978,
  71.17583859709698,
  70.45565981523414,
  59.43904440685432,
  53.600373133029585,
  56.142675390472796,
  73.24616030891995,
  71.7007670809734,
  69.63476143421474,
  61.66036003787198,
  56.0,
  61.29437168288782,
  71.19691004531025,
  61.98386886924694,
  81.2219182240853,
  81.09870529176159],
 [75.66372975210778,
  80.72174428244226,
  79.39773296511683,
  73.59347797189639,
  72.86288492778748,
  72.24956747275377,
  70.178344238091,
  68.6804193347711,
  60.30754513325841,
  69.31089380465383,
  70.7248188403477,
  70.71067811865476,
  62.00806399170998,
  67.26812023536856,
  69.42621983083913,
  71.449282711585,
  50.44799302251776,
  48.507731342539614,
  63.89053137985315,
  49.8196748283246,
  45.89117562233506,
  42.95346318982906,
  52.46903848937962,
  46.52956049652737,
  52.3450093132096,
  50.35871324805669,
  44.40720662234904,
  52.08646657242167,
  48.093658625644196,
  40.311288741492746,
  52.009614495783374,
  50.00999900019995,
  46.010868281309364,
  54.0,
  50.039984012787215,
  47.0,
  48.041648597857254,
  14.035668847618199,
  11.045361017187261,
  9.055385138137417,
  0.0,
  12.649110640673518,
  9.848857801796104,
  5.0,
  14.7648230602334,
  7.615773105863909,
  15.811388300841896,
  14.212670403551895,
  11.40175425099138,
  17.029386365926403,
  16.64331697709324,
  14.866068747318506,
  25.079872407968907,
  28.071337695236398,
  26.0,
  30.0,
  34.0,
  25.079872407968907,
  31.064449134018133,
  22.561028345356956,
  33.37663853655727,
  28.160255680657446,
  30.083217912982647,
  34.92849839314596,
  32.28002478313795,
  26.92582403567252,
  36.76955262170047,
  55.569775957799216,
  43.01162633521314,
  58.309518948453004,
  59.36328831862332,
  51.40038910358559,
  51.73973328110612,
  48.010415536631214,
  50.44799302251776,
  51.73973328110612,
  46.400431032480725,
  46.09772228646444,
  48.104053883222775,
  43.840620433565945,
  46.75467891024384,
  77.62087348130012,
  76.48529270389177,
  71.5122367151245,
  70.8801805866774,
  71.06335201775947,
  72.56721022610694,
  71.19691004531025,
  75.13321502504735,
  68.65857557508748,
  65.96969000988257,
  66.70832032063167,
  68.4470598345904,
  65.92419889539804,
  63.953107821277925,
  69.72087205421343,
  65.76473218982953,
  60.8276253029822,
  55.0,
  65.19202405202648,
  63.12685640834652,
  67.26812023536856,
  69.35416353759881,
  65.0,
  67.02984409947557,
  71.17583859709698,
  70.11419257183242,
  69.00724599634447,
  61.09828148156051,
  54.70831746635972,
  55.226805085936306,
  73.54590403278758,
  72.53275122315436,
  71.16881339463234,
  62.6418390534633,
  55.731499172371095,
  60.07495318350236,
  72.42237223399962,
  63.245553203367585,
  81.39410298049853,
  80.15609770940699],
 [79.02531240052139,
  76.55063683601855,
  75.47184905645283,
  69.33974329343886,
  68.5054742338158,
  68.26419266350405,
  66.37017402418047,
  64.66065264130884,
  54.817880294662984,
  64.89992295835181,
  66.2872536767062,
  66.03029607687671,
  55.97320787662612,
  62.00806399170998,
  64.03124237432849,
  66.40030120413611,
  40.311288741492746,
  38.48376280978771,
  65.37583651472461,
  39.21734310225516,
  35.4682957019364,
  32.69556544854363,
  41.48493702538308,
  35.73513677041127,
  41.23105625617661,
  39.293765408777,
  33.52610922848042,
  40.607881008493905,
  36.6742416417845,
  28.0178514522438,
  40.311288741492746,
  38.3275357934736,
  34.36568055487916,
  42.190046219457976,
  38.05259518088089,
  35.22782990761707,
  36.05551275463989,
  3.605551275463989,
  3.1622776601683795,
  4.242640687119285,
  12.649110640673518,
  0.0,
  3.0,
  9.0,
  3.1622776601683795,
  9.486832980505138,
  5.0990195135927845,
  5.0990195135927845,
  7.0710678118654755,
  7.0710678118654755,
  10.44030650891055,
  12.206555615733702,
  37.48332962798263,
  40.44749683231337,
  38.2099463490856,
  42.190046219457976,
  46.17358552246078,
  37.05401462729781,
  43.04648650006177,
  34.0147027033899,
  45.0111097397076,
  39.20459156782532,
  41.19465984809196,
  46.17358552246078,
  43.289721643826724,
  37.48332962798263,
  47.07440918375928,
  65.60487786742691,
  53.75872022286245,
  68.41052550594829,
  69.65629906907199,
  62.24146527838174,
  62.96824596572466,
  58.728187440104094,
  61.5223536610881,
  62.96824596572466,
  57.28001396647874,
  57.3149195236284,
  59.54829972383762,
  55.44366510251645,
  58.60034129593445,
  79.40403012442127,
  79.05694150420949,
  72.18032973047436,
  72.44308110509934,
  73.17103251970687,
  75.05997601918082,
  74.00675644831355,
  78.00640999302557,
  71.06335201775947,
  68.11754546370561,
  69.06518659932803,
  71.02816342831905,
  69.00724599634447,
  67.00746227100382,
  73.02739212103907,
  69.02897942168927,
  64.03124237432849,
  60.207972893961475,
  69.35416353759881,
  67.47592163134935,
  71.25307010929424,
  73.17103251970687,
  70.03570517957252,
  71.7007670809734,
  75.32595834106593,
  74.43117626371358,
  73.824115301167,
  61.814237842102365,
  56.22277118748239,
  59.413803110051795,
  76.05918747922567,
  74.33034373659252,
  72.00694410957877,
  64.28063471995279,
  59.07622195096772,
  64.6297145282261,
  73.68174807915459,
  64.4980619863884,
  84.0535543567314,
  84.29116205154607],
 [79.00632886041473,
  78.2368199762746,
  77.10382610480494,
  71.02816342831905,
  70.21395872616783,
  69.8927750200262,
  67.95586803212802,
  66.2872536767062,
  56.72741841473134,
  66.61080993352356,
  68.00735254367721,
  67.80117993073571,
  58.0,
  63.89053137985315,
  65.94694837519019,
  68.24954212300622,
  43.08131845707603,
  41.23105625617661,
  65.76473218982953,
  42.05948168962618,
  38.27531841800928,
  35.4682957019364,
  44.384682042344295,
  38.600518131237564,
  44.14748010928823,
  42.20189569201838,
  36.40054944640259,
  43.56604182158393,
  39.6232255123179,
  31.016124838541646,
  43.289721643826724,
  41.30375285612676,
  37.33630940518894,
  45.17742799230607,
  41.048751503547585,
  38.2099463490856,
  39.05124837953327,
  5.830951894845301,
  3.605551275463989,
  3.0,
  9.848857801796104,
  3.0,
  0.0,
  6.0,
  5.0,
  6.708203932499369,
  6.4031242374328485,
  5.385164807134504,
  5.385164807134504,
  8.06225774829855,
  10.0,
  10.770329614269007,
  34.52535300326414,
  37.48332962798263,
  35.22782990761707,
  39.20459156782532,
  43.18564576337837,
  34.058772731852805,
  40.049968789001575,
  31.016124838541646,
  42.01190307520001,
  36.22154055254967,
  38.2099463490856,
  43.18564576337837,
  40.311288741492746,
  34.52535300326414,
  44.14748010928823,
  63.8200595424354,
  51.73973328110612,
  66.61080993352356,
  67.80117993073571,
  60.207972893961475,
  60.81118318204309,
  56.72741841473134,
  59.413803110051795,
  60.81118318204309,
  55.226805085936306,
  55.154328932550705,
  57.3149195236284,
  53.150729063673246,
  56.22277118748239,
  79.76214641043707,
  79.22752047110903,
  72.78049189171504,
  72.83543093852057,
  73.43704787094863,
  75.23961722390672,
  74.10802925459562,
  78.10249675906654,
  71.25307010929424,
  68.3593446428504,
  69.26037828369117,
  71.17583859709698,
  69.02897942168927,
  67.02984409947557,
  73.00684899377592,
  69.00724599634447,
  64.00781202322104,
  59.682493245507096,
  69.1158447825099,
  67.1863081289633,
  71.06335201775947,
  73.02739212103907,
  69.58448102845921,
  71.34423592694787,
  75.1065909225016,
  74.16872656315464,
  73.43704787094863,
  62.369864518050704,
  56.568542494923804,
  59.135437767890075,
  76.23647421018367,
  74.67261881037788,
  72.56721022610694,
  64.6297145282261,
  59.0,
  64.28063471995279,
  74.14849964766651,
  64.93843238021688,
  84.2140130857092,
  84.09518416651456],
 [79.30952023559341,
  81.83520025025906,
  80.60397012554654,
  74.65252842335616,
  73.87827826905551,
  73.40980860893181,
  71.40028011149536,
  69.81403870282824,
  60.81118318204309,
  70.2922470831599,
  71.7007670809734,
  71.58910531638176,
  62.289646009589745,
  67.89698078707183,
  70.00714249274856,
  72.18032973047436,
  48.703182647543684,
  46.8187996428785,
  66.94027188471824,
  47.80167361086848,
  43.965895873961216,
  41.10960958218893,
  50.21951811795888,
  44.384682042344295,
  50.00999900019995,
  48.05205510693585,
  42.20189569201838,
  49.49747468305833,
  45.5411901469428,
  37.013511046643494,
  49.25444142409901,
  47.265209192385896,
  43.289721643826724,
  51.15662224971465,
  47.042533945356304,
  44.181444068749045,
  45.044422518220834,
  11.40175425099138,
  8.54400374531753,
  6.708203932499369,
  5.0,
  9.0,
  6.0,
  0.0,
  10.44030650891055,
  3.0,
  11.180339887498949,
  9.433981132056603,
  6.4031242374328485,
  12.206555615733702,
  11.661903789690601,
  10.198039027185569,
  28.635642126552707,
  31.575306807693888,
  29.274562336608895,
  33.24154027718932,
  37.21558813185679,
  28.071337695236398,
  34.058772731852805,
  25.019992006393608,
  36.013886210738214,
  30.265491900843113,
  32.2490309931942,
  37.21558813185679,
  34.36568055487916,
  28.635642126552707,
  38.3275357934736,
  60.53924347066124,
  48.010415536631214,
  63.28506932918696,
  64.35060217278468,
  56.4003546088143,
  56.72741841473134,
  53.009433122794285,
  55.44366510251645,
  56.72741841473134,
  51.40038910358559,
  51.088159097779204,
  53.075418038862395,
  48.79549159502341,
  51.66236541235796,
  80.8084154033477,
  79.90619500389191,
  74.33034373659252,
  73.97972695272672,
  74.33034373659252,
  75.95393340703298,
  74.67261881037788,
  78.63841300535,
  72.00694410957877,
  69.2314957226839,
  70.03570517957252,
  71.84705978674423,
  69.46221994724903,
  67.47592163134935,
  73.33484846919642,
  69.35416353759881,
  64.38167441127949,
  59.07622195096772,
  69.02897942168927,
  67.00746227100382,
  71.06335201775947,
  73.10950690573696,
  69.06518659932803,
  71.00704190430693,
  75.02666192761077,
  74.00675644831355,
  73.02739212103907,
  63.89053137985315,
  57.723478758647246,
  59.033888572581766,
  76.94153624668537,
  75.7099729229908,
  74.04052944165107,
  65.73431371817918,
  59.30430001273095,
  64.0,
  75.43208866258443,
  66.21933252457322,
  84.8528137423857,
  84.02380615040002],
 [82.05485969764375,
  78.54934754662193,
  77.52418977325722,
  71.34423592694787,
  70.49113419430843,
  70.32780389006896,
  68.47627326307996,
  66.73080248281148,
  56.61271941887264,
  66.88796603276258,
  68.26419266350405,
  67.95586803212802,
  57.62811813689564,
  63.8200595424354,
  65.80273550544841,
  68.24221567329127,
  40.70626487409524,
  38.948684188300895,
  68.26419266350405,
  39.395431207184416,
  35.77708763999664,
  33.12099032335839,
  41.43669871020132,
  35.84689665786984,
  41.10960958218893,
  39.21734310225516,
  33.61547262794322,
  40.26164427839479,
  36.40054944640259,
  27.073972741361768,
  39.81205847478876,
  37.8549864614954,
  33.95585369269929,
  41.593268686170845,
  37.33630940518894,
  34.713109915419565,
  35.35533905932738,
  6.082762530298219,
  6.324555320336759,
  7.211102550927978,
  14.7648230602334,
  3.1622776601683795,
  5.0,
  10.44030650891055,
  0.0,
  10.0,
  2.0,
  2.8284271247461903,
  6.324555320336759,
  4.0,
  8.06225774829855,
  10.63014581273465,
  39.05124837953327,
  41.97618372363071,
  39.6232255123179,
  43.56604182158393,
  47.51841748206689,
  38.3275357934736,
  44.28317965096906,
  35.05709628591621,
  46.04345773288535,
  40.01249804748511,
  42.01190307520001,
  47.01063709417264,
  44.04543109109048,
  38.118237105091836,
  47.51841748206689,
  68.60029154456998,
  56.639209034025185,
  71.40028011149536,
  72.62231062146122,
  65.11528238439882,
  65.76473218982953,
  61.61980201201558,
  64.35060217278468,
  65.76473218982953,
  60.14149981501958,
  60.108235708594876,
  62.289646009589745,
  58.137767414994535,
  61.220911460055866,
  82.29823813423954,
  82.02438661763951,
  74.96665925596525,
  75.32595834106593,
  76.10519036176179,
  78.02563681252464,
  77.0,
  81.0,
  74.02702209328699,
  71.06335201775947,
  72.02777242147643,
  74.00675644831355,
  72.02777242147643,
  70.02856560004639,
  76.05918747922567,
  72.06247289678588,
  67.06713054842886,
  63.34824385884742,
  72.44308110509934,
  70.57619995437555,
  74.33034373659252,
  76.23647421018367,
  73.16419889536138,
  74.81310045707235,
  78.40918313565064,
  77.52418977325722,
  76.94153624668537,
  64.6297145282261,
  59.135437767890075,
  62.51399843235114,
  79.02531240052139,
  77.23341245859851,
  74.81310045707235,
  67.1863081289633,
  62.12889826803627,
  67.7421582177598,
  76.53103945458993,
  67.36467917239716,
  87.02298546935747,
  87.36704184073076],
 [82.29823813423954,
  84.20213774008353,
  83.00602387778854,
  77.00649323271382,
  76.21679604916491,
  75.8023746329889,
  73.81734213584231,
  72.20110802473879,
  62.96824596572466,
  72.62231062146122,
  74.02702209328699,
  73.87827826905551,
  64.35060217278468,
  70.09279563550022,
  72.18032973047436,
  74.41102068914255,
  49.76946855251722,
  47.92702786528704,
  69.85699678629192,
  48.703182647543684,
  44.94441010848846,
  42.15447781671598,
  50.96076922496363,
  45.221676218380054,
  50.695167422546305,
  48.76474136094644,
  43.01162633521314,
  50.00999900019995,
  46.09772228646444,
  37.05401462729781,
  49.64876634922564,
  47.67598976424087,
  43.73785545725808,
  51.478150704935004,
  47.265209192385896,
  44.553338819890925,
  45.27692569068709,
  12.529964086141668,
  10.0,
  8.48528137423857,
  7.615773105863909,
  9.486832980505138,
  6.708203932499369,
  3.0,
  10.0,
  0.0,
  10.198039027185569,
  8.246211251235321,
  4.47213595499958,
  10.770329614269007,
  9.219544457292887,
  7.280109889280518,
  29.410882339705484,
  32.28002478313795,
  29.832867780352597,
  33.734255586866,
  37.656340767525464,
  28.442925306655784,
  34.36568055487916,
  25.079872407968907,
  36.05551275463989,
  30.01666203960727,
  32.01562118716424,
  37.013511046643494,
  34.058772731852805,
  28.160255680657446,
  37.656340767525464,
  63.13477647065839,
  50.47771785649585,
  65.86349520030045,
  66.88796603276258,
  58.82176467941097,
  59.033888572581766,
  55.47071299343465,
  57.8013840664737,
  59.033888572581766,
  53.823786563191554,
  53.41348144429457,
  55.31726674375732,
  50.99019513592785,
  53.74011537017761,
  83.74365647617735,
  82.87339742040264,
  77.20103626247513,
  76.90253571892151,
  77.27871634544663,
  78.91767862779544,
  77.64663547121665,
  81.61494961096282,
  74.96665925596525,
  72.18032973047436,
  72.99315036357864,
  74.81310045707235,
  72.44308110509934,
  70.45565981523414,
  76.32168761236873,
  72.3394774656273,
  67.36467917239716,
  62.07253821135398,
  72.02777242147643,
  70.00714249274856,
  74.06078584514209,
  76.10519036176179,
  72.06247289678588,
  74.00675644831355,
  78.02563681252464,
  77.00649323271382,
  76.02631123499285,
  66.76076692189807,
  60.63827174318213,
  62.03224967708329,
  79.90619500389191,
  78.64477096412705,
  76.92203845452875,
  68.65857557508748,
  62.289646009589745,
  67.0,
  78.33900688673555,
  69.12307863514182,
  87.82368700982668,
  87.02298546935747],
 [84.0535543567314,
  80.26207074328447,
  79.25906888173743,
  73.06161783043132,
  72.20110802473879,
  72.069410986909,
  70.2353187506115,
  68.47627326307996,
  58.249463516842795,
  68.60029154456998,
  69.9714227381436,
  69.6419413859206,
  59.20304046246274,
  65.45991139621256,
  67.42403132415029,
  69.8927750200262,
  41.677331968349414,
  39.96248240537617,
  70.25667228100119,
  40.24922359499622,
  36.71511950137164,
  34.132096331752024,
  42.15447781671598,
  36.6742416417845,
  41.78516483155236,
  39.92492955535426,
  34.438350715445125,
  40.80441152620633,
  37.0,
  27.294688127912362,
  40.26164427839479,
  38.3275357934736,
  34.48187929913333,
  41.97618372363071,
  37.656340767525464,
  35.17101079013795,
  35.6931365951495,
  8.06225774829855,
  8.246211251235321,
  8.94427190999916,
  15.811388300841896,
  5.0990195135927845,
  6.4031242374328485,
  11.180339887498949,
  2.0,
  10.198039027185569,
  0.0,
  2.0,
  6.0,
  2.0,
  6.4031242374328485,
  9.433981132056603,
  39.56008088970496,
  42.44997055358225,
  40.024992192379,
  43.93176527297759,
  47.853944456021594,
  38.63935817272331,
  44.553338819890925,
  35.22782990761707,
  46.17358552246078,
  40.01249804748511,
  42.01190307520001,
  47.01063709417264,
  44.0,
  38.01315561749642,
  47.265209192385896,
  70.21395872616783,
  58.137767414994535,
  73.00684899377592,
  74.2024258363566,
  66.60330322138685,
  67.17886572427373,
  63.12685640834652,
  65.79513659838392,
  67.17886572427373,
  61.61980201201558,
  61.5223536610881,
  63.655321851358195,
  59.464274989274024,
  62.48199740725323,
  84.29116205154607,
  84.02380615040002,
  76.94153624668537,
  77.31752712031083,
  78.10249675906654,
  80.02499609497022,
  79.0,
  83.0,
  76.02631123499285,
  73.06161783043132,
  74.02702209328699,
  76.00657866263946,
  74.02702209328699,
  72.02777242147643,
  78.05767098754612,
  74.06078584514209,
  69.06518659932803,
  65.30696746902278,
  74.43117626371358,
  72.56031973468694,
  76.32168761236873,
  78.23042886243178,
  75.13321502504735,
  76.79192665899195,
  80.39900496896712,
  79.51100552753688,
  78.91767862779544,
  66.61080993352356,
  61.1310068623117,
  64.4980619863884,
  81.02468759581859,
  79.22752047110903,
  76.79192665899195,
  69.18092222571191,
  64.1248781675256,
  69.72087205421343,
  78.5175139698144,
  69.35416353759881,
  89.02246907382428,
  89.35882720806042],
 [84.00595217006948,
  81.30190649671138,
  80.26207074328447,
  74.09453421137081,
  73.24616030891995,
  73.06161783043132,
  71.19691004531025,
  69.46221994724903,
  59.405386961116584,
  69.6419413859206,
  71.02112361825881,
  70.7248188403477,
  60.440052945046304,
  66.61080993352356,
  68.60029154456998,
  71.02816342831905,
  43.41658669218482,
  41.677331968349414,
  70.45565981523414,
  42.04759208325728,
  38.47076812334269,
  35.84689665786984,
  44.01136216933077,
  38.48376280978771,
  43.657759905886145,
  41.78516483155236,
  36.24913792078372,
  42.720018726587654,
  38.897300677553446,
  29.274562336608895,
  42.20189569201838,
  40.26164427839479,
  36.40054944640259,
  43.93176527297759,
  39.6232255123179,
  37.107950630558946,
  37.656340767525464,
  8.54400374531753,
  8.0,
  8.246211251235321,
  14.212670403551895,
  5.0990195135927845,
  5.385164807134504,
  9.433981132056603,
  2.8284271247461903,
  8.246211251235321,
  2.0,
  0.0,
  4.0,
  2.8284271247461903,
  5.385164807134504,
  7.810249675906654,
  37.64306044943742,
  40.52159917870962,
  38.07886552931954,
  41.97618372363071,
  45.89117562233506,
  36.6742416417845,
  42.579337712087536,
  33.24154027718932,
  44.181444068749045,
  38.01315561749642,
  40.01249804748511,
  45.0111097397076,
  42.0,
  36.013886210738214,
  45.27692569068709,
  69.06518659932803,
  56.85068161420758,
  71.84705978674423,
  73.00684899377592,
  65.29931086925804,
  65.79513659838392,
  61.84658438426491,
  64.4437739428721,
  65.79513659838392,
  60.30754513325841,
  60.14149981501958,
  62.22539674441618,
  58.0,
  60.959002616512684,
  84.48076704197234,
  84.09518416651456,
  77.27871634544663,
  77.52418977325722,
  78.23042886243178,
  80.09993757800315,
  79.02531240052139,
  83.02409288875127,
  76.10519036176179,
  73.17103251970687,
  74.10802925459562,
  76.05918747922567,
  74.0,
  72.0,
  78.00640999302557,
  74.00675644831355,
  69.00724599634447,
  64.93843238021688,
  74.24284477308234,
  72.3394774656273,
  76.16429609731846,
  78.10249675906654,
  74.81310045707235,
  76.53103945458993,
  80.22468448052632,
  79.30952023559341,
  78.63841300535,
  66.91038783328041,
  61.29437168288782,
  64.28063471995279,
  81.09870529176159,
  79.40403012442127,
  77.10382610480494,
  69.35416353759881,
  64.03124237432849,
  69.46221994724903,
  78.77182237323191,
  69.58448102845921,
  89.08984229416954,
  89.20201791439474],
 [84.0535543567314,
  83.48652585896721,
  82.37718130647589,
  76.27581530209953,
  75.45197147855052,
  75.16648189186454,
  73.24616030891995,
  71.56116265125938,
  61.84658438426491,
  71.84705978674423,
  73.23933369440222,
  73.00684899377592,
  63.0317380372777,
  69.02897942168927,
  71.06335201775947,
  73.40980860893181,
  46.95742752749558,
  45.17742799230607,
  71.02112361825881,
  45.69463863518345,
  42.04759208325728,
  39.35733730830886,
  47.75981574503821,
  42.15447781671598,
  47.43416490252569,
  45.5411901469428,
  39.92492955535426,
  46.57252408878007,
  42.720018726587654,
  33.24154027718932,
  46.09772228646444,
  44.14748010928823,
  40.26164427839479,
  47.853944456021594,
  43.56604182158393,
  41.0,
  41.593268686170845,
  10.63014581273465,
  8.94427190999916,
  8.246211251235321,
  11.40175425099138,
  7.0710678118654755,
  5.385164807134504,
  6.4031242374328485,
  6.324555320336759,
  4.47213595499958,
  6.0,
  4.0,
  0.0,
  6.324555320336759,
  5.385164807134504,
  5.385164807134504,
  33.83784863137726,
  36.68787265568828,
  34.20526275297414,
  38.07886552931954,
  41.97618372363071,
  32.7566787083184,
  38.63935817272331,
  29.274562336608895,
  40.19950248448356,
  34.0147027033899,
  36.013886210738214,
  41.012193308819754,
  38.0,
  32.01562118716424,
  41.30375285612676,
  66.88796603276258,
  54.405882034941776,
  69.6419413859206,
  70.7248188403477,
  62.80127387243033,
  63.12685640834652,
  59.405386961116584,
  61.84658438426491,
  63.12685640834652,
  57.8013840664737,
  57.48912940721924,
  59.464274989274024,
  55.17245689653489,
  58.0,
  85.0,
  84.38009243891595,
  78.10249675906654,
  78.08969202141856,
  78.63841300535,
  80.39900496896712,
  79.22752047110903,
  83.21658488546619,
  76.4198926981712,
  73.55270219373317,
  74.43117626371358,
  76.32168761236873,
  74.10802925459562,
  72.11102550927978,
  78.05767098754612,
  74.06078584514209,
  69.06518659932803,
  64.38167441127949,
  74.02702209328699,
  72.06247289678588,
  76.00657866263946,
  78.0,
  74.33034373659252,
  76.16429609731846,
  80.02499609497022,
  79.05694150420949,
  78.23042886243178,
  67.68308503607086,
  61.814237842102365,
  64.03124237432849,
  81.39410298049853,
  79.90619500389191,
  77.87810988975015,
  69.87131027825369,
  64.03124237432849,
  69.1158447825099,
  79.42921376924235,
  70.21395872616783,
  89.35882720806042,
  89.02246907382428],
 [86.05230967266364,
  81.98780397107853,
  81.00617260431454,
  74.79304780526061,
  73.92563831310488,
  73.824115301167,
  72.00694410957877,
  70.2353187506115,
  59.90826320300064,
  70.32780389006896,
  71.69379331573968,
  71.34423592694787,
  60.802960454241045,
  67.11929677819934,
  69.06518659932803,
  71.56116265125938,
  42.720018726587654,
  41.048751503547585,
  72.24956747275377,
  41.182520563948,
  37.73592452822641,
  35.22782990761707,
  42.95346318982906,
  37.589892258425,
  42.5440947723653,
  40.718546143004666,
  35.35533905932738,
  41.43669871020132,
  37.69615364994153,
  27.65863337187866,
  40.80441152620633,
  38.897300677553446,
  35.11409973215888,
  42.44997055358225,
  38.07886552931954,
  35.73513677041127,
  36.138621999185304,
  10.04987562112089,
  10.198039027185569,
  10.770329614269007,
  17.029386365926403,
  7.0710678118654755,
  8.06225774829855,
  12.206555615733702,
  4.0,
  10.770329614269007,
  2.0,
  2.8284271247461903,
  6.324555320336759,
  0.0,
  5.0,
  8.54400374531753,
  40.162171256046406,
  43.01162633521314,
  40.52159917870962,
  44.384682042344295,
  48.27007354458868,
  39.05124837953327,
  44.91102314577124,
  35.510561809129406,
  46.389654018972806,
  40.11234224026316,
  42.1070065428546,
  47.095647357266465,
  44.04543109109048,
  38.01315561749642,
  47.095647357266465,
  71.84705978674423,
  59.665735560705194,
  74.63243262818116,
  75.8023746329889,
  68.11754546370561,
  68.62215385719105,
  64.66065264130884,
  67.26812023536856,
  68.62215385719105,
  63.12685640834652,
  62.96824596572466,
  65.05382386916237,
  60.8276253029822,
  63.7808748764079,
  86.28441342444185,
  86.02325267042627,
  78.91767862779544,
  79.30952023559341,
  80.09993757800315,
  82.02438661763951,
  81.0,
  85.0,
  78.02563681252464,
  75.05997601918082,
  76.02631123499285,
  78.00640999302557,
  76.02631123499285,
  74.02702209328699,
  80.05623023850174,
  76.05918747922567,
  71.06335201775947,
  67.26812023536856,
  76.4198926981712,
  74.54528824815154,
  78.31347265956222,
  80.22468448052632,
  77.10382610480494,
  78.77182237323191,
  82.38931969618392,
  81.49846624323675,
  80.89499366462674,
  68.59300255857006,
  63.12685640834652,
  66.48308055437865,
  83.02409288875127,
  81.2219182240853,
  78.77182237323191,
  71.17583859709698,
  66.12110101926616,
  71.7007670809734,
  80.50465825031493,
  71.34423592694787,
  91.02197536858887,
  91.35097153287424],
 [89.00561780022652,
  86.6083136886985,
  85.58621384311844,
  79.40403012442127,
  78.54934754662193,
  78.39005038906404,
  76.53757247260981,
  74.79304780526061,
  64.63745044476924,
  74.94664769020693,
  76.32168761236873,
  76.00657866263946,
  65.60487786742691,
  71.84705978674423,
  73.81734213584231,
  76.27581530209953,
  47.70744176750625,
  46.04345773288535,
  75.66372975210778,
  46.14108798023731,
  42.720018726587654,
  40.22437072223753,
  47.853944456021594,
  42.5440947723653,
  47.4236228055175,
  45.617978911828175,
  40.311288741492746,
  46.238512086787566,
  42.5440947723653,
  32.28002478313795,
  45.5411901469428,
  43.657759905886145,
  39.92492955535426,
  47.12748667179272,
  42.720018726587654,
  40.496913462633174,
  40.80441152620633,
  13.92838827718412,
  13.152946437965905,
  13.0,
  16.64331697709324,
  10.44030650891055,
  10.0,
  11.661903789690601,
  8.06225774829855,
  9.219544457292887,
  6.4031242374328485,
  5.385164807134504,
  5.385164807134504,
  5.0,
  0.0,
  4.0,
  37.57658845611187,
  40.311288741492746,
  37.69615364994153,
  41.43669871020132,
  45.221676218380054,
  36.05551275463989,
  41.7612260356422,
  32.28002478313795,
  42.95346318982906,
  36.49657518178932,
  38.47076812334269,
  43.41658669218482,
  40.311288741492746,
  34.23448553724738,
  43.0,
  72.20110802473879,
  59.64059020499378,
  74.94664769020693,
  76.00657866263946,
  68.00735254367721,
  68.24954212300622,
  64.63745044476924,
  67.00746227100382,
  68.24954212300622,
  63.00793600809346,
  62.625873247404705,
  64.53681120105021,
  60.207972893961475,
  62.93647591023825,
  89.67719888578144,
  89.20201791439474,
  82.56512580987206,
  82.73451516749222,
  83.38465086573187,
  85.21150157109074,
  84.09518416651456,
  88.09086218218096,
  81.2219182240853,
  78.31347265956222,
  79.22752047110903,
  81.15417426109393,
  79.02531240052139,
  77.02596964660685,
  83.00602387778854,
  79.00632886041473,
  74.00675644831355,
  69.58448102845921,
  79.1012010022604,
  77.1621668954417,
  81.05553651663777,
  83.02409288875127,
  79.51100552753688,
  81.30190649671138,
  85.09406559801923,
  84.1486779456457,
  83.38465086573187,
  72.18032973047436,
  66.48308055437865,
  69.1158447825099,
  86.20904824900923,
  84.5931439302264,
  82.37718130647589,
  74.54528824815154,
  69.0,
  74.24284477308234,
  84.01190391843289,
  74.81310045707235,
  94.19129471453293,
  94.08506789071261],
 [89.1403387922662,
  88.75246475450696,
  87.66413177577246,
  81.54140052758476,
  80.7093550959243,
  80.45495634204272,
  78.54934754662193,
  76.8505042273634,
  67.00746227100382,
  77.10382610480494,
  78.49203781276162,
  78.2368199762746,
  68.11754546370561,
  74.2024258363566,
  76.21679604916491,
  78.60025445251433,
  51.10772935672255,
  49.39635614091387,
  76.2954782408499,
  49.64876634922564,
  46.14108798023731,
  43.56604182158393,
  51.478150704935004,
  46.06517122512408,
  51.07837115648854,
  49.24428900898052,
  43.829214001622255,
  49.9799959983992,
  46.238512086787566,
  36.138621999185304,
  49.33558553417604,
  47.43416490252569,
  43.657759905886145,
  50.96076922496363,
  46.57252408878007,
  44.27188724235731,
  44.64302857109943,
  15.811388300841896,
  14.317821063276353,
  13.601470508735444,
  14.866068747318506,
  12.206555615733702,
  10.770329614269007,
  10.198039027185569,
  10.63014581273465,
  7.280109889280518,
  9.433981132056603,
  7.810249675906654,
  5.385164807134504,
  8.54400374531753,
  4.0,
  0.0,
  34.0,
  36.6742416417845,
  34.0147027033899,
  37.69615364994153,
  41.43669871020132,
  32.31098884280702,
  37.94733192202055,
  28.460498941515414,
  39.05124837953327,
  32.55764119219941,
  34.52535300326414,
  39.45883931389772,
  36.345563690772494,
  30.265491900843113,
  39.0,
  70.2353187506115,
  57.4543296888929,
  72.94518489934754,
  73.92563831310488,
  65.73431371817918,
  65.80273550544841,
  62.433965115151864,
  64.63745044476924,
  65.80273550544841,
  60.74537019394976,
  60.21627686929839,
  62.00806399170998,
  57.62811813689564,
  60.207972893961475,
  90.2551937563706,
  89.56003573022959,
  83.43260753446461,
  83.3606621854697,
  83.86298349093002,
  85.58621384311844,
  84.38009243891595,
  88.36288813749809,
  81.61494961096282,
  78.77182237323191,
  79.63039620647382,
  81.49846624323675,
  79.22752047110903,
  77.23341245859851,
  83.15046602394962,
  79.1580697086532,
  74.16872656315464,
  69.18092222571191,
  79.0,
  77.00649323271382,
  81.00617260431454,
  83.02409288875127,
  79.1580697086532,
  81.05553651663777,
  85.0,
  84.00595217006948,
  83.09632964216891,
  73.00684899377592,
  67.08203932499369,
  69.0,
  86.57944328765345,
  85.15867542417507,
  83.19855767980596,
  75.13321502504735,
  69.1158447825099,
  74.02702209328699,
  84.72307831990054,
  75.50496672405067,
  94.53041838477179,
  94.0],
 [80.95677859203639,
  98.00510190801293,
  96.40020746865642,
  91.2633551870629,
  90.69729874698585,
  89.62700485902673,
  87.41853350405736,
  86.2670273047588,
  80.06247560499239,
  87.36704184073076,
  88.75246475450696,
  89.04493247793498,
  82.29216244576394,
  86.45229898620394,
  88.68483523128404,
  90.21086409075129,
  74.67261881037788,
  72.69112738154499,
  73.60027173862879,
  74.33034373659252,
  70.34912934784623,
  67.36467917239716,
  77.1621668954417,
  71.17583859709698,
  77.10382610480494,
  75.1065909225016,
  69.1158447825099,
  77.00649323271382,
  73.00684899377592,
  65.37583651472461,
  77.00649323271382,
  75.0066663703967,
  71.00704190430693,
  79.02531240052139,
  75.1065909225016,
  72.02777242147643,
  73.10950690573696,
  39.11521443121589,
  36.124783736376884,
  34.132096331752024,
  25.079872407968907,
  37.48332962798263,
  34.52535300326414,
  28.635642126552707,
  39.05124837953327,
  29.410882339705484,
  39.56008088970496,
  37.64306044943742,
  33.83784863137726,
  40.162171256046406,
  37.57658845611187,
  34.0,
  0.0,
  3.0,
  2.23606797749979,
  5.385164807134504,
  9.219544457292887,
  4.0,
  7.211102550927978,
  7.615773105863909,
  10.63014581273465,
  10.198039027185569,
  10.770329614269007,
  13.45362404707371,
  12.529964086141668,
  12.0,
  18.35755975068582,
  46.09772228646444,
  33.0,
  48.25971404805462,
  48.507731342539614,
  39.45883931389772,
  38.07886552931954,
  37.12142238654117,
  37.656340767525464,
  38.07886552931954,
  35.12833614050059,
  33.37663853655727,
  33.95585369269929,
  29.614185789921695,
  30.675723300355934,
  85.75546629807339,
  83.24061508662703,
  82.00609733428362,
  79.88116173416608,
  79.07591289387686,
  79.75587752636166,
  77.89736837660179,
  81.41252974819048,
  76.32168761236873,
  74.33034373659252,
  74.62573282722254,
  75.8023746329889,
  72.56031973468694,
  70.83078426785913,
  75.59100475585703,
  72.069410986909,
  67.7421582177598,
  58.60034129593445,
  69.77822009767804,
  67.54257916307313,
  72.01388754955533,
  74.24957912338628,
  67.77905281132217,
  70.3846574190711,
  75.23961722390672,
  73.92563831310488,
  71.8679344353238,
  72.34638899074369,
  65.29931086925804,
  60.90155991434045,
  80.62257748298549,
  80.99382692526635,
  81.40024569987489,
  72.20110802473879,
  62.96824596572466,
  64.40496875241847,
  81.8413098624405,
  73.40980860893181,
  87.65842800324451,
  83.57032966310472],
 [82.29823813423954,
  100.43903623591775,
  98.812954616285,
  93.74433316206373,
  93.1933474020544,
  92.0869154657707,
  89.87213138676528,
  88.75246475450696,
  82.73451516749222,
  89.88882021697692,
  91.26883367283708,
  91.58602513484249,
  85.0,
  89.06739021662193,
  91.30169768410661,
  92.78469701410896,
  77.64663547121665,
  75.66372975210778,
  75.43208866258443,
  77.31752712031083,
  73.33484846919642,
  70.34912934784623,
  80.15609770940699,
  74.16872656315464,
  80.09993757800315,
  78.10249675906654,
  72.11102550927978,
  80.00624975587844,
  76.00657866263946,
  68.3593446428504,
  80.00624975587844,
  78.00640999302557,
  74.00675644831355,
  82.02438661763951,
  78.10249675906654,
  75.02666192761077,
  76.10519036176179,
  42.1070065428546,
  39.11521443121589,
  37.12142238654117,
  28.071337695236398,
  40.44749683231337,
  37.48332962798263,
  31.575306807693888,
  41.97618372363071,
  32.28002478313795,
  42.44997055358225,
  40.52159917870962,
  36.68787265568828,
  43.01162633521314,
  40.311288741492746,
  36.6742416417845,
  3.0,
  0.0,
  2.8284271247461903,
  2.8284271247461903,
  6.324555320336759,
  5.0,
  5.0,
  9.219544457292887,
  8.602325267042627,
  10.04987562112089,
  10.04987562112089,
  11.661903789690601,
  11.40175425099138,
  12.36931687685298,
  17.08800749063506,
  46.0,
  33.13608305156178,
  48.041648597857254,
  48.16637831516918,
  39.11521443121589,
  37.48332962798263,
  37.0,
  37.21558813185679,
  37.48332962798263,
  35.0,
  33.06055050963308,
  33.37663853655727,
  29.154759474226502,
  29.832867780352597,
  87.36704184073076,
  84.72307831990054,
  83.86894538504703,
  81.60882305241266,
  80.7093550959243,
  81.30190649671138,
  79.40403012442127,
  82.85529554590944,
  77.93587107359485,
  76.02631123499285,
  76.27581530209953,
  77.38862965578342,
  74.09453421137081,
  72.40165743959182,
  77.02596964660685,
  73.57309290766564,
  69.33974329343886,
  59.941638282582836,
  71.11961754677819,
  68.8839603971781,
  73.35529974037323,
  75.59100475585703,
  68.94200461257273,
  71.58910531638176,
  76.48529270389177,
  75.15317691222374,
  73.00684899377592,
  74.33034373659252,
  67.26812023536856,
  62.433965115151864,
  82.15229759416349,
  82.66196223173026,
  83.24061508662703,
  74.02702209328699,
  64.63745044476924,
  65.76473218982953,
  83.60023923410746,
  75.28612089887484,
  89.06739021662193,
  84.69356528095862],
 [83.19254774317228,
  100.0,
  98.40731680114035,
  93.23089616645332,
  92.65527507918802,
  91.60785992479029,
  89.40357934669059,
  88.23264701911646,
  81.88406438373708,
  89.30845424706443,
  90.69729874698585,
  90.97252332435328,
  84.07734534343957,
  88.32326986700618,
  90.55385138137417,
  92.11405973031478,
  75.95393340703298,
  73.97972695272672,
  75.8023746329889,
  75.53806987208503,
  71.56814934033156,
  68.59300255857006,
  78.31347265956222,
  72.3394774656273,
  78.23042886243178,
  76.23647421018367,
  70.25667228100119,
  78.05767098754612,
  74.06078584514209,
  66.18912297349165,
  78.00640999302557,
  76.00657866263946,
  72.00694410957877,
  80.0,
  76.02631123499285,
  73.0,
  74.02702209328699,
  40.01249804748511,
  37.013511046643494,
  35.014282800023196,
  26.0,
  38.2099463490856,
  35.22782990761707,
  29.274562336608895,
  39.6232255123179,
  29.832867780352597,
  40.024992192379,
  38.07886552931954,
  34.20526275297414,
  40.52159917870962,
  37.69615364994153,
  34.0147027033899,
  2.23606797749979,
  2.8284271247461903,
  0.0,
  4.0,
  8.0,
  2.23606797749979,
  5.385164807134504,
  6.4031242374328485,
  8.602325267042627,
  8.06225774829855,
  8.54400374531753,
  11.313708498984761,
  10.295630140987,
  10.04987562112089,
  16.1245154965971,
  48.041648597857254,
  35.014282800023196,
  50.15974481593781,
  50.35871324805669,
  41.30375285612676,
  39.81205847478876,
  39.05124837953327,
  39.45883931389772,
  39.81205847478876,
  37.05401462729781,
  35.22782990761707,
  35.6931365951495,
  31.400636936215164,
  32.28002478313795,
  87.98295289429652,
  85.47514258543241,
  84.20213774008353,
  82.09750300709517,
  81.30190649671138,
  81.98780397107853,
  80.1311425102625,
  83.64807230295268,
  78.54934754662193,
  76.55063683601855,
  76.8505042273634,
  78.03204469959762,
  74.79304780526061,
  73.06161783043132,
  77.82673062643708,
  74.30343195303969,
  69.9714227381436,
  60.83584469702052,
  72.01388754955533,
  69.77822009767804,
  74.24957912338628,
  76.48529270389177,
  70.00714249274856,
  72.61542535852834,
  77.47257579298626,
  76.15773105863909,
  74.09453421137081,
  74.51845409024533,
  67.47592163134935,
  63.13477647065839,
  82.85529554590944,
  83.21658488546619,
  83.60023923410746,
  74.4043009509531,
  65.19202405202648,
  66.64082832618455,
  84.0535543567314,
  75.6042326857432,
  89.89438247187641,
  85.7962703152066],
 [85.0,
  103.22790320451152,
  101.6070863670443,
  96.51942809610924,
  95.96353474106714,
  94.86832980505137,
  92.65527507918802,
  91.52595260361949,
  85.42247947700886,
  92.64987857520376,
  94.03190947758107,
  94.33981132056604,
  87.66413177577246,
  91.7877987534291,
  94.02127418834527,
  95.524865872714,
  79.90619500389191,
  77.92945527847606,
  78.24321056807422,
  79.51100552753688,
  75.53806987208503,
  72.56031973468694,
  82.29823813423954,
  76.32168761236873,
  82.21921916437786,
  80.22468448052632,
  74.24284477308234,
  82.05485969764375,
  78.05767098754612,
  70.178344238091,
  82.00609733428362,
  80.00624975587844,
  76.00657866263946,
  84.0,
  80.02499609497022,
  77.0,
  78.02563681252464,
  44.01136216933077,
  41.012193308819754,
  39.01281840626232,
  30.0,
  42.190046219457976,
  39.20459156782532,
  33.24154027718932,
  43.56604182158393,
  33.734255586866,
  43.93176527297759,
  41.97618372363071,
  38.07886552931954,
  44.384682042344295,
  41.43669871020132,
  37.69615364994153,
  5.385164807134504,
  2.8284271247461903,
  4.0,
  0.0,
  4.0,
  5.385164807134504,
  2.23606797749979,
  9.433981132056603,
  5.830951894845301,
  8.54400374531753,
  8.06225774829855,
  8.94427190999916,
  9.055385138137417,
  11.180339887498949,
  14.560219778561036,
  48.041648597857254,
  35.35533905932738,
  50.0,
  50.039984012787215,
  41.012193308819754,
  39.20459156782532,
  39.05124837953327,
  39.05124837953327,
  39.20459156782532,
  37.05401462729781,
  35.0,
  35.12833614050059,
  31.016124838541646,
  31.400636936215164,
  90.13878188659973,
  87.46427842267951,
  86.68333173107735,
  84.40379138403677,
  83.48652585896721,
  84.05950273467003,
  82.15229759416349,
  85.58621384311844,
  80.7093550959243,
  78.81624198095213,
  79.05694150420949,
  80.15609770940699,
  76.8505042273634,
  75.16648189186454,
  79.75587752636166,
  76.32168761236873,
  72.11102550927978,
  62.64982043070834,
  73.824115301167,
  71.58910531638176,
  76.05918747922567,
  78.29431652425353,
  71.58910531638176,
  74.24957912338628,
  79.1580697086532,
  77.82030583337487,
  75.64390259630977,
  77.15568676384133,
  70.09279563550022,
  65.19202405202648,
  84.90583018850944,
  85.44588931013593,
  86.05230967266364,
  76.83749084919418,
  67.42403132415029,
  68.47627326307996,
  86.40023148117139,
  78.10249675906654,
  91.7877987534291,
  87.32124598286491],
 [86.95401083331349,
  106.50821564555478,
  104.86181383134664,
  99.85990186255943,
  99.32270636667126,
  98.18350166906862,
  95.96353474106714,
  94.87360012142472,
  89.0,
  96.04165762834376,
  97.41663102366043,
  97.75479527879949,
  91.28526715741155,
  95.29428104561154,
  97.52948272189288,
  98.97979591815695,
  83.86298349093002,
  81.88406438373708,
  80.8084154033477,
  83.48652585896721,
  79.51100552753688,
  76.53103945458993,
  86.28441342444185,
  80.30566605165541,
  86.20904824900923,
  84.2140130857092,
  78.23042886243178,
  86.05230967266364,
  82.05485969764375,
  74.16872656315464,
  86.00581375697809,
  84.00595217006948,
  80.00624975587844,
  88.0,
  84.02380615040002,
  81.0,
  82.02438661763951,
  48.010415536631214,
  45.0111097397076,
  43.01162633521314,
  34.0,
  46.17358552246078,
  43.18564576337837,
  37.21558813185679,
  47.51841748206689,
  37.656340767525464,
  47.853944456021594,
  45.89117562233506,
  41.97618372363071,
  48.27007354458868,
  45.221676218380054,
  41.43669871020132,
  9.219544457292887,
  6.324555320336759,
  8.0,
  4.0,
  0.0,
  9.219544457292887,
  3.605551275463989,
  13.0,
  5.0990195135927845,
  10.63014581273465,
  9.433981132056603,
  8.0,
  9.486832980505138,
  13.45362404707371,
  14.0,
  48.373546489791295,
  36.138621999185304,
  50.15974481593781,
  50.039984012787215,
  41.10960958218893,
  39.0,
  39.45883931389772,
  39.05124837953327,
  39.0,
  37.48332962798263,
  35.22782990761707,
  35.014282800023196,
  31.144823004794873,
  31.016124838541646,
  92.41753080449618,
  89.58794561770016,
  89.27485648266257,
  86.83317338436964,
  85.80209787645055,
  86.2670273047588,
  84.31488599292535,
  87.66413177577246,
  83.00602387778854,
  81.21576201698781,
  81.40024569987489,
  82.41965784932621,
  79.05694150420949,
  77.42092740338364,
  81.83520025025906,
  78.49203781276162,
  74.4043009509531,
  64.66065264130884,
  75.8023746329889,
  73.57309290766564,
  78.03204469959762,
  80.26207074328447,
  73.35529974037323,
  76.05918747922567,
  81.00617260431454,
  79.64923100695951,
  77.3692445355388,
  79.90619500389191,
  72.83543093852057,
  67.42403132415029,
  87.09190547921202,
  87.80091115700337,
  88.61715409558128,
  79.39773296511683,
  69.81403870282824,
  70.49113419430843,
  88.86506625215557,
  80.72174428244226,
  93.81364506296512,
  89.0],
 [84.581321815162,
  100.42410069301094,
  98.85848471426213,
  93.60021367496978,
  93.00537618869137,
  92.00543462209176,
  89.81091247727082,
  88.60022573334675,
  81.98780397107853,
  89.62700485902673,
  91.02197536858887,
  91.2633551870629,
  84.11896337925236,
  88.50988645343524,
  90.7358804442873,
  92.35799911215054,
  75.31268153505093,
  73.348483283569,
  76.84399781375251,
  74.81310045707235,
  70.8590149522275,
  67.89698078707183,
  77.52418977325722,
  71.56814934033156,
  77.41446893184762,
  75.4254598925323,
  69.46221994724903,
  77.1621668954417,
  73.17103251970687,
  65.06919393998976,
  77.05841939723393,
  75.05997601918082,
  71.06335201775947,
  79.02531240052139,
  75.0,
  72.02777242147643,
  73.0,
  39.01281840626232,
  36.013886210738214,
  34.0147027033899,
  25.079872407968907,
  37.05401462729781,
  34.058772731852805,
  28.071337695236398,
  38.3275357934736,
  28.442925306655784,
  38.63935817272331,
  36.6742416417845,
  32.7566787083184,
  39.05124837953327,
  36.05551275463989,
  32.31098884280702,
  4.0,
  5.0,
  2.23606797749979,
  5.385164807134504,
  9.219544457292887,
  0.0,
  6.0,
  4.242640687119285,
  8.54400374531753,
  6.324555320336759,
  7.211102550927978,
  10.816653826391969,
  9.219544457292887,
  8.0,
  15.0,
  50.08991914547278,
  37.0,
  52.23983154643591,
  52.46903848937962,
  43.41658669218482,
  41.97618372363071,
  41.10960958218893,
  41.593268686170845,
  41.97618372363071,
  39.11521443121589,
  37.33630940518894,
  37.8549864614954,
  33.54101966249684,
  34.48187929913333,
  89.1852005660132,
  86.76981041814025,
  85.21150157109074,
  83.21658488546619,
  82.49242389456137,
  83.24061508662703,
  81.41252974819048,
  84.970583144992,
  79.75587752636166,
  77.69813382572325,
  78.03204469959762,
  79.25906888173743,
  76.05918747922567,
  74.30343195303969,
  79.1580697086532,
  75.59100475585703,
  71.19691004531025,
  62.24146527838174,
  73.40980860893181,
  71.17583859709698,
  75.64390259630977,
  77.87810988975015,
  71.5122367151245,
  74.09453421137081,
  78.92401408950256,
  77.62087348130012,
  75.61084578286372,
  75.43208866258443,
  68.41052550594829,
  64.41273166075166,
  84.11896337925236,
  84.38009243891595,
  84.62860036654276,
  75.45197147855052,
  66.37017402418047,
  68.0294054067798,
  85.14693182963201,
  76.60939890117922,
  91.23595782365635,
  87.31551981177229],
 [87.23531395025755,
  105.20931517693668,
  103.60019305001319,
  98.4733466477097,
  97.9081201943945,
  96.83491106000976,
  94.62557793746889,
  93.47726996441435,
  87.23531395025755,
  94.57801012920498,
  95.96353474106714,
  96.25487000666512,
  89.44271909999159,
  93.64827814754524,
  95.88013350011565,
  97.41663102366043,
  81.21576201698781,
  79.2464510246358,
  80.43009387039157,
  80.75270893288967,
  76.79192665899195,
  73.824115301167,
  83.48652585896721,
  77.52418977325722,
  83.38465086573187,
  81.39410298049853,
  75.4254598925323,
  83.15046602394962,
  79.1580697086532,
  71.06335201775947,
  83.05419917138447,
  81.05553651663777,
  77.05841939723393,
  85.0235261559999,
  81.0,
  78.02563681252464,
  79.0,
  45.0111097397076,
  42.01190307520001,
  40.01249804748511,
  31.064449134018133,
  43.04648650006177,
  40.049968789001575,
  34.058772731852805,
  44.28317965096906,
  34.36568055487916,
  44.553338819890925,
  42.579337712087536,
  38.63935817272331,
  44.91102314577124,
  41.7612260356422,
  37.94733192202055,
  7.211102550927978,
  5.0,
  5.385164807134504,
  2.23606797749979,
  3.605551275463989,
  6.0,
  0.0,
  9.486832980505138,
  3.605551275463989,
  7.211102550927978,
  6.324555320336759,
  6.708203932499369,
  7.0,
  10.0,
  12.36931687685298,
  50.08991914547278,
  37.48332962798263,
  52.009614495783374,
  52.009614495783374,
  43.0,
  41.10960958218893,
  41.10960958218893,
  41.012193308819754,
  41.10960958218893,
  39.11521443121589,
  37.013511046643494,
  37.05401462729781,
  33.0,
  33.24154027718932,
  92.35799911215054,
  89.69392398596462,
  88.86506625215557,
  86.6083136886985,
  85.70297544426332,
  86.28441342444185,
  84.38009243891595,
  87.8179936004006,
  82.92767933567175,
  81.02468759581859,
  81.27115109311545,
  82.37718130647589,
  79.07591289387686,
  77.38862965578342,
  81.98780397107853,
  78.54934754662193,
  74.33034373659252,
  64.88451279003334,
  76.05918747922567,
  73.824115301167,
  78.29431652425353,
  80.52949770115296,
  73.824115301167,
  76.48529270389177,
  81.39410298049853,
  80.05623023850174,
  77.87810988975015,
  79.30952023559341,
  72.24956747275377,
  67.4166151627327,
  87.13208364316786,
  87.65842800324451,
  88.23831367382311,
  79.02531240052139,
  69.63476143421474,
  70.71067811865476,
  88.60022573334675,
  80.2807573457052,
  94.02127418834527,
  89.55445270895244],
 [86.16263691415206,
  100.00499987500625,
  98.49365461794989,
  93.08598175880189,
  92.45539465060976,
  91.54780172128658,
  89.37561188601732,
  88.09086218218096,
  80.99382692526635,
  89.02246907382428,
  90.4267659490264,
  90.60353193998566,
  83.00602387778854,
  87.65842800324451,
  89.87213138676528,
  91.60785992479029,
  73.00684899377592,
  71.06335201775947,
  77.69813382572325,
  72.3671196055225,
  68.4470598345904,
  65.5133574166368,
  74.96665925596525,
  69.05070600652827,
  74.81310045707235,
  72.83543093852057,
  66.91038783328041,
  74.43117626371358,
  70.45565981523414,
  62.0,
  74.24284477308234,
  72.24956747275377,
  68.26419266350405,
  76.16429609731846,
  72.06247289678588,
  69.18092222571191,
  70.06425622241343,
  36.22154055254967,
  33.24154027718932,
  31.25699921617557,
  22.561028345356956,
  34.0147027033899,
  31.016124838541646,
  25.019992006393608,
  35.05709628591621,
  25.079872407968907,
  35.22782990761707,
  33.24154027718932,
  29.274562336608895,
  35.510561809129406,
  32.28002478313795,
  28.460498941515414,
  7.615773105863909,
  9.219544457292887,
  6.4031242374328485,
  9.433981132056603,
  13.0,
  4.242640687119285,
  9.486832980505138,
  0.0,
  11.0,
  5.830951894845301,
  7.615773105863909,
  12.36931687685298,
  9.848857801796104,
  5.830951894845301,
  15.0,
  53.33854141237835,
  40.11234224026316,
  55.57877292636101,
  55.90169943749474,
  46.87216658103186,
  45.60701700396552,
  44.40720662234904,
  45.12205669071391,
  45.60701700396552,
  42.42640687119285,
  40.792156108742276,
  41.48493702538308,
  37.107950630558946,
  38.27531841800928,
  90.35485598461214,
  88.14193099768123,
  85.98255637046388,
  84.21995013059554,
  83.64807230295268,
  84.52810183601665,
  82.76472678623425,
  86.4060183089118,
  80.95060222135473,
  78.77182237323191,
  79.17701686727027,
  80.49844718999243,
  77.38862965578342,
  75.58438992278762,
  80.62257748298549,
  76.96752561957543,
  72.44998274671983,
  63.90618123468183,
  75.02666192761077,
  72.80109889280519,
  77.25283166331187,
  79.47955712005447,
  73.37574531137656,
  75.8946638440411,
  80.65358020571685,
  79.37883848986453,
  77.4919350642375,
  76.02631123499285,
  69.06518659932803,
  65.79513659838392,
  85.42833253669417,
  85.47514258543241,
  85.44003745317531,
  76.32168761236873,
  67.53517601961218,
  69.6419413859206,
  86.09297300012354,
  77.38862965578342,
  92.70382947861431,
  89.1852005660132],
 [90.82400563727631,
  108.60018416190647,
  107.00467279516349,
  101.83319694480774,
  101.25709851659784,
  100.20977996183805,
  98.00510190801293,
  96.83491106000976,
  90.4267659490264,
  97.9081201943945,
  99.29753269845128,
  99.56907150315303,
  92.59049627256569,
  96.89685237405806,
  99.12618221237011,
  100.70253224224304,
  83.74365647617735,
  81.78630692236935,
  84.02380615040002,
  83.18653737234169,
  79.2464510246358,
  76.2954782408499,
  85.84287972802403,
  79.90619500389191,
  85.70880934886448,
  83.72574275573791,
  77.78174593052023,
  85.37564055396598,
  81.39410298049853,
  73.0,
  85.21150157109074,
  83.21658488546619,
  79.22752047110903,
  87.14355971613737,
  83.05419917138447,
  80.15609770940699,
  81.05553651663777,
  47.16990566028302,
  44.181444068749045,
  42.190046219457976,
  33.37663853655727,
  45.0111097397076,
  42.01190307520001,
  36.013886210738214,
  46.04345773288535,
  36.05551275463989,
  46.17358552246078,
  44.181444068749045,
  40.19950248448356,
  46.389654018972806,
  42.95346318982906,
  39.05124837953327,
  10.63014581273465,
  8.602325267042627,
  8.602325267042627,
  5.830951894845301,
  5.0990195135927845,
  8.54400374531753,
  3.605551275463989,
  11.0,
  0.0,
  6.708203932499369,
  5.0,
  3.1622776601683795,
  4.47213595499958,
  9.433981132056603,
  9.055385138137417,
  53.23532661682466,
  40.792156108742276,
  55.08175741568164,
  55.00909015790027,
  46.04345773288535,
  44.01136216933077,
  44.28317965096906,
  44.01136216933077,
  44.01136216933077,
  42.2965719651132,
  40.11234224026316,
  40.0,
  36.05551275463989,
  36.05551275463989,
  95.96353474106714,
  93.29523031752481,
  92.45539465060976,
  90.21086409075129,
  89.30845424706443,
  89.88882021697692,
  87.98295289429652,
  91.4166286842826,
  86.53323061113575,
  84.62860036654276,
  84.87638069569178,
  85.98255637046388,
  82.68010643437755,
  80.99382692526635,
  85.58621384311844,
  82.15229759416349,
  77.93587107359485,
  68.47627326307996,
  79.64923100695951,
  77.41446893184762,
  81.88406438373708,
  84.11896337925236,
  77.38862965578342,
  80.05623023850174,
  84.970583144992,
  83.6301381082203,
  81.43709228600933,
  82.87943049997388,
  75.82216034906945,
  71.02112361825881,
  90.7358804442873,
  91.2633551870629,
  91.8313671900838,
  82.61961026293939,
  73.23933369440222,
  74.30343195303969,
  92.20086767487604,
  83.86894538504703,
  97.6165969494942,
  93.1074647920348],
 [90.87353850269065,
  105.75916035975324,
  104.23531071570709,
  98.85848471426213,
  98.2344135219425,
  97.30878685915265,
  95.13148795220224,
  93.86160024205851,
  86.82165628459296,
  94.81033698917012,
  96.21330469326995,
  96.40020746865642,
  88.83692925805124,
  93.47726996441435,
  95.69221493935649,
  97.41663102366043,
  78.587530817554,
  76.6550715869472,
  82.92767933567175,
  77.87810988975015,
  73.97972695272672,
  71.06335201775947,
  80.4114419718985,
  74.52516353554684,
  80.23091673413684,
  78.26237921249263,
  72.3671196055225,
  79.76214641043707,
  75.8023746329889,
  67.06713054842886,
  79.51100552753688,
  77.52418977325722,
  73.55270219373317,
  81.39410298049853,
  77.23341245859851,
  74.43117626371358,
  75.23961722390672,
  41.593268686170845,
  38.63935817272331,
  36.6742416417845,
  28.160255680657446,
  39.20459156782532,
  36.22154055254967,
  30.265491900843113,
  40.01249804748511,
  30.01666203960727,
  40.01249804748511,
  38.01315561749642,
  34.0147027033899,
  40.11234224026316,
  36.49657518178932,
  32.55764119219941,
  10.198039027185569,
  10.04987562112089,
  8.06225774829855,
  8.54400374531753,
  10.63014581273465,
  6.324555320336759,
  7.211102550927978,
  5.830951894845301,
  6.708203932499369,
  0.0,
  2.0,
  7.0,
  4.123105625617661,
  2.8284271247461903,
  9.219544457292887,
  56.00892785976178,
  43.04648650006177,
  58.077534382926416,
  58.215118311311535,
  49.16299421312742,
  47.51841748206689,
  47.01063709417264,
  47.265209192385896,
  47.51841748206689,
  45.0111097397076,
  43.104524124504614,
  43.41658669218482,
  39.20459156782532,
  39.81205847478876,
  95.38343671728336,
  93.02150289046077,
  91.2633551870629,
  89.35882720806042,
  88.68483523128404,
  89.47066558375433,
  87.65842800324451,
  91.23595782365635,
  85.95929269136641,
  83.86298349093002,
  84.21995013059554,
  85.47514258543241,
  82.29823813423954,
  80.52949770115296,
  85.42833253669417,
  81.8413098624405,
  77.41446893184762,
  68.54195795277518,
  79.7057087039567,
  77.47257579298626,
  81.93900170248598,
  84.17244204607586,
  77.83315488916018,
  80.4114419718985,
  85.23496934944014,
  83.93449827097318,
  81.93289937503738,
  81.40024569987489,
  74.4043009509531,
  70.66116330771806,
  90.35485598461214,
  90.55385138137417,
  90.69729874698585,
  81.54140052758476,
  72.56031973468694,
  74.32361670424818,
  91.26883367283708,
  82.66196223173026,
  97.50897394599124,
  93.637599285757],
 [91.706052144883,
  107.29864864013899,
  105.75916035975324,
  100.42410069301094,
  99.80981915623332,
  98.85848471426213,
  96.67471230885562,
  95.42536350467836,
  88.50988645343524,
  96.40020746865642,
  97.8008179924892,
  98.00510190801293,
  90.55385138137417,
  95.13148795220224,
  97.3498844375277,
  99.04544411531506,
  80.52328855678958,
  78.587530817554,
  84.0535543567314,
  79.83107164506812,
  75.92759709091287,
  73.00684899377592,
  82.37718130647589,
  76.48529270389177,
  82.20097323024831,
  80.23091673413684,
  74.33034373659252,
  81.74350127074322,
  77.78174593052023,
  69.06518659932803,
  81.49846624323675,
  79.51100552753688,
  75.53806987208503,
  83.38465086573187,
  79.22752047110903,
  76.4198926981712,
  77.23341245859851,
  43.56604182158393,
  40.607881008493905,
  38.63935817272331,
  30.083217912982647,
  41.19465984809196,
  38.2099463490856,
  32.2490309931942,
  42.01190307520001,
  32.01562118716424,
  42.01190307520001,
  40.01249804748511,
  36.013886210738214,
  42.1070065428546,
  38.47076812334269,
  34.52535300326414,
  10.770329614269007,
  10.04987562112089,
  8.54400374531753,
  8.06225774829855,
  9.433981132056603,
  7.211102550927978,
  6.324555320336759,
  7.615773105863909,
  5.0,
  2.0,
  0.0,
  5.0,
  2.23606797749979,
  4.47213595499958,
  7.810249675906654,
  56.00892785976178,
  43.18564576337837,
  58.008620049092706,
  58.077534382926416,
  49.040799340956916,
  47.265209192385896,
  47.01063709417264,
  47.095647357266465,
  47.265209192385896,
  45.0111097397076,
  43.01162633521314,
  43.18564576337837,
  39.05124837953327,
  39.45883931389772,
  96.38464608017192,
  93.94147114027967,
  92.41753080449618,
  90.4267659490264,
  89.69392398596462,
  90.4267659490264,
  88.58893836140041,
  92.13034245024816,
  86.95401083331349,
  84.90583018850944,
  85.23496934944014,
  86.45229898620394,
  83.24061508662703,
  81.49233092751734,
  86.31338250816034,
  82.76472678623425,
  78.39005038906404,
  69.35416353759881,
  80.52949770115296,
  78.29431652425353,
  82.76472678623425,
  85.0,
  78.54934754662193,
  81.15417426109393,
  86.00581375697809,
  84.69356528095862,
  82.63776376451628,
  82.61961026293939,
  75.6042326857432,
  71.58910531638176,
  91.30169768410661,
  91.58602513484249,
  91.83681179135085,
  82.66196223173026,
  73.57309290766564,
  75.15317691222374,
  92.35799911215054,
  83.81527307120105,
  98.38699100999075,
  94.33981132056604],
 [93.94147114027967,
  111.21151019566275,
  109.63576058932597,
  104.4030650891055,
  103.81233067415451,
  102.80077820717118,
  100.60318086422517,
  99.40321926376429,
  92.78469701410896,
  100.43903623591775,
  101.83319694480774,
  102.07840124139877,
  94.89467845985885,
  99.32270636667126,
  101.54801819828883,
  103.1746092796091,
  85.37564055396598,
  83.43260753446461,
  87.00574693662483,
  84.72307831990054,
  80.8084154033477,
  77.87810988975015,
  87.29833904490967,
  81.39410298049853,
  87.13208364316786,
  85.15867542417507,
  79.2464510246358,
  86.70063436907483,
  82.73451516749222,
  74.06078584514209,
  86.46964785403026,
  84.48076704197234,
  80.50465825031493,
  88.36288813749809,
  84.2140130857092,
  81.39410298049853,
  82.21921916437786,
  48.507731342539614,
  45.5411901469428,
  43.56604182158393,
  34.92849839314596,
  46.17358552246078,
  43.18564576337837,
  37.21558813185679,
  47.01063709417264,
  37.013511046643494,
  47.01063709417264,
  45.0111097397076,
  41.012193308819754,
  47.095647357266465,
  43.41658669218482,
  39.45883931389772,
  13.45362404707371,
  11.661903789690601,
  11.313708498984761,
  8.94427190999916,
  8.0,
  10.816653826391969,
  6.708203932499369,
  12.36931687685298,
  3.1622776601683795,
  7.0,
  5.0,
  0.0,
  3.1622776601683795,
  9.219544457292887,
  6.0,
  56.32051136131489,
  43.93176527297759,
  58.137767414994535,
  58.034472514187634,
  49.09175083453431,
  47.0,
  47.38143096192854,
  47.042533945356304,
  47.0,
  45.39823785126467,
  43.18564576337837,
  43.01162633521314,
  39.11521443121589,
  39.01281840626232,
  99.02019995940222,
  96.38464608017192,
  95.42536350467836,
  93.23089616645332,
  92.35799911215054,
  92.9623579735368,
  91.0659101969557,
  94.51454914456292,
  89.58794561770016,
  87.65842800324451,
  87.92041856133307,
  89.04493247793498,
  85.75546629807339,
  84.05950273467003,
  88.68483523128404,
  85.23496934944014,
  80.99382692526635,
  71.58910531638176,
  82.76472678623425,
  80.52949770115296,
  85.0,
  87.23531395025755,
  80.52949770115296,
  83.19254774317228,
  88.10221336606703,
  86.76404785393545,
  84.581321815162,
  85.7962703152066,
  78.74642849044012,
  74.09453421137081,
  93.81364506296512,
  94.30270409696638,
  94.81033698917012,
  85.60373823613078,
  76.27581530209953,
  77.41446893184762,
  95.2102935611481,
  86.83317338436964,
  100.72239075796404,
  96.25487000666512],
 [93.47192091746055,
  109.48059188732951,
  107.93516572461452,
  102.61578825892242,
  102.00490184299969,
  101.04454463255303,
  98.85848471426213,
  97.6165969494942,
  90.7358804442873,
  98.60020283954795,
  100.0,
  100.20977996183805,
  92.78469701410896,
  97.3498844375277,
  99.56907150315303,
  101.25709851659784,
  82.71033792700887,
  80.77747210701756,
  86.02325267042627,
  82.0,
  78.10249675906654,
  75.18643494673756,
  84.52810183601665,
  78.64477096412705,
  84.34453153583817,
  82.37718130647589,
  76.48529270389177,
  83.86298349093002,
  79.90619500389191,
  71.11258679024411,
  83.60023923410746,
  81.61494961096282,
  77.64663547121665,
  85.47514258543241,
  81.30190649671138,
  78.5175139698144,
  79.30952023559341,
  45.70557952810576,
  42.7551166528639,
  40.792156108742276,
  32.28002478313795,
  43.289721643826724,
  40.311288741492746,
  34.36568055487916,
  44.04543109109048,
  34.058772731852805,
  44.0,
  42.0,
  38.0,
  44.04543109109048,
  40.311288741492746,
  36.345563690772494,
  12.529964086141668,
  11.40175425099138,
  10.295630140987,
  9.055385138137417,
  9.486832980505138,
  9.219544457292887,
  7.0,
  9.848857801796104,
  4.47213595499958,
  4.123105625617661,
  2.23606797749979,
  3.1622776601683795,
  0.0,
  6.082762530298219,
  5.830951894845301,
  57.07889277132134,
  44.40720662234904,
  59.00847396772772,
  59.00847396772772,
  50.0,
  48.093658625644196,
  48.093658625644196,
  48.010415536631214,
  48.093658625644196,
  46.09772228646444,
  44.01136216933077,
  44.04543109109048,
  40.0,
  40.19950248448356,
  98.2700361249552,
  95.77055915050303,
  94.40338976964757,
  92.35799911215054,
  91.58602513484249,
  92.28217596047462,
  90.4267659490264,
  93.94147114027967,
  88.83692925805124,
  86.82165628459296,
  87.13208364316786,
  88.32326986700618,
  85.0881895447306,
  83.35466393669883,
  88.11923740024082,
  84.59905436823747,
  80.26207074328447,
  71.11258679024411,
  82.29216244576394,
  80.05623023850174,
  84.52810183601665,
  86.76404785393545,
  80.23091673413684,
  82.85529554590944,
  87.72684879784524,
  86.4060183089118,
  84.30895563343196,
  84.64632301523794,
  77.62087348130012,
  73.43023900274328,
  93.15041599477696,
  93.49331526906082,
  93.81364506296512,
  84.62860036654276,
  75.47184905645283,
  76.92203845452875,
  94.30270409696638,
  85.80209787645055,
  100.18482919085105,
  96.00520819205592],
 [91.92388155425118,
  105.55093557141026,
  104.06248123122954,
  98.59513172565875,
  97.94896630388705,
  97.082439194738,
  94.92101980067429,
  93.60555539069249,
  86.2670273047588,
  94.49338601193207,
  95.90099061010788,
  96.04686356149273,
  88.20430828479978,
  93.00537618869137,
  95.2102935611481,
  97.0051545022222,
  77.20103626247513,
  75.28612089887484,
  83.52843827104634,
  76.40026177965623,
  72.53275122315436,
  69.6419413859206,
  78.85429601486528,
  73.00684899377592,
  78.64477096412705,
  76.68767828015137,
  70.83078426785913,
  78.08969202141856,
  74.14849964766651,
  65.19202405202648,
  77.78174593052023,
  75.8023746329889,
  71.84705978674423,
  79.63039620647382,
  75.4254598925323,
  72.69112738154499,
  73.43704787094863,
  40.024992192379,
  37.107950630558946,
  35.17101079013795,
  26.92582403567252,
  37.48332962798263,
  34.52535300326414,
  28.635642126552707,
  38.118237105091836,
  28.160255680657446,
  38.01315561749642,
  36.013886210738214,
  32.01562118716424,
  38.01315561749642,
  34.23448553724738,
  30.265491900843113,
  12.0,
  12.36931687685298,
  10.04987562112089,
  11.180339887498949,
  13.45362404707371,
  8.0,
  10.0,
  5.830951894845301,
  9.433981132056603,
  2.8284271247461903,
  4.47213595499958,
  9.219544457292887,
  6.082762530298219,
  0.0,
  9.848857801796104,
  58.077534382926416,
  45.0,
  60.207972893961475,
  60.40695324215582,
  51.35172830587107,
  49.8196748283246,
  49.09175083453431,
  49.49747468305833,
  49.8196748283246,
  47.095647357266465,
  45.27692569068709,
  45.70557952810576,
  41.43669871020132,
  42.20189569201838,
  96.17692030835673,
  93.94147114027967,
  91.80958555619343,
  90.0499861188218,
  89.47066558375433,
  90.33825324855468,
  88.56635930193812,
  92.19544457292888,
  86.76981041814025,
  84.59905436823747,
  85.0,
  86.31338250816034,
  83.19254774317228,
  81.39410298049853,
  86.4060183089118,
  82.76472678623425,
  78.26237921249263,
  69.6419413859206,
  80.77747210701756,
  78.54934754662193,
  83.00602387778854,
  85.23496934944014,
  79.05694150420949,
  81.59656855530139,
  86.37708029332781,
  85.09406559801923,
  83.16850365372699,
  81.8413098624405,
  74.88658090739622,
  71.58910531638176,
  91.23595782365635,
  91.30169768410661,
  91.26883367283708,
  82.15229759416349,
  73.35529974037323,
  75.39230729988306,
  91.92388155425118,
  83.21658488546619,
  98.48857801796105,
  94.86832980505137],
 [99.28242543370907,
  114.97825881443848,
  113.45483682946268,
  108.07404868885037,
  107.44766167767449,
  106.5269918846862,
  104.35037134576953,
  103.07764064044152,
  95.96353474106714,
  104.01922899156675,
  105.4229576515476,
  105.60303025955268,
  97.94386147176351,
  102.6498904042279,
  104.86181383134664,
  106.60675400742676,
  86.97700845625813,
  85.07055894961546,
  91.83681179135085,
  86.1278120005379,
  82.28000972289685,
  79.40403012442127,
  88.52683209061533,
  82.71033792700887,
  88.29496021857646,
  86.34813257969161,
  80.52328855678958,
  87.66413177577246,
  83.74365647617735,
  74.54528824815154,
  87.29833904490967,
  85.32877591996736,
  81.39410298049853,
  89.10667763978185,
  84.8528137423857,
  82.20097323024831,
  82.87339742040264,
  49.72926703662542,
  46.84015371452148,
  44.9221548904324,
  36.76955262170047,
  47.07440918375928,
  44.14748010928823,
  38.3275357934736,
  47.51841748206689,
  37.656340767525464,
  47.265209192385896,
  45.27692569068709,
  41.30375285612676,
  47.095647357266465,
  43.0,
  39.0,
  18.35755975068582,
  17.08800749063506,
  16.1245154965971,
  14.560219778561036,
  14.0,
  15.0,
  12.36931687685298,
  15.0,
  9.055385138137417,
  9.219544457292887,
  7.810249675906654,
  6.0,
  5.830951894845301,
  9.848857801796104,
  0.0,
  62.289646009589745,
  49.8196748283246,
  64.1248781675256,
  64.03124237432849,
  55.08175741568164,
  53.0,
  53.33854141237835,
  53.03772242470448,
  53.0,
  51.35172830587107,
  49.16299421312742,
  49.01020301937138,
  45.09988913511872,
  45.0111097397076,
  104.10091258005379,
  101.59724405711013,
  100.20977996183805,
  98.18350166906862,
  97.41663102366043,
  98.11218069128827,
  96.25487000666512,
  99.76472322419383,
  94.66784036831093,
  92.64987857520376,
  92.9623579735368,
  94.15412895885129,
  90.91754506144565,
  89.1852005660132,
  93.94147114027967,
  90.4267659490264,
  86.09297300012354,
  76.92203845452875,
  88.10221336606703,
  85.86617494683223,
  90.33825324855468,
  92.57429448826494,
  86.00581375697809,
  88.63972021616495,
  93.5200513259055,
  92.19544457292888,
  90.07774419910837,
  90.4267659490264,
  83.40863264674707,
  79.25906888173743,
  98.97979591815695,
  99.32270636667126,
  99.62429422585637,
  90.44335243676011,
  81.30190649671138,
  82.73451516749222,
  100.12492197250393,
  91.60785992479029,
  106.00471687618433,
  101.76934705499491],
 [46.61544808322666,
  82.87339742040264,
  80.89499366462674,
  78.23042886243178,
  78.16009211867653,
  76.23647421018367,
  74.16872656315464,
  74.06078584514209,
  74.54528824815154,
  76.02631123499285,
  77.05841939723393,
  78.02563681252464,
  77.64663547121665,
  78.05767098754612,
  80.02499609497022,
  80.00624975587844,
  85.0,
  83.19254774317228,
  48.76474136094644,
  86.31338250816034,
  82.76472678623425,
  80.1311425102625,
  89.89438247187641,
  84.59905436823747,
  90.35485598461214,
  88.58893836140041,
  83.35466393669883,
  91.7877987534291,
  88.32326986700618,
  86.21484790916237,
  92.78469701410896,
  91.0659101969557,
  87.66413177577246,
  95.01578816175763,
  92.64987857520376,
  89.04493247793498,
  90.97252332435328,
  64.53681120105021,
  62.625873247404705,
  61.40032573203501,
  55.569775957799216,
  65.60487786742691,
  63.8200595424354,
  60.53924347066124,
  68.60029154456998,
  63.13477647065839,
  70.21395872616783,
  69.06518659932803,
  66.88796603276258,
  71.84705978674423,
  72.20110802473879,
  70.2353187506115,
  46.09772228646444,
  46.0,
  48.041648597857254,
  48.041648597857254,
  48.373546489791295,
  50.08991914547278,
  50.08991914547278,
  53.33854141237835,
  53.23532661682466,
  56.00892785976178,
  56.00892785976178,
  56.32051136131489,
  57.07889277132134,
  58.077534382926416,
  62.289646009589745,
  0.0,
  13.341664064126334,
  2.8284271247461903,
  4.47213595499958,
  7.615773105863909,
  10.816653826391969,
  9.0,
  9.848857801796104,
  10.816653826391969,
  11.0,
  13.152946437965905,
  13.92838827718412,
  17.26267650163207,
  18.384776310850235,
  55.072679252057455,
  50.774009099144415,
  56.302753041036986,
  52.0,
  49.658836071740545,
  48.76474136094644,
  46.52956049652737,
  48.54894437575342,
  47.01063709417264,
  46.8187996428785,
  46.238512086787566,
  46.09772228646444,
  42.5440947723653,
  41.78516483155236,
  43.41658669218482,
  41.6293165929973,
  39.84971769034255,
  28.861739379323623,
  37.12142238654117,
  35.34119409414458,
  38.948684188300895,
  40.8166632639171,
  32.7566787083184,
  35.510561809129406,
  40.22437072223753,
  38.8329756778952,
  35.805027579936315,
  50.80354318352215,
  45.17742799230607,
  33.734255586866,
  49.24428900898052,
  51.894122981316485,
  55.362442142665635,
  47.53945729601885,
  37.656340767525464,
  33.24154027718932,
  54.230987451824994,
  49.47726750741192,
  53.600373133029585,
  45.967379738244816],
 [53.150729063673246,
  82.86132994346639,
  80.95677859203639,
  77.3692445355388,
  77.12976079309465,
  75.43208866258443,
  73.24616030891995,
  72.78049189171504,
  71.11258679024411,
  74.52516353554684,
  75.7099729229908,
  76.48529270389177,
  74.06078584514209,
  75.66372975210778,
  77.78174593052023,
  78.26237921249263,
  77.4919350642375,
  75.58438992278762,
  51.10772935672255,
  78.43468620451031,
  74.67261881037788,
  71.8679344353238,
  81.93289937503738,
  76.32168761236873,
  82.28000972289685,
  80.4114419718985,
  74.84650960465692,
  83.38465086573187,
  79.7057087039567,
  76.32168761236873,
  84.17244204607586,
  82.34682750416071,
  78.7210264160726,
  86.4060183089118,
  83.6301381082203,
  80.05623023850174,
  81.8413098624405,
  53.075418038862395,
  50.91168824543142,
  49.51767361255979,
  43.01162633521314,
  53.75872022286245,
  51.73973328110612,
  48.010415536631214,
  56.639209034025185,
  50.47771785649585,
  58.137767414994535,
  56.85068161420758,
  54.405882034941776,
  59.665735560705194,
  59.64059020499378,
  57.4543296888929,
  33.0,
  33.13608305156178,
  35.014282800023196,
  35.35533905932738,
  36.138621999185304,
  37.0,
  37.48332962798263,
  40.11234224026316,
  40.792156108742276,
  43.04648650006177,
  43.18564576337837,
  43.93176527297759,
  44.40720662234904,
  45.0,
  49.8196748283246,
  13.341664064126334,
  0.0,
  15.811388300841896,
  16.55294535724685,
  8.48528137423857,
  9.848857801796104,
  5.0,
  8.06225774829855,
  9.848857801796104,
  3.605551275463989,
  5.0,
  8.0,
  7.211102550927978,
  10.770329614269007,
  60.207972893961475,
  56.568542494923804,
  59.36328831862332,
  55.80322571321482,
  54.037024344425184,
  53.81449618829484,
  51.66236541235796,
  54.45181356024793,
  51.22499389946279,
  50.21951811795888,
  50.0,
  50.44799302251776,
  46.861498055439924,
  45.60701700396552,
  48.79549159502341,
  46.09772228646444,
  43.01162633521314,
  32.01562118716424,
  42.42640687119285,
  40.311288741492746,
  44.553338819890925,
  46.69047011971501,
  39.05124837953327,
  41.86884283091664,
  46.861498055439924,
  45.45327270945405,
  42.80186911806539,
  51.894122981316485,
  45.31004303683677,
  36.05551275463989,
  54.48853090330111,
  56.22277118748239,
  58.52349955359813,
  49.73932046178355,
  39.44616584663204,
  37.53664875824692,
  58.008620049092706,
  51.478150704935004,
  60.207972893961475,
  54.08326913195984],
 [47.16990566028302,
  84.5931439302264,
  82.60750571225353,
  80.09993757800315,
  80.05623023850174,
  78.10249675906654,
  76.05918747922567,
  76.00657866263946,
  76.79192665899195,
  78.0,
  79.00632886041473,
  80.0,
  79.90619500389191,
  80.15609770940699,
  82.09750300709517,
  82.00609733428362,
  87.66413177577246,
  85.86617494683223,
  50.21951811795888,
  89.00561780022652,
  85.47514258543241,
  82.85529554590944,
  92.59049627256569,
  87.32124598286491,
  93.05912099305473,
  91.30169768410661,
  86.09297300012354,
  94.51454914456292,
  91.0659101969557,
  89.02246907382428,
  95.524865872714,
  93.81364506296512,
  90.4267659490264,
  97.75479527879949,
  95.4148835350125,
  91.80958555619343,
  93.74433316206373,
  67.35725647619564,
  65.43699259593154,
  64.20280367709809,
  58.309518948453004,
  68.41052550594829,
  66.61080993352356,
  63.28506932918696,
  71.40028011149536,
  65.86349520030045,
  73.00684899377592,
  71.84705978674423,
  69.6419413859206,
  74.63243262818116,
  74.94664769020693,
  72.94518489934754,
  48.25971404805462,
  48.041648597857254,
  50.15974481593781,
  50.0,
  50.15974481593781,
  52.23983154643591,
  52.009614495783374,
  55.57877292636101,
  55.08175741568164,
  58.077534382926416,
  58.008620049092706,
  58.137767414994535,
  59.00847396772772,
  60.207972893961475,
  64.1248781675256,
  2.8284271247461903,
  15.811388300841896,
  0.0,
  2.0,
  9.055385138137417,
  11.704699910719626,
  11.180339887498949,
  11.180339887498949,
  11.704699910719626,
  13.152946437965905,
  15.0,
  15.297058540778355,
  19.026297590440446,
  19.6468827043885,
  55.90169943749474,
  51.478150704935004,
  57.56735185849702,
  53.14132102234569,
  50.695167422546305,
  49.658836071740545,
  47.4236228055175,
  49.24428900898052,
  48.104053883222775,
  48.08326112068523,
  47.43416490252569,
  47.16990566028302,
  43.657759905886145,
  43.01162633521314,
  44.28317965096906,
  42.720018726587654,
  41.23105625617661,
  30.4138126514911,
  38.07886552931954,
  36.40054944640259,
  39.81205847478876,
  41.593268686170845,
  33.54101966249684,
  36.235341863986875,
  40.8166632639171,
  39.44616584663204,
  36.359317925395686,
  52.46903848937962,
  47.042533945356304,
  35.35533905932738,
  50.08991914547278,
  52.92447448959697,
  56.61271941887264,
  49.03060268852505,
  39.319206502675,
  34.48187929913333,
  55.362442142665635,
  50.99019513592785,
  54.08326913195984,
  46.09772228646444],
 [48.877397639399746,
  86.57944328765345,
  84.5931439302264,
  82.09750300709517,
  82.05485969764375,
  80.09993757800315,
  78.05767098754612,
  78.00640999302557,
  78.77182237323191,
  80.0,
  81.00617260431454,
  82.0,
  81.88406438373708,
  82.15229759416349,
  84.09518416651456,
  84.00595217006948,
  89.47066558375433,
  87.66413177577246,
  52.172789842982326,
  90.78546139112804,
  87.23531395025755,
  84.59905436823747,
  94.36630754670864,
  89.06739021662193,
  94.82615672903758,
  93.05912099305473,
  87.8179936004006,
  96.25487000666512,
  92.78469701410896,
  90.60353193998566,
  97.24710792614863,
  95.524865872714,
  92.11405973031478,
  99.4786409235671,
  97.09788875150684,
  93.49331526906082,
  95.4148835350125,
  68.6804193347711,
  66.70832032063167,
  65.43699259593154,
  59.36328831862332,
  69.65629906907199,
  67.80117993073571,
  64.35060217278468,
  72.62231062146122,
  66.88796603276258,
  74.2024258363566,
  73.00684899377592,
  70.7248188403477,
  75.8023746329889,
  76.00657866263946,
  73.92563831310488,
  48.507731342539614,
  48.16637831516918,
  50.35871324805669,
  50.039984012787215,
  50.039984012787215,
  52.46903848937962,
  52.009614495783374,
  55.90169943749474,
  55.00909015790027,
  58.215118311311535,
  58.077534382926416,
  58.034472514187634,
  59.00847396772772,
  60.40695324215582,
  64.03124237432849,
  4.47213595499958,
  16.55294535724685,
  2.0,
  0.0,
  9.055385138137417,
  11.180339887498949,
  11.704699910719626,
  11.0,
  11.180339887498949,
  13.601470508735444,
  15.132745950421556,
  15.033296378372908,
  19.026297590440446,
  19.235384061671343,
  57.697486947006624,
  53.23532661682466,
  59.481089431852205,
  55.02726596879042,
  52.55473337388365,
  51.478150704935004,
  49.24428900898052,
  51.0,
  49.9799959983992,
  50.0,
  49.33558553417604,
  49.040799340956916,
  45.5411901469428,
  44.9221548904324,
  46.09772228646444,
  44.598206241955516,
  43.174066289845804,
  32.38826948140329,
  39.92492955535426,
  38.27531841800928,
  41.6293165929973,
  43.382023926967726,
  35.34119409414458,
  38.01315561749642,
  42.5440947723653,
  41.182520563948,
  38.07886552931954,
  54.45181356024793,
  49.040799340956916,
  37.33630940518894,
  51.894122981316485,
  54.78138369920935,
  58.52349955359813,
  50.99019513592785,
  41.30375285612676,
  36.40054944640259,
  57.245087125446844,
  52.952809179494906,
  55.758407437802596,
  47.634021455258214],
 [53.2634959423431,
  87.0976463516667,
  85.14693182963201,
  82.03657720797473,
  81.88406438373708,
  80.06247560499239,
  77.92945527847606,
  77.64663547121665,
  77.02596964660685,
  79.51100552753688,
  80.62257748298549,
  81.49846624323675,
  80.05623023850174,
  81.09870529176159,
  83.15046602394962,
  83.38465086573187,
  85.09406559801923,
  83.21658488546619,
  53.85164807134504,
  86.16263691415206,
  82.46211251235322,
  79.7057087039567,
  89.69392398596462,
  84.17244204607586,
  90.07774419910837,
  88.23831367382311,
  82.76472678623425,
  91.28526715741155,
  87.66413177577246,
  84.59905436823747,
  92.13576938409967,
  90.33825324855468,
  86.76981041814025,
  94.37160589923221,
  91.706052144883,
  88.11923740024082,
  89.94442728707543,
  61.554853586049575,
  59.39696961966999,
  58.0,
  51.40038910358559,
  62.24146527838174,
  60.207972893961475,
  56.4003546088143,
  65.11528238439882,
  58.82176467941097,
  66.60330322138685,
  65.29931086925804,
  62.80127387243033,
  68.11754546370561,
  68.00735254367721,
  65.73431371817918,
  39.45883931389772,
  39.11521443121589,
  41.30375285612676,
  41.012193308819754,
  41.10960958218893,
  43.41658669218482,
  43.0,
  46.87216658103186,
  46.04345773288535,
  49.16299421312742,
  49.040799340956916,
  49.09175083453431,
  50.0,
  51.35172830587107,
  55.08175741568164,
  7.615773105863909,
  8.48528137423857,
  9.055385138137417,
  9.055385138137417,
  0.0,
  3.605551275463989,
  3.605551275463989,
  2.23606797749979,
  3.605551275463989,
  5.0,
  6.082762530298219,
  6.324555320336759,
  10.0,
  10.770329614269007,
  61.29437168288782,
  57.201398584300364,
  61.741396161732524,
  57.706152185014034,
  55.569775957799216,
  54.91812087098393,
  52.69724850502159,
  55.0,
  52.839379254491625,
  52.32590180780452,
  51.884487084291386,
  51.97114584074513,
  48.373546489791295,
  47.41307836451879,
  49.64876634922564,
  47.50789408087881,
  45.18849411078001,
  34.0147027033899,
  43.266615305567875,
  41.340053217188775,
  45.221676218380054,
  47.20169488482379,
  39.20459156782532,
  42.01190307520001,
  46.861498055439924,
  45.45327270945405,
  42.5205832509386,
  55.362442142665635,
  49.24428900898052,
  38.62641583165593,
  55.47071299343465,
  57.8013840664737,
  60.83584469702052,
  52.55473337388365,
  42.37924020083418,
  38.948684188300895,
  59.941638282582836,
  54.42425929675111,
  60.30754513325841,
  53.075418038862395],
 [56.85068161420758,
  90.47098982546837,
  88.52683209061533,
  85.32877591996736,
  85.15867542417507,
  83.3606621854697,
  81.21576201698781,
  80.89499366462674,
  80.0,
  82.73451516749222,
  83.86298349093002,
  84.71717653463199,
  83.00602387778854,
  84.2140130857092,
  86.28441342444185,
  86.57944328765345,
  87.28115489611719,
  85.38149682454625,
  57.38466694161429,
  88.25531145489205,
  84.50443775329198,
  81.70679286326198,
  91.76055797563569,
  86.16263691415206,
  92.11405973031478,
  90.24965373894794,
  84.69356528095862,
  93.23089616645332,
  89.55445270895244,
  86.09297300012354,
  94.02127418834527,
  92.19544457292888,
  88.56635930193812,
  96.25487000666512,
  93.47192091746055,
  89.89994438263018,
  91.67878707749138,
  62.48199740725323,
  60.207972893961475,
  58.728187440104094,
  51.73973328110612,
  62.96824596572466,
  60.81118318204309,
  56.72741841473134,
  65.76473218982953,
  59.033888572581766,
  67.17886572427373,
  65.79513659838392,
  63.12685640834652,
  68.62215385719105,
  68.24954212300622,
  65.80273550544841,
  38.07886552931954,
  37.48332962798263,
  39.81205847478876,
  39.20459156782532,
  39.0,
  41.97618372363071,
  41.10960958218893,
  45.60701700396552,
  44.01136216933077,
  47.51841748206689,
  47.265209192385896,
  47.0,
  48.093658625644196,
  49.8196748283246,
  53.0,
  10.816653826391969,
  9.848857801796104,
  11.704699910719626,
  11.180339887498949,
  3.605551275463989,
  0.0,
  6.0,
  2.0,
  0.0,
  6.324555320336759,
  5.656854249492381,
  4.123105625617661,
  8.54400374531753,
  8.06225774829855,
  64.89992295835181,
  60.802960454241045,
  65.30696746902278,
  61.29437168288782,
  59.16924876994806,
  58.52349955359813,
  56.302753041036986,
  58.60034129593445,
  56.43580423808985,
  55.90169943749474,
  55.47071299343465,
  55.569775957799216,
  51.97114584074513,
  51.0,
  53.25410782277739,
  51.10772935672255,
  48.75448697299562,
  37.57658845611187,
  46.87216658103186,
  44.94441010848846,
  48.82622246293481,
  50.80354318352215,
  42.80186911806539,
  45.60701700396552,
  50.44799302251776,
  49.040799340956916,
  46.09772228646444,
  58.82176467941097,
  52.630789467763066,
  42.15447781671598,
  59.07622195096772,
  61.40032573203501,
  64.40496875241847,
  56.08029957123981,
  45.880278987817846,
  42.5440947723653,
  63.52952069707436,
  57.9396237474839,
  63.89053137985315,
  56.586217403180434],
 [52.3450093132096,
  84.64632301523794,
  82.71033792700887,
  79.42921376924235,
  79.2464510246358,
  77.46612162745726,
  75.31268153505093,
  74.96665925596525,
  74.0,
  76.79192665899195,
  77.92945527847606,
  78.77182237323191,
  77.00649323271382,
  78.23042886243178,
  80.30566605165541,
  80.62257748298549,
  81.59656855530139,
  79.71198153351854,
  51.894122981316485,
  82.63776376451628,
  78.92401408950256,
  76.15773105863909,
  86.16263691415206,
  80.62257748298549,
  86.53900854527974,
  84.69356528095862,
  79.20227269466452,
  87.72684879784524,
  84.09518416651456,
  80.99382692526635,
  88.56635930193812,
  86.76404785393545,
  83.18653737234169,
  90.80198235721508,
  88.11923740024082,
  84.53401682163222,
  86.35392289873113,
  58.0,
  55.86591089385369,
  54.48853090330111,
  48.010415536631214,
  58.728187440104094,
  56.72741841473134,
  53.009433122794285,
  61.61980201201558,
  55.47071299343465,
  63.12685640834652,
  61.84658438426491,
  59.405386961116584,
  64.66065264130884,
  64.63745044476924,
  62.433965115151864,
  37.12142238654117,
  37.0,
  39.05124837953327,
  39.05124837953327,
  39.45883931389772,
  41.10960958218893,
  41.10960958218893,
  44.40720662234904,
  44.28317965096906,
  47.01063709417264,
  47.01063709417264,
  47.38143096192854,
  48.093658625644196,
  49.09175083453431,
  53.33854141237835,
  9.0,
  5.0,
  11.180339887498949,
  11.704699910719626,
  3.605551275463989,
  6.0,
  0.0,
  4.0,
  6.0,
  2.0,
  4.47213595499958,
  6.4031242374328485,
  8.54400374531753,
  10.63014581273465,
  60.0,
  56.08029957123981,
  59.941638282582836,
  56.08029957123981,
  54.08326913195984,
  53.600373133029585,
  51.40038910358559,
  53.907327887774215,
  51.31276644267,
  50.60632371551998,
  50.24937810560445,
  50.47771785649585,
  46.87216658103186,
  45.79301256742124,
  48.41487374764082,
  46.04345773288535,
  43.41658669218482,
  32.2490309931942,
  42.01190307520001,
  40.0,
  44.04543109109048,
  46.09772228646444,
  38.2099463490856,
  41.036569057366385,
  45.967379738244816,
  44.553338819890925,
  41.72529209005013,
  53.14132102234569,
  46.84015371452148,
  36.6742416417845,
  54.20332093147061,
  56.302753041036986,
  59.0592922409336,
  50.566787519082126,
  40.311288741492746,
  37.44329045369811,
  58.309518948453004,
  52.392747589718944,
  59.413803110051795,
  52.630789467763066],
 [55.31726674375732,
  88.52683209061533,
  86.58521813797087,
  83.3606621854697,
  83.18653737234169,
  81.39410298049853,
  79.2464510246358,
  78.91767862779544,
  78.0,
  80.75270893288967,
  81.88406438373708,
  82.73451516749222,
  81.00617260431454,
  82.21921916437786,
  84.29116205154607,
  84.5931439302264,
  85.38149682454625,
  83.48652585896721,
  55.54277630799526,
  86.37708029332781,
  82.63776376451628,
  79.84985911070852,
  89.88882021697692,
  84.30895563343196,
  90.24965373894794,
  88.3911760301898,
  82.85529554590944,
  91.3892772703669,
  87.72684879784524,
  84.38009243891595,
  92.19544457292888,
  90.37698822156003,
  86.76404785393545,
  94.42986815621421,
  91.67878707749138,
  88.10221336606703,
  89.89438247187641,
  60.959002616512684,
  58.728187440104094,
  57.28001396647874,
  50.44799302251776,
  61.5223536610881,
  59.413803110051795,
  55.44366510251645,
  64.35060217278468,
  57.8013840664737,
  65.79513659838392,
  64.4437739428721,
  61.84658438426491,
  67.26812023536856,
  67.00746227100382,
  64.63745044476924,
  37.656340767525464,
  37.21558813185679,
  39.45883931389772,
  39.05124837953327,
  39.05124837953327,
  41.593268686170845,
  41.012193308819754,
  45.12205669071391,
  44.01136216933077,
  47.265209192385896,
  47.095647357266465,
  47.042533945356304,
  48.010415536631214,
  49.49747468305833,
  53.03772242470448,
  9.848857801796104,
  8.06225774829855,
  11.180339887498949,
  11.0,
  2.23606797749979,
  2.0,
  4.0,
  0.0,
  2.0,
  4.47213595499958,
  4.47213595499958,
  4.123105625617661,
  8.06225774829855,
  8.54400374531753,
  63.245553203367585,
  59.20304046246274,
  63.50590523722971,
  59.53990258641679,
  57.4543296888929,
  56.859475903318,
  54.644304369257,
  57.0087712549569,
  54.70831746635972,
  54.120236510939236,
  53.71219600798314,
  53.85164807134504,
  50.24937810560445,
  49.24428900898052,
  51.61395160225576,
  49.39635614091387,
  46.95742752749558,
  35.77708763999664,
  45.221676218380054,
  43.266615305567875,
  47.20169488482379,
  49.20365840057018,
  41.23105625617661,
  44.04543109109048,
  48.91829923454004,
  47.50789408087881,
  44.598206241955516,
  56.92099788303083,
  50.695167422546305,
  40.311288741492746,
  57.42821606144492,
  59.682493245507096,
  62.609903369994115,
  54.230987451824994,
  44.01136216933077,
  40.8166632639171,
  61.773780845922005,
  56.08029957123981,
  62.369864518050704,
  55.226805085936306],
 [56.85068161420758,
  90.47098982546837,
  88.52683209061533,
  85.32877591996736,
  85.15867542417507,
  83.3606621854697,
  81.21576201698781,
  80.89499366462674,
  80.0,
  82.73451516749222,
  83.86298349093002,
  84.71717653463199,
  83.00602387778854,
  84.2140130857092,
  86.28441342444185,
  86.57944328765345,
  87.28115489611719,
  85.38149682454625,
  57.38466694161429,
  88.25531145489205,
  84.50443775329198,
  81.70679286326198,
  91.76055797563569,
  86.16263691415206,
  92.11405973031478,
  90.24965373894794,
  84.69356528095862,
  93.23089616645332,
  89.55445270895244,
  86.09297300012354,
  94.02127418834527,
  92.19544457292888,
  88.56635930193812,
  96.25487000666512,
  93.47192091746055,
  89.89994438263018,
  91.67878707749138,
  62.48199740725323,
  60.207972893961475,
  58.728187440104094,
  51.73973328110612,
  62.96824596572466,
  60.81118318204309,
  56.72741841473134,
  65.76473218982953,
  59.033888572581766,
  67.17886572427373,
  65.79513659838392,
  63.12685640834652,
  68.62215385719105,
  68.24954212300622,
  65.80273550544841,
  38.07886552931954,
  37.48332962798263,
  39.81205847478876,
  39.20459156782532,
  39.0,
  41.97618372363071,
  41.10960958218893,
  45.60701700396552,
  44.01136216933077,
  47.51841748206689,
  47.265209192385896,
  47.0,
  48.093658625644196,
  49.8196748283246,
  53.0,
  10.816653826391969,
  9.848857801796104,
  11.704699910719626,
  11.180339887498949,
  3.605551275463989,
  0.0,
  6.0,
  2.0,
  0.0,
  6.324555320336759,
  5.656854249492381,
  4.123105625617661,
  8.54400374531753,
  8.06225774829855,
  64.89992295835181,
  60.802960454241045,
  65.30696746902278,
  61.29437168288782,
  59.16924876994806,
  58.52349955359813,
  56.302753041036986,
  58.60034129593445,
  56.43580423808985,
  55.90169943749474,
  55.47071299343465,
  55.569775957799216,
  51.97114584074513,
  51.0,
  53.25410782277739,
  51.10772935672255,
  48.75448697299562,
  37.57658845611187,
  46.87216658103186,
  44.94441010848846,
  48.82622246293481,
  50.80354318352215,
  42.80186911806539,
  45.60701700396552,
  50.44799302251776,
  49.040799340956916,
  46.09772228646444,
  58.82176467941097,
  52.630789467763066,
  42.15447781671598,
  59.07622195096772,
  61.40032573203501,
  64.40496875241847,
  56.08029957123981,
  45.880278987817846,
  42.5440947723653,
  63.52952069707436,
  57.9396237474839,
  63.89053137985315,
  56.586217403180434],
 [53.74011537017761,
  85.16454661418682,
  83.24061508662703,
  79.83107164506812,
  79.62411694957753,
  77.87810988975015,
  75.7099729229908,
  75.31268153505093,
  74.02702209328699,
  77.10382610480494,
  78.26237921249263,
  79.07591289387686,
  77.00649323271382,
  78.40918313565064,
  80.50465825031493,
  80.89499366462674,
  80.95677859203639,
  79.05694150420949,
  52.773099207835045,
  81.93289937503738,
  78.18567643756751,
  75.39230729988306,
  85.44003745317531,
  79.84985911070852,
  85.7962703152066,
  83.93449827097318,
  78.39005038906404,
  86.92525524840292,
  83.2586331859946,
  79.92496481075234,
  87.72684879784524,
  85.90692637965812,
  82.29216244576394,
  89.96110270555825,
  87.20665112249179,
  83.6301381082203,
  85.42247947700886,
  56.639209034025185,
  54.45181356024793,
  53.03772242470448,
  46.400431032480725,
  57.28001396647874,
  55.226805085936306,
  51.40038910358559,
  60.14149981501958,
  53.823786563191554,
  61.61980201201558,
  60.30754513325841,
  57.8013840664737,
  63.12685640834652,
  63.00793600809346,
  60.74537019394976,
  35.12833614050059,
  35.0,
  37.05401462729781,
  37.05401462729781,
  37.48332962798263,
  39.11521443121589,
  39.11521443121589,
  42.42640687119285,
  42.2965719651132,
  45.0111097397076,
  45.0111097397076,
  45.39823785126467,
  46.09772228646444,
  47.095647357266465,
  51.35172830587107,
  11.0,
  3.605551275463989,
  13.152946437965905,
  13.601470508735444,
  5.0,
  6.324555320336759,
  2.0,
  4.47213595499958,
  6.324555320336759,
  0.0,
  2.8284271247461903,
  5.385164807134504,
  6.708203932499369,
  9.219544457292887,
  61.220911460055866,
  57.38466694161429,
  60.90155991434045,
  57.14017850864661,
  55.21775076911409,
  54.817880294662984,
  52.630789467763066,
  55.226805085936306,
  52.43090691567332,
  51.62363799656123,
  51.31276644267,
  51.61395160225576,
  48.010415536631214,
  46.87216658103186,
  49.678969393496885,
  47.20169488482379,
  44.41846462902562,
  33.28663395418648,
  43.278170016764804,
  41.23105625617661,
  45.34313619501854,
  47.4236228055175,
  39.59797974644666,
  42.42640687119285,
  47.38143096192854,
  45.967379738244816,
  43.18564576337837,
  53.85164807134504,
  47.43416490252569,
  37.589892258425,
  55.44366510251645,
  57.42821606144492,
  60.03332407921454,
  51.42956348249516,
  41.14608122288197,
  38.600518131237564,
  59.36328831862332,
  53.225933528685054,
  60.81118318204309,
  54.20332093147061],
 [56.568542494923804,
  87.64131445842195,
  85.72630868059116,
  82.22530024268686,
  82.0,
  80.2807573457052,
  78.10249675906654,
  77.6659513557904,
  76.10519036176179,
  79.42921376924235,
  80.60397012554654,
  81.39410298049853,
  79.05694150420949,
  80.62257748298549,
  82.73451516749222,
  83.18653737234169,
  82.28000972289685,
  80.36168241145776,
  55.47071299343465,
  83.16850365372699,
  79.37883848986453,
  76.55063683601855,
  86.64871608973787,
  80.99382692526635,
  86.97700845625813,
  85.09406559801923,
  79.47955712005447,
  88.02272433866155,
  84.30895563343196,
  80.62257748298549,
  88.76936408468859,
  86.92525524840292,
  83.2586331859946,
  91.0,
  88.14193099768123,
  84.581321815162,
  86.33075929238663,
  56.85068161420758,
  54.56189146281496,
  53.075418038862395,
  46.09772228646444,
  57.3149195236284,
  55.154328932550705,
  51.088159097779204,
  60.108235708594876,
  53.41348144429457,
  61.5223536610881,
  60.14149981501958,
  57.48912940721924,
  62.96824596572466,
  62.625873247404705,
  60.21627686929839,
  33.37663853655727,
  33.06055050963308,
  35.22782990761707,
  35.0,
  35.22782990761707,
  37.33630940518894,
  37.013511046643494,
  40.792156108742276,
  40.11234224026316,
  43.104524124504614,
  43.01162633521314,
  43.18564576337837,
  44.01136216933077,
  45.27692569068709,
  49.16299421312742,
  13.152946437965905,
  5.0,
  15.0,
  15.132745950421556,
  6.082762530298219,
  5.656854249492381,
  4.47213595499958,
  4.47213595499958,
  5.656854249492381,
  2.8284271247461903,
  0.0,
  3.0,
  4.123105625617661,
  6.4031242374328485,
  64.03124237432849,
  60.207972893961475,
  63.631753079732135,
  59.90826320300064,
  58.008620049092706,
  57.62811813689564,
  55.44366510251645,
  58.05170109479997,
  55.21775076911409,
  54.378304497290095,
  54.08326913195984,
  54.405882034941776,
  50.80354318352215,
  49.64876634922564,
  52.49761899362675,
  50.0,
  47.16990566028302,
  36.05551275463989,
  46.09772228646444,
  44.04543109109048,
  48.16637831516918,
  50.24937810560445,
  42.42640687119285,
  45.254833995939045,
  50.20956084253277,
  48.79549159502341,
  46.010868281309364,
  56.462376853972415,
  49.9799959983992,
  40.311288741492746,
  58.25804665451803,
  60.21627686929839,
  62.76941930590086,
  54.120236510939236,
  43.829214001622255,
  41.400483088968905,
  62.12889826803627,
  55.90169943749474,
  63.63961030678928,
  57.0087712549569],
 [58.728187440104094,
  90.52071586106685,
  88.60022573334675,
  85.14693182963201,
  84.92938243034621,
  83.19855767980596,
  81.02468759581859,
  80.60397012554654,
  79.1012010022604,
  82.37718130647589,
  83.54639429682169,
  84.34453153583817,
  82.05485969764375,
  83.60023923410746,
  85.70880934886448,
  86.14522621712709,
  85.16454661418682,
  83.24061508662703,
  58.137767414994535,
  86.02325267042627,
  82.21921916437786,
  79.37883848986453,
  89.49301648732151,
  83.81527307120105,
  89.81091247727082,
  87.92041856133307,
  82.28000972289685,
  90.82400563727631,
  87.09190547921202,
  83.24061508662703,
  91.54780172128658,
  89.69392398596462,
  86.00581375697809,
  93.77632963600144,
  90.87353850269065,
  87.32124598286491,
  89.05054744357274,
  59.20304046246274,
  56.85068161420758,
  55.31726674375732,
  48.104053883222775,
  59.54829972383762,
  57.3149195236284,
  53.075418038862395,
  62.289646009589745,
  55.31726674375732,
  63.655321851358195,
  62.22539674441618,
  59.464274989274024,
  65.05382386916237,
  64.53681120105021,
  62.00806399170998,
  33.95585369269929,
  33.37663853655727,
  35.6931365951495,
  35.12833614050059,
  35.014282800023196,
  37.8549864614954,
  37.05401462729781,
  41.48493702538308,
  40.0,
  43.41658669218482,
  43.18564576337837,
  43.01162633521314,
  44.04543109109048,
  45.70557952810576,
  49.01020301937138,
  13.92838827718412,
  8.0,
  15.297058540778355,
  15.033296378372908,
  6.324555320336759,
  4.123105625617661,
  6.4031242374328485,
  4.123105625617661,
  4.123105625617661,
  5.385164807134504,
  3.0,
  0.0,
  4.47213595499958,
  4.47213595499958,
  66.40030120413611,
  62.48199740725323,
  66.24198064671678,
  62.433965115151864,
  60.4648658313239,
  60.0,
  57.8013840664737,
  60.30754513325841,
  57.688820407423826,
  56.938563381947034,
  56.60388679233962,
  56.859475903318,
  53.25410782277739,
  52.15361924162119,
  54.817880294662984,
  52.43090691567332,
  49.73932046178355,
  38.58756276314948,
  48.41487374764082,
  46.400431032480725,
  50.44799302251776,
  52.49761899362675,
  44.598206241955516,
  47.4236228055175,
  52.3450093132096,
  50.93132631298737,
  48.08326112068523,
  59.23681287847955,
  52.81098370604357,
  42.941821107167776,
  60.60528029800704,
  62.68173577685928,
  65.36818798161687,
  56.79788728465171,
  46.51881339845203,
  43.829214001622255,
  64.66065264130884,
  58.60034129593445,
  65.79513659838392,
  58.89821729050889],
 [60.14149981501958,
  89.81091247727082,
  87.92041856133307,
  84.20213774008353,
  83.93449827097318,
  82.28000972289685,
  80.0812087820857,
  79.55501241279521,
  77.41446893184762,
  81.25269226308751,
  82.46211251235322,
  83.19855767980596,
  80.30566605165541,
  82.20097323024831,
  84.34453153583817,
  84.92938243034621,
  82.22530024268686,
  80.2807573457052,
  58.309518948453004,
  82.96987405076617,
  79.12016177940993,
  76.24303246854758,
  86.40023148117139,
  80.65358020571685,
  86.68333173107735,
  84.77027781009096,
  79.05694150420949,
  87.5956619930462,
  83.81527307120105,
  79.60527620704548,
  88.25531145489205,
  86.37708029332781,
  82.63776376451628,
  90.47651629014017,
  87.46427842267951,
  83.93449827097318,
  85.61541917201598,
  55.21775076911409,
  52.80151512977634,
  51.22499389946279,
  43.840620433565945,
  55.44366510251645,
  53.150729063673246,
  48.79549159502341,
  58.137767414994535,
  50.99019513592785,
  59.464274989274024,
  58.0,
  55.17245689653489,
  60.8276253029822,
  60.207972893961475,
  57.62811813689564,
  29.614185789921695,
  29.154759474226502,
  31.400636936215164,
  31.016124838541646,
  31.144823004794873,
  33.54101966249684,
  33.0,
  37.107950630558946,
  36.05551275463989,
  39.20459156782532,
  39.05124837953327,
  39.11521443121589,
  40.0,
  41.43669871020132,
  45.09988913511872,
  17.26267650163207,
  7.211102550927978,
  19.026297590440446,
  19.026297590440446,
  10.0,
  8.54400374531753,
  8.54400374531753,
  8.06225774829855,
  8.54400374531753,
  6.708203932499369,
  4.123105625617661,
  4.47213595499958,
  0.0,
  4.0,
  67.35725647619564,
  63.655321851358195,
  66.57326790837296,
  63.00793600809346,
  61.220911460055866,
  60.959002616512684,
  58.7962583843564,
  61.5223536610881,
  58.412327466040935,
  57.42821606144492,
  57.201398584300364,
  57.62811813689564,
  54.037024344425184,
  52.80151512977634,
  55.90169943749474,
  53.2634959423431,
  50.21951811795888,
  39.20459156782532,
  49.51767361255979,
  47.4236228055175,
  51.62363799656123,
  53.74011537017761,
  46.010868281309364,
  48.83646178829912,
  53.81449618829484,
  52.40229002629561,
  49.678969393496885,
  59.033888572581766,
  52.392747589718944,
  43.266615305567875,
  61.61980201201558,
  63.41135544995076,
  65.73431371817918,
  56.938563381947034,
  46.647615158762406,
  44.68780594300866,
  65.21502894272147,
  58.66856057549052,
  67.20863039818622,
  60.802960454241045],
 [62.93647591023825,
  93.60555539069249,
  91.706052144883,
  88.05679985100527,
  87.80091115700337,
  86.1278120005379,
  83.93449827097318,
  83.43260753446461,
  81.39410298049853,
  85.14693182963201,
  86.34813257969161,
  87.0976463516667,
  84.29116205154607,
  86.14522621712709,
  88.28363381737297,
  88.84255736976509,
  86.12200647918046,
  84.17244204607586,
  61.773780845922005,
  86.83317338436964,
  82.96987405076617,
  80.0812087820857,
  90.24965373894794,
  84.48076704197234,
  90.52071586106685,
  88.60022573334675,
  82.86132994346639,
  91.39474820797966,
  87.5956619930462,
  83.19254774317228,
  92.02716990106781,
  90.13878188659973,
  86.37708029332781,
  94.2443632266673,
  91.18113840043894,
  87.66413177577246,
  89.31965069345043,
  58.52349955359813,
  56.0357029044876,
  54.405882034941776,
  46.75467891024384,
  58.60034129593445,
  56.22277118748239,
  51.66236541235796,
  61.220911460055866,
  53.74011537017761,
  62.48199740725323,
  60.959002616512684,
  58.0,
  63.7808748764079,
  62.93647591023825,
  60.207972893961475,
  30.675723300355934,
  29.832867780352597,
  32.28002478313795,
  31.400636936215164,
  31.016124838541646,
  34.48187929913333,
  33.24154027718932,
  38.27531841800928,
  36.05551275463989,
  39.81205847478876,
  39.45883931389772,
  39.01281840626232,
  40.19950248448356,
  42.20189569201838,
  45.0111097397076,
  18.384776310850235,
  10.770329614269007,
  19.6468827043885,
  19.235384061671343,
  10.770329614269007,
  8.06225774829855,
  10.63014581273465,
  8.54400374531753,
  8.06225774829855,
  9.219544457292887,
  6.4031242374328485,
  4.47213595499958,
  4.0,
  0.0,
  70.43436661176133,
  66.60330322138685,
  69.9714227381436,
  66.2872536767062,
  64.40496875241847,
  64.03124237432849,
  61.84658438426491,
  64.4437739428721,
  61.61168720299745,
  60.74537019394976,
  60.4648658313239,
  60.802960454241045,
  57.201398584300364,
  56.0357029044876,
  58.89821729050889,
  56.4003546088143,
  53.53503525729669,
  42.43819034784589,
  52.49761899362675,
  50.44799302251776,
  54.56189146281496,
  56.639209034025185,
  48.79549159502341,
  51.62363799656123,
  56.568542494923804,
  55.154328932550705,
  52.3450093132096,
  62.64982043070834,
  56.08029957123981,
  46.647615158762406,
  64.66065264130884,
  66.61080993352356,
  69.1158447825099,
  60.41522986797286,
  50.11985634456667,
  47.80167361086848,
  68.5054742338158,
  62.177166226839255,
  70.00714249274856,
  63.25345840347388],
 [10.0,
  37.16180835212409,
  35.34119409414458,
  36.61966684720111,
  37.20215047547655,
  35.0,
  34.058772731852805,
  35.38361202590826,
  44.40720662234904,
  37.53664875824692,
  37.64306044943742,
  39.05124837953327,
  47.01063709417264,
  42.42640687119285,
  43.18564576337837,
  41.23105625617661,
  69.35416353759881,
  68.54195795277518,
  14.035668847618199,
  72.09022125087424,
  70.57619995437555,
  69.57010852370435,
  75.15317691222374,
  72.80109889280519,
  76.05918747922567,
  75.23961722390672,
  73.05477397131553,
  78.79086241436883,
  77.25283166331187,
  82.46211251235322,
  80.62257748298549,
  79.84985911070852,
  78.43468620451031,
  82.34682750416071,
  82.63776376451628,
  79.71198153351854,
  81.93289937503738,
  76.23647421018367,
  76.53103945458993,
  76.79192665899195,
  77.62087348130012,
  79.40403012442127,
  79.76214641043707,
  80.8084154033477,
  82.29823813423954,
  83.74365647617735,
  84.29116205154607,
  84.48076704197234,
  85.0,
  86.28441342444185,
  89.67719888578144,
  90.2551937563706,
  85.75546629807339,
  87.36704184073076,
  87.98295289429652,
  90.13878188659973,
  92.41753080449618,
  89.1852005660132,
  92.35799911215054,
  90.35485598461214,
  95.96353474106714,
  95.38343671728336,
  96.38464608017192,
  99.02019995940222,
  98.2700361249552,
  96.17692030835673,
  104.10091258005379,
  55.072679252057455,
  60.207972893961475,
  55.90169943749474,
  57.697486947006624,
  61.29437168288782,
  64.89992295835181,
  60.0,
  63.245553203367585,
  64.89992295835181,
  61.220911460055866,
  64.03124237432849,
  66.40030120413611,
  67.35725647619564,
  70.43436661176133,
  0.0,
  5.0,
  9.433981132056603,
  7.0,
  6.708203932499369,
  6.4031242374328485,
  8.602325267042627,
  7.0710678118654755,
  9.433981132056603,
  11.704699910719626,
  11.180339887498949,
  10.0,
  13.45362404707371,
  15.0,
  11.661903789690601,
  14.142135623730951,
  18.027756377319946,
  28.284271247461902,
  18.027756377319946,
  20.0,
  16.1245154965971,
  14.317821063276353,
  22.360679774997898,
  19.697715603592208,
  15.524174696260024,
  16.76305461424021,
  19.924858845171276,
  18.110770276274835,
  23.194827009486403,
  25.0,
  5.830951894845301,
  5.0990195135927845,
  8.94427190999916,
  15.132745950421556,
  22.825424421026653,
  22.67156809750927,
  6.324555320336759,
  15.0,
  7.0710678118654755,
  15.811388300841896],
 [5.0,
  41.78516483155236,
  39.92492955535426,
  40.8166632639171,
  41.340053217188775,
  39.11521443121589,
  38.01315561749642,
  39.20459156782532,
  47.50789408087881,
  41.400483088968905,
  41.617304093369626,
  43.01162633521314,
  50.24937810560445,
  46.09772228646444,
  47.01063709417264,
  45.221676218380054,
  71.58910531638176,
  70.66116330771806,
  14.560219778561036,
  74.24284477308234,
  72.49827584156743,
  71.30918594402827,
  77.41446893184762,
  74.73285756613352,
  78.29431652425353,
  77.3692445355388,
  74.84650960465692,
  80.95060222135473,
  79.20227269466452,
  83.81527307120105,
  82.73451516749222,
  81.85963596303125,
  80.23091673413684,
  84.53401682163222,
  84.581321815162,
  81.54140052758476,
  83.773504164503,
  76.00657866263946,
  76.10519036176179,
  76.23647421018367,
  76.48529270389177,
  79.05694150420949,
  79.22752047110903,
  79.90619500389191,
  82.02438661763951,
  82.87339742040264,
  84.02380615040002,
  84.09518416651456,
  84.38009243891595,
  86.02325267042627,
  89.20201791439474,
  89.56003573022959,
  83.24061508662703,
  84.72307831990054,
  85.47514258543241,
  87.46427842267951,
  89.58794561770016,
  86.76981041814025,
  89.69392398596462,
  88.14193099768123,
  93.29523031752481,
  93.02150289046077,
  93.94147114027967,
  96.38464608017192,
  95.77055915050303,
  93.94147114027967,
  101.59724405711013,
  50.774009099144415,
  56.568542494923804,
  51.478150704935004,
  53.23532661682466,
  57.201398584300364,
  60.802960454241045,
  56.08029957123981,
  59.20304046246274,
  60.802960454241045,
  57.38466694161429,
  60.207972893961475,
  62.48199740725323,
  63.655321851358195,
  66.60330322138685,
  5.0,
  0.0,
  12.806248474865697,
  8.602325267042627,
  6.324555320336759,
  4.0,
  5.385164807134504,
  2.23606797749979,
  8.0,
  11.045361017187261,
  10.0,
  8.06225774829855,
  10.770329614269007,
  12.649110640673518,
  7.810249675906654,
  11.180339887498949,
  15.811388300841896,
  25.0,
  14.142135623730951,
  16.278820596099706,
  12.041594578792296,
  10.0,
  18.027756377319946,
  15.264337522473747,
  10.770329614269007,
  12.083045973594572,
  15.231546211727817,
  19.313207915827967,
  23.08679276123039,
  22.360679774997898,
  3.0,
  6.4031242374328485,
  12.041594578792296,
  15.297058540778355,
  20.8806130178211,
  19.209372712298546,
  9.219544457292887,
  15.811388300841896,
  5.0,
  11.180339887498949],
 [17.0,
  29.832867780352597,
  27.892651361962706,
  28.178005607210743,
  28.653097563788805,
  26.419689627245813,
  25.238858928247925,
  26.40075756488817,
  35.0,
  28.600699292150182,
  28.844410203711913,
  30.23243291566195,
  37.64306044943742,
  33.301651610693426,
  34.20526275297414,
  32.449961479175904,
  60.00833275470999,
  59.23681287847955,
  8.48528137423857,
  62.76941930590086,
  61.35144660071187,
  60.440052945046304,
  65.79513659838392,
  63.56886030125127,
  66.70832032063167,
  65.92419889539804,
  63.89053137985315,
  69.46221994724903,
  68.00735254367721,
  73.54590403278758,
  71.30918594402827,
  70.57619995437555,
  69.26037828369117,
  73.00684899377592,
  73.38937252763509,
  70.51950084905593,
  72.73238618387272,
  68.8839603971781,
  69.42621983083913,
  69.85699678629192,
  71.5122367151245,
  72.18032973047436,
  72.78049189171504,
  74.33034373659252,
  74.96665925596525,
  77.20103626247513,
  76.94153624668537,
  77.27871634544663,
  78.10249675906654,
  78.91767862779544,
  82.56512580987206,
  83.43260753446461,
  82.00609733428362,
  83.86894538504703,
  84.20213774008353,
  86.68333173107735,
  89.27485648266257,
  85.21150157109074,
  88.86506625215557,
  85.98255637046388,
  92.45539465060976,
  91.2633551870629,
  92.41753080449618,
  95.42536350467836,
  94.40338976964757,
  91.80958555619343,
  100.20977996183805,
  56.302753041036986,
  59.36328831862332,
  57.56735185849702,
  59.481089431852205,
  61.741396161732524,
  65.30696746902278,
  59.941638282582836,
  63.50590523722971,
  65.30696746902278,
  60.90155991434045,
  63.631753079732135,
  66.24198064671678,
  66.57326790837296,
  69.9714227381436,
  9.433981132056603,
  12.806248474865697,
  0.0,
  5.0990195135927845,
  8.246211251235321,
  10.770329614269007,
  12.36931687685298,
  13.892443989449804,
  10.0,
  9.486832980505138,
  10.198039027185569,
  11.0,
  14.142135623730951,
  14.560219778561036,
  15.132745950421556,
  15.132745950421556,
  16.55294535724685,
  27.730849247724095,
  20.09975124224178,
  21.37755832643195,
  19.0,
  18.110770276274835,
  25.079872407968907,
  23.0,
  20.396078054371138,
  21.213203435596427,
  24.08318915758459,
  10.44030650891055,
  17.0,
  23.323807579381203,
  11.180339887498949,
  6.708203932499369,
  1.0,
  9.899494936611665,
  20.0,
  23.08679276123039,
  3.605551275463989,
  8.602325267042627,
  16.401219466856727,
  23.853720883753127],
 [12.206555615733702,
  34.92849839314596,
  32.984845004941285,
  33.1058907144937,
  33.54101966249684,
  31.304951684997057,
  30.01666203960727,
  31.064449134018133,
  38.948684188300895,
  33.28663395418648,
  33.61547262794322,
  34.9857113690718,
  41.72529209005013,
  37.8021163428716,
  38.8329756778952,
  37.21558813185679,
  63.06346010171025,
  62.16912416947821,
  7.0710678118654755,
  65.7419196555744,
  64.07807737440318,
  62.96824596572466,
  68.8839603971781,
  66.30987860040162,
  69.77105417004964,
  68.87670143089025,
  66.48308055437865,
  72.44998274671983,
  70.77428911688199,
  75.69015788066504,
  74.24957912338628,
  73.40980860893181,
  71.8679344353238,
  76.02631123499285,
  76.15773105863909,
  73.16419889536138,
  75.39230729988306,
  69.26037828369117,
  69.58448102845921,
  69.87131027825369,
  70.8801805866774,
  72.44308110509934,
  72.83543093852057,
  73.97972695272672,
  75.32595834106593,
  76.90253571892151,
  77.31752712031083,
  77.52418977325722,
  78.08969202141856,
  79.30952023559341,
  82.73451516749222,
  83.3606621854697,
  79.88116173416608,
  81.60882305241266,
  82.09750300709517,
  84.40379138403677,
  86.83317338436964,
  83.21658488546619,
  86.6083136886985,
  84.21995013059554,
  90.21086409075129,
  89.35882720806042,
  90.4267659490264,
  93.23089616645332,
  92.35799911215054,
  90.0499861188218,
  98.18350166906862,
  52.0,
  55.80322571321482,
  53.14132102234569,
  55.02726596879042,
  57.706152185014034,
  61.29437168288782,
  56.08029957123981,
  59.53990258641679,
  61.29437168288782,
  57.14017850864661,
  59.90826320300064,
  62.433965115151864,
  63.00793600809346,
  66.2872536767062,
  7.0,
  8.602325267042627,
  5.0990195135927845,
  0.0,
  3.1622776601683795,
  5.830951894845301,
  7.280109889280518,
  9.219544457292887,
  5.0990195135927845,
  5.656854249492381,
  5.830951894845301,
  6.082762530298219,
  9.486832980505138,
  10.295630140987,
  10.04987562112089,
  10.44030650891055,
  12.806248474865697,
  23.853720883753127,
  15.297058540778355,
  16.76305461424021,
  14.035668847618199,
  13.038404810405298,
  20.223748416156685,
  18.027756377319946,
  15.297058540778355,
  16.1245154965971,
  19.026297590440446,
  11.180339887498949,
  16.278820596099706,
  19.849433241279208,
  6.4031242374328485,
  2.23606797749979,
  4.123105625617661,
  8.246211251235321,
  17.029386365926403,
  18.788294228055936,
  2.23606797749979,
  8.0,
  13.0,
  19.209372712298546],
 [9.219544457292887,
  38.07886552931954,
  36.138621999185304,
  36.24913792078372,
  36.6742416417845,
  34.438350715445125,
  33.12099032335839,
  34.132096331752024,
  41.72529209005013,
  36.359317925395686,
  36.71511950137164,
  38.07886552931954,
  44.553338819890925,
  40.80441152620633,
  41.88078318274385,
  40.311288741492746,
  65.36818798161687,
  64.41273166075166,
  8.246211251235321,
  68.0,
  66.2117814289874,
  65.0,
  71.19691004531025,
  68.4470598345904,
  72.069410986909,
  71.11961754677819,
  68.54195795277518,
  74.70609078247904,
  72.91776189653656,
  77.4919350642375,
  76.47875521999558,
  75.58438992278762,
  73.92563831310488,
  78.29431652425353,
  78.29431652425353,
  75.23961722390672,
  77.47257579298626,
  70.06425622241343,
  70.25667228100119,
  70.45565981523414,
  71.06335201775947,
  73.17103251970687,
  73.43704787094863,
  74.33034373659252,
  76.10519036176179,
  77.27871634544663,
  78.10249675906654,
  78.23042886243178,
  78.63841300535,
  80.09993757800315,
  83.38465086573187,
  83.86298349093002,
  79.07591289387686,
  80.7093550959243,
  81.30190649671138,
  83.48652585896721,
  85.80209787645055,
  82.49242389456137,
  85.70297544426332,
  83.64807230295268,
  89.30845424706443,
  88.68483523128404,
  89.69392398596462,
  92.35799911215054,
  91.58602513484249,
  89.47066558375433,
  97.41663102366043,
  49.658836071740545,
  54.037024344425184,
  50.695167422546305,
  52.55473337388365,
  55.569775957799216,
  59.16924876994806,
  54.08326913195984,
  57.4543296888929,
  59.16924876994806,
  55.21775076911409,
  58.008620049092706,
  60.4648658313239,
  61.220911460055866,
  64.40496875241847,
  6.708203932499369,
  6.324555320336759,
  8.246211251235321,
  3.1622776601683795,
  0.0,
  2.8284271247461903,
  4.123105625617661,
  6.4031242374328485,
  2.8284271247461903,
  5.0990195135927845,
  4.47213595499958,
  3.605551275463989,
  7.211102550927978,
  8.48528137423857,
  7.0,
  8.06225774829855,
  11.40175425099138,
  22.02271554554524,
  12.649110640673518,
  14.317821063276353,
  11.180339887498949,
  10.0,
  17.46424919657298,
  15.132745950421556,
  12.165525060596439,
  13.038404810405298,
  16.0,
  13.0,
  17.0,
  18.439088914585774,
  3.605551275463989,
  2.23606797749979,
  7.280109889280518,
  9.055385138137417,
  16.1245154965971,
  16.64331697709324,
  5.0,
  9.486832980505138,
  11.180339887498949,
  16.278820596099706],
 [6.4031242374328485,
  40.52159917870962,
  38.600518131237564,
  38.91015291668744,
  39.35733730830886,
  37.12142238654117,
  35.84689665786984,
  36.89173349139343,
  44.553338819890925,
  39.11521443121589,
  39.44616584663204,
  40.8166632639171,
  47.38143096192854,
  43.60045871318328,
  44.654227123532216,
  43.04648650006177,
  68.09552114493287,
  67.11929677819934,
  10.770329614269007,
  70.71067811865476,
  68.87670143089025,
  67.62396025078685,
  73.92563831310488,
  71.11258679024411,
  74.79304780526061,
  73.824115301167,
  71.17583859709698,
  77.41446893184762,
  75.58438992278762,
  80.00624975587844,
  79.17701686727027,
  78.26237921249263,
  76.55716818169282,
  81.00617260431454,
  80.95677859203639,
  77.87810988975015,
  80.11242100947892,
  72.00694410957877,
  72.11102550927978,
  72.24956747275377,
  72.56721022610694,
  75.05997601918082,
  75.23961722390672,
  75.95393340703298,
  78.02563681252464,
  78.91767862779544,
  80.02499609497022,
  80.09993757800315,
  80.39900496896712,
  82.02438661763951,
  85.21150157109074,
  85.58621384311844,
  79.75587752636166,
  81.30190649671138,
  81.98780397107853,
  84.05950273467003,
  86.2670273047588,
  83.24061508662703,
  86.28441342444185,
  84.52810183601665,
  89.88882021697692,
  89.47066558375433,
  90.4267659490264,
  92.9623579735368,
  92.28217596047462,
  90.33825324855468,
  98.11218069128827,
  48.76474136094644,
  53.81449618829484,
  49.658836071740545,
  51.478150704935004,
  54.91812087098393,
  58.52349955359813,
  53.600373133029585,
  56.859475903318,
  58.52349955359813,
  54.817880294662984,
  57.62811813689564,
  60.0,
  60.959002616512684,
  64.03124237432849,
  6.4031242374328485,
  4.0,
  10.770329614269007,
  5.830951894845301,
  2.8284271247461903,
  0.0,
  2.23606797749979,
  3.605551275463989,
  4.0,
  7.0710678118654755,
  6.0,
  4.123105625617661,
  7.211102550927978,
  8.94427190999916,
  5.385164807134504,
  7.810249675906654,
  12.083045973594572,
  21.93171219946131,
  11.661903789690601,
  13.601470508735444,
  9.848857801796104,
  8.246211251235321,
  16.15549442140351,
  13.601470508735444,
  10.0,
  11.045361017187261,
  14.142135623730951,
  15.652475842498529,
  19.1049731745428,
  18.867962264113206,
  1.0,
  4.123105625617661,
  9.848857801796104,
  11.40175425099138,
  17.08800749063506,
  16.278820596099706,
  7.280109889280518,
  12.083045973594572,
  9.0,
  13.45362404707371],
 [5.830951894845301,
  42.20189569201838,
  40.26164427839479,
  40.311288741492746,
  40.718546143004666,
  38.48376280978771,
  37.12142238654117,
  38.07886552931954,
  45.27692569068709,
  40.311288741492746,
  40.70626487409524,
  42.05948168962618,
  48.16637831516918,
  44.654227123532216,
  45.79301256742124,
  44.294469180700204,
  68.26419266350405,
  67.23094525588644,
  10.816653826391969,
  70.83078426785913,
  68.8839603971781,
  67.54257916307313,
  74.09453421137081,
  71.11961754677819,
  74.94664769020693,
  73.92563831310488,
  71.11258679024411,
  77.52418977325722,
  75.59100475585703,
  79.71198153351854,
  79.25906888173743,
  78.29431652425353,
  76.48529270389177,
  81.12336284942828,
  80.95060222135473,
  77.82030583337487,
  80.05623023850174,
  71.00704190430693,
  71.02816342831905,
  71.11258679024411,
  71.19691004531025,
  74.00675644831355,
  74.10802925459562,
  74.67261881037788,
  77.0,
  77.64663547121665,
  79.0,
  79.02531240052139,
  79.22752047110903,
  81.0,
  84.09518416651456,
  84.38009243891595,
  77.89736837660179,
  79.40403012442127,
  80.1311425102625,
  82.15229759416349,
  84.31488599292535,
  81.41252974819048,
  84.38009243891595,
  82.76472678623425,
  87.98295289429652,
  87.65842800324451,
  88.58893836140041,
  91.0659101969557,
  90.4267659490264,
  88.56635930193812,
  96.25487000666512,
  46.52956049652737,
  51.66236541235796,
  47.4236228055175,
  49.24428900898052,
  52.69724850502159,
  56.302753041036986,
  51.40038910358559,
  54.644304369257,
  56.302753041036986,
  52.630789467763066,
  55.44366510251645,
  57.8013840664737,
  58.7962583843564,
  61.84658438426491,
  8.602325267042627,
  5.385164807134504,
  12.36931687685298,
  7.280109889280518,
  4.123105625617661,
  2.23606797749979,
  0.0,
  4.0,
  3.605551275463989,
  6.708203932499369,
  5.385164807134504,
  3.1622776601683795,
  5.385164807134504,
  7.280109889280518,
  3.1622776601683795,
  5.830951894845301,
  10.44030650891055,
  19.849433241279208,
  9.433981132056603,
  11.40175425099138,
  7.615773105863909,
  6.082762530298219,
  13.92838827718412,
  11.40175425099138,
  8.06225774829855,
  9.0,
  12.041594578792296,
  15.811388300841896,
  18.439088914585774,
  17.0,
  2.8284271247461903,
  6.0,
  11.40175425099138,
  11.180339887498949,
  15.524174696260024,
  14.142135623730951,
  9.055385138137417,
  12.206555615733702,
  10.198039027185569,
  12.806248474865697],
 [3.1622776601683795,
  43.32435804486894,
  41.43669871020132,
  42.05948168962618,
  42.5440947723653,
  40.311288741492746,
  39.11521443121589,
  40.22437072223753,
  48.104053883222775,
  42.43819034784589,
  42.720018726587654,
  44.10215414239989,
  50.91168824543142,
  47.01063709417264,
  48.010415536631214,
  46.32493928760188,
  71.69379331573968,
  70.71067811865476,
  14.317821063276353,
  74.30343195303969,
  72.44998274671983,
  71.17583859709698,
  77.52418977325722,
  74.68600939935136,
  78.39005038906404,
  77.41446893184762,
  74.73285756613352,
  81.00617260431454,
  79.1580697086532,
  83.48652585896721,
  82.76472678623425,
  81.8413098624405,
  80.11242100947892,
  84.59905436823747,
  84.52810183601665,
  81.43709228600933,
  83.6719785830358,
  75.0066663703967,
  75.02666192761077,
  75.1065909225016,
  75.13321502504735,
  78.00640999302557,
  78.10249675906654,
  78.63841300535,
  81.0,
  81.61494961096282,
  83.0,
  83.02409288875127,
  83.21658488546619,
  85.0,
  88.09086218218096,
  88.36288813749809,
  81.41252974819048,
  82.85529554590944,
  83.64807230295268,
  85.58621384311844,
  87.66413177577246,
  84.970583144992,
  87.8179936004006,
  86.4060183089118,
  91.4166286842826,
  91.23595782365635,
  92.13034245024816,
  94.51454914456292,
  93.94147114027967,
  92.19544457292888,
  99.76472322419383,
  48.54894437575342,
  54.45181356024793,
  49.24428900898052,
  51.0,
  55.0,
  58.60034129593445,
  53.907327887774215,
  57.0087712549569,
  58.60034129593445,
  55.226805085936306,
  58.05170109479997,
  60.30754513325841,
  61.5223536610881,
  64.4437739428721,
  7.0710678118654755,
  2.23606797749979,
  13.892443989449804,
  9.219544457292887,
  6.4031242374328485,
  3.605551275463989,
  4.0,
  0.0,
  7.280109889280518,
  10.44030650891055,
  9.219544457292887,
  7.0710678118654755,
  9.219544457292887,
  11.180339887498949,
  5.830951894845301,
  9.486832980505138,
  14.317821063276353,
  23.021728866442675,
  12.041594578792296,
  14.212670403551895,
  9.899494936611665,
  7.810249675906654,
  15.811388300841896,
  13.038404810405298,
  8.54400374531753,
  9.848857801796104,
  13.0,
  19.235384061671343,
  22.360679774997898,
  20.615528128088304,
  2.8284271247461903,
  7.211102550927978,
  13.038404810405298,
  14.866068747318506,
  19.4164878389476,
  17.204650534085253,
  10.295630140987,
  15.652475842498529,
  6.324555320336759,
  10.0],
 [9.433981132056603,
  39.6232255123179,
  37.656340767525464,
  37.33630940518894,
  37.69615364994153,
  35.4682957019364,
  34.0147027033899,
  34.88552708502482,
  41.773197148410844,
  37.12142238654117,
  37.57658845611187,
  38.91015291668744,
  44.68780594300866,
  41.340053217188775,
  42.5440947723653,
  41.14608122288197,
  64.66065264130884,
  63.631753079732135,
  7.211102550927978,
  67.23094525588644,
  65.29931086925804,
  63.97655820689325,
  70.49113419430843,
  67.53517601961218,
  71.34423592694787,
  70.32780389006896,
  67.54257916307313,
  73.92563831310488,
  72.00694410957877,
  76.21679604916491,
  75.66372975210778,
  74.70609078247904,
  72.91776189653656,
  77.52418977325722,
  77.3692445355388,
  74.24957912338628,
  76.48529270389177,
  68.00735254367721,
  68.11754546370561,
  68.26419266350405,
  68.65857557508748,
  71.06335201775947,
  71.25307010929424,
  72.00694410957877,
  74.02702209328699,
  74.96665925596525,
  76.02631123499285,
  76.10519036176179,
  76.4198926981712,
  78.02563681252464,
  81.2219182240853,
  81.61494961096282,
  76.32168761236873,
  77.93587107359485,
  78.54934754662193,
  80.7093550959243,
  83.00602387778854,
  79.75587752636166,
  82.92767933567175,
  80.95060222135473,
  86.53323061113575,
  85.95929269136641,
  86.95401083331349,
  89.58794561770016,
  88.83692925805124,
  86.76981041814025,
  94.66784036831093,
  47.01063709417264,
  51.22499389946279,
  48.104053883222775,
  49.9799959983992,
  52.839379254491625,
  56.43580423808985,
  51.31276644267,
  54.70831746635972,
  56.43580423808985,
  52.43090691567332,
  55.21775076911409,
  57.688820407423826,
  58.412327466040935,
  61.61168720299745,
  9.433981132056603,
  8.0,
  10.0,
  5.0990195135927845,
  2.8284271247461903,
  4.0,
  3.605551275463989,
  7.280109889280518,
  0.0,
  3.1622776601683795,
  2.0,
  1.0,
  4.47213595499958,
  5.656854249492381,
  5.385164807134504,
  5.385164807134504,
  8.602325267042627,
  19.209372712298546,
  10.198039027185569,
  11.704699910719626,
  9.0,
  8.246211251235321,
  15.132745950421556,
  13.0,
  10.770329614269007,
  11.40175425099138,
  14.142135623730951,
  12.206555615733702,
  15.132745950421556,
  15.620499351813308,
  5.0,
  5.0,
  9.0,
  7.615773105863909,
  13.416407864998739,
  13.892443989449804,
  7.280109889280518,
  8.602325267042627,
  13.0,
  16.401219466856727],
 [12.529964086141668,
  38.2099463490856,
  36.22154055254967,
  35.4400902933387,
  35.73513677041127,
  33.52610922848042,
  31.953090617340916,
  32.69556544854363,
  39.05124837953327,
  34.92849839314596,
  35.4682957019364,
  36.76955262170047,
  42.01190307520001,
  38.948684188300895,
  40.24922359499622,
  39.0,
  61.587336360651285,
  60.53924347066124,
  4.242640687119285,
  64.1404708432983,
  62.177166226839255,
  60.83584469702052,
  67.4166151627327,
  64.41273166075166,
  68.26419266350405,
  67.23094525588644,
  64.40496875241847,
  70.83078426785913,
  68.8839603971781,
  73.05477397131553,
  72.56031973468694,
  71.58910531638176,
  69.77822009767804,
  74.43117626371358,
  74.24284477308234,
  71.11258679024411,
  73.348483283569,
  65.03076195155643,
  65.19202405202648,
  65.37583651472461,
  65.96969000988257,
  68.11754546370561,
  68.3593446428504,
  69.2314957226839,
  71.06335201775947,
  72.18032973047436,
  73.06161783043132,
  73.17103251970687,
  73.55270219373317,
  75.05997601918082,
  78.31347265956222,
  78.77182237323191,
  74.33034373659252,
  76.02631123499285,
  76.55063683601855,
  78.81624198095213,
  81.21576201698781,
  77.69813382572325,
  81.02468759581859,
  78.77182237323191,
  84.62860036654276,
  83.86298349093002,
  84.90583018850944,
  87.65842800324451,
  86.82165628459296,
  84.59905436823747,
  92.64987857520376,
  46.8187996428785,
  50.21951811795888,
  48.08326112068523,
  50.0,
  52.32590180780452,
  55.90169943749474,
  50.60632371551998,
  54.120236510939236,
  55.90169943749474,
  51.62363799656123,
  54.378304497290095,
  56.938563381947034,
  57.42821606144492,
  60.74537019394976,
  11.704699910719626,
  11.045361017187261,
  9.486832980505138,
  5.656854249492381,
  5.0990195135927845,
  7.0710678118654755,
  6.708203932499369,
  10.44030650891055,
  3.1622776601683795,
  0.0,
  1.4142135623730951,
  3.605551275463989,
  5.0990195135927845,
  5.0990195135927845,
  7.810249675906654,
  6.082762530298219,
  7.211102550927978,
  18.35755975068582,
  11.045361017187261,
  12.041594578792296,
  10.44030650891055,
  10.295630140987,
  16.0312195418814,
  14.317821063276353,
  13.038404810405298,
  13.416407864998739,
  15.811388300841896,
  9.219544457292887,
  12.041594578792296,
  14.212670403551895,
  8.06225774829855,
  6.708203932499369,
  8.54400374531753,
  4.47213595499958,
  11.40175425099138,
  13.601470508735444,
  7.810249675906654,
  5.656854249492381,
  16.0312195418814,
  19.4164878389476],
 [11.180339887498949,
  39.319206502675,
  37.33630940518894,
  36.68787265568828,
  37.0,
  34.785054261852174,
  33.24154027718932,
  34.0147027033899,
  40.45985664828782,
  36.24913792078372,
  36.76955262170047,
  38.07886552931954,
  43.41658669218482,
  40.311288741492746,
  41.593268686170845,
  40.311288741492746,
  62.96824596572466,
  61.91122676865643,
  5.656854249492381,
  65.5133574166368,
  63.52952069707436,
  62.16912416947821,
  68.79680225126747,
  65.76473218982953,
  69.6419413859206,
  68.60029154456998,
  65.7419196555744,
  72.20110802473879,
  70.2353187506115,
  74.33034373659252,
  73.92563831310488,
  72.94518489934754,
  71.11258679024411,
  75.8023746329889,
  75.59100475585703,
  72.44998274671983,
  74.68600939935136,
  66.007575322837,
  66.12110101926616,
  66.27216610312357,
  66.70832032063167,
  69.06518659932803,
  69.26037828369117,
  70.03570517957252,
  72.02777242147643,
  72.99315036357864,
  74.02702209328699,
  74.10802925459562,
  74.43117626371358,
  76.02631123499285,
  79.22752047110903,
  79.63039620647382,
  74.62573282722254,
  76.27581530209953,
  76.8505042273634,
  79.05694150420949,
  81.40024569987489,
  78.03204469959762,
  81.27115109311545,
  79.17701686727027,
  84.87638069569178,
  84.21995013059554,
  85.23496934944014,
  87.92041856133307,
  87.13208364316786,
  85.0,
  92.9623579735368,
  46.238512086787566,
  50.0,
  47.43416490252569,
  49.33558553417604,
  51.884487084291386,
  55.47071299343465,
  50.24937810560445,
  53.71219600798314,
  55.47071299343465,
  51.31276644267,
  54.08326913195984,
  56.60388679233962,
  57.201398584300364,
  60.4648658313239,
  11.180339887498949,
  10.0,
  10.198039027185569,
  5.830951894845301,
  4.47213595499958,
  6.0,
  5.385164807134504,
  9.219544457292887,
  2.0,
  1.4142135623730951,
  0.0,
  2.23606797749979,
  4.0,
  4.47213595499958,
  6.4031242374328485,
  5.0,
  7.0710678118654755,
  18.027756377319946,
  10.0,
  11.180339887498949,
  9.219544457292887,
  8.94427190999916,
  15.0,
  13.152946437965905,
  11.661903789690601,
  12.083045973594572,
  14.560219778561036,
  10.63014581273465,
  13.152946437965905,
  14.142135623730951,
  7.0,
  6.4031242374328485,
  9.219544457292887,
  5.830951894845301,
  11.661903789690601,
  13.0,
  8.06225774829855,
  7.0710678118654755,
  15.0,
  18.027756377319946],
 [8.94427190999916,
  40.607881008493905,
  38.63935817272331,
  38.27531841800928,
  38.62641583165593,
  36.40054944640259,
  34.92849839314596,
  35.77708763999664,
  42.5205832509386,
  38.01315561749642,
  38.48376280978771,
  39.81205847478876,
  45.45327270945405,
  42.190046219457976,
  43.41658669218482,
  42.04759208325728,
  65.19202405202648,
  64.1404708432983,
  7.810249675906654,
  67.7421582177598,
  65.76473218982953,
  64.40496875241847,
  71.02112361825881,
  68.0,
  71.8679344353238,
  70.83078426785913,
  67.97793759742936,
  74.43117626371358,
  72.47068372797375,
  76.55063683601855,
  76.15773105863909,
  75.17978451685,
  73.348483283569,
  78.03204469959762,
  77.82673062643708,
  74.68600939935136,
  76.92203845452875,
  68.0,
  68.06614430096654,
  68.18357573492314,
  68.4470598345904,
  71.02816342831905,
  71.17583859709698,
  71.84705978674423,
  74.00675644831355,
  74.81310045707235,
  76.00657866263946,
  76.05918747922567,
  76.32168761236873,
  78.00640999302557,
  81.15417426109393,
  81.49846624323675,
  75.8023746329889,
  77.38862965578342,
  78.03204469959762,
  80.15609770940699,
  82.41965784932621,
  79.25906888173743,
  82.37718130647589,
  80.49844718999243,
  85.98255637046388,
  85.47514258543241,
  86.45229898620394,
  89.04493247793498,
  88.32326986700618,
  86.31338250816034,
  94.15412895885129,
  46.09772228646444,
  50.44799302251776,
  47.16990566028302,
  49.040799340956916,
  51.97114584074513,
  55.569775957799216,
  50.47771785649585,
  53.85164807134504,
  55.569775957799216,
  51.61395160225576,
  54.405882034941776,
  56.859475903318,
  57.62811813689564,
  60.802960454241045,
  10.0,
  8.06225774829855,
  11.0,
  6.082762530298219,
  3.605551275463989,
  4.123105625617661,
  3.1622776601683795,
  7.0710678118654755,
  1.0,
  3.605551275463989,
  2.23606797749979,
  0.0,
  3.605551275463989,
  5.0,
  4.47213595499958,
  4.47213595499958,
  8.06225774829855,
  18.439088914585774,
  9.219544457292887,
  10.770329614269007,
  8.0,
  7.280109889280518,
  14.142135623730951,
  12.0,
  9.848857801796104,
  10.44030650891055,
  13.152946437965905,
  12.806248474865697,
  15.297058540778355,
  15.0,
  5.0990195135927845,
  5.830951894845301,
  10.0,
  8.06225774829855,
  13.0,
  13.038404810405298,
  8.246211251235321,
  9.219544457292887,
  13.038404810405298,
  15.811388300841896],
 [10.04987562112089,
  43.289721643826724,
  41.30375285612676,
  40.52159917870962,
  40.80441152620633,
  38.600518131237564,
  37.0,
  37.69615364994153,
  43.60045871318328,
  39.92492955535426,
  40.496913462633174,
  41.78516483155236,
  46.61544808322666,
  43.829214001622255,
  45.18849411078001,
  44.01136216933077,
  65.21502894272147,
  64.07027391856539,
  8.94427190999916,
  67.67569726275453,
  65.5133574166368,
  64.00781202322104,
  71.02816342831905,
  67.7421582177598,
  71.84705978674423,
  70.7248188403477,
  67.60177512462228,
  74.33034373659252,
  72.20110802473879,
  75.76938695805846,
  76.00657866263946,
  74.94664769020693,
  72.94518489934754,
  77.93587107359485,
  77.52418977325722,
  74.30343195303969,
  76.53757247260981,
  66.06814663663572,
  66.0,
  66.03029607687671,
  65.92419889539804,
  69.00724599634447,
  69.02897942168927,
  69.46221994724903,
  72.02777242147643,
  72.44308110509934,
  74.02702209328699,
  74.0,
  74.10802925459562,
  76.02631123499285,
  79.02531240052139,
  79.22752047110903,
  72.56031973468694,
  74.09453421137081,
  74.79304780526061,
  76.8505042273634,
  79.05694150420949,
  76.05918747922567,
  79.07591289387686,
  77.38862965578342,
  82.68010643437755,
  82.29823813423954,
  83.24061508662703,
  85.75546629807339,
  85.0881895447306,
  83.19254774317228,
  90.91754506144565,
  42.5440947723653,
  46.861498055439924,
  43.657759905886145,
  45.5411901469428,
  48.373546489791295,
  51.97114584074513,
  46.87216658103186,
  50.24937810560445,
  51.97114584074513,
  48.010415536631214,
  50.80354318352215,
  53.25410782277739,
  54.037024344425184,
  57.201398584300364,
  13.45362404707371,
  10.770329614269007,
  14.142135623730951,
  9.486832980505138,
  7.211102550927978,
  7.211102550927978,
  5.385164807134504,
  9.219544457292887,
  4.47213595499958,
  5.0990195135927845,
  4.0,
  3.605551275463989,
  0.0,
  2.0,
  4.123105625617661,
  1.0,
  5.0990195135927845,
  14.866068747318506,
  6.0,
  7.280109889280518,
  5.385164807134504,
  5.656854249492381,
  11.0,
  9.219544457292887,
  8.48528137423857,
  8.602325267042627,
  10.770329614269007,
  13.601470508735444,
  14.317821063276353,
  11.661903789690601,
  8.06225774829855,
  9.433981132056603,
  13.152946437965905,
  8.602325267042627,
  10.198039027185569,
  9.433981132056603,
  11.704699910719626,
  10.295630140987,
  15.524174696260024,
  16.15549442140351],
 [12.041594578792296,
  43.104524124504614,
  41.10960958218893,
  40.024992192379,
  40.26164427839479,
  38.07886552931954,
  36.40054944640259,
  37.0,
  42.43819034784589,
  39.21734310225516,
  39.84971769034255,
  41.10960958218893,
  45.48626166217664,
  42.95346318982906,
  44.384682042344295,
  43.32435804486894,
  63.60031446463138,
  62.42595614005443,
  8.246211251235321,
  66.03029607687671,
  63.812224534175265,
  62.26556030423239,
  69.40461079784254,
  66.03786792439622,
  70.21395872616783,
  69.06518659932803,
  65.86349520030045,
  72.67048919609665,
  70.49113419430843,
  73.92563831310488,
  74.33034373659252,
  73.24616030891995,
  71.19691004531025,
  76.27581530209953,
  75.8023746329889,
  72.56031973468694,
  74.79304780526061,
  64.07027391856539,
  64.0,
  64.03124237432849,
  63.953107821277925,
  67.00746227100382,
  67.02984409947557,
  67.47592163134935,
  70.02856560004639,
  70.45565981523414,
  72.02777242147643,
  72.0,
  72.11102550927978,
  74.02702209328699,
  77.02596964660685,
  77.23341245859851,
  70.83078426785913,
  72.40165743959182,
  73.06161783043132,
  75.16648189186454,
  77.42092740338364,
  74.30343195303969,
  77.38862965578342,
  75.58438992278762,
  80.99382692526635,
  80.52949770115296,
  81.49233092751734,
  84.05950273467003,
  83.35466393669883,
  81.39410298049853,
  89.1852005660132,
  41.78516483155236,
  45.60701700396552,
  43.01162633521314,
  44.9221548904324,
  47.41307836451879,
  51.0,
  45.79301256742124,
  49.24428900898052,
  51.0,
  46.87216658103186,
  49.64876634922564,
  52.15361924162119,
  52.80151512977634,
  56.0357029044876,
  15.0,
  12.649110640673518,
  14.560219778561036,
  10.295630140987,
  8.48528137423857,
  8.94427190999916,
  7.280109889280518,
  11.180339887498949,
  5.656854249492381,
  5.0990195135927845,
  4.47213595499958,
  5.0,
  2.0,
  0.0,
  6.082762530298219,
  2.23606797749979,
  3.1622776601683795,
  13.601470508735444,
  6.324555320336759,
  7.0,
  6.4031242374328485,
  7.211102550927978,
  11.180339887498949,
  9.848857801796104,
  10.0,
  9.899494936611665,
  11.661903789690601,
  12.529964086141668,
  12.529964086141668,
  10.0,
  9.848857801796104,
  10.63014581273465,
  13.601470508735444,
  7.615773105863909,
  8.246211251235321,
  8.54400374531753,
  12.529964086141668,
  9.486832980505138,
  17.46424919657298,
  18.027756377319946],
 [6.0,
  44.91102314577124,
  42.95346318982906,
  42.720018726587654,
  43.08131845707603,
  40.85339643163099,
  39.395431207184416,
  40.24922359499622,
  46.861498055439924,
  42.485291572496,
  42.95346318982906,
  44.28317965096906,
  49.8196748283246,
  46.647615158762406,
  47.885279575251516,
  46.51881339845203,
  69.06518659932803,
  67.95586803212802,
  12.041594578792296,
  71.56116265125938,
  69.46221994724903,
  68.0,
  74.88658090739622,
  71.69379331573968,
  75.71657678474378,
  74.62573282722254,
  71.58910531638176,
  78.23042886243178,
  76.15773105863909,
  79.84985911070852,
  79.92496481075234,
  78.89233169326408,
  76.94153624668537,
  81.83520025025906,
  81.49233092751734,
  78.29431652425353,
  80.52949770115296,
  70.11419257183242,
  70.00714249274856,
  70.00714249274856,
  69.72087205421343,
  73.02739212103907,
  73.00684899377592,
  73.33484846919642,
  76.05918747922567,
  76.32168761236873,
  78.05767098754612,
  78.00640999302557,
  78.05767098754612,
  80.05623023850174,
  83.00602387778854,
  83.15046602394962,
  75.59100475585703,
  77.02596964660685,
  77.82673062643708,
  79.75587752636166,
  81.83520025025906,
  79.1580697086532,
  81.98780397107853,
  80.62257748298549,
  85.58621384311844,
  85.42833253669417,
  86.31338250816034,
  88.68483523128404,
  88.11923740024082,
  86.4060183089118,
  93.94147114027967,
  43.41658669218482,
  48.79549159502341,
  44.28317965096906,
  46.09772228646444,
  49.64876634922564,
  53.25410782277739,
  48.41487374764082,
  51.61395160225576,
  53.25410782277739,
  49.678969393496885,
  52.49761899362675,
  54.817880294662984,
  55.90169943749474,
  58.89821729050889,
  11.661903789690601,
  7.810249675906654,
  15.132745950421556,
  10.04987562112089,
  7.0,
  5.385164807134504,
  3.1622776601683795,
  5.830951894845301,
  5.385164807134504,
  7.810249675906654,
  6.4031242374328485,
  4.47213595499958,
  4.123105625617661,
  6.082762530298219,
  0.0,
  4.0,
  9.0,
  17.204650534085253,
  6.4031242374328485,
  8.48528137423857,
  4.47213595499958,
  3.0,
  10.770329614269007,
  8.246211251235321,
  5.385164807134504,
  6.082762530298219,
  9.0,
  16.97056274847714,
  18.384776310850235,
  14.866068747318506,
  5.830951894845301,
  9.055385138137417,
  14.142135623730951,
  12.041594578792296,
  14.035668847618199,
  11.40175425099138,
  12.0,
  13.45362404707371,
  12.083045973594572,
  12.083045973594572],
 [10.0,
  44.28317965096906,
  42.2965719651132,
  41.48493702538308,
  41.7612260356422,
  39.56008088970496,
  37.94733192202055,
  38.62641583165593,
  44.40720662234904,
  40.85339643163099,
  41.43669871020132,
  42.720018726587654,
  47.43416490252569,
  44.721359549995796,
  46.09772228646444,
  44.94441010848846,
  65.80273550544841,
  64.63745044476924,
  9.848857801796104,
  68.24221567329127,
  66.03786792439622,
  64.4980619863884,
  71.61005515987263,
  68.26419266350405,
  72.42237223399962,
  71.28113354878694,
  68.09552114493287,
  74.88658090739622,
  72.71863585079137,
  76.15773105863909,
  76.55063683601855,
  75.47184905645283,
  73.43023900274328,
  78.49203781276162,
  78.03204469959762,
  74.79304780526061,
  77.02596964660685,
  66.12110101926616,
  66.007575322837,
  66.007575322837,
  65.76473218982953,
  69.02897942168927,
  69.00724599634447,
  69.35416353759881,
  72.06247289678588,
  72.3394774656273,
  74.06078584514209,
  74.00675644831355,
  74.06078584514209,
  76.05918747922567,
  79.00632886041473,
  79.1580697086532,
  72.069410986909,
  73.57309290766564,
  74.30343195303969,
  76.32168761236873,
  78.49203781276162,
  75.59100475585703,
  78.54934754662193,
  76.96752561957543,
  82.15229759416349,
  81.8413098624405,
  82.76472678623425,
  85.23496934944014,
  84.59905436823747,
  82.76472678623425,
  90.4267659490264,
  41.6293165929973,
  46.09772228646444,
  42.720018726587654,
  44.598206241955516,
  47.50789408087881,
  51.10772935672255,
  46.04345773288535,
  49.39635614091387,
  51.10772935672255,
  47.20169488482379,
  50.0,
  52.43090691567332,
  53.2634959423431,
  56.4003546088143,
  14.142135623730951,
  11.180339887498949,
  15.132745950421556,
  10.44030650891055,
  8.06225774829855,
  7.810249675906654,
  5.830951894845301,
  9.486832980505138,
  5.385164807134504,
  6.082762530298219,
  5.0,
  4.47213595499958,
  1.0,
  2.23606797749979,
  4.0,
  0.0,
  5.0,
  14.142135623730951,
  5.0,
  6.324555320336759,
  4.47213595499958,
  5.0,
  10.0,
  8.246211251235321,
  7.810249675906654,
  7.810249675906654,
  9.848857801796104,
  14.422205101855956,
  14.7648230602334,
  11.180339887498949,
  8.602325267042627,
  10.295630140987,
  14.142135623730951,
  9.433981132056603,
  10.04987562112089,
  8.602325267042627,
  12.649110640673518,
  11.180339887498949,
  15.811388300841896,
  15.811388300841896],
 [15.0,
  44.0,
  42.0,
  40.44749683231337,
  40.607881008493905,
  38.47076812334269,
  36.6742416417845,
  37.107950630558946,
  41.677331968349414,
  39.293765408777,
  40.024992192379,
  41.23105625617661,
  44.77722635447622,
  42.720018726587654,
  44.27188724235731,
  43.41658669218482,
  61.84658438426491,
  60.60528029800704,
  9.055385138137417,
  64.20280367709809,
  61.85466837676846,
  60.207972893961475,
  67.62396025078685,
  64.07027391856539,
  68.41052550594829,
  67.20119046564577,
  63.812224534175265,
  70.80254232723568,
  68.5054742338158,
  71.58910531638176,
  72.42237223399962,
  71.28113354878694,
  69.1158447825099,
  74.4043009509531,
  73.78346698278686,
  70.49113419430843,
  72.71863585079137,
  61.1310068623117,
  61.00819617067857,
  61.00819617067857,
  60.8276253029822,
  64.03124237432849,
  64.00781202322104,
  64.38167441127949,
  67.06713054842886,
  67.36467917239716,
  69.06518659932803,
  69.00724599634447,
  69.06518659932803,
  71.06335201775947,
  74.00675644831355,
  74.16872656315464,
  67.7421582177598,
  69.33974329343886,
  69.9714227381436,
  72.11102550927978,
  74.4043009509531,
  71.19691004531025,
  74.33034373659252,
  72.44998274671983,
  77.93587107359485,
  77.41446893184762,
  78.39005038906404,
  80.99382692526635,
  80.26207074328447,
  78.26237921249263,
  86.09297300012354,
  39.84971769034255,
  43.01162633521314,
  41.23105625617661,
  43.174066289845804,
  45.18849411078001,
  48.75448697299562,
  43.41658669218482,
  46.95742752749558,
  48.75448697299562,
  44.41846462902562,
  47.16990566028302,
  49.73932046178355,
  50.21951811795888,
  53.53503525729669,
  18.027756377319946,
  15.811388300841896,
  16.55294535724685,
  12.806248474865697,
  11.40175425099138,
  12.083045973594572,
  10.44030650891055,
  14.317821063276353,
  8.602325267042627,
  7.211102550927978,
  7.0710678118654755,
  8.06225774829855,
  5.0990195135927845,
  3.1622776601683795,
  9.0,
  5.0,
  0.0,
  11.180339887498949,
  7.0710678118654755,
  6.708203932499369,
  8.06225774829855,
  9.486832980505138,
  11.180339887498949,
  10.63014581273465,
  12.083045973594572,
  11.661903789690601,
  12.727922061357855,
  12.36931687685298,
  10.63014581273465,
  7.0710678118654755,
  13.0,
  13.45362404707371,
  15.652475842498529,
  8.0,
  5.0990195135927845,
  7.0,
  15.0,
  10.0,
  20.615528128088304,
  20.615528128088304],
 [22.360679774997898,
  54.230987451824994,
  52.23983154643591,
  50.00999900019995,
  50.039984012787215,
  48.010415536631214,
  46.04345773288535,
  46.17358552246078,
  48.703182647543684,
  48.25971404805462,
  49.16299421312742,
  50.24937810560445,
  51.86520991955976,
  50.99019513592785,
  52.773099207835045,
  52.3450093132096,
  65.19202405202648,
  63.702433234531945,
  19.924858845171276,
  67.20863039818622,
  64.35060217278468,
  62.289646009589745,
  70.76722405181653,
  66.48308055437865,
  71.449282711585,
  70.00714249274856,
  65.85590330410783,
  73.53910524340094,
  70.76722405181653,
  72.11102550927978,
  74.96665925596525,
  73.59347797189639,
  70.9365914038728,
  77.07788269017254,
  75.82216034906945,
  72.34638899074369,
  74.51845409024533,
  57.723478758647246,
  57.0701322935211,
  56.71860364994893,
  55.0,
  60.207972893961475,
  59.682493245507096,
  59.07622195096772,
  63.34824385884742,
  62.07253821135398,
  65.30696746902278,
  64.93843238021688,
  64.38167441127949,
  67.26812023536856,
  69.58448102845921,
  69.18092222571191,
  58.60034129593445,
  59.941638282582836,
  60.83584469702052,
  62.64982043070834,
  64.66065264130884,
  62.24146527838174,
  64.88451279003334,
  63.90618123468183,
  68.47627326307996,
  68.54195795277518,
  69.35416353759881,
  71.58910531638176,
  71.11258679024411,
  69.6419413859206,
  76.92203845452875,
  28.861739379323623,
  32.01562118716424,
  30.4138126514911,
  32.38826948140329,
  34.0147027033899,
  37.57658845611187,
  32.2490309931942,
  35.77708763999664,
  37.57658845611187,
  33.28663395418648,
  36.05551275463989,
  38.58756276314948,
  39.20459156782532,
  42.43819034784589,
  28.284271247461902,
  25.0,
  27.730849247724095,
  23.853720883753127,
  22.02271554554524,
  21.93171219946131,
  19.849433241279208,
  23.021728866442675,
  19.209372712298546,
  18.35755975068582,
  18.027756377319946,
  18.439088914585774,
  14.866068747318506,
  13.601470508735444,
  17.204650534085253,
  14.142135623730951,
  11.180339887498949,
  0.0,
  11.180339887498949,
  8.94427190999916,
  13.416407864998739,
  15.652475842498529,
  10.0,
  12.165525060596439,
  16.76305461424021,
  15.524174696260024,
  14.035668847618199,
  22.090722034374522,
  17.26267650163207,
  5.0,
  22.67156809750927,
  24.20743687382041,
  26.832815729997478,
  18.681541692269406,
  9.0,
  5.830951894845301,
  26.076809620810597,
  20.615528128088304,
  29.154759474226502,
  25.495097567963924],
 [11.180339887498949,
  49.25444142409901,
  47.265209192385896,
  46.32493928760188,
  46.57252408878007,
  44.384682042344295,
  42.720018726587654,
  43.32435804486894,
  48.54894437575342,
  45.5411901469428,
  46.17358552246078,
  47.43416490252569,
  51.62363799656123,
  49.24428900898052,
  50.695167422546305,
  49.64876634922564,
  68.8839603971781,
  67.62396025078685,
  14.560219778561036,
  71.21797525905943,
  68.81860213634101,
  67.11929677819934,
  74.65252842335616,
  71.02816342831905,
  75.43208866258443,
  74.2024258363566,
  70.7248188403477,
  77.80102827083971,
  75.45197147855052,
  78.26237921249263,
  79.40403012442127,
  78.2368199762746,
  76.00657866263946,
  81.40024569987489,
  80.7093550959243,
  77.38862965578342,
  79.61155694998057,
  66.61080993352356,
  66.27216610312357,
  66.12110101926616,
  65.19202405202648,
  69.35416353759881,
  69.1158447825099,
  69.02897942168927,
  72.44308110509934,
  72.02777242147643,
  74.43117626371358,
  74.24284477308234,
  74.02702209328699,
  76.4198926981712,
  79.1012010022604,
  79.0,
  69.77822009767804,
  71.11961754677819,
  72.01388754955533,
  73.824115301167,
  75.8023746329889,
  73.40980860893181,
  76.05918747922567,
  75.02666192761077,
  79.64923100695951,
  79.7057087039567,
  80.52949770115296,
  82.76472678623425,
  82.29216244576394,
  80.77747210701756,
  88.10221336606703,
  37.12142238654117,
  42.42640687119285,
  38.07886552931954,
  39.92492955535426,
  43.266615305567875,
  46.87216658103186,
  42.01190307520001,
  45.221676218380054,
  46.87216658103186,
  43.278170016764804,
  46.09772228646444,
  48.41487374764082,
  49.51767361255979,
  52.49761899362675,
  18.027756377319946,
  14.142135623730951,
  20.09975124224178,
  15.297058540778355,
  12.649110640673518,
  11.661903789690601,
  9.433981132056603,
  12.041594578792296,
  10.198039027185569,
  11.045361017187261,
  10.0,
  9.219544457292887,
  6.0,
  6.324555320336759,
  6.4031242374328485,
  5.0,
  7.0710678118654755,
  11.180339887498949,
  0.0,
  2.23606797749979,
  2.23606797749979,
  4.47213595499958,
  5.0,
  3.605551275463989,
  6.0,
  5.0990195135927845,
  5.656854249492381,
  18.788294228055936,
  17.69180601295413,
  10.0,
  12.206555615733702,
  14.866068747318506,
  19.1049731745428,
  13.92838827718412,
  10.770329614269007,
  5.385164807134504,
  17.46424919657298,
  15.811388300841896,
  18.027756377319946,
  15.0],
 [13.416407864998739,
  50.08991914547278,
  48.093658625644196,
  46.87216658103186,
  47.07440918375928,
  44.91102314577124,
  43.174066289845804,
  43.68065933568311,
  48.373546489791295,
  45.880278987817846,
  46.57252408878007,
  47.80167361086848,
  51.478150704935004,
  49.39635614091387,
  50.92150822589606,
  50.0,
  68.01470429252781,
  66.70832032063167,
  15.132745950421556,
  70.2922470831599,
  67.80117993073571,
  66.03029607687671,
  73.7563556583431,
  70.0,
  74.51845409024533,
  73.24616030891995,
  69.63476143421474,
  76.83749084919418,
  74.4043009509531,
  76.94153624668537,
  78.40918313565064,
  77.20103626247513,
  74.88658090739622,
  80.43009387039157,
  79.63039620647382,
  76.27581530209953,
  78.49203781276162,
  64.77653896280658,
  64.38167441127949,
  64.19501538281614,
  63.12685640834652,
  67.47592163134935,
  67.1863081289633,
  67.00746227100382,
  70.57619995437555,
  70.00714249274856,
  72.56031973468694,
  72.3394774656273,
  72.06247289678588,
  74.54528824815154,
  77.1621668954417,
  77.00649323271382,
  67.54257916307313,
  68.8839603971781,
  69.77822009767804,
  71.58910531638176,
  73.57309290766564,
  71.17583859709698,
  73.824115301167,
  72.80109889280519,
  77.41446893184762,
  77.47257579298626,
  78.29431652425353,
  80.52949770115296,
  80.05623023850174,
  78.54934754662193,
  85.86617494683223,
  35.34119409414458,
  40.311288741492746,
  36.40054944640259,
  38.27531841800928,
  41.340053217188775,
  44.94441010848846,
  40.0,
  43.266615305567875,
  44.94441010848846,
  41.23105625617661,
  44.04543109109048,
  46.400431032480725,
  47.4236228055175,
  50.44799302251776,
  20.0,
  16.278820596099706,
  21.37755832643195,
  16.76305461424021,
  14.317821063276353,
  13.601470508735444,
  11.40175425099138,
  14.212670403551895,
  11.704699910719626,
  12.041594578792296,
  11.180339887498949,
  10.770329614269007,
  7.280109889280518,
  7.0,
  8.48528137423857,
  6.324555320336759,
  6.708203932499369,
  8.94427190999916,
  2.23606797749979,
  0.0,
  4.47213595499958,
  6.708203932499369,
  4.47213595499958,
  4.47213595499958,
  8.06225774829855,
  7.0,
  6.708203932499369,
  18.973665961010276,
  17.029386365926403,
  8.06225774829855,
  14.212670403551895,
  16.55294535724685,
  20.396078054371138,
  14.317821063276353,
  9.433981132056603,
  3.1622776601683795,
  18.973665961010276,
  16.278820596099706,
  20.248456731316587,
  17.029386365926403],
 [8.94427190999916,
  48.507731342539614,
  46.52956049652737,
  45.880278987817846,
  46.17358552246078,
  43.965895873961216,
  42.37924020083418,
  43.08131845707603,
  48.82622246293481,
  45.31004303683677,
  45.880278987817846,
  47.16990566028302,
  51.86520991955976,
  49.193495504995376,
  50.566787519082126,
  49.39635614091387,
  69.81403870282824,
  68.60029154456998,
  14.317821063276353,
  72.20110802473879,
  69.8927750200262,
  68.26419266350405,
  75.6042326857432,
  72.11102550927978,
  76.40026177965623,
  75.21303078589507,
  71.8679344353238,
  78.81624198095213,
  76.55063683601855,
  79.62411694957753,
  80.44874144447506,
  79.32212806020776,
  77.17512552629896,
  82.41965784932621,
  81.83520025025906,
  78.54934754662193,
  80.77747210701756,
  68.46897107449476,
  68.18357573492314,
  68.06614430096654,
  67.26812023536856,
  71.25307010929424,
  71.06335201775947,
  71.06335201775947,
  74.33034373659252,
  74.06078584514209,
  76.32168761236873,
  76.16429609731846,
  76.00657866263946,
  78.31347265956222,
  81.05553651663777,
  81.00617260431454,
  72.01388754955533,
  73.35529974037323,
  74.24957912338628,
  76.05918747922567,
  78.03204469959762,
  75.64390259630977,
  78.29431652425353,
  77.25283166331187,
  81.88406438373708,
  81.93900170248598,
  82.76472678623425,
  85.0,
  84.52810183601665,
  83.00602387778854,
  90.33825324855468,
  38.948684188300895,
  44.553338819890925,
  39.81205847478876,
  41.6293165929973,
  45.221676218380054,
  48.82622246293481,
  44.04543109109048,
  47.20169488482379,
  48.82622246293481,
  45.34313619501854,
  48.16637831516918,
  50.44799302251776,
  51.62363799656123,
  54.56189146281496,
  16.1245154965971,
  12.041594578792296,
  19.0,
  14.035668847618199,
  11.180339887498949,
  9.848857801796104,
  7.615773105863909,
  9.899494936611665,
  9.0,
  10.44030650891055,
  9.219544457292887,
  8.0,
  5.385164807134504,
  6.4031242374328485,
  4.47213595499958,
  4.47213595499958,
  8.06225774829855,
  13.416407864998739,
  2.23606797749979,
  4.47213595499958,
  0.0,
  2.23606797749979,
  6.324555320336759,
  4.0,
  4.123105625617661,
  3.605551275463989,
  5.385164807134504,
  18.867962264113206,
  18.601075237738275,
  12.041594578792296,
  10.295630140987,
  13.341664064126334,
  18.0,
  13.892443989449804,
  12.36931687685298,
  7.615773105863909,
  16.1245154965971,
  15.652475842498529,
  15.811388300841896,
  13.038404810405298],
 [6.708203932499369,
  47.853944456021594,
  45.89117562233506,
  45.5411901469428,
  45.880278987817846,
  43.657759905886145,
  42.15447781671598,
  42.95346318982906,
  49.20365840057018,
  45.18849411078001,
  45.69463863518345,
  47.01063709417264,
  52.20153254455275,
  49.24428900898052,
  50.53711507397311,
  49.24428900898052,
  70.80254232723568,
  69.63476143421474,
  14.422205101855956,
  73.23933369440222,
  71.02112361825881,
  69.46221994724903,
  76.60939890117922,
  73.24616030891995,
  77.42092740338364,
  76.27581530209953,
  73.06161783043132,
  79.88116173416608,
  77.69813382572325,
  81.02468759581859,
  81.54140052758476,
  80.45495634204272,
  78.39005038906404,
  83.48652585896721,
  83.00602387778854,
  79.75587752636166,
  81.98780397107853,
  70.34912934784623,
  70.11419257183242,
  70.02856560004639,
  69.35416353759881,
  73.17103251970687,
  73.02739212103907,
  73.10950690573696,
  76.23647421018367,
  76.10519036176179,
  78.23042886243178,
  78.10249675906654,
  78.0,
  80.22468448052632,
  83.02409288875127,
  83.02409288875127,
  74.24957912338628,
  75.59100475585703,
  76.48529270389177,
  78.29431652425353,
  80.26207074328447,
  77.87810988975015,
  80.52949770115296,
  79.47955712005447,
  84.11896337925236,
  84.17244204607586,
  85.0,
  87.23531395025755,
  86.76404785393545,
  85.23496934944014,
  92.57429448826494,
  40.8166632639171,
  46.69047011971501,
  41.593268686170845,
  43.382023926967726,
  47.20169488482379,
  50.80354318352215,
  46.09772228646444,
  49.20365840057018,
  50.80354318352215,
  47.4236228055175,
  50.24937810560445,
  52.49761899362675,
  53.74011537017761,
  56.639209034025185,
  14.317821063276353,
  10.0,
  18.110770276274835,
  13.038404810405298,
  10.0,
  8.246211251235321,
  6.082762530298219,
  7.810249675906654,
  8.246211251235321,
  10.295630140987,
  8.94427190999916,
  7.280109889280518,
  5.656854249492381,
  7.211102550927978,
  3.0,
  5.0,
  9.486832980505138,
  15.652475842498529,
  4.47213595499958,
  6.708203932499369,
  2.23606797749979,
  0.0,
  8.06225774829855,
  5.385164807134504,
  2.8284271247461903,
  3.1622776601683795,
  6.0,
  19.209372712298546,
  19.72308292331602,
  14.142135623730951,
  8.54400374531753,
  12.041594578792296,
  17.11724276862369,
  14.212670403551895,
  14.142135623730951,
  9.848857801796104,
  15.0,
  15.811388300841896,
  13.601470508735444,
  11.180339887498949],
 [14.142135623730951,
  54.230987451824994,
  52.23983154643591,
  51.19570294468082,
  51.419840528729765,
  49.24428900898052,
  47.53945729601885,
  48.08326112068523,
  52.839379254491625,
  50.28916384272063,
  50.96076922496363,
  52.20153254455275,
  55.94640292279746,
  53.85164807134504,
  55.362442142665635,
  54.405882034941776,
  72.18032973047436,
  70.83784299369935,
  19.4164878389476,
  74.41102068914255,
  71.84010022264724,
  70.0,
  77.89736837660179,
  74.02702209328699,
  78.64477096412705,
  77.33692520394123,
  73.60027173862879,
  80.91971329657564,
  78.40918313565064,
  80.62257748298549,
  82.46211251235322,
  81.21576201698781,
  78.81624198095213,
  84.50443775329198,
  83.60023923410746,
  80.2122185206219,
  82.41965784932621,
  67.468511173732,
  66.91038783328041,
  66.61080993352356,
  65.0,
  70.03570517957252,
  69.58448102845921,
  69.06518659932803,
  73.16419889536138,
  72.06247289678588,
  75.13321502504735,
  74.81310045707235,
  74.33034373659252,
  77.10382610480494,
  79.51100552753688,
  79.1580697086532,
  67.77905281132217,
  68.94200461257273,
  70.00714249274856,
  71.58910531638176,
  73.35529974037323,
  71.5122367151245,
  73.824115301167,
  73.37574531137656,
  77.38862965578342,
  77.83315488916018,
  78.54934754662193,
  80.52949770115296,
  80.23091673413684,
  79.05694150420949,
  86.00581375697809,
  32.7566787083184,
  39.05124837953327,
  33.54101966249684,
  35.34119409414458,
  39.20459156782532,
  42.80186911806539,
  38.2099463490856,
  41.23105625617661,
  42.80186911806539,
  39.59797974644666,
  42.42640687119285,
  44.598206241955516,
  46.010868281309364,
  48.79549159502341,
  22.360679774997898,
  18.027756377319946,
  25.079872407968907,
  20.223748416156685,
  17.46424919657298,
  16.15549442140351,
  13.92838827718412,
  15.811388300841896,
  15.132745950421556,
  16.0312195418814,
  15.0,
  14.142135623730951,
  11.0,
  11.180339887498949,
  10.770329614269007,
  10.0,
  11.180339887498949,
  10.0,
  5.0,
  4.47213595499958,
  6.324555320336759,
  8.06225774829855,
  0.0,
  2.8284271247461903,
  7.810249675906654,
  6.4031242374328485,
  4.123105625617661,
  23.40939982143925,
  21.400934559032695,
  11.180339887498949,
  16.55294535724685,
  19.6468827043885,
  24.08318915758459,
  18.681541692269406,
  13.45362404707371,
  5.830951894845301,
  22.360679774997898,
  20.615528128088304,
  21.213203435596427,
  15.811388300841896],
 [11.313708498984761,
  52.46903848937962,
  50.48762224545735,
  49.72926703662542,
  50.0,
  47.80167361086848,
  46.17358552246078,
  46.8187996428785,
  52.15361924162119,
  49.040799340956916,
  49.64876634922564,
  50.92150822589606,
  55.226805085936306,
  52.80151512977634,
  54.230987451824994,
  53.14132102234569,
  72.34638899074369,
  71.06335201775947,
  18.027756377319946,
  74.65252842335616,
  72.20110802473879,
  70.45565981523414,
  78.10249675906654,
  74.4043009509531,
  78.87331614684399,
  77.62087348130012,
  74.06078584514209,
  81.21576201698781,
  78.81624198095213,
  81.41252974819048,
  82.80096617793781,
  81.60882305241266,
  79.32212806020776,
  84.81155581640984,
  84.0535543567314,
  80.7093550959243,
  82.92767933567175,
  69.05070600652827,
  68.59300255857006,
  68.3593446428504,
  67.02984409947557,
  71.7007670809734,
  71.34423592694787,
  71.00704190430693,
  74.81310045707235,
  74.00675644831355,
  76.79192665899195,
  76.53103945458993,
  76.16429609731846,
  78.77182237323191,
  81.30190649671138,
  81.05553651663777,
  70.3846574190711,
  71.58910531638176,
  72.61542535852834,
  74.24957912338628,
  76.05918747922567,
  74.09453421137081,
  76.48529270389177,
  75.8946638440411,
  80.05623023850174,
  80.4114419718985,
  81.15417426109393,
  83.19254774317228,
  82.85529554590944,
  81.59656855530139,
  88.63972021616495,
  35.510561809129406,
  41.86884283091664,
  36.235341863986875,
  38.01315561749642,
  42.01190307520001,
  45.60701700396552,
  41.036569057366385,
  44.04543109109048,
  45.60701700396552,
  42.42640687119285,
  45.254833995939045,
  47.4236228055175,
  48.83646178829912,
  51.62363799656123,
  19.697715603592208,
  15.264337522473747,
  23.0,
  18.027756377319946,
  15.132745950421556,
  13.601470508735444,
  11.40175425099138,
  13.038404810405298,
  13.0,
  14.317821063276353,
  13.152946437965905,
  12.0,
  9.219544457292887,
  9.848857801796104,
  8.246211251235321,
  8.246211251235321,
  10.63014581273465,
  12.165525060596439,
  3.605551275463989,
  4.47213595499958,
  4.0,
  5.385164807134504,
  2.8284271247461903,
  0.0,
  5.0,
  3.605551275463989,
  2.23606797749979,
  22.360679774997898,
  21.213203435596427,
  12.36931687685298,
  13.92838827718412,
  17.26267650163207,
  22.0,
  17.46424919657298,
  13.892443989449804,
  7.0710678118654755,
  20.09975124224178,
  19.313207915827967,
  18.384776310850235,
  13.341664064126334],
 [6.4031242374328485,
  50.21951811795888,
  48.27007354458868,
  48.104053883222775,
  48.46648326421054,
  46.238512086787566,
  44.77722635447622,
  45.617978911828175,
  52.009614495783374,
  47.853944456021594,
  48.33218389437829,
  49.658836071740545,
  55.0,
  51.97114584074513,
  53.23532661682466,
  51.894122981316485,
  73.60027173862879,
  72.42237223399962,
  17.204650534085253,
  76.02631123499285,
  73.78346698278686,
  72.20110802473879,
  79.40403012442127,
  76.00657866263946,
  80.2122185206219,
  79.05694150420949,
  75.8023746329889,
  82.66196223173026,
  80.45495634204272,
  83.6719785830358,
  84.31488599292535,
  83.21658488546619,
  81.12336284942828,
  86.2670273047588,
  85.75546629807339,
  82.49242389456137,
  84.72307831990054,
  72.56031973468694,
  72.24956747275377,
  72.11102550927978,
  71.17583859709698,
  75.32595834106593,
  75.1065909225016,
  75.02666192761077,
  78.40918313565064,
  78.02563681252464,
  80.39900496896712,
  80.22468448052632,
  80.02499609497022,
  82.38931969618392,
  85.09406559801923,
  85.0,
  75.23961722390672,
  76.48529270389177,
  77.47257579298626,
  79.1580697086532,
  81.00617260431454,
  78.92401408950256,
  81.39410298049853,
  80.65358020571685,
  84.970583144992,
  85.23496934944014,
  86.00581375697809,
  88.10221336606703,
  87.72684879784524,
  86.37708029332781,
  93.5200513259055,
  40.22437072223753,
  46.861498055439924,
  40.8166632639171,
  42.5440947723653,
  46.861498055439924,
  50.44799302251776,
  45.967379738244816,
  48.91829923454004,
  50.44799302251776,
  47.38143096192854,
  50.20956084253277,
  52.3450093132096,
  53.81449618829484,
  56.568542494923804,
  15.524174696260024,
  10.770329614269007,
  20.396078054371138,
  15.297058540778355,
  12.165525060596439,
  10.0,
  8.06225774829855,
  8.54400374531753,
  10.770329614269007,
  13.038404810405298,
  11.661903789690601,
  9.848857801796104,
  8.48528137423857,
  10.0,
  5.385164807134504,
  7.810249675906654,
  12.083045973594572,
  16.76305461424021,
  6.0,
  8.06225774829855,
  4.123105625617661,
  2.8284271247461903,
  7.810249675906654,
  5.0,
  0.0,
  1.4142135623730951,
  4.47213595499958,
  22.02271554554524,
  22.47220505424423,
  16.0,
  10.04987562112089,
  14.035668847618199,
  19.4164878389476,
  17.029386365926403,
  16.492422502470642,
  11.180339887498949,
  17.11724276862369,
  18.601075237738275,
  13.45362404707371,
  9.0],
 [7.810249675906654,
  50.99019513592785,
  49.03060268852505,
  48.703182647543684,
  49.040799340956916,
  46.8187996428785,
  45.31004303683677,
  46.09772228646444,
  52.20153254455275,
  48.33218389437829,
  48.84669896727925,
  50.15974481593781,
  55.21775076911409,
  52.354560450833695,
  53.665631459994955,
  52.392747589718944,
  73.40980860893181,
  72.20110802473879,
  17.4928556845359,
  75.8023746329889,
  73.49829930005184,
  71.8679344353238,
  79.20227269466452,
  75.71657678474378,
  80.0,
  78.81624198095213,
  75.47184905645283,
  82.41965784932621,
  80.15609770940699,
  83.19254774317228,
  84.0535543567314,
  82.92767933567175,
  80.77747210701756,
  86.02325267042627,
  85.44003745317531,
  82.15229759416349,
  84.38009243891595,
  71.7007670809734,
  71.34423592694787,
  71.17583859709698,
  70.11419257183242,
  74.43117626371358,
  74.16872656315464,
  74.00675644831355,
  77.52418977325722,
  77.00649323271382,
  79.51100552753688,
  79.30952023559341,
  79.05694150420949,
  81.49846624323675,
  84.1486779456457,
  84.00595217006948,
  73.92563831310488,
  75.15317691222374,
  76.15773105863909,
  77.82030583337487,
  79.64923100695951,
  77.62087348130012,
  80.05623023850174,
  79.37883848986453,
  83.6301381082203,
  83.93449827097318,
  84.69356528095862,
  86.76404785393545,
  86.4060183089118,
  85.09406559801923,
  92.19544457292888,
  38.8329756778952,
  45.45327270945405,
  39.44616584663204,
  41.182520563948,
  45.45327270945405,
  49.040799340956916,
  44.553338819890925,
  47.50789408087881,
  49.040799340956916,
  45.967379738244816,
  48.79549159502341,
  50.93132631298737,
  52.40229002629561,
  55.154328932550705,
  16.76305461424021,
  12.083045973594572,
  21.213203435596427,
  16.1245154965971,
  13.038404810405298,
  11.045361017187261,
  9.0,
  9.848857801796104,
  11.40175425099138,
  13.416407864998739,
  12.083045973594572,
  10.44030650891055,
  8.602325267042627,
  9.899494936611665,
  6.082762530298219,
  7.810249675906654,
  11.661903789690601,
  15.524174696260024,
  5.0990195135927845,
  7.0,
  3.605551275463989,
  3.1622776601683795,
  6.4031242374328485,
  3.605551275463989,
  1.4142135623730951,
  0.0,
  3.1622776601683795,
  22.20360331117452,
  22.20360331117452,
  15.033296378372908,
  11.180339887498949,
  15.0,
  20.223748416156685,
  17.204650534085253,
  15.811388300841896,
  10.04987562112089,
  18.027756377319946,
  18.867962264113206,
  14.866068747318506,
  10.04987562112089],
 [10.816653826391969,
  53.75872022286245,
  51.78802950489621,
  51.24451190127583,
  51.54609587543949,
  49.33558553417604,
  47.75981574503821,
  48.46648326421054,
  54.08326913195984,
  50.695167422546305,
  51.264022471905186,
  52.55473337388365,
  57.14017850864661,
  54.56189146281496,
  55.94640292279746,
  54.78138369920935,
  74.51845409024533,
  73.24616030891995,
  19.697715603592208,
  76.83749084919418,
  74.4043009509531,
  72.67048919609665,
  80.2807573457052,
  76.60939890117922,
  81.05553651663777,
  79.81227975693966,
  76.27581530209953,
  83.40863264674707,
  81.02468759581859,
  83.64807230295268,
  85.0,
  83.81527307120105,
  81.54140052758476,
  87.00574693662483,
  86.2670273047588,
  82.92767933567175,
  85.14693182963201,
  71.19691004531025,
  70.71067811865476,
  70.45565981523414,
  69.00724599634447,
  73.824115301167,
  73.43704787094863,
  73.02739212103907,
  76.94153624668537,
  76.02631123499285,
  78.91767862779544,
  78.63841300535,
  78.23042886243178,
  80.89499366462674,
  83.38465086573187,
  83.09632964216891,
  71.8679344353238,
  73.00684899377592,
  74.09453421137081,
  75.64390259630977,
  77.3692445355388,
  75.61084578286372,
  77.87810988975015,
  77.4919350642375,
  81.43709228600933,
  81.93289937503738,
  82.63776376451628,
  84.581321815162,
  84.30895563343196,
  83.16850365372699,
  90.07774419910837,
  35.805027579936315,
  42.80186911806539,
  36.359317925395686,
  38.07886552931954,
  42.5205832509386,
  46.09772228646444,
  41.72529209005013,
  44.598206241955516,
  46.09772228646444,
  43.18564576337837,
  46.010868281309364,
  48.08326112068523,
  49.678969393496885,
  52.3450093132096,
  19.924858845171276,
  15.231546211727817,
  24.08318915758459,
  19.026297590440446,
  16.0,
  14.142135623730951,
  12.041594578792296,
  13.0,
  14.142135623730951,
  15.811388300841896,
  14.560219778561036,
  13.152946437965905,
  10.770329614269007,
  11.661903789690601,
  9.0,
  9.848857801796104,
  12.727922061357855,
  14.035668847618199,
  5.656854249492381,
  6.708203932499369,
  5.385164807134504,
  6.0,
  4.123105625617661,
  2.23606797749979,
  4.47213595499958,
  3.1622776601683795,
  0.0,
  24.186773244895647,
  23.345235059857504,
  14.560219778561036,
  14.317821063276353,
  18.027756377319946,
  23.08679276123039,
  19.235384061671343,
  16.1245154965971,
  9.219544457292887,
  21.0,
  21.02379604162864,
  17.804493814764857,
  11.704699910719626],
 [21.633307652783937,
  32.14031735997639,
  30.14962686336267,
  28.160255680657446,
  28.284271247461902,
  26.1725046566048,
  24.331050121192877,
  24.73863375370596,
  30.0,
  26.92582403567252,
  27.65863337187866,
  28.861739379323623,
  33.015148038438355,
  30.463092423455635,
  31.953090617340916,
  31.04834939252005,
  52.478567053607705,
  51.478150704935004,
  5.0,
  55.072679252057455,
  53.225933528685054,
  52.0,
  58.309518948453004,
  55.46169849544819,
  59.16924876994806,
  58.180752831155424,
  55.54277630799526,
  61.773780845922005,
  59.93329625508679,
  64.5600495662759,
  63.52952069707436,
  62.609903369994115,
  60.92618484691127,
  65.36818798161687,
  65.30696746902278,
  62.24146527838174,
  64.47480127925948,
  58.54912467321779,
  59.033888572581766,
  59.43904440685432,
  61.09828148156051,
  61.814237842102365,
  62.369864518050704,
  63.89053137985315,
  64.6297145282261,
  66.76076692189807,
  66.61080993352356,
  66.91038783328041,
  67.68308503607086,
  68.59300255857006,
  72.18032973047436,
  73.00684899377592,
  72.34638899074369,
  74.33034373659252,
  74.51845409024533,
  77.15568676384133,
  79.90619500389191,
  75.43208866258443,
  79.30952023559341,
  76.02631123499285,
  82.87943049997388,
  81.40024569987489,
  82.61961026293939,
  85.7962703152066,
  84.64632301523794,
  81.8413098624405,
  90.4267659490264,
  50.80354318352215,
  51.894122981316485,
  52.46903848937962,
  54.45181356024793,
  55.362442142665635,
  58.82176467941097,
  53.14132102234569,
  56.92099788303083,
  58.82176467941097,
  53.85164807134504,
  56.462376853972415,
  59.23681287847955,
  59.033888572581766,
  62.64982043070834,
  18.110770276274835,
  19.313207915827967,
  10.44030650891055,
  11.180339887498949,
  13.0,
  15.652475842498529,
  15.811388300841896,
  19.235384061671343,
  12.206555615733702,
  9.219544457292887,
  10.63014581273465,
  12.806248474865697,
  13.601470508735444,
  12.529964086141668,
  16.97056274847714,
  14.422205101855956,
  12.36931687685298,
  22.090722034374522,
  18.788294228055936,
  18.973665961010276,
  18.867962264113206,
  19.209372712298546,
  23.40939982143925,
  22.360679774997898,
  22.02271554554524,
  22.20360331117452,
  24.186773244895647,
  0.0,
  7.0710678118654755,
  17.11724276862369,
  16.55294535724685,
  13.341664064126334,
  10.198039027185569,
  5.0,
  13.152946437965905,
  19.235384061671343,
  12.0,
  3.605551275463989,
  24.041630560342615,
  28.600699292150182],
 [24.041630560342615,
  37.8549864614954,
  35.90264614203248,
  33.06055050963308,
  33.015148038438355,
  31.064449134018133,
  29.017236257093817,
  29.017236257093817,
  31.78049716414141,
  31.064449134018133,
  32.01562118716424,
  33.06055050963308,
  34.92849839314596,
  33.734255586866,
  35.510561809129406,
  35.12833614050059,
  51.22499389946279,
  50.0,
  9.219544457292887,
  53.600373133029585,
  51.31276644267,
  49.73932046178355,
  57.0087712549569,
  53.53503525729669,
  57.8013840664737,
  56.61271941887264,
  53.33854141237835,
  60.21627686929839,
  57.982756057296896,
  61.465437442517235,
  61.85466837676846,
  60.74537019394976,
  58.66856057549052,
  63.8200595424354,
  63.28506932918696,
  60.03332407921454,
  62.26556030423239,
  53.08483775994799,
  53.33854141237835,
  53.600373133029585,
  54.70831746635972,
  56.22277118748239,
  56.568542494923804,
  57.723478758647246,
  59.135437767890075,
  60.63827174318213,
  61.1310068623117,
  61.29437168288782,
  61.814237842102365,
  63.12685640834652,
  66.48308055437865,
  67.08203932499369,
  65.29931086925804,
  67.26812023536856,
  67.47592163134935,
  70.09279563550022,
  72.83543093852057,
  68.41052550594829,
  72.24956747275377,
  69.06518659932803,
  75.82216034906945,
  74.4043009509531,
  75.6042326857432,
  78.74642849044012,
  77.62087348130012,
  74.88658090739622,
  83.40863264674707,
  45.17742799230607,
  45.31004303683677,
  47.042533945356304,
  49.040799340956916,
  49.24428900898052,
  52.630789467763066,
  46.84015371452148,
  50.695167422546305,
  52.630789467763066,
  47.43416490252569,
  49.9799959983992,
  52.81098370604357,
  52.392747589718944,
  56.08029957123981,
  23.194827009486403,
  23.08679276123039,
  17.0,
  16.278820596099706,
  17.0,
  19.1049731745428,
  18.439088914585774,
  22.360679774997898,
  15.132745950421556,
  12.041594578792296,
  13.152946437965905,
  15.297058540778355,
  14.317821063276353,
  12.529964086141668,
  18.384776310850235,
  14.7648230602334,
  10.63014581273465,
  17.26267650163207,
  17.69180601295413,
  17.029386365926403,
  18.601075237738275,
  19.72308292331602,
  21.400934559032695,
  21.213203435596427,
  22.47220505424423,
  22.20360331117452,
  23.345235059857504,
  7.0710678118654755,
  0.0,
  12.36931687685298,
  20.09975124224178,
  18.110770276274835,
  16.55294535724685,
  8.06225774829855,
  8.54400374531753,
  16.1245154965971,
  17.72004514666935,
  8.54400374531753,
  28.071337695236398,
  30.463092423455635],
 [20.615528128088304,
  49.25444142409901,
  47.265209192385896,
  45.0111097397076,
  45.044422518220834,
  43.01162633521314,
  41.048751503547585,
  41.19465984809196,
  44.01136216933077,
  43.289721643826724,
  44.181444068749045,
  45.27692569068709,
  47.16990566028302,
  46.09772228646444,
  47.853944456021594,
  47.38143096192854,
  61.5223536610881,
  60.108235708594876,
  15.231546211727817,
  63.655321851358195,
  60.959002616512684,
  59.033888572581766,
  67.17886572427373,
  63.12685640834652,
  67.89698078707183,
  66.52818951391959,
  62.625873247404705,
  70.09279563550022,
  67.47592163134935,
  69.46221994724903,
  71.58910531638176,
  70.2922470831599,
  67.80117993073571,
  73.66138744281159,
  72.62231062146122,
  69.20260110718382,
  71.40028011149536,
  56.71860364994893,
  56.32051136131489,
  56.142675390472796,
  55.226805085936306,
  59.413803110051795,
  59.135437767890075,
  59.033888572581766,
  62.51399843235114,
  62.03224967708329,
  64.4980619863884,
  64.28063471995279,
  64.03124237432849,
  66.48308055437865,
  69.1158447825099,
  69.0,
  60.90155991434045,
  62.433965115151864,
  63.13477647065839,
  65.19202405202648,
  67.42403132415029,
  64.41273166075166,
  67.4166151627327,
  65.79513659838392,
  71.02112361825881,
  70.66116330771806,
  71.58910531638176,
  74.09453421137081,
  73.43023900274328,
  71.58910531638176,
  79.25906888173743,
  33.734255586866,
  36.05551275463989,
  35.35533905932738,
  37.33630940518894,
  38.62641583165593,
  42.15447781671598,
  36.6742416417845,
  40.311288741492746,
  42.15447781671598,
  37.589892258425,
  40.311288741492746,
  42.941821107167776,
  43.266615305567875,
  46.647615158762406,
  25.0,
  22.360679774997898,
  23.323807579381203,
  19.849433241279208,
  18.439088914585774,
  18.867962264113206,
  17.0,
  20.615528128088304,
  15.620499351813308,
  14.212670403551895,
  14.142135623730951,
  15.0,
  11.661903789690601,
  10.0,
  14.866068747318506,
  11.180339887498949,
  7.0710678118654755,
  5.0,
  10.0,
  8.06225774829855,
  12.041594578792296,
  14.142135623730951,
  11.180339887498949,
  12.36931687685298,
  16.0,
  15.033296378372908,
  14.560219778561036,
  17.11724276862369,
  12.36931687685298,
  0.0,
  19.72308292331602,
  20.518284528683193,
  22.47220505424423,
  13.92838827718412,
  4.0,
  5.385164807134504,
  22.02271554554524,
  15.811388300841896,
  26.92582403567252,
  25.0],
 [5.830951894845301,
  40.80441152620633,
  38.897300677553446,
  39.35733730830886,
  39.824615503479755,
  37.589892258425,
  36.359317925395686,
  37.44329045369811,
  45.27692569068709,
  39.66106403010388,
  39.96248240537617,
  41.340053217188775,
  48.08326112068523,
  44.204072210600685,
  45.221676218380054,
  43.56604182158393,
  68.96375859826666,
  68.0,
  11.704699910719626,
  71.58910531638176,
  69.77822009767804,
  68.54195795277518,
  74.79304780526061,
  72.01388754955533,
  75.66372975210778,
  74.70609078247904,
  72.09022125087424,
  78.29431652425353,
  76.48529270389177,
  80.95677859203639,
  80.06247560499239,
  79.1580697086532,
  77.47257579298626,
  81.88406438373708,
  81.85963596303125,
  78.79086241436883,
  81.02468759581859,
  73.00684899377592,
  73.10950690573696,
  73.24616030891995,
  73.54590403278758,
  76.05918747922567,
  76.23647421018367,
  76.94153624668537,
  79.02531240052139,
  79.90619500389191,
  81.02468759581859,
  81.09870529176159,
  81.39410298049853,
  83.02409288875127,
  86.20904824900923,
  86.57944328765345,
  80.62257748298549,
  82.15229759416349,
  82.85529554590944,
  84.90583018850944,
  87.09190547921202,
  84.11896337925236,
  87.13208364316786,
  85.42833253669417,
  90.7358804442873,
  90.35485598461214,
  91.30169768410661,
  93.81364506296512,
  93.15041599477696,
  91.23595782365635,
  98.97979591815695,
  49.24428900898052,
  54.48853090330111,
  50.08991914547278,
  51.894122981316485,
  55.47071299343465,
  59.07622195096772,
  54.20332093147061,
  57.42821606144492,
  59.07622195096772,
  55.44366510251645,
  58.25804665451803,
  60.60528029800704,
  61.61980201201558,
  64.66065264130884,
  5.830951894845301,
  3.0,
  11.180339887498949,
  6.4031242374328485,
  3.605551275463989,
  1.0,
  2.8284271247461903,
  2.8284271247461903,
  5.0,
  8.06225774829855,
  7.0,
  5.0990195135927845,
  8.06225774829855,
  9.848857801796104,
  5.830951894845301,
  8.602325267042627,
  13.0,
  22.67156809750927,
  12.206555615733702,
  14.212670403551895,
  10.295630140987,
  8.54400374531753,
  16.55294535724685,
  13.92838827718412,
  10.04987562112089,
  11.180339887498949,
  14.317821063276353,
  16.55294535724685,
  20.09975124224178,
  19.72308292331602,
  0.0,
  4.47213595499958,
  10.295630140987,
  12.36931687685298,
  18.027756377319946,
  16.97056274847714,
  7.615773105863909,
  13.0,
  8.0,
  12.806248474865697],
 [10.295630140987,
  36.40054944640259,
  34.48187929913333,
  34.88552708502482,
  35.35533905932738,
  33.12099032335839,
  31.906112267087632,
  33.015148038438355,
  41.10960958218893,
  35.22782990761707,
  35.510561809129406,
  36.89173349139343,
  43.86342439892262,
  39.824615503479755,
  40.80441152620633,
  39.11521443121589,
  65.29931086925804,
  64.40496875241847,
  9.0,
  67.97793759742936,
  66.30987860040162,
  65.19202405202648,
  71.11961754677819,
  68.54195795277518,
  72.00694410957877,
  71.11258679024411,
  68.70953354520753,
  74.68600939935136,
  73.00684899377592,
  77.88452991448301,
  76.48529270389177,
  75.64390259630977,
  74.09453421137081,
  78.26237921249263,
  78.39005038906404,
  75.39230729988306,
  77.62087348130012,
  71.17583859709698,
  71.449282711585,
  71.7007670809734,
  72.53275122315436,
  74.33034373659252,
  74.67261881037788,
  75.7099729229908,
  77.23341245859851,
  78.64477096412705,
  79.22752047110903,
  79.40403012442127,
  79.90619500389191,
  81.2219182240853,
  84.5931439302264,
  85.15867542417507,
  80.99382692526635,
  82.66196223173026,
  83.21658488546619,
  85.44588931013593,
  87.80091115700337,
  84.38009243891595,
  87.65842800324451,
  85.47514258543241,
  91.2633551870629,
  90.55385138137417,
  91.58602513484249,
  94.30270409696638,
  93.49331526906082,
  91.30169768410661,
  99.32270636667126,
  51.894122981316485,
  56.22277118748239,
  52.92447448959697,
  54.78138369920935,
  57.8013840664737,
  61.40032573203501,
  56.302753041036986,
  59.682493245507096,
  61.40032573203501,
  57.42821606144492,
  60.21627686929839,
  62.68173577685928,
  63.41135544995076,
  66.61080993352356,
  5.0990195135927845,
  6.4031242374328485,
  6.708203932499369,
  2.23606797749979,
  2.23606797749979,
  4.123105625617661,
  6.0,
  7.211102550927978,
  5.0,
  6.708203932499369,
  6.4031242374328485,
  5.830951894845301,
  9.433981132056603,
  10.63014581273465,
  9.055385138137417,
  10.295630140987,
  13.45362404707371,
  24.20743687382041,
  14.866068747318506,
  16.55294535724685,
  13.341664064126334,
  12.041594578792296,
  19.6468827043885,
  17.26267650163207,
  14.035668847618199,
  15.0,
  18.027756377319946,
  13.341664064126334,
  18.110770276274835,
  20.518284528683193,
  4.47213595499958,
  0.0,
  5.830951894845301,
  10.04987562112089,
  18.027756377319946,
  18.867962264113206,
  3.1622776601683795,
  10.04987562112089,
  10.770329614269007,
  17.204650534085253],
 [16.1245154965971,
  30.805843601498726,
  28.861739379323623,
  29.068883707497267,
  29.5296461204668,
  27.294688127912362,
  26.076809620810597,
  27.202941017470888,
  35.608987629529715,
  29.410882339705484,
  29.68164415931166,
  31.064449134018133,
  38.28837943815329,
  34.058772731852805,
  35.0,
  33.28663395418648,
  60.41522986797286,
  59.61543424315552,
  7.810249675906654,
  63.15853069855251,
  61.68468205316454,
  60.728905802755904,
  66.2117814289874,
  63.90618123468183,
  67.11929677819934,
  66.30987860040162,
  64.19501538281614,
  69.85699678629192,
  68.35202996254024,
  73.7563556583431,
  71.69379331573968,
  70.9365914038728,
  69.57010852370435,
  73.40980860893181,
  73.7360156233031,
  70.83784299369935,
  73.05477397131553,
  68.73136110975832,
  69.2314957226839,
  69.63476143421474,
  71.16881339463234,
  72.00694410957877,
  72.56721022610694,
  74.04052944165107,
  74.81310045707235,
  76.92203845452875,
  76.79192665899195,
  77.10382610480494,
  77.87810988975015,
  78.77182237323191,
  82.37718130647589,
  83.19855767980596,
  81.40024569987489,
  83.24061508662703,
  83.60023923410746,
  86.05230967266364,
  88.61715409558128,
  84.62860036654276,
  88.23831367382311,
  85.44003745317531,
  91.8313671900838,
  90.69729874698585,
  91.83681179135085,
  94.81033698917012,
  93.81364506296512,
  91.26883367283708,
  99.62429422585637,
  55.362442142665635,
  58.52349955359813,
  56.61271941887264,
  58.52349955359813,
  60.83584469702052,
  64.40496875241847,
  59.0592922409336,
  62.609903369994115,
  64.40496875241847,
  60.03332407921454,
  62.76941930590086,
  65.36818798161687,
  65.73431371817918,
  69.1158447825099,
  8.94427190999916,
  12.041594578792296,
  1.0,
  4.123105625617661,
  7.280109889280518,
  9.848857801796104,
  11.40175425099138,
  13.038404810405298,
  9.0,
  8.54400374531753,
  9.219544457292887,
  10.0,
  13.152946437965905,
  13.601470508735444,
  14.142135623730951,
  14.142135623730951,
  15.652475842498529,
  26.832815729997478,
  19.1049731745428,
  20.396078054371138,
  18.0,
  17.11724276862369,
  24.08318915758459,
  22.0,
  19.4164878389476,
  20.223748416156685,
  23.08679276123039,
  10.198039027185569,
  16.55294535724685,
  22.47220505424423,
  10.295630140987,
  5.830951894845301,
  0.0,
  9.219544457292887,
  19.209372712298546,
  22.135943621178654,
  2.8284271247461903,
  8.06225774829855,
  15.811388300841896,
  23.021728866442675],
 [17.0,
  36.0,
  34.0,
  32.55764119219941,
  32.7566787083184,
  30.59411708155671,
  28.861739379323623,
  29.410882339705484,
  35.0,
  31.622776601683793,
  32.28002478313795,
  33.52610922848042,
  38.01315561749642,
  35.34119409414458,
  36.76955262170047,
  35.73513677041127,
  57.14017850864661,
  56.08029957123981,
  1.4142135623730951,
  59.682493245507096,
  57.706152185014034,
  56.36488268416781,
  62.96824596572466,
  59.941638282582836,
  63.812224534175265,
  62.76941930590086,
  59.93329625508679,
  66.37017402418047,
  64.41273166075166,
  68.62215385719105,
  68.09552114493287,
  67.11929677819934,
  65.30696746902278,
  69.9714227381436,
  69.77105417004964,
  66.64082832618455,
  68.87670143089025,
  61.1310068623117,
  61.40032573203501,
  61.66036003787198,
  62.6418390534633,
  64.28063471995279,
  64.6297145282261,
  65.73431371817918,
  67.1863081289633,
  68.65857557508748,
  69.18092222571191,
  69.35416353759881,
  69.87131027825369,
  71.17583859709698,
  74.54528824815154,
  75.13321502504735,
  72.20110802473879,
  74.02702209328699,
  74.4043009509531,
  76.83749084919418,
  79.39773296511683,
  75.45197147855052,
  79.02531240052139,
  76.32168761236873,
  82.61961026293939,
  81.54140052758476,
  82.66196223173026,
  85.60373823613078,
  84.62860036654276,
  82.15229759416349,
  90.44335243676011,
  47.53945729601885,
  49.73932046178355,
  49.03060268852505,
  50.99019513592785,
  52.55473337388365,
  56.08029957123981,
  50.566787519082126,
  54.230987451824994,
  56.08029957123981,
  51.42956348249516,
  54.120236510939236,
  56.79788728465171,
  56.938563381947034,
  60.41522986797286,
  15.132745950421556,
  15.297058540778355,
  9.899494936611665,
  8.246211251235321,
  9.055385138137417,
  11.40175425099138,
  11.180339887498949,
  14.866068747318506,
  7.615773105863909,
  4.47213595499958,
  5.830951894845301,
  8.06225774829855,
  8.602325267042627,
  7.615773105863909,
  12.041594578792296,
  9.433981132056603,
  8.0,
  18.681541692269406,
  13.92838827718412,
  14.317821063276353,
  13.892443989449804,
  14.212670403551895,
  18.681541692269406,
  17.46424919657298,
  17.029386365926403,
  17.204650534085253,
  19.235384061671343,
  5.0,
  8.06225774829855,
  13.92838827718412,
  12.36931687685298,
  10.04987562112089,
  9.219544457292887,
  0.0,
  10.295630140987,
  15.0,
  9.848857801796104,
  2.0,
  20.223748416156685,
  23.853720883753127],
 [20.024984394500787,
  45.27692569068709,
  43.289721643826724,
  41.012193308819754,
  41.048751503547585,
  39.01281840626232,
  37.05401462729781,
  37.21558813185679,
  40.311288741492746,
  39.319206502675,
  40.19950248448356,
  41.30375285612676,
  43.46262762420146,
  42.20189569201838,
  43.93176527297759,
  43.41658669218482,
  58.728187440104094,
  57.38466694161429,
  11.661903789690601,
  60.959002616512684,
  58.412327466040935,
  56.61271941887264,
  64.4437739428721,
  60.60528029800704,
  65.19202405202648,
  63.89053137985315,
  60.21627686929839,
  67.47592163134935,
  65.0,
  67.53517601961218,
  69.02897942168927,
  67.80117993073571,
  65.45991139621256,
  71.06335201775947,
  70.21395872616783,
  66.85057965343307,
  69.06518659932803,
  56.22277118748239,
  56.0357029044876,
  56.0,
  55.731499172371095,
  59.07622195096772,
  59.0,
  59.30430001273095,
  62.12889826803627,
  62.289646009589745,
  64.1248781675256,
  64.03124237432849,
  64.03124237432849,
  66.12110101926616,
  69.0,
  69.1158447825099,
  62.96824596572466,
  64.63745044476924,
  65.19202405202648,
  67.42403132415029,
  69.81403870282824,
  66.37017402418047,
  69.63476143421474,
  67.53517601961218,
  73.23933369440222,
  72.56031973468694,
  73.57309290766564,
  76.27581530209953,
  75.47184905645283,
  73.35529974037323,
  81.30190649671138,
  37.656340767525464,
  39.44616584663204,
  39.319206502675,
  41.30375285612676,
  42.37924020083418,
  45.880278987817846,
  40.311288741492746,
  44.01136216933077,
  45.880278987817846,
  41.14608122288197,
  43.829214001622255,
  46.51881339845203,
  46.647615158762406,
  50.11985634456667,
  22.825424421026653,
  20.8806130178211,
  20.0,
  17.029386365926403,
  16.1245154965971,
  17.08800749063506,
  15.524174696260024,
  19.4164878389476,
  13.416407864998739,
  11.40175425099138,
  11.661903789690601,
  13.0,
  10.198039027185569,
  8.246211251235321,
  14.035668847618199,
  10.04987562112089,
  5.0990195135927845,
  9.0,
  10.770329614269007,
  9.433981132056603,
  12.36931687685298,
  14.142135623730951,
  13.45362404707371,
  13.892443989449804,
  16.492422502470642,
  15.811388300841896,
  16.1245154965971,
  13.152946437965905,
  8.54400374531753,
  4.0,
  18.027756377319946,
  18.027756377319946,
  19.209372712298546,
  10.295630140987,
  0.0,
  7.810249675906654,
  19.1049731745428,
  12.083045973594572,
  25.709920264364882,
  25.317977802344327],
 [16.55294535724685,
  51.0,
  49.0,
  47.38143096192854,
  47.51841748206689,
  45.39823785126467,
  43.56604182158393,
  43.93176527297759,
  47.853944456021594,
  46.09772228646444,
  46.87216658103186,
  48.05205510693585,
  50.99019513592785,
  49.33558553417604,
  50.96076922496363,
  50.21951811795888,
  66.48308055437865,
  65.11528238439882,
  16.0312195418814,
  68.6804193347711,
  66.06814663663572,
  64.20280367709809,
  72.18032973047436,
  68.24954212300622,
  72.91776189653656,
  71.58910531638176,
  67.80117993073571,
  75.16648189186454,
  72.62231062146122,
  74.79304780526061,
  76.69419795525604,
  75.43208866258443,
  73.00684899377592,
  78.74642849044012,
  77.80102827083971,
  74.4043009509531,
  76.60939890117922,
  61.98386886924694,
  61.5223536610881,
  61.29437168288782,
  60.07495318350236,
  64.6297145282261,
  64.28063471995279,
  64.0,
  67.7421582177598,
  67.0,
  69.72087205421343,
  69.46221994724903,
  69.1158447825099,
  71.7007670809734,
  74.24284477308234,
  74.02702209328699,
  64.40496875241847,
  65.76473218982953,
  66.64082832618455,
  68.47627326307996,
  70.49113419430843,
  68.0294054067798,
  70.71067811865476,
  69.6419413859206,
  74.30343195303969,
  74.32361670424818,
  75.15317691222374,
  77.41446893184762,
  76.92203845452875,
  75.39230729988306,
  82.73451516749222,
  33.24154027718932,
  37.53664875824692,
  34.48187929913333,
  36.40054944640259,
  38.948684188300895,
  42.5440947723653,
  37.44329045369811,
  40.8166632639171,
  42.5440947723653,
  38.600518131237564,
  41.400483088968905,
  43.829214001622255,
  44.68780594300866,
  47.80167361086848,
  22.67156809750927,
  19.209372712298546,
  23.08679276123039,
  18.788294228055936,
  16.64331697709324,
  16.278820596099706,
  14.142135623730951,
  17.204650534085253,
  13.892443989449804,
  13.601470508735444,
  13.0,
  13.038404810405298,
  9.433981132056603,
  8.54400374531753,
  11.40175425099138,
  8.602325267042627,
  7.0,
  5.830951894845301,
  5.385164807134504,
  3.1622776601683795,
  7.615773105863909,
  9.848857801796104,
  5.830951894845301,
  7.0710678118654755,
  11.180339887498949,
  10.04987562112089,
  9.219544457292887,
  19.235384061671343,
  16.1245154965971,
  5.385164807134504,
  16.97056274847714,
  18.867962264113206,
  22.135943621178654,
  15.0,
  7.810249675906654,
  0.0,
  21.02379604162864,
  17.0,
  23.323807579381203,
  20.09975124224178],
 [13.416407864998739,
  33.24154027718932,
  31.32091952673165,
  31.76476034853718,
  32.2490309931942,
  30.01666203960727,
  28.844410203711913,
  30.0,
  38.41874542459709,
  32.202484376209235,
  32.449961479175904,
  33.83784863137726,
  41.10960958218893,
  36.87817782917155,
  37.8021163428716,
  36.05551275463989,
  63.071388124885914,
  62.24146527838174,
  8.54400374531753,
  65.79513659838392,
  64.25729530566937,
  63.245553203367585,
  68.87670143089025,
  66.48308055437865,
  69.77822009767804,
  68.94200461257273,
  66.73080248281148,
  72.49827584156743,
  70.9365914038728,
  76.15773105863909,
  74.32361670424818,
  73.53910524340094,
  72.11102550927978,
  76.05918747922567,
  76.32168761236873,
  73.38937252763509,
  75.61084578286372,
  70.45565981523414,
  70.8590149522275,
  71.19691004531025,
  72.42237223399962,
  73.68174807915459,
  74.14849964766651,
  75.43208866258443,
  76.53103945458993,
  78.33900688673555,
  78.5175139698144,
  78.77182237323191,
  79.42921376924235,
  80.50465825031493,
  84.01190391843289,
  84.72307831990054,
  81.8413098624405,
  83.60023923410746,
  84.0535543567314,
  86.40023148117139,
  88.86506625215557,
  85.14693182963201,
  88.60022573334675,
  86.09297300012354,
  92.20086767487604,
  91.26883367283708,
  92.35799911215054,
  95.2102935611481,
  94.30270409696638,
  91.92388155425118,
  100.12492197250393,
  54.230987451824994,
  58.008620049092706,
  55.362442142665635,
  57.245087125446844,
  59.941638282582836,
  63.52952069707436,
  58.309518948453004,
  61.773780845922005,
  63.52952069707436,
  59.36328831862332,
  62.12889826803627,
  64.66065264130884,
  65.21502894272147,
  68.5054742338158,
  6.324555320336759,
  9.219544457292887,
  3.605551275463989,
  2.23606797749979,
  5.0,
  7.280109889280518,
  9.055385138137417,
  10.295630140987,
  7.280109889280518,
  7.810249675906654,
  8.06225774829855,
  8.246211251235321,
  11.704699910719626,
  12.529964086141668,
  12.0,
  12.649110640673518,
  15.0,
  26.076809620810597,
  17.46424919657298,
  18.973665961010276,
  16.1245154965971,
  15.0,
  22.360679774997898,
  20.09975124224178,
  17.11724276862369,
  18.027756377319946,
  21.0,
  12.0,
  17.72004514666935,
  22.02271554554524,
  7.615773105863909,
  3.1622776601683795,
  2.8284271247461903,
  9.848857801796104,
  19.1049731745428,
  21.02379604162864,
  0.0,
  9.219544457292887,
  13.038404810405298,
  20.248456731316587],
 [18.027756377319946,
  34.0,
  32.0,
  30.59411708155671,
  30.805843601498726,
  28.635642126552707,
  26.92582403567252,
  27.51363298439521,
  33.421549934136806,
  29.732137494637012,
  30.364452901377952,
  31.622776601683793,
  36.40054944640259,
  33.54101966249684,
  34.92849839314596,
  33.83784863137726,
  56.08029957123981,
  55.072679252057455,
  1.4142135623730951,
  58.66856057549052,
  56.79788728465171,
  55.54277630799526,
  61.91122676865643,
  59.033888572581766,
  62.76941930590086,
  61.773780845922005,
  59.09314681077663,
  65.36818798161687,
  63.50590523722971,
  68.00735254367721,
  67.11929677819934,
  66.18912297349165,
  64.47480127925948,
  68.96375859826666,
  68.87670143089025,
  65.79513659838392,
  68.0294054067798,
  61.29437168288782,
  61.66036003787198,
  61.98386886924694,
  63.245553203367585,
  64.4980619863884,
  64.93843238021688,
  66.21933252457322,
  67.36467917239716,
  69.12307863514182,
  69.35416353759881,
  69.58448102845921,
  70.21395872616783,
  71.34423592694787,
  74.81310045707235,
  75.50496672405067,
  73.40980860893181,
  75.28612089887484,
  75.6042326857432,
  78.10249675906654,
  80.72174428244226,
  76.60939890117922,
  80.2807573457052,
  77.38862965578342,
  83.86894538504703,
  82.66196223173026,
  83.81527307120105,
  86.83317338436964,
  85.80209787645055,
  83.21658488546619,
  91.60785992479029,
  49.47726750741192,
  51.478150704935004,
  50.99019513592785,
  52.952809179494906,
  54.42425929675111,
  57.9396237474839,
  52.392747589718944,
  56.08029957123981,
  57.9396237474839,
  53.225933528685054,
  55.90169943749474,
  58.60034129593445,
  58.66856057549052,
  62.177166226839255,
  15.0,
  15.811388300841896,
  8.602325267042627,
  8.0,
  9.486832980505138,
  12.083045973594572,
  12.206555615733702,
  15.652475842498529,
  8.602325267042627,
  5.656854249492381,
  7.0710678118654755,
  9.219544457292887,
  10.295630140987,
  9.486832980505138,
  13.45362404707371,
  11.180339887498949,
  10.0,
  20.615528128088304,
  15.811388300841896,
  16.278820596099706,
  15.652475842498529,
  15.811388300841896,
  20.615528128088304,
  19.313207915827967,
  18.601075237738275,
  18.867962264113206,
  21.02379604162864,
  3.605551275463989,
  8.54400374531753,
  15.811388300841896,
  13.0,
  10.04987562112089,
  8.06225774829855,
  2.0,
  12.083045973594572,
  17.0,
  9.219544457292887,
  0.0,
  20.615528128088304,
  25.0],
 [7.0710678118654755,
  43.829214001622255,
  42.05948168962618,
  43.60045871318328,
  44.204072210600685,
  42.01190307520001,
  41.10960958218893,
  42.44997055358225,
  51.40038910358559,
  44.598206241955516,
  44.68780594300866,
  46.09772228646444,
  54.037024344425184,
  49.49747468305833,
  50.24937810560445,
  48.27007354458868,
  76.02631123499285,
  75.15317691222374,
  19.4164878389476,
  78.7210264160726,
  77.07788269017254,
  75.96051605933177,
  81.8413098624405,
  79.30952023559341,
  82.73451516749222,
  81.85963596303125,
  79.47955712005447,
  85.42833253669417,
  83.773504164503,
  88.60022573334675,
  87.23531395025755,
  86.4060183089118,
  84.86459803710851,
  89.0,
  89.15716460273958,
  86.16263691415206,
  88.3911760301898,
  81.00617260431454,
  81.09870529176159,
  81.2219182240853,
  81.39410298049853,
  84.0535543567314,
  84.2140130857092,
  84.8528137423857,
  87.02298546935747,
  87.82368700982668,
  89.02246907382428,
  89.08984229416954,
  89.35882720806042,
  91.02197536858887,
  94.19129471453293,
  94.53041838477179,
  87.65842800324451,
  89.06739021662193,
  89.89438247187641,
  91.7877987534291,
  93.81364506296512,
  91.23595782365635,
  94.02127418834527,
  92.70382947861431,
  97.6165969494942,
  97.50897394599124,
  98.38699100999075,
  100.72239075796404,
  100.18482919085105,
  98.48857801796105,
  106.00471687618433,
  53.600373133029585,
  60.207972893961475,
  54.08326913195984,
  55.758407437802596,
  60.30754513325841,
  63.89053137985315,
  59.413803110051795,
  62.369864518050704,
  63.89053137985315,
  60.81118318204309,
  63.63961030678928,
  65.79513659838392,
  67.20863039818622,
  70.00714249274856,
  7.0710678118654755,
  5.0,
  16.401219466856727,
  13.0,
  11.180339887498949,
  9.0,
  10.198039027185569,
  6.324555320336759,
  13.0,
  16.0312195418814,
  15.0,
  13.038404810405298,
  15.524174696260024,
  17.46424919657298,
  12.083045973594572,
  15.811388300841896,
  20.615528128088304,
  29.154759474226502,
  18.027756377319946,
  20.248456731316587,
  15.811388300841896,
  13.601470508735444,
  21.213203435596427,
  18.384776310850235,
  13.45362404707371,
  14.866068747318506,
  17.804493814764857,
  24.041630560342615,
  28.071337695236398,
  26.92582403567252,
  8.0,
  10.770329614269007,
  15.811388300841896,
  20.223748416156685,
  25.709920264364882,
  23.323807579381203,
  13.038404810405298,
  20.615528128088304,
  0.0,
  10.0],
 [7.0710678118654755,
  52.92447448959697,
  51.07837115648854,
  51.97114584074513,
  52.478567053607705,
  50.24937810560445,
  49.09175083453431,
  50.21951811795888,
  57.982756057296896,
  52.43090691567332,
  52.69724850502159,
  54.08326913195984,
  60.8276253029822,
  57.0087712549569,
  58.008620049092706,
  56.302753041036986,
  80.99382692526635,
  79.92496481075234,
  23.600847442411894,
  83.52843827104634,
  81.49233092751734,
  80.06247560499239,
  86.82165628459296,
  83.72574275573791,
  87.66413177577246,
  86.6083136886985,
  83.64807230295268,
  90.21086409075129,
  88.19297024139736,
  91.92388155425118,
  91.92388155425118,
  90.91754506144565,
  89.00561780022652,
  93.81364506296512,
  93.53608929178085,
  90.35485598461214,
  92.59049627256569,
  81.49846624323675,
  81.2219182240853,
  81.09870529176159,
  80.15609770940699,
  84.29116205154607,
  84.09518416651456,
  84.02380615040002,
  87.36704184073076,
  87.02298546935747,
  89.35882720806042,
  89.20201791439474,
  89.02246907382428,
  91.35097153287424,
  94.08506789071261,
  94.0,
  83.57032966310472,
  84.69356528095862,
  85.7962703152066,
  87.32124598286491,
  89.0,
  87.31551981177229,
  89.55445270895244,
  89.1852005660132,
  93.1074647920348,
  93.637599285757,
  94.33981132056604,
  96.25487000666512,
  96.00520819205592,
  94.86832980505137,
  101.76934705499491,
  45.967379738244816,
  54.08326913195984,
  46.09772228646444,
  47.634021455258214,
  53.075418038862395,
  56.586217403180434,
  52.630789467763066,
  55.226805085936306,
  56.586217403180434,
  54.20332093147061,
  57.0087712549569,
  58.89821729050889,
  60.802960454241045,
  63.25345840347388,
  15.811388300841896,
  11.180339887498949,
  23.853720883753127,
  19.209372712298546,
  16.278820596099706,
  13.45362404707371,
  12.806248474865697,
  10.0,
  16.401219466856727,
  19.4164878389476,
  18.027756377319946,
  15.811388300841896,
  16.15549442140351,
  18.027756377319946,
  12.083045973594572,
  15.811388300841896,
  20.615528128088304,
  25.495097567963924,
  15.0,
  17.029386365926403,
  13.038404810405298,
  11.180339887498949,
  15.811388300841896,
  13.341664064126334,
  9.0,
  10.04987562112089,
  11.704699910719626,
  28.600699292150182,
  30.463092423455635,
  25.0,
  12.806248474865697,
  17.204650534085253,
  23.021728866442675,
  23.853720883753127,
  25.317977802344327,
  20.09975124224178,
  20.248456731316587,
  25.0,
  10.0,
  0.0]]
    data['demands'] = [0,
 25,
 7,
 13,
 6,
 14,
 5,
 11,
 19,
 5,
 15,
 15,
 17,
 13,
 12,
 18,
 13,
 18,
 12,
 17,
 4,
 7,
 12,
 13,
 8,
 16,
 15,
 6,
 5,
 9,
 11,
 10,
 3,
 7,
 2,
 4,
 4,
 18,
 14,
 12,
 17,
 20,
 14,
 16,
 10,
 9,
 11,
 7,
 13,
 5,
 4,
 21,
 13,
 11,
 12,
 14,
 10,
 8,
 16,
 19,
 5,
 17,
 7,
 16,
 14,
 17,
 13,
 17,
 13,
 14,
 16,
 7,
 13,
 9,
 11,
 35,
 5,
 28,
 7,
 3,
 10,
 7,
 12,
 11,
 10,
 8,
 11,
 21,
 4,
 15,
 16,
 4,
 16,
 7,
 10,
 9,
 11,
 17,
 12,
 11,
 7,
 9,
 11,
 12,
 7,
 8,
 6,
 5,
 12,
 13,
 7,
 7,
 8,
 11,
 13,
 11,
 10,
 7,
 4,
 20,
 13]
    data['num_vehicles'] = 7
    data['vehicle_capacities'] = [200]*data['num_vehicles']
    data['depot'] = 0
    return data

In [5]:
# def data_CMT2():
#     data = {}
#     data["distance_matrix"] = []
#     data['demands'] = []
#     data['num_vehicles'] = 10
#     data['vehicle_capacities'] = [140]*data['num_vehicles']
#     data['depot'] = 0
#     return data